In [1]:

import sys
import os
import io
import warnings
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.exceptions import NotFoundError
import pandas as pd
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor  # Use ProcessPoolExecutor for multiprocessing
from unstructured.partition.html import partition_html
from unstructured.partition.text import partition_text
from unstructured.cleaners.core import (
    clean,
    clean_non_ascii_chars,
    replace_unicode_quotes,
    group_broken_paragraphs,
    clean_bullets,
    clean_extra_whitespace,    
    group_broken_paragraphs
)

from lxml import etree
import time
from typing import List, Tuple

from haystack_integrations.components.embedders.fastembed import FastembedSparseDocumentEmbedder
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack.components.preprocessors import DocumentSplitter
from haystack_integrations.components.embedders.cohere import CohereDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import NLTKDocumentSplitter
from haystack.utils import Secret
from qdrant_client import QdrantClient
# import pynvml  # For GPU temperature monitoring
import contextlib  # For suppressing stdout and stderr
import logging  # For adjusting logging levels
from joblib import Parallel, delayed
from math import ceil
import warnings
import cohere
import httpx
import re
from bs4 import BeautifulSoup

cvdr_html = """<div role="main" id="content">
            <ul class="pageactions">
               <li><a href="#" data-decorator="init-modal" data-handler="open-modal" data-modal="modal-1"><img src="/cb-common/2.0.0/images/icon-permalink.svg" alt="" role="presentation"/>Permalink</a></li>
               <li><a href="" data-decorator="init-printtrigger"><img src="/cb-common/2.0.0/images/icon-print.svg" alt="" role="presentation"/>Printen</a></li>
            </ul>
            <div id="modal-1" class="modal modal--off-screen" hidden="" role="alert" tabindex="0">
               <div class="modal__inner">
                  <div class="modal__content">
                     <h2>Permanente link</h2>
                     <h3>Naar de actuele versie van de regeling</h3>
                     <div class="row">
                        <div class="copydata " data-decorator="init-copydata" data-config="{       &quot;triggerLabel&quot;: &quot;Kopieer link&quot;,       &quot;triggerCopiedlabel&quot;: &quot;&quot;,       &quot;triggerClass&quot;: &quot;copydata__trigger&quot;       }">
                           <p class="copydata__datafield js-copydata__datafield">http://lokaleregelgeving.overheid.nl/CVDR390943</p>
                        </div>
                     </div>
                     <h3>Naar de door u bekeken versie</h3>
                     <div class="row">
                        <div class="copydata " data-decorator="init-copydata" data-config="{       &quot;triggerLabel&quot;: &quot;Kopieer link&quot;,       &quot;triggerCopiedlabel&quot;: &quot;&quot;,       &quot;triggerClass&quot;: &quot;copydata__trigger&quot;       }">
                           <p class="copydata__datafield js-copydata__datafield">http://lokaleregelgeving.overheid.nl/CVDR390943/1</p>
                        </div>
                     </div>
                  </div><button type="button" data-handler="close-modal" class="modal__close"><span class="visually-hidden">sluiten</span></button></div>
            </div>
            <div xmlns="http://www.w3.org/1999/xhtml" xmlns:cl="https://koop.overheid.nl/namespaces/component-library" xmlns:local="urn:local" xmlns:utils="https://koop.overheid.nl/namespaces/utils" id="PaginaContainer">
               <div class="inhoud">
                  <div id="inhoud">
                     <div id="cvdr_meta">
                        <div class="js-scrollContentElement">
                           <h1 class="h1">Regeling piketdienst gemeente Westervoort 2016</h1>
                           <p><span class="label label--info label--lg"><i>Geldend van 12-01-2016 t/m heden met terugwerkende kracht vanaf 01-01-2016</i></span></p>
                        </div>
                        <section class="section-chapter js-scrollSection" id="intitule">
                           <h4 class="Intitulé">Intitulé</h4>Regeling piketdienst gemeente Westervoort 2016</section>
                        <div class="regeling"><!--TCID regeling-->
                           <div class="aanhef"><!--TCID aanhef-->
                              <p>Burgemeester en wethouders van Westervoort:</p>
                              <p/>
                              <p>gelet op het bepaalde in de Collectieve Arbeidsvoorwaardenregeling en de Uitwerkingsovereenkomst
                                 (CAR/UWO) laatstelijk gewijzigd op 16 december 2014;</p>
                              <p/>
                              <p>gelet op de overeenstemming met de commissie voor Georganiseerd Overleg van de gemeente
                                 Westervoort d.d. 9 december 2015; </p>
                              <p/>
                              <p/>
                              <p/>
                              <p> B E S L U I T E N</p>
                              <p/>
                              <p/>
                              <p>vast te stellen de “Regeling piketdienst gemeente Westervoort 2016”.</p>
                              <p/>
                           </div>
                           <div class="regeling-tekst"><!--TCID regeling-tekst-->
                              <section class="section-chapter js-scrollSection" id="artikel_1">
                                 <div class="artikel">
                                    <container>
                                       <h2 class="docArtikel">Artikel 1 Piketdienst</h2>
                                       <ul class="list-custom list-custom--size-xs whitespace-small">
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">a.</span> Onder piketdienst wordt verstaan het zich op last van het college van burgemeester
                                                en wethouders beschikbaar houden van de medewerk(st)er buiten de voor hem/haar geldende
                                                werktijd.</p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">b.</span> De medewerk(st)er die een piketdienst heeft, moet vanaf maandagavond 16.15 uur tot
                                                de daaropvolgende maandag 9.00 uur, buiten de reguliere werktijden, beschikbaar zijn
                                                en draagt tijdens deze periodes een mobiele telefoon bij zich.</p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">c.</span> Wanneer de omstandigheden dit vereisen kunnen nadere aanwijzingen aan de geconsigneerde
                                                medewerk(st)er worden gegeven wat betreft zijn standplaats.</p>
                                          </li>
                                       </ul>
                                    </container>
                                 </div>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_2">
                                 <h2 class="docArtikel">Artikel 2 Onderscheiden piketdiensten</h2>
                                 <p>De volgende piketdiensten vallen onder de werking van deze regeling:</p>
                                 <ul class="list-custom list-custom--size-xs ">
                                    <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>beveiliging gemeentehuis;</p>
                                    </li>
                                    <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>beveiliging van andere gebouwen die eigendom zijn van de gemeente;</p>
                                    </li>
                                    <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>dringende milieumeldingen;</p>
                                    </li>
                                    <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>bereikbaarheid in de openbare ruimte (wateroverlast, ongelukken, wegafzettingen, storingen
                                          aan verkeersvoorzieningen etc.) en</p>
                                    </li>
                                    <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>rioolstoringen (pompen en gemalen).</p>
                                    </li>
                                 </ul>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_3">
                                 <h2 class="docArtikel">Artikel 3 Functies piketdiensten</h2>
                                 <p>De piketdiensten zijn als volgt verdeeld over verschillende functies:</p>
                                 <table class="table ">
                                    <colgroup>
                                       <col style="width: 50%;"/>
                                       <col style="width: 50%;"/>
                                    </colgroup>
                                    <tbody>
                                       <tr>
                                          <td>
                                             <p>beveiliging gemeentehuis</p>
                                          </td>
                                          <td>
                                             <p>bode</p>
                                          </td>
                                       </tr>
                                       <tr>
                                          <td>
                                             <p>beveiliging van andere gebouwen die eigendom zijn van de gemeente</p>
                                             <p>dringende milieumeldingen</p>
                                             <p>bereikbaarheid in de openbare ruimte (wateroverlast, ongelukken, wegafzettingen, storingen
                                                aan verkeersvoorzieningen etc.)</p>
                                             <p>rioolstoringen (pompen en gemalen)</p>
                                          </td>
                                          <td>
                                             <p>medewerker buitendienst</p>
                                             <p>senior medewerker buitendienst</p>
                                             <p>coördinator buitendienst</p>
                                             <p>opzichter cultuurtechniek</p>
                                             <p>civieltechnisch medewerker</p>
                                          </td>
                                       </tr>
                                    </tbody>
                                 </table>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_4">
                                 <div class="artikel">
                                    <container>
                                       <h2 class="docArtikel">Artikel 4 Rooster piketdiensten</h2>
                                       <ul class="list-custom list-custom--size-xs whitespace-small">
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">a.</span> De afdelingsmanager Ondersteuning stelt vooraf per half jaar een rooster op voor
                                                de bodes. In overleg met de afdelingsmanager kunnen de bodes van dienst ruilen.</p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">b.</span> De coördinator buitendienst stelt vooraf per half jaar een rooster op voor de medewerkers
                                                van afdeling Openbare Ruimte. Hij registreert en overlegt met de aangewezen ambtenaren.
                                                Ruiling van dienst is mogelijk in overleg met de coördinator buitendienst.</p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">c.</span> De afdelingsmanager van Openbare Ruimte stelt een instructie op over de te nemen
                                                maatregelen voor de piketdiensten die door de medewerk(st)ers van afdeling Openbare
                                                Ruimte worden uitgevoerd.</p>
                                          </li>
                                       </ul>
                                    </container>
                                 </div>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_5">
                                 <div class="artikel">
                                    <container>
                                       <h2 class="docArtikel">Artikel 5 Bereikbaarheid</h2>
                                       <ul class="list-custom list-custom--size-xs whitespace-small">
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">a.</span> Van de medewerk(st)er die wordt aangewezen voor het verrichten van piketdiensten,
                                                wordt verlangd dat hij of zij zich niet buiten een straal van 15 kilometer om Westervoort
                                                begeeft en te allen tijde binnen 30 minuten na oproep op de plaats van de te verrichten
                                                werkzaamheden aanwezig kan zijn. </p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">b.</span> De aangewezen ambtenaar dient tijdens de piketdienstperiode te allen tijde op het
                                                afgesproken telefoonnummer bereikbaar te zijn. </p>
                                          </li>
                                       </ul>
                                    </container>
                                 </div>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_6">
                                 <div class="artikel">
                                    <container>
                                       <h2 class="docArtikel">Artikel 6 Toelage</h2>
                                       <ul class="list-custom list-custom--size-xs whitespace-small">
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">a.</span> De medewerk(st)er die bij rooster, conform artikel 3 van deze regeling, deelneemt
                                                aan de piketregeling ontvangt een toelage beschikbaarheidsdienst. De vergoeding bedraagt
                                                overeenkomstig het gestelde in artikel 3:13 lid 2 en 3 CAR-UWO. De vergoeding wordt
                                                op declaratiebasis uitbetaald, op basis van een opgave, ondertekend door de afdelingsmanager.
                                                </p>
                                          </li>
                                          <li class="list-custom__item">
                                             <p class="lid labeled"><span class="list-custom__itembullet">b.</span> De medewerk(st)er ontvangt een vergoeding over de uren gedurende welke hij/zij tijdens
                                                deze beschikbaarheidsdienst – na een oproep daartoe – werkzaamheden heeft verricht.</p>
                                          </li>
                                       </ul>
                                    </container>
                                 </div>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_7">
                                 <h2 class="docArtikel">Artikel 7 Ziekte of verlof</h2>
                                 <p>Bij ziekte of verlof van de medewerk(st)er die een piketdienst heeft, zal de vervanging
                                    van deze medewerk(st)er worden geregeld door de coördinator buitendienst. De coördinator
                                    buitendienst deelt dit mede aan de afdelingsmanager. In het geval dat een ingeroosterde
                                    ambtenaar moet worden vervangen als gevolg van verlof wordt hem over de vervangingsperiode
                                    geen vergoeding toegekend en uitbetaald. In deze situatie wordt de uitbetaling gedaan
                                    aan de vervang(st)er. In het geval dat een ingeroosterde ambtenaar moet worden vervangen
                                    als gevolg van ziekte wordt hem voor de betreffende week wel de vergoeding uitbetaald.
                                    De vervang(st)er ontvangt tevens de vergoeding.</p>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_8">
                                 <h2 class="docArtikel">Artikel 8 Onvoorzien</h2>
                                 <p>In gevallen, waarin deze regeling niet voorziet, wordt door burgemeester en wethouders
                                    incidenteel beslist.</p>
                              </section>
                              <section class="section-chapter js-scrollSection" id="artikel_9">
                                 <h2 class="docArtikel">Artikel 9 Citeertitel en inwerkingtreding</h2>
                                 <p>Deze regeling wordt aangehaald als “Regeling piketdienst gemeente Westervoort 2016”
                                    en treedt in werking na bekendmaking en werkt terug tot 1 januari 2016. Onder gelijke
                                    datum wordt de regeling Piketdienst Gemeente Westervoort 2011 ingetrokken.</p>
                                 <p/>
                                 <p/>
                                 <p>Aldus vastgesteld door burgemeester en wethouders van Westervoort op ………….</p>
                                 <p>de secretaris, de burgemeester,</p>
                                 <p>P.J.E. Breukers drs. A.J. van Hout</p>
                              </section>
                           </div>
                           <section class="section-chapter js-scrollSection" id="nota-toelichting"><!--TCID nota-toelichting-->
                              <h2 class="docToelichting">Toelichting </h2>
                              <h3 class="hoofdkop">Regeling piketdienst gemeente Westervoort 2016</h3>
                              <p/>
                              <p><strong class="tussenkop">Artikel 1</strong></p>
                              <p>Het begrip piketdienst: De regeling verstaat onder piketdienst het zich op last van
                                 burgemeester en wethouders beschikbaar houden buiten de normale diensttijd.</p>
                              <p/>
                              <p>Voor de uitvoering is aansluiting gezocht in de CAR/UWO art. 3:13 (toelage beschikbaarheidsdienst),
                                 art. 3:12 (buitendagvenstertoelage voor de ambtenaar die valt onder de standaardregeling
                                 voor de werktijden) en art. 3:18 (overwerkvergoeding voor de ambtenaar die valt onder
                                 de bijzondere regeling voor de werktijden).</p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 2</strong></p>
                              <p>Er blijken een aantal situaties te bestaan waarvoor medewerk(st)ers buiten kantooruren
                                 bereikbaar moeten zijn en waarvoor uniforme richtlijnen noodzakelijk zijn, nl.:</p>
                              <ul class="list-custom list-custom--size-xs ">
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>beveiliging gemeentehuis;</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>beveiliging andere gebouwen die eigendom zijn van de gemeente;</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>dringende milieumeldingen;</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>bereikbaarheid in de openbare ruimte (wateroverlast, ongelukken, wegafzettingen, storingen
                                       aan verkeersvoorzieningen etc.) en</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>rioolstoringen (storingen van pompen en gemalen).</p>
                                    <p/>
                                 </li>
                              </ul>
                              <p>Bij beveiliging gaat het om meldingen bij scholen, het gemeentehuis en gebouwen die
                                 eigendom zijn van de gemeente.</p>
                              <p>Een melding bij de school gebeurt in eerste instantie bij de directeur van de school,
                                 vervolgens bij een leerkracht. Zijn deze personen niet te bereiken dan wordt contact
                                 opgenomen met de aangewezen ambtenaar voor de piketdienst.</p>
                              <p>Meldingen bij het gemeentehuis en andere gebouwen die eigendom zijn van de gemeente
                                 gaan direct naar de aangewezen ambtenaar voor de piketdienst. </p>
                              <p>Verwacht wordt dat de aangewezen medewerker maatregelen treft of laat treffen die
                                 de onveilige situatie opheffen.</p>
                              <p>Bij dringende milieumeldingen wordt verwacht dat de medewerker direct contact opneemt
                                 met een extern gespecialiseerd bedrijf, tenzij hij dit zelf adequaat kan oplossen.</p>
                              <p>Bij rioolstoringen wordt verwacht dat de medewerker de storing zelf verhelpt danwel
                                 externen inschakelt om de storing te verhelpen.</p>
                              <p>Bij bereikbaarheid in de openbare ruimte gaat het om meldingen in de openbare ruimte
                                 waar acute actie noodzakelijk is.</p>
                              <p>Bij rioolstoringen wordt verwacht dat de medewerker de storing zelf verhelpt danwel
                                 externen inschakelt om de storing te verhelpen.</p>
                              <p>Ten behoeve van de instructie is een draaiboek opgesteld met verantwoordelijkheden
                                 en bevoegdheden. </p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 3</strong></p>
                              <p>Bij de verschillende piketdiensten is aansluiting gezocht bij de inhoud van de verschillende
                                 functies.</p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 4</strong></p>
                              <p>In het rooster wordt vastgelegd welke medewerk(st)er wanneer en voor welke piketdienst
                                 bereikbaar moet zijn buiten de kantooruren. </p>
                              <p>Via <strong class="vet">een instructie</strong> kunnen uitvoeringszaken worden geregeld. Te denken valt dan aan:</p>
                              <ul class="list-custom list-custom--size-xs ">
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>vervanging tussen aangewezen ambtenaren onderling;</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>registratie van meldingen, zoals datum melding, tijdstip binnenkomst; </p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>aard van de melding, naam van de melder; </p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>afhandeling van de melding; </p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>tijdbeslag daadwerkelijk verrichte werkzaamheden en</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>taken, bevoegdheden en verantwoordelijkheden.</p>
                                 </li>
                              </ul>
                              <p>Het bereikbaar zijn geldt voor de duur van een piketdienst. Tegenover deze dienst
                                 staat een toelage beschikbaarheidsdienst. Dit is een bruto-vergoeding<strong class="vet">. </strong>Bij een feitelijke “uitruk” start de overwerkvergoeding of buitendagvenstertoelage
                                 (zie toelichting artikel 5).</p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 5</strong></p>
                              <p>De bereikbaarheid en tijdige aanwezigheid en de wijze waarop de werkzaamheden dienen
                                 te worden verricht zijn belangrijke bepalingen in de regeling en zijn daarom in een
                                 apart artikel opgenomen.</p>
                              <p>Het begrip bereikbaar: Door gebruik van de mobiele telefoon is het niet nodig dat
                                 men thuis bereikbaar is. Wel dient men binnen een bepaalde tijd op de standplaats
                                 aanwezig te kunnen zijn. Opgenomen is dat de medewerker zich niet buiten een straal
                                 van 15 kilometer om Westervoort begeeft en derhalve binnen 30 minuten na de oproep
                                 aanwezig moet zijn op de standplaats.</p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 6</strong></p>
                              <p>Op declaratiebasis (eens per kwartaal) worden de vergoedingen berekend en via de salarisadministratie
                                 uitbetaald. Mocht er vanuit de fiscale wetgeving aanleiding toe zijn, dan kan de frequentie
                                 van de in te dienen declaraties wijzigen.</p>
                              <p>Naast de toelage beschikbaarheidsdienst kent de regeling een vergoeding voor overwerk
                                 of een buitendagvenstertoelage toe wanneer als gevolg van een calamiteit feitelijke
                                 werkzaamheden moeten worden verricht. Voor deze vergoeding geldt het volgende:</p>
                              <ul class="list-custom list-custom--size-xs ">
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>De ambtenaar die valt onder de standaardregeling voor de werktijden, ontvangt over
                                       de tijdens deze dienst gewerkte uren die buiten het dagvenster vallen, een buitendagvenstertoelage
                                       op grond van artikel 3:12 CAR-UWO.</p>
                                 </li>
                                 <li class="list-custom__item"><span class="list-custom__itembullet"><!--ins-->• </span><p>De ambtenaar die valt onder de bijzondere regeling voor de werktijden heeft over alle
                                       tijdens deze dienst gewerkte uren recht op een overwerkvergoeding op grond van artikel
                                       3:18 CAR-UWO.</p>
                                    <p/>
                                 </li>
                              </ul>
                              <p><strong class="tussenkop">Artikel 7</strong></p>
                              <p>Er is een duidelijk onderscheid in geen piketdienst kunnen draaien als gevolg van
                                 verlof, of geen piketdienst kunnen draaien als gevolg van ziekte. Ziekte wordt aangemerkt
                                 als een overmachtsituatie, waarbij de medewerker zich toch al van tevoren beperkingen
                                 op heeft moeten leggen als gevolg van de piketdienst. Daarom wordt over de betreffende
                                 week waarin de medewerker wel piket heeft maar waarin deze ziek wordt wel de piketvergoeding
                                 uitbetaald. Mocht de ziekte langdurig zijn, dan zal de afdelingsmanager bij de bodes
                                 en de coördinator buitendienst (in overleg met de afdelingsmanager Openbare Ruimte)
                                 het rooster voor piket aan moeten passen en wordt de medewerker automatisch uitgeroosterd.
                                 Een piketvergoeding is dan niet meer van toepassing.</p>
                              <p/>
                              <p><strong class="tussenkop">Artikel 8</strong></p>
                              <p>Het betreffen hier incidentele niet voorzienbare situaties.</p>
                           </section>
                        </div>
                     </div>
                     <div id="voetregel"/>
                  </div>
               </div>
            </div>
            <div xmlns:cl="https://koop.overheid.nl/namespaces/component-library" xmlns:local="urn:local" xmlns:utils="https://koop.overheid.nl/namespaces/utils" class="row row--large"><a name="fout-in-regeling"/><div class="alert alert--success alert--lg icon--document " role="alert">
                  <h2 class="alert__heading">Ziet u een fout in deze regeling?</h2>
                  <div class="alert__inner">
                     <p>Bent u van mening dat de inhoud niet juist is? Neem dan contact op met de organisatie
                        die de regelgeving heeft gepubliceerd. Deze organisatie is namelijk zelf verantwoordelijk
                        voor de inhoud van de regelgeving. De naam van de organisatie ziet u bovenaan de regelgeving.
                        De contactgegevens van de organisatie kunt u hier opzoeken: <a href="https://organisaties.overheid.nl/">organisaties.overheid.nl</a>.</p>
                     <p>Werkt de website of een link niet goed? Stuur dan een e-mail naar <a href="mailto:regelgeving@overheid.nl">regelgeving@overheid.nl</a></p>
                  </div>
               </div>
            </div>
         </div>
      """
oor_txt = """

 
 
 
 
 
TER INFORMATIE 
Aan 
de voorzitter en de leden van het overleg van secretarissen-generaal 
Auditdienst Rijk 
 
 
 
 
Datum 
8 april 2022 
 
Notanummer 
2022-7351 
 
Bijlagen 
1. 
Versterken eerste en 
tweede lijn van 
verantwoordelijkheden 
 
Pagina 1 van 8 
 
 
Hoofdlijnennota rijksbreed beeld 2021 
 
1. Inleiding 
In deze nota schetsen wij ons rijksbrede beeld over 2021. Dit is gebaseerd op de 
kennis en inzichten die wij hebben opgedaan bij de controle van de jaarverslagen 
2021 in het kader van onze wettelijke taak en bij onze overige onderzoeken voor 
de rijksdienst. Waar relevant besteden we aandacht aan de bijzondere 
omstandigheden waarin de rijksdienst het afgelopen jaar verkeerde.  
 
Belangrijke aandachtspunten: 
-  
schep helderheid over de criteria op grond waarvan het geld uit de nieuwe 
fondsen mag worden besteed en over de wijze van verantwoording; 
-  
investeer in fraudebeheersing door te verkennen welke typen activiteiten, 
werkgebieden en processen kwetsbaar zijn voor omvangrijke fraude en hoe 
deze risico’s kunnen worden gemitigeerd; 
-  
richt adequaat risicomanagement in voor het signaleren van kwetsbaarheden 
in de informatiebeveiliging; 
-  
houd vanuit het rijksbrede programma Open op Orde zicht op de inhoudelijke 
voortgang op het gebied van de informatiehuishouding. 
 
 
Op verzoek van het overleg van Secretarissen-Generaal gaan wij apart in op 
goede praktijken en knelpunten in het functioneren van de eerste en tweede lijn 
van verantwoordelijkheid op de departementen en uitvoeringsorganisaties (bijlage 
1). 
 
2. Uitkomsten controle jaarverslagen 2021 
 
Getrouwheid: goedkeurende controleverklaringen, één verklaring met 
beperking 
Bij de jaarverslagen 2021 van alle begrotingshoofdstukken hebben wij op één 
uitzondering na goedkeurende controleverklaringen afgegeven over het getrouwe 
beeld daarvan. Bij de financiële verantwoording van het ministerie van VWS 
hebben wij – net als vorig jaar – een controleverklaring met beperking afgegeven. 
Sinds het uitbreken van de COVID-19-pandemie in 2020 staat het financieel 
beheer bij VWS onder grote druk. Dit heeft ertoe geleid dat geen zekerheid kon 
worden verkregen over de getrouwe verantwoording van omvangrijke bedragen. 
De ministers moeten in de bedrijfsvoeringparagraaf van hun jaarverslag melding 
maken van fouten en onzekerheden ten aanzien van de comptabele 
rechtmatigheid indien het bedrag daarvan een bepaalde grens 
(rapporteringstolerantie) overschrijdt. Indien dit juist en volledig gebeurt, staat 
dit een goedkeurende controleverklaring van de Auditdienst Rijk niet in de weg. 


 
 
 
Pagina 2 van 8 
 
Rechtmatigheid: veel overschrijdingen van de rapporteringstoleranties, 
vooral ten aanzien van de aangegane verplichtingen 
In de jaarverslagen 2021 zien we zes overschrijdingen van de 
rapporteringstoleranties voor de comptabele rechtmatigheid op hoofdstukniveau. 
Hierbij gaat het om vijf overschrijdingen voor de aangegane verplichtingen (EZK, 
VWS, DEF, FIN en BZK) en één overschrijding voor de uitgaven en ontvangsten 
(VWS). Voor het totaal van de in 2021 op alle begrotingshoofdstukken aangegane 
verplichtingen komen wij tot een bedrag aan rechtmatigheidsfouten en 
-onzekerheden van meer dan € 12 miljard. Dit is uitzonderlijk hoog. Deze fouten 
en onzekerheden zijn vooral gerelateerd aan de noodmaatregelen als gevolg van 
de coronacrisis. Zij worden voor het grootste deel veroorzaakt door het niet tijdig 
informeren van de Eerste Kamer over in uitvoering genomen nieuw beleid (vooral 
in de eerste maanden van het jaar) en door de niet-naleving van procedures voor 
inkoop en aanbesteding. 
 
 
 
Financieel en materieelbeheer: een solide basis zorgt voor stevig beheer, 
een gebrek daaraan leidt tot hardnekkige bevindingen 
Rijksbreed is het aantal bevindingen dat wij in onze auditrapporten bij de 
jaarverslagen rapporteren over het gevoerde financieel en materieelbeheer in 
2021 ten opzichte van vorig jaar iets toegenomen. Hoewel het aantal ernstige 
bevindingen is verminderd, is er sprake van een behoorlijke toename van de als 
gemiddeld aangemerkte bevindingen.  
 
 
De belangrijkste onderwerpen betreffen (1) beheer van IT-systemen en 
informatiebeveiliging, (2) interne beheersing/controle (waaronder 
materieelbeheer), (3) beheer van inkopen en aanbestedingen en (4) beheer van 
3
1
16
4
1
19
8
5
38
6
5
36
0
10
20
30
40
Hoofdstukken
Agentschappen
Artikelen en voorschotten
A antal overschrijdingen
2018
2019
2020
2021
0
23
33
56
1
22
40
63
4
17
33
54
2
25
30
57
0
10
20
30
40
50
60
70
Ernstig
Gemiddeld
Licht
Totaal
A antal beheerbevindingen
2018
2019
2020
2021


 
 
 
Pagina 3 van 8 
 
voorschotten en verplichtingen. Over de eerste twee onderwerpen hebben wij de 
meeste nieuwe bevindingen. Bij enkele departementen is het beheer verbeterd, 
bij andere ministeries is het echter verslechterd of onvoldoende verbeterd. Onze 
bevindingen kunnen bijvoorbeeld leiden tot risico’s voor de continuïteit van IT-
systemen, binnendringing in systemen van buitenaf of ongeautoriseerde 
transacties door niet-bevoegde medewerkers, risico’s op onrechtmatige 
aanbestedingen die leiden tot juridische procedures van potentiële leveranciers en 
risico’s op ontbrekend zicht op verstrekte voorschotten en aangegane 
verplichtingen waardoor de ambtelijke en politieke leiding en de Staten-Generaal 
hierover niet juist of volledig geïnformeerd zijn. Het duurzaam oplossen van 
bevindingen vraagt tijd. Bijna de helft van de beheerbevindingen betreft 
hardnekkige bevindingen waarvan wij al meerdere jaren melding maken. 
 
3. Inrichting en verantwoording fondsen  
Het coalitieakkoord voorziet in het oprichten van nieuwe fondsen voor het 
realiseren van ambitieuze doelstellingen, zoals een Klimaat- en transitiefonds, een 
Stikstoffonds en een Onderzoeks- en ontwikkelfonds. Hiermee geeft het kabinet 
het signaal af de aanpak van de grote uitdagingen van de komende decennia 
daadwerkelijk te willen borgen met beleid en investeringen gericht op de lange 
termijn. Het Stikstof- en het Klimaat- en transitiefonds worden ingericht als 
begrotingsfondsen conform de Comptabiliteitswet (artikel 2.11), met een aparte 
instellingswet. De gelden voor het Onderzoeks- en ontwikkelingsfonds worden aan 
de reguliere begroting van het ministerie van OCW toegevoegd. 
 
Het is zaak helderheid te scheppen over de criteria op grond waarvan het geld uit 
de fondsen mag worden besteed en hoe daar transparant verantwoording over 
kan worden afgelegd. De doelmatige en doeltreffende besteding vraagt onder 
meer om antwoord op de volgende vragen: 
- 
hanteren we een tijdshorizon van enkele jaren of van een langere periode, 
met alle onzekerheden en aannames die daar bij horen? 
- 
kijken we ook vanuit het perspectief van (elementen van) brede welvaart? 
- 
is er oog voor de onderlinge samenhang tussen de investeringen? 
 
Wij adviseren de ervaringen met, en lessen van de inrichting en aansturing van 
fondsen ter harte te nemen: 
- 
formuleer en hanteer heldere doelen, criteria en randvoorwaarden voor de 
toekenning van de gelden, ook rekening houdend met de doelen en criteria 
die gelden voor al bestaande fondsen, zoals het Nationaal Groeifonds, het 
Mobiliteitsfonds en het Deltafonds; 
- 
bepaal vooraf het perspectief op doelmatigheid; 
- 
benoem onafhankelijke, materiedeskundige adviescommissies die een 
eenduidig mandaat meekrijgen en transparant rapporteren aan de ministers 
en de beide Kamers; 
- 
bied ruimte voor pilots, laat deze tussentijds onafhankelijk evalueren op basis 
van de vooraf bepaalde dimensies van welvaart en bepaal vooraf de criteria 
voor stopzetten, opschalen en afschalen; 
- 
laat onafhankelijke partijen zoals de planbureaus adviseren over de 
samenhang van de voorgenomen investeringen, zodat parlement en kabinet 
uiteindelijke ook integrale afwegingen (over de grenzen van de fondsen en 
beleidsvelden heen) kunnen maken.  
 
4. Beheersing frauderisico’s 
De aandacht voor fraude in de publieke sector is de laatste jaren toegenomen. Er 
kan sprake zijn van interne fraude (door ambtenaren of bestuurders binnen de 
overheid), of externe fraude (jegens de overheid). Het gaat dus om fraude van 
zowel derden als medewerkers van het ministerie met betrekking tot 
programmagelden (subsidies, uitkeringen, bijdragen, opdrachten, etc.) en 


 
 
 
Pagina 4 van 8 
 
apparaatsgelden (personele en materiële uitgaven). Frauderisico’s die 
voortvloeien uit door derden aangeleverde gegevens waarvan de uitvoering van 
wet- en regelgeving afhankelijk is, vallen onder de reikwijdte van het 
departementale beleid voor misbruik en oneigenlijk gebruik (M&O).  
 
Fraudebeheersing is een gezamenlijke verantwoordelijkheid voor de eerste en 
tweede lijn.1 Frauderisicobeheersing is een verplicht aandachtsveld in de jaarlijkse 
wettelijke controles van accountants. De ADR heeft in de controleverklaringen 
2021 een passage opgenomen over de controleaanpak frauderisico’s. Vorig jaar 
adviseerde de ADR in zijn tussentijds beeld het management en de interne 
toezichthouders een eigen fraudeanalyse te laten maken en adequate interne 
beheersmaatregelen te nemen. Uiteraard dient de meeste aandacht daarbij uit te 
gaan naar de grootste frauderisicofactoren (proportionaliteit). 
 
Wij adviseren meer specifiek:  
- 
voor het identificeren en analyseren van frauderisico’s de zogenoemde 
fraudedriehoek te hanteren. Dit is een beproefd instrument waarmee drie 
essentiële onderdelen van fraude in beeld worden gebracht: de gelegenheid, 
de motivatie en de rationalisatie.2  
- 
ten aanzien van ‘de gelegenheid’ te verkennen welke typen activiteiten, 
werkgebieden en processen kwetsbaar zijn voor fraude.3 Ook kunnen 
omstandigheden die de kwetsbaarheid verhogen of ‘rode vlaggen’ binnen de 
fraudedriehoek in kaart worden gebracht. Onderzoek laat namelijk zien dat 
bestedings-, prestatie- en tijdsdruk en de samenloop van publieke en private 
activiteiten extra kwetsbaarheden meebrengen;4  
- 
te zorgen dat alle betrokkenen zich bewust zijn van het belang van goede 
interne controles, goed voorbeeldgedrag van leidinggevenden (‘tone at the 
top’) en vroege signalering van fraude. 
 
5. Informatiebeveiliging  
 
Kosten en beperkte IT-capaciteit maken risicoanalyses urgenter  
In onze eerdere onderzoeken identificeerden wij naast verbeteringen op het 
gebied van de departementale informatiebeveiliging ook punten van zorg. Uit ons 
onderzoek naar de sturing en beheersing en onze DigiD-assesments blijkt dat veel 
organisaties hun (kritieke) systemen nog niet periodiek laten onderzoeken op 
kwetsbaarheden.  
 
Wij beseffen dat dergelijk onderzoek kostbaar kan zijn en ook steeds vaker stuit 
op capaciteitsproblemen op IT-gebied. Juist om die reden adviseren wij:  
- 
een adequaat risicomanagement in te richten om goede keuzes te kunnen 
maken voor het inzetten van dergelijke onderzoeken; 
- 
te bezien of voor de belangrijkste systemen quick scans kunnen worden 
uitgevoerd naar de belangrijkste kwetsbaarheden en de te nemen 
maatregelen; 
 
1 De eerstelijnsrol betreft het lijnmanagement van de (beleids)directies, die verantwoordelijk is voor de 
eigen processen. De tweedelijnsrol heeft betrekking op het adviseren, ondersteunen, coördineren en 
bewaken dat het lijnmanagement deze verantwoordelijkheden ook daadwerkelijk neemt. De directie FEZ 
van de departementen vervult deze tweedelijnsrol op het terrein van het begrotings- en financieel 
beheer; voor de andere terreinen van de bedrijfsvoering is de tweedelijnsrol elders in de departementale 
organisatie belegd. 
2 Gelegenheid: de mogelijkheid tot het plegen van fraude moet aanwezig zijn. Motivatie: de fraudeur(s) 
moet(en) verleiding, dan wel pressie ervaren om tot fraude over te gaan. Rationalisatie: het 
rechtvaardigen van de fraude, bijvoorbeeld vanuit de opvatting dat een hogere beloning voor prestaties 
van de betrokkene(n) op zijn plaats zou zijn. 
3 Denk aan de aanschaf, het beheer en het gebruik van goederen, het uit- of aanbesteden van 
opdrachten of orders, het verlenen van vergunningen of machtigingen, het omgaan met (vertrouwelijke) 
informatie of budgetten en het handhaven via toezicht, controles, inspecties of vervolging en opsporing. 
4 Het recente wereldwijde 2020 ACFE Report to the Nations concludeert op basis van analyses van meer 
dan 18.000 fraudegevallen sinds 1996 dat de werkwijzen van de fraudeurs vrij constant is gebleven. 


 
 
 
Pagina 5 van 8 
 
- 
(geautomatiseerde) kwetsbaarhedenscans of periodieke penetratietesten te 
laten uitvoeren. 
 
Het afgelopen jaar hebben twee departementen zogenoemde ‘red-teaming’ laten 
uitvoeren door externe partijen die de opdracht kregen om de digitale systemen 
binnen te dringen. Wij adviseren: 
- 
andere departementen dit goede voorbeeld te laten navolgen omdat hiermee 
het bewustzijn toeneemt over het belang van informatiebeveiliging bij het 
management en tegelijkertijd ook kwetsbaarheden zichtbaar worden; 
- 
de geanonimiseerde bevindingen en uitkomsten van mogelijke 
oorzakenanalyses in het kader van red teaming onderling te delen, om 
hiervan te kunnen leren.  
 
6. Informatiehuishouding 
 
Combineer decentrale invulling met centraal zicht op de voortgang 
Het rijksbrede programma ‘Open op Orde’ met de actieplannen voor verbetering 
van de informatiehuishouding loopt door tot 2026. Het heeft een totaalbudget van 
787 miljoen euro. De ADR voert diverse onderzoeken uit, rijksbreed en bij 
departementen. In ons tussentijds beeld constateerden wij op basis van de eerste 
fase van ons rijksbrede onderzoek dat de geplande nulmetingen van de stand van 
de informatiehuishouding en de uitvoeringstoetsen van de haalbaarheid van het 
actieplan vanwege tijdsdruk waren uitgesteld. Wij adviseerden de invulling en 
sturing van het programma en de rol van de Regeringscommissaris 
Informatiehuishouding te verduidelijken.  
 
In januari 2022 is de Regeringscommissaris gestart als onafhankelijke aanjager 
en adviseur voor het programma. Hij richt zich op de rijksbrede 
informatiehuishouding en daarmee zowel op de deelnemers als de 
programmaorganisatie. Ook hebben alle deelnemers inmiddels de nulmeting en 
de uitvoeringstoets uitgevoerd. Dit heeft geleid tot bijgestelde actieplannen die 
richting geven aan de tweede tranche van budgettoekenningen.  
Op basis van de tweede fase van ons onderzoek constateren wij dat er een 
duidelijke keuze is gemaakt voor een sterkere decentrale verantwoordelijkheid 
voor rijksonderdelen om hun informatiehuishouding op orde te brengen. 
Deelnemers zijn zelf verantwoordelijk voor het realiseren van acties en de op te 
leveren resultaten binnen het toegekende budget, inspelend op de 
organisatiespecifieke probleemanalyse binnen elk van de actielijnen. Daarbij richt 
de centrale sturing zich op het formuleren van rijksbrede actielijnen, 
doelstellingen en prioriteiten. Hierdoor is het centrale beoordelings- en 
verantwoordingsproces versimpeld, met vermindering van de uitvoeringslast. Een 
sterkere decentrale verantwoordelijkheid is overigens niet vrijblijvend en vraagt 
om gefundeerd vertrouwen. Wij adviseren:  
- 
vanuit het centrale programma voldoende zicht te houden op de inhoudelijke 
voortgang van de deelnemers, en waar nodig interventies uit te voeren om 
het gewenste ontwikkelniveau en de beoogde effecten te bereiken; 
- 
de deelnemers transparant te laten aangeven welke vorderingen ze boeken 
om tot het gewenste ontwikkelniveau te komen; 
- 
te verduidelijken op welke onderdelen rijksbrede standaarden en 
voorzieningen worden ontwikkeld; 
- 
een explicietere escalatiemogelijkheid via de ambtelijk-bestuurlijke lijn te 
overwegen als de individuele deelnemers onvoldoende prioriteit geven aan de 
informatiehuishouding. 
 
 


 
 
 
Pagina 6 van 8 
 
Bijlage 1. Versterken eerste en tweede lijn van 
verantwoordelijkheden 
 
In ons tussentijds beeld van 2021 hebben wij geadviseerd over manieren om de 
eerste- en tweedelijnsrollen van verantwoordelijkheden te versterken en de 
samenwerking met de derde lijn te bevorderen. Wij pleitten onder meer voor 
stimuleren van de onderlinge mobiliteit van medewerkers om de kennis van 
controle-, verantwoordings- en uitvoeringsvraagstukken te bevorderen. Op 
verzoek van het overleg van Secretarissen-Generaal hebben wij voor deze 
hoofdlijnennotitie een aantal recente voorbeelden verzameld van goede praktijken 
en verbeterpunten in dit samenspel. Deze staat hieronder opgesomd. 
 
Samenspel beleid, FEZ en uitvoering bij Sociale Zaken en 
Werkgelegenheid  
De samenwerking tussen de beleidsafdelingen en de directie FEZ van SZW met 
het onderdeel Uitvoering van Beleid van het ministerie en UWV verloopt goed. Dit 
was vorig jaar al zichtbaar bij het voortraject van de tijdelijke Noodmaatregel 
Overbrugging Werkgelegenheid (NOW) en dit jaar bij specifieke vragen rond de 
reviews van de NOW. Op het gebied van het echte concerncontrol en het 
verkrijgen van inzicht kan FEZ nog wel een stap maken. De opzet van de 
afspraken tussen het ministerie en de ADR over de aan te leveren informatie voor 
de jaarafsluiting en controle is goed, mede dankzij stimulansen door de tweede 
lijn. 
 
Visievorming op intern beheer en interne controle bij Defensie  
Defensie is bijna geheel zelfvoorzienend op het gebied van ICT. Passend bij het 
actuele besturingsmodel wil de Hoofddirectie Financiën en Control als 
concerncontroller een moderne, Defensiebrede visie ontwikkelen op het gebied 
van de interne beheersing en interne controle. In overeenstemming met het 
huidige model van de drie lijnen van verantwoordelijkheid zullen geen aparte 
functies maar rollen met bijbehorende expertise in de organisatie worden 
aangewezen. Er zal een risico- en themagericht roulerend programma worden 
uitgevoerd met de inzet van moderne technieken, waaronder data-analyse. 
Defensie heeft de ADR gevraagd hier op te reflecteren op basis van de kennis 
vanuit de jaarcontroles en de gevraagde onderzoeken bij Defensie en de 
interdepartementale ontwikkelingen en innovaties. 
 
Verbeterplannen financieel beheer bij Infrastructuur en Waterstaat 
Bij de uitvoering van controles op de werkzaamheden van de tweede lijn bij IenW 
loopt de ADR regelmatig tegen niet verklaarde afwijkingen aan. Hierdoor is het 
nog niet mogelijk volledig te steunen op de werkzaamheden van de tweede lijn.  
IenW is zich ervan bewust dat er op diverse vlakken verbeteringen nodig zijn in 
de aansturing en processen om in control te kunnen zijn. Het heeft een 
reorganisatie van FEZ doorgevoerd, is bezig diverse verbeterplannen uit te voeren 
en zal ook zelf data-analyses maken ter ondersteuning van de processen. De 
effecten van de geplande acties zullen in de komende jaren zichtbaar moeten 
gaan worden.  
 
Doorontwikkeling van toezicht bij Buitenlandse Zaken 
De departementsleiding van BZ heeft behoefte aan een effectievere beheersing 
omdat de eerste en tweede lijn er nu nog onvoldoende in slagen om hardnekkige 
en terugkerende fouten te detecteren of te voorkomen. Uit gesprekken met de 
concerncontroller FEZ blijkt dat er behoefte is aan verdere doorontwikkeling in het 
toezicht en dat daarvoor stappen voorzien zijn. Aangezien dit een taai proces 
blijkt te zijn met lange doorlooptijden, heeft de departementsleiding de ADR 
gevraagd te onderzoeken hoe het toezichtmodel ook in de praktijk goed kan gaan 
functioneren.  


 
 
 
Pagina 7 van 8 
 
 
Omgaan met grote veranderingen bij Financiën 
Bij Financiën zorgen grote ontwikkelingen zoals de ontvlechting van Douane en 
Toeslagen uit de Belastingdienst, de zorg voor modernere dienstverlening en IT 
en de gevolgen van wijzigingen en naleving van wetgeving voor onverminderd 
hoge druk op de uitvoering. Dit vraagt om een sterke toezichts- en adviesrol, en 
meer kennis van de comptabele wet- en regelgeving en control. De directie FEZ 
vervult als concerncontroller de centrale controlfunctie, met ondersteuning vanuit 
de decentrale controllerfuncties bij de DG’s van de uitvoeringsorganisaties. Ze 
geeft actief invulling aan haar rol. Er is een reguliere planning en control-cyclus 
met jaarplannen en viermaandelijkse rapportages. Daarnaast is er ook een 
commissie die de voortgang van de verbetermaatregelen voor onvolkomenheden 
en bevindingen uit de onderzoeken van de Algemene Rekenkamer en ADR 
monitort. Om beleids- en bedrijfsvoeringsrisico’s op concernniveau duidelijker en 
vollediger in beeld te krijgen en hier beter op te anticiperen, wordt een control-
raamwerk opgesteld. Aan de hand hiervan kan de kwaliteit van risicomanagement 
binnen alle lagen van de organisatie toenemen. Zo kunnen tekortkomingen in de 
bedrijfsvoering tijdiger worden gesignaleerd en kan effectieve sturing verbeteren.  
 
Opbouw Management Control Statement bij de Belastingdienst 
De Belastingdienst investeert in een traject om het Management Control 
Statement vanuit de dienstonderdelen op te bouwen. Verder besteedt de 
Belastingdienst veel aandacht aan het verbeteren van het financieel beheer 
rondom verplichtingen en prestatieverklaringen. Dit gebeurt onder meer door het 
versterken van de tweede lijn met het uitvoeren van IC-maatregelen. Deze acties 
zijn er op gericht de interne beheersing te verbeteren.  
 
Risicosignalering en samenspel rondom subsidies bij Economische Zaken 
en Klimaat en Landbouw, Natuur en Voedselkwaliteit 
De directie FEZ van LNV stelt brede, hoogwaardige toezichtrapportages op van 
het financieel en materieel beheer die uitmonden in concrete acties om de 
gesignaleerde risico’s te ondervangen. Ook heeft de afdeling Financiële Diensten 
en Administratie die onder FEZ valt een uitgebreid plan voor de intern controle 
opgesteld, dat een goede basis biedt voor het monitoren van onder meer het 
subsidieproces.  
De uitvoering van de controles van de COVID-19-subsidieregelingen, zoals de 
regeling voor de Tegemoetkoming Vaste Lasten (TVL) bij EZK vergt een goed 
samenspel tussen de tweede en derde lijn, inclusief de samenwerking met de 
Rijksdienst voor Ondernemend Nederland. Dit is nog in ontwikkeling en kan 
verder verbeteren. 
 
Vorming SSO voor financiële administraties bij Justitie en Veiligheid 
Het ministerie van JenV is van plan op korte termijn één Shared Service 
Organisatie (SSO) voor het hele J&V-conglomeraat te ontwikkelen, waardoor het 
makkelijker wordt kwaliteitsslagen te realiseren in de administratieve verwerking. 
Al vóór de SSO-vorming bouwde het ministerie routine op met tertaalgewijze 
(zachte) afsluiting van het ondersteunende software-systeem (Enterprise 
Resource Planning) voor alle eigen organisaties. Het werkt met een uitgebreide 
checklist voor uit te voeren werkzaamheden en bijbehorende bewijsstukken en 
draaiboeken met te hanteren deadlines. De SSO’s voeren een deel van de 
werkzaamheden rond de tertaalafsluitingen uit en de JenV-organisaties 
rapporteren hierover in een controlememo. De ADR beschouwt de tweede 
tertaalafsluiting als een ‘systeemtest’ voor de jaarafsluiting, en kan op grond van 
de bevindingen waar nodig early warnings afgeven in het najaar. Ook voert de 
directie FEZ mede door gebrek aan capaciteit nog maar beperkt controles en 
reviews uit op de opgeleverde balansdossiers. De ADR en FEZ zijn daarom in 
gesprek over de juiste verantwoordelijkheidsafbakening in het nieuwe model. 
 


 
 
 
Pagina 8 van 8 
 
Versterkte monitoring bij Binnenlandse Zaken en Koninkrijksrelaties 
Vanaf 2020 heeft BZK de follow up van bevindingen en onvolkomenheden meer 
aandacht gegeven. Inmiddels zijn aanvullende verbeterplannen opgesteld en is 
een monitoringscommissie actief om de voortgang bij te houden. Daarvoor voert 
de directie FEZ en de Chief Information Officer als tweede lijn de gesprekken met 
de spelers van de eerste lijn. Ook neemt de tweede lijn reflecties van de ADR mee 
in de rapportages. Hierdoor houdt BZK zelf grip op de interne beheersing en kan 
de ADR waarde toevoegen vanuit de onafhankelijke auditrol.  
 
Provincies, gemeenten en waterschappen verantwoorden zich jaarlijks aan het 
Rijk over de besteding van specifieke uitkeringen via de zogenoemde Single 
information Single audit (SiSa), die onderdeel uitmaakt van hun jaarrekening. 
Hun accountant geeft daartoe een controle-verklaring af. De ADR doet jaarlijks 
onderzoek naar de kwaliteit van deze accountantscontroles en rapporteert 
hierover aan BZK, dat de systeemverantwoordelijkheid voor de uitkeringen heeft. 
Om deze taak goed te kunnen vervullen heeft de ADR informatie nodig over de 
lopende regelingen, het totaalbedrag van verleningen per regeling en mede 
overheid, de risico-inschatting op onjuiste besteding per individuele SiSa-regeling, 
en de geconstateerde fouten en onzekerheden per regeling in voorgaande jaren. 
De ADR is in gesprek met BZK om deze informatie in de toekomst aangeleverd te 
krijgen.  
 
Kwetsbaarheden bij FEZ en DG-Control bij Onderwijs, Cultuur en 
Wetenschappen  
Onder meer door veel verloop en uitval van medewerkers bij de directie FEZ en 
het DG-control is de tweede lijn bij OC&W kwetsbaar. Een positieve ontwikkeling 
is dat de eerste en tweede lijn sterk betrokken zijn bij het maken van keuzes 
rondom de inrichting van het Nationaal Programma Onderwijs. 
 
Investeringen in de financiële functie bij Volksgezondheid, Welzijn en 
Sport  
VWS is dit jaar gestart met kwalitatieve en kwantitatieve investeringen in de 
financiële functie, waar omvangrijke en weerbarstige problemen zijn blootgelegd 
die vragen om meerjarige inspanningen. Positief is dat de Dienst Uitvoering 
Subsidies aan Instellingen is gestart met een meer risicogerichte interne controle. 
Ook gaat de Dienst meer gebruik maken van data-analyse, door de scripts die de 
ADR al heeft gebouwd over te nemen en in goede samenspraak verder door te 
ontwikkelen. 
 """
openbesluitvorming_html_1 = """<p>Burgemeester en Wethouders

Postadres Postbus 16200, 3500 CE Utrecht
Telefoon 030 - 286 10 00 Fax 030 - 286 12 24
www.utrecht.nl




leden van de cie M&S en S&R




Behandeld door    Carolien Koek                         Datum              7 juni 2011
Doorkiesnummer    030 – 2865020                         Ons kenmerk        11.065181 + 11.065182
E-mail            c.koek@utrecht.nl                     Onderwerp          Voorhangprocedure Gerrit Rietveld
Bijlage(n)        1                                                        College
Uw kenmerk                                              Verzonden
Uw brief van                                            Bij antwoord datum, kenmerk en onderwerp vermelden



Geachte leden van de commissie,

Op 7 juni 2011 staat de voorhangprocedure Gerrit Rietveld College op de agenda in de raadscommissie
Mens en Samenleving. Tot vrijdag 3 juni 10:00uur was u in de gelegenheid schriftelijk vragen in te
dienen. Naar aanleiding van de gestelde vragen volgt hieronder op enkele thema's een korte
toelichting. In de bijlagen zijn de vragen en antwoorden per fractie terug te vinden.

Geen exploitatietekort
Het schoolbestuur heeft bij het tot stand komen van de intentieovereenkomst aangegeven dat zij,
binnen haar lumpsum rijksbekostiging, een jaarlijks budget beschikbaar heeft van ongeveer € 23,00
voor het exploitatiedeel groot en klein onderhoud. Mitros heeft aangegeven circa € 28,00 nodig te
hebben op basis van de door Mitros opgestelde opstal exploitatie. Op basis van deze gegevens is bij
het tot stand komen van de intentieovereenkomst geconcludeerd dat er sprake was van een
exploitatietekort op het onderdeel onderhoudslasten.


Bij het tot stand komen van de samenwerkingsovereenkomst is ICS, als onafhankelijke partij, gevraagd
om op basis van zijn ervaringscijfers uit de ‘markt’ de totale exploitatielasten te prognosticeren. ICS
heeft de totale exploitatielasten geprognosticeerd op basis van kengetallen, waar de onderhoudslasten
een onderdeel van zijn. ICS heeft aangegeven rekening te moeten houden met een onderhoudslast
(groot en klein onderhoud) van € 22,00 plus € 7,50 dit maakt in totaal € 29,50. De € 29,50 is
onderdeel van de totale € 70,75 per m2 bvo per jaar aan totale exploitatielasten, zoals opgenomen in
de samenwerkingsovereenkomst.


Door de focus van enkel het exploitatiedeel onderhoudslasten te verplaatsen naar de totale
exploitatielasten garandeert het schoolbestuur Mitros kostendekkende onderhoudslasten, mits de
totale exploitatielasten niet hoger zijn dan maximaal € 70,75 plus een eventuele, op voorhand
geaccepteerde, geringe overschrijding van maximaal € 2,50 per m2 bvo per jaar.

Onroerend zaak belasting
In de Wet op het voortgezet onderwijs is opgenomen dat de gemeente het schoolbestuur het bedrag
dat is uitgegeven voor de belastingen met betrekking tot de in de gemeente gelegen gebouwen en
terreinen vergoed. Dit geldt voor zowel de OZB eigenarendeel als OZB gebruikersdeel.
</p><p>Burgemeester en Wethouders



Datum 07-06-2011
Ons kenmerk 11.065181+11.06518211.065181 + 11.065182




In de discussie over de totale exploitatielasten heeft Mitros aangegeven dat de kosten voor het OZB
eigenarendeel en OZB gebruikersdeel worden doorbelast aan de huurder, in deze het schoolbestuur.
Dit is gebruikelijk in een privaatrechtelijke huurovereenkomst. De gemeente vergoed deze kosten net
als bij alle andere onderwijsvoorzieningen in Utrecht rechtstreeks aan het schoolbestuur.

De aanslag 2011 van het gebruikersdeel is 0,1728% van de gecorrigeerde vervangingswaarde. De
aanslag 2011 van het eigenaardeel is 0,2143% van de gecorrigeerde vervangingswaarde.


Opstalverzekering algemeen
Op grond van de Verordening voorzieningen huisvesting onderwijs kunnen schoolbesturen een
voorziening aanvragen zoals genoemd in artikel 2 sub e: “herstel en vervanging in verband met schade
aan een gebouw, onderwijsleerpakket of leer- en hulpmiddelen en meubilair ingeval van bijzondere
omstandigheden”. Bij schade zoals hierboven omschreven is de gemeente volgens de verordening
verplicht deze schade te vergoeden. Het is de keuze van de gemeente om of een verzekering af te
sluiten of te sparen voor deze vergoeding van de schade. Op het moment dat de gemeente besluit
geen verzekering af te sluiten draagt de gemeente volledig het risico. De gemeente heeft besloten voor
alle onderwijsvoorzieningen een opstalverzekering af te sluiten.


Bij de decentralisatie van de onderwijshuisvesting ingaande 1 januari 1997 is de vergoeding voor
verzekeringen niet overgegaan van het Rijk naar de schoolbesturen maar is onderdeel gaan uitmaken
van de Gemeentefondsuitkering. Vervolgens is dit begrotingstechnisch verwerkt in de begroting van
Maatschappelijk Vastgoed van de DMO.


Opstalverzekering Gerrit Rietveld College
Ingeval van het Gerrit Rietveld College hebben we in verband met de afwijkende
eigendomsverhoudingen (Mitros als eigenaar) deze huisvestingsvoorziening in de overeenkomst
huisvestingskosten gesplitst. Willibrord behoudt het recht om voor schade aan het onderwijsleerpakket
of leer- en hulpmiddelen en meubilair ingeval van bijzondere omstandigheden een aanvraag te doen
conform de verordening. Voor schade aan het gebouw kan niet meer conform de verordening een
aanvraag worden gedaan. Schade aan het gebouw is voor rekening en risico van Mitros. Mitros heeft
aangegeven bij verzekering van dit risico hiervoor een vergoeding te willen ontvangen van Willibrord
binnen de vergoeding voor de kostendekkende exploitatie. De gemeente zal de premie voor de
verzekering van schade aan het gebouw vergoeden aan Willibrord zoals zij dit ook doet voor alle
andere onderwijsvoorzieningen in de gemeente, omdat het van algemeen belang is dat
onderwijsgebouwen goed verzekerd zijn.


Op basis van het Definitief Ontwerp (DO) wordt een Prognose Lifecycle Costst (PLC) opgesteld waarvan
een meerjaren onderhoudsplan (MOP) deel uitmaakt. Op basis van het DO en PLC wordt de
exploitatiebijdrage en dus ook de vergoeding van de verzekeringspremie definitief overeen gekomen.




                                                                                                         2/3
</p><p>Burgemeester en Wethouders



Datum 07-06-2011
Ons kenmerk 11.065181+11.06518211.065181 + 11.065182




Ontwerpproces
De Europese aanbesteding van het ontwerp heeft plaats gevonden, de architect is geselecteerd. Om te
komen tot een ontwerp van het schoolgebouw worden de volgende fasen doorlopen.
- Schets Ontwerp (SO), op basis van het SO wordt een bouwkostenraming op hoofdlijnen uitgewerkt
- Voorlopig Ontwerp (VO), op basis van het VO wordt een eerste doorrekening van de
bouwkostenraming en de exploitatieraming uitgewerkt.
- Definitief Ontwerp (DO), op basis van het DO wordt een PLC en een MOP opgesteld.

Wij vertrouwen u met deze brief voldoende te hebben geïnformeerd.




Hoogachtend,

Burgemeester en wethouders van Utrecht,

De secretaris,                                    De burgemeester,




                                                                                                      3/3
</p>"""
openbesluitvorming_html_2 = """<p>                                Notulen Raadsvergadering
                                 Gemeenteraad Almelo


Datum:                      10 november 2011 (voortzetting van de vergadering op dinsdag 8
                            november 2011 te 19.00 uur)
Aanvang:                    10.00 uur
Locatie:                    Raadzaal

Voorzitter:                 afwisselend mevrouw J.H.M. Hermans-Vloedbeld, burgemeester, en de
                            heer J.J. Stapel, vicevoorzitter.
Griffier:                   mevrouw C.M. Steenbergen, griffier.

Aanwezige leden:            mevrouw A. Abdullatif, R.B.M. Borgman, J. Brand, J. Buitenweg, U. Çete,
                            F.J. Cornelissen (later), mevrouw C.N. Davis, mevrouw I. Dikmen-Yalçi,
                            mevrouw M.J.C. van Doorn, F.J. Gerritsen, J.J.H. Hammink (later), A.
                            Heite, mevrouw H.J. Heitink, F.E. Van der Horst, A.G.P. Hümmels, H.C.
                            Isendoorn, G.F. Kamphuis, mevrouw N. Kirkagaç, B. Kozijn (later), A.
                            Maathuis, H.K. Nijhuis, H.G. de Olde, J. Pauwels, mevrouw H.R.
                            Rettensberger-Stevens, mevrouw H.E. Rozema-Jungjohann, J.L. Schuur,
                            mevrouw I.A.M. ten Seldam-Even, L.K.J. Simmering, J.J. Stapel, G.W.L.
                            Stork, J. Veenstra, G.P. Vos en mevrouw M. Wessels.

Afwezige leden:             F. Akse en M.M. Geerdink.

Aanwezige wethouders:       J. Andela, mevrouw J.M.M. Kuik-Verweg, E.J. van Marle, T.J. Schouten
                            en G.H. van Woudenbergh.


         De VOORZITTER heropent de vergadering en spreekt als volgt:
         Dames en heren! Ik heet u allemaal hartelijk welkom, zowel de raadsleden als de mensen op de
publieke tribune en de mensen die thuis naar ons kijken via internet, nu of op een later moment. Dit
laatste kan immers ook; dat is het mooie van de internetuitzending.
         Bericht van verhindering is ingekomen van de heer Akse. De heren Geerdink, Kozijn en
Hammink hebben laten weten dat zij later ter vergadering zullen komen.
         Vandaag gaan wij verder met de begrotingsbehandeling. Straks zal eerst het college het woord
krijgen om te reageren op de vragen, opmerkingen, moties en amendementen die de raad afgelopen
dinsdagavond naar voren heeft gebracht. Daarvoor hebben wij met elkaar een bepaalde tijd
afgesproken, maar er zijn veel vragen gesteld en moties en amendementen ingediend. Daar willen wij
als college zorgvuldig op reageren. Ik zal proberen daarbij de tijd te bewaken, maar ik meen dat u er alle
belang bij heeft dat wij goed antwoorden. Ik vraag daar op voorhand enige souplesse in. Dat kan ook,
omdat op de agenda een tijdschema is aangegeven waarmee flexibel kan worden omgegaan.

        De heer SCHUUR (PvdA): Voorzitter! Ik wil een ordevoorstel doen. Op de agenda is geen
schorsing gepland na de beantwoording in de eerste termijn door het college. Het lijkt mij echter nuttig
dat de diverse fracties zich kunnen beraden op de inbreng van het college. Daarom stel ik voor om na
de beantwoording door het college in de eerste termijn de vergadering te schorsen, zodat de fracties
zich daarop kunnen beraden.




INT - 1201417                                                                                           1
</p><p>       De VOORZITTER: Dat lijkt mij logisch. Wij zullen dan bekijken hoeveel tijd de raad nodig heeft.
       Dames en heren! Ik stel voor om voor de inbreng van de raad in de tweede termijn de spreek-
volgorde aan te houden die wij afgelopen dinsdag hebben bepaald bij loting. Tenslotte gaat het om één,
doorlopende vergadering. Ik stel vast dat u daarmee akkoord gaat.
       De leden van het college zullen in de volgende volgorde reageren: als eerste zal ik als
burgemeester spreken en vervolgens zullen successievelijk mevrouw Kuik en de heren Schouten, Van
Woudenbergh, Andela en Van Marle het woord voeren.

Reactie van het college.

                (De burgemeester draagt het voorzitterschap over aan de heer Stapel.)

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Mijnheer de voorzitter! Op de eerste plaats
dankt het college alle fracties voor hun bijdragen, vragen en positieve maar ook kritische opstelling. Ik
ben van mening dat wij uiteindelijk alleen op die manier tot een goed product komen.
         De aanzet tot en de kaders voor de voor ons liggende begroting voor de jaarschijf 2012 zijn
door de raad meegegeven bij de behandeling van de voorjaarsnota. Toen hebben wij uitgebreid met
elkaar gesproken over bezuinigingen - ook in afrondende zin, want wij hadden daarvoor al een proces
gevolgd met elkaar - en over "Het verschil maken". Wat dat betreft, is de begroting 2012 voor het
college een logische uitwerking van de voorjaarsnota die wij onlangs hebben beetgepakt.
         In de bijdragen en in de moties en amendementen verwijzen de verschillende fracties een aantal
keren naar "Het verschil maken". Samen met onze partners hebben wij de visie ontwikkeld op de wijze
waarop wij willen inspelen op de grote ontwikkelingen in onze samenleving. Tegelijkertijd is er ook de
noodzaak tot bezuinigen. Beide aspecten botsen soms, in belangen en posities. Bezuinigingen gaan niet
zonder slag of stoot, omdat ze de samenleving raken. Wij menen echter dat het aantal
bezwaarschriften dat is ingediend - ongeveer 15 - een goede indicatie is van de wijze waarop het
bezuinigingsproces tot nu toe is verlopen. Dit zeg ik als reactie op de inbreng van de fractie van de
ChristenUnie.
         "Het verschil maken" kent twee belangrijke inhoudelijke doelen: het versterken van het
samenspel met de stad en het optimaliseren van onze dienstverlening. In beide gevallen gaat het om
een brede en meerjarige doelstelling waar wij langer aan moeten werken dan alleen vandaag om
daadwerkelijk grote veranderingen te realiseren. Dat betekent: een lange adem, maar ook
kortetermijnresultaten. Ik noem enkele voorbeelden, als reactie op een vraag van de PvdA-fractie. Wij
hebben gerealiseerd c.q. te maken gehad met:
- De totstandkoming van de Sociale Agenda 2020.
- Het evenementenfonds, ook een zeer interactief proces samen met de stad.
- Verschillende bezuinigingsmaatregelen die samen met de stad tot stand zijn gekomen en waar zeker
    ook met de stad handen en voeten aan wordt gegeven, zoals het beleidsterrein sport.
- Het college heeft een uitvoeringsagenda "Het verschil maken" vastgesteld. Dat betekent onder meer
    dienstverleningsprocessen optimaliseren door digitalisering, het toepassen van Lean Management en
    deregulering.
- Begin dit jaar startten wij met het meldpunt openbare ruimte en parkeervergunningen.
- Ook gaan wij, analoog aan de Sociale Agenda 2020, een dergelijke agenda ontwikkelen voor het
    programma Aantrekkelijke stad. Dat is dan een agenda 2020, zo men wil (mijnheer Veenstra; bij de
    behandeling van moties en amendementen kom ik daar op terug).
- Daarnaast zullen wij ook in de reguliere beleidstrajecten voor de detailhandel (de structuurvisie), het
    integraal beheer van de openbare ruimte en de schuldhulpverlening structurele samenwerking met de
    stad zoeken.
         Dit laatste vraagt een omslag van onze partners en onze medewerkers, maar ook van onszelf.
Daarom willen wij de komende tijd ook nadenken over de wijze waarop raad en college samenwerken
en hoe wij die samenwerking kunnen verbeteren. Daarover hebben wij afspraken gemaakt met elkaar.
Ook binnen de organisatie is een dergelijke omslag in werkwijze nodig. De zorg die de PvdA-fractie
heeft dat in tijden van bezuinigingen talenten elders hun heil zoeken, delen wij. Daarom wordt gewerkt
aan het vasthouden van talenten door met hen afspraken te maken over hun ontwikkeling en
perspectieven en tegelijkertijd ook aandacht te hebben voor slecht functionerende medewerkers. Want:
het mooie hoort erbij, maar ook het vervelende.
         Ik begrijp (de fractie van GroenLinks refereerde daaraan) dat tijden van bezuinigingen leiden tot
pijnlijke keuzes. De economische omstandigheden en de grote ontwikkelingen in onze samenleving




INT - 1201417                                                                                           2
</p><p>vragen een omslag van onze partners, van onze medewerkers en van onszelf. Dat moeten wij willen
leren, en dat moeten wij onszelf gunnen.
         Ik hoop dat ik inmiddels heb laten zien dat wij wel degelijk stappen hebben gezet voor zowel de
korte als de langere termijn. Ik heb dat gedaan, omdat een aantal fracties daar expliciet om gevraagd
heeft.
         Diverse fracties hebben opmerkingen gemaakt over een artikel 12-status. Het is volgens mij
goed om daar even met elkaar over te spreken. Het is niet anders: het college bestuurt de stad. Het
college neemt dus ook zijn verantwoordelijkheid en zorgt voor een sluitende begroting. Een dergelijke
begroting ligt nu ook voor. Derhalve zien wij als college geen aanleiding om zelfs maar te overwegen
om een artikel 12-status aan te vragen. Die status brengt namelijk met zich mee dat de bevoegdheden
van het gemeentebestuur op veel punten drastisch worden ingeperkt, waarbij men het
gemeentebestuur moet begrijpen als: college en raad. Daarnaast - en daarover bestaat nogal eens een
misvatting - leidt de artikel 12-status tot extra verplichtingen om de financiële positie te verbeteren.
Daarbij moet men echt denken aan minder uitgaven en het genereren van meer inkomsten.
         Dan ga ik nu in op een aantal vragen met betrekking tot mijn portefeuille. Daarbij begin ik met
de vragen die zijn gesteld over de transitie bij Stadstoezicht, namelijk van toezicht naar handhaving.
Het is duidelijk, want het besluit daartoe is genomen, dat Stadstoezicht overgaat van een enkel
toezichthoudende organisatie naar een handhavende organisatie. Dat wil zeggen: een overgang van
toezichthouders zonder bevoegdheden naar toezichthouders met bevoegdheden. Dat is nodig om
effectiever te kunnen optreden in de openbare ruimte. Dit is een landelijk proces, want dat gebeurt niet
alleen in Almelo zo. Overigens blijft preventie natuurlijk wel een kerntaak van Stadstoezicht, zo zeg ik
met nadruk richting de heer Hümmels. Belangrijk is overigens dat de organisatieontwikkeling bij
Stadstoezicht ook een budgettair neutrale operatie is.
         Dan de planning voor de nota integrale veiligheid c.q. het lokale integraal veiligheidsplan, waar
de VVD-fractie om vroeg. Overigens is dit proces al geschetst, ook wat betreft de termijnen, tijdens de
informatieve raadsbijeenkomst. De aftrap voor het proces vond plaats op 5 juni jl. Ik noem nu de
volgende data:
- de periode januari 2012 tot en met mei 2012 gebruiken wij om veldwerk te doen en de opbrengst te
    analyseren;
- van juni 2012 tot september 2012 hebben wij rondetafelgesprekken met partners, waarbij men ook
    mag begrijpen dat wij rondetafelgesprekken zullen hebben met buurtbewoners/wijkbewoners;
- in de periode september 2012 - december 2012 zorgen wij voor de regionale afstemming en de
    vaststelling van het lokale integraal veiligheidsplan 2013-2016.
         Daarmee voldoen wij ook aan de wettelijke verplichtingen. De raad wordt uiteraard in dit hele
proces meegenomen.
         Ik kom nu bij de opmerking die door de heer Kamphuis is gemaakt over de aanpak van de zware
criminaliteit en het slechts geringe aantal politieauto's dat wij beschikbaar zouden hebben op bepaalde
dagen. Deze zaken staan geheel los van elkaar. De inzet van voertuigen heeft alleen te maken met de
organisatie van de noodhulp in Twente. De aanpak van de zware criminaliteit vraagt een totaal andere
inzet (daar hebben wij al tijdens de informatieve raadsbijeenkomst over gesproken).
         De VVD-fractie heeft gevraagd of de raad regelmatig geïnformeerd kan worden over
ontwikkelingen in de Veiligheidsregio Twente en de ontwikkelingen met betrekking tot de landelijke
politie. Daar nemen wij de raad in mee. Dat zullen wij op gezette tijden doen, ook op momenten dat wij
meer weten, vooral over de ontwikkeling van de landelijke politie. Ik dacht dat er gisteravond een
algemene radenbijeenkomst is georganiseerd in Zenderen om gezamenlijk te kijken naar de
ontwikkelingen ten aanzien van de veiligheidsregio en annexe onderwerpen.
         Ik ga nu in op enkele moties en amendementen.
         Motie F van de PvdA-fractie betreft het bedrag van € 125.000,-- ten behoeve van het proces
van "Het verschil maken". Ons voorstel om dit bedrag daarvoor beschikbaar te stellen, volgt uit een
besluit dat reeds is genomen door de raad. Zoals men weet (ik ga nu inhoudelijk op de zaak in), zijn er
twee soorten ambtenaren. De ene soort wordt bekostigd uit het personeelsbudget, de andere soort
wordt bekostigd uit projectgelden. Het is belangrijk om dit onderscheid vast te stellen. De
procesbegeleiding en communicatie rond "Het verschil maken" behoren tot de taken van ambtenaren
van de tweede soort. Daarom is er ook een projectbudget nodig. Dat budget hebben wij bij de
voorjaarsnota gevraagd, en daarmee heeft de raad toen ingestemd. Nu wordt het budget in de
begroting verwerkt en vragen wij de raad opnieuw om daarmee in te stemmen, want zonder dat budget
zijn wij niet in staat om aan de slag te gaan met "Het verschil maken".
         Amendement K van de ChristenUnie-fractie betreft het opstellen van een agenda 2020 per
programma. Het college omarmt dit amendement, omdat het in lijn is met de vastgestelde



INT - 1201417                                                                                           3
</p><p>uitvoeringsagenda voor "Het verschil maken". Ook biedt het amendement ons de ruimte om gefaseerd
en samen met de partners gezamenlijke agenda's voor 2020 te ontwikkelen en daarbij te starten met
het programma Aantrekkelijke stad. Het ontwikkelen van dergelijke agenda's voor deelprogramma's
raadt het college nadrukkelijk af.
        Motie P van de fractie van D66 heeft als titel: Extra bezuinigen door focus op samenwerking.
Samenwerking en uitbesteding hebben volop de aandacht van het college. In regioverband worden op
dit moment de mogelijkheden van verdergaande samenwerking in kaart gebracht (de raad heeft daar
onlangs een brief over gekregen). Daarnaast zijn wij zelf bezig om sourcing als strategie in te zetten.
Tijdens de behandeling van de voorjaarsnota hebben wij daarover al gesproken met de raad. Toen
hebben wij zelfs twee zaken afgesproken, namelijk dat wij nog dit jaar een kaderstellend stuk ter zake
zouden voorleggen en dat de raad dat op zeer korte termijn zou ontvangen. Kortom, wij zijn daarmee
aan de slag. Wij ontraden derhalve motie P.
        Motie V, ook van de D66-fractie, betreft het houden van een peiling onder Almeloërs. Ik zeg het
heel recht: het college voelt er niet voor om de twee door de D66-fractie geformuleerde vragen voor te
leggen aan de bevolking. De reden daarvoor is dat ze een complexe problematiek betreffen waarin
verschillende lagen en onderwerpen zijn verwerkt. Een dergelijke complexe problematiek zou dan in een
te sterk versimpelde vorm worden voorgelegd. Het is onmogelijk om daar dan de mening van het
publiek over te vragen. Wij vinden het veeleer een zaak voor raad en college om daar zelf opvattingen
over te hebben en uitgangspunten over te delen. Daarom ontraden wij motie V.
        Dan motie HA van de fractie van ALA-AOV met betrekking tot het beperken van gemeentelijke
publicaties. De wet biedt de mogelijkheid om over te gaan op digitale publicaties. Vanuit het oogpunt
van dienstverlening zijn wij echter van mening dat wij de betrokken informatie moeten blijven
aanbieden via een huis-aan-huisblad. Wel zijn wij bezig de teksten in te korten en verwijzen wij naar de
website van onze gemeente (www.Almelo.nl). Overigens voorziet het contract met de uitgever in de
mogelijkheid om in 2012 het aantal pagina's terug te brengen tot drie per week. Daarnaast publiceren
wij ook algemene informatie, dus niet alleen de wettelijk voorgeschreven teksten. Het gaat daarbij om
informatie over wegwerkzaamheden, gemeentelijke voorzieningen en raadsbijeenkomsten. Het
informeren hierover is niet wettelijk noodzakelijk, maar is wat het college betreft wel essentieel voor
onze dienstverlening naar de burgers en het vormgeven van het democratisch proces. Derhalve
ontraden wij motie HA.
        Amendement J van de fractie van de ChristenUnie gaat over het kiezen voor sociale stijging. Uit
de overwegingen van de heer Veenstra blijkt dat het hem vooral gaat om het behouden van het sociaal
kapitaal dat al aanwezig is in de stad en het realiseren van sociale stijging voor de individuele
bewoners. Die doelstelling onderschrijft het college. Het aantrekken van kansrijke bevolkingsgroepen -
 nieuw sociaal kapitaal, als men dat zo wil noemen - lijkt de heer Veenstra te willen loslaten. Voor het
college is en blijft het aantrekken van kansrijke bevolkingsgroepen een belangrijk uitgangspunt van
beleid, niet alleen omdat dat nodig is om de stad meer in balans te laten komen, maar ook omdat het
een wezenlijke bijdrage kan leveren aan de sociale stijging van kwetsbare groepen. Voorts vragen wij
ons af of het aanpassen van de doelstelling een wezenlijke verandering zou betekenen voor de
maatregelen die wij nemen en het beleid dat wij uitvoeren, want wij verbeteren deze stad toch niet
enkel en alleen voor nieuwkomers? Ook al zou er in de komende tien jaar helemaal niemand meer naar
Almelo verhuizen, dan nog blijft steeds overeind staan dat de huidige binnenstad om een
kwaliteitsimpuls vraagt. Ook ten behoeve van de mensen die al in Almelo wonen, is het noodzakelijk en
wenselijk om te investeren in aantrekkelijke woonmilieus en goede stedelijke voorzieningen. Kortom,
amendement J lijkt ons niet zinvol.

                (De heer Stapel draagt het voorzitterschap over aan de burgemeester.)

        De heer HÜMMELS (Leefbaar Almelo): Voorzitter! Een punt van orde. Ik heb afgelopen dinsdag
moeite gedaan om de door mij aangekondigde moties en amendementen meegenomen te krijgen. Ik
vraag mij nu af of er ook op de aangekondigde moties en amendementen gereageerd wordt. Ik zou dat
zeer op prijs stellen. U hebt inhoudelijk wel op mijn inbreng gereageerd, maar niet bij de behandeling
van moties en amendementen.

       De VOORZITTER: Daar zal ik op een later moment naar kijken. Ik heb gereageerd op de moties
en amendementen die zijn ingediend. Volgens mij is de lijst met die moties en amendementen
gehanteerd door het college.




INT - 1201417                                                                                         4
</p><p>       De heer VEENSTRA (ChristenUnie): Voorzitter! Ik heb een amendement (L) ingediend over het
formuleren van een besparingstaakstelling per maatschappelijke opgave. Daar hebt u niet op
gereageerd.

      De VOORZITTER: Nee. Per portefeuillehouder wordt gereageerd op de moties en
amendementen. Misschien moet u nog even geduld hebben.

          Mevrouw KUIK-VERWEG (wethouder): Voorzitter! Ik zei het al bij de voorjaarsnotabehandeling:
dit zijn ingewikkelde tijden. De consequenties daarvan zijn groot voor het sociale beleid en zeker voor
degenen die moeten rondkomen van een minimuminkomen of een uitkering. Den Haag heeft in
sneltempo een aantal regelingen doorgevoerd die met name de mensen aan de onderkant raken. Wij
hebben daarmee echter wel te dealen. Je kunt twee dingen doen: of je legt je hoofd in de schoot en
laat het aan anderen over, of je haalt het meest haalbare eruit. Degenen die mij een beetje kennen,
weten dat ik voor dit laatste ga. Dit neemt niet weg dat ik mij grote zorgen maak over het toenemende
aantal gezinnen met een minimuminkomen en de toename van het aantal werkende armen. Als je
voltijds werkt en toch niet in staat bent om aan al je verplichtingen te voldoen omdat de middelen niet
toereikend zijn, dan is dat een groot probleem. Dat betreft niet alleen Almelo. De opeenstapeling van
kosten maakt het voor mensen erg zwaar om het hoofd boven water te houden.
          Zaken die mijn portefeuille betreffen en waarover veel gesproken is tijdens de algemene
beschouwingen van afgelopen dinsdag, zijn: de Stadsbank, Almelo Sociaal, welzijn en Soweco. Ik
waardeer het ten zeerste dat de raad met suggesties is gekomen omtrent deze onderwerpen, maar
tegelijkertijd vraag ik de raad de ruimte en het vertrouwen om deze ingewikkelde zaken zorgvuldig te
laten verlopen. Ik doel hierbij met name op het terugdringen van de overschrijdingen e.d.
          Ik ga nu over tot de beantwoording, te beginnen bij de inbreng van de CDA-fractie. Die vroeg of
niet zorgvuldiger omgegaan zou kunnen worden met de welzijnsaccommodaties, in die zin dat ze
optimaal benut worden. Wij hebben dat bij de voorjaarsnotabehandeling reeds toegezegd, en op dit
moment bekijken wij hoe dat kan. De raad heeft ons toen als datum 1 januari 2014 meegegeven om te
bekijken hoe zaken in elkaar geschoven kunnen worden. Inmiddels vinden daarover gesprekken plaats.
Wat betreft de nog te ontwikkelen multifunctionele accommodaties in de binnenstad en
Ossenkoppelerhoek, hebben wij bekeken hoe 't Veurbrook en Goossenmaat verder ontwikkeld kunnen
worden (van de eerste accommodatie weet de raad dat), opdat daarvoor geen nieuwe accommodaties
nodig zijn. Met de beide welzijnsaccommodaties die er nu zijn in de binnenstad, gaan wij in gesprek
over de wijze waarop invulling kan worden gegeven aan de wens om ze zo optimaal mogelijk te laten
gebruiken.


                                                      (De heer Cornelissen komt, te 10.25 uur, ter
                                                      vergadering.)

         De fractie van de SP heeft gesproken over peuterspeelzalen en kinderopvang. Overigens stelde
de SP-fractie dat de raad een stom besluit heeft genomen met betrekking tot Scoop. Die opmerking laat
ik voor rekening van de heer Cornelissen. Voorts stelde hij dat door de bezuinigingen op het
peuterspeelzaalwerk, ook kwaliteit weggehaald wordt. Dat is niet het geval. Wij hebben gezegd: Wij
gaan bezuinigen op het peuterspeelzaalwerk daar waar dat kan. Dat doen wij, omdat ook kinderopvang
mogelijk is voor kinderen uit gezinnen met tweeverdieners. Die kunnen via de belastingsystematiek
kosten terugvorderen. Dat geldt voor Scoop, maar het geldt bijvoorbeeld ook voor De Cirkel. Beide
instellingen volgen de Piramidemethode, dus doen wat de heer Cornelissen bedoelt met de
voorbereiding op het basisonderwijs. Ook in de kinderopvang wordt die methode gebruikt. Uiteraard is
het aan de ouders om de keuze te maken uit de kinderopvangvoorzieningen. Voorts zeg ik nog maar
een keer dat de kinderopvang die georganiseerd wordt door Scoop, niet door de gemeente
gesubsidieerd wordt. Verder meld ik nog dat wij zeer binnenkort bij de raad terug zullen komen op de
kinderopvang. Overigens is er al eens een informatieve raadsbijeenkomst geweest over hoe wij denken
om te gaan met kinderen van 0-12 jaar daar waar het gaat om de kindercentra e.d.
         De fracties van de PvdA, het CDA, de ChristenUnie en de VVD hebben vragen gesteld over de
individuele voorzieningen in het kader van de WMO. Juist om de risico's op dit gebied in te dammen,
zijn al maatregelen doorgevoerd, onder andere het niet meer ondersteunen van mensen met een
persoonsgebonden budget van de Sociale Verzekeringsbank en het aanpassen van de normtijden voor
de huishoudelijke hulp. Vrij onlangs is nog in een informatieve bijeenkomst, waarbij ook De Klup
aanwezig was, met de raad gesproken over hoe zodanig kan worden omgegaan met het



INT - 1201417                                                                                          5
</p><p>dienstverleningmodel, dat eerst de voorliggende voorzieningen aan de orde komen voordat overgegaan
wordt naar de individuele voorzieningen. Verder proberen wij in het kader van het project Wonen, Zorg
en Service in de Wijk eenvoudige woningaanpassingen te laten uitvoeren door de klussendienst. Dit in
plaats van direct naar de corporaties of andere partijen te stappen.
          De VVD-fractie vroeg of de eigen bijdragen voor de WMO-voorzieningen niet verhoogd kunnen
worden. Wij zitten al aan de maximale eigen bijdragen, dus dat is niet het geval.
          Mijn collega de heer Van Marle zal ingaan op wat er gezegd is over Soweco. Ik kan alvast
zeggen dat wij de raad op 30 november a.s. zullen informeren over de stand van zaken met betrekking
tot de toekomst van Soweco (de raad heeft daarover al bericht ontvangen).
          Dan ga ik nu over tot de behandeling van de moties en amendementen die betrekking hebben
op mijn portefeuille. De fractie van D66 heeft amendement N ingediend, waarin gevraagd wordt om het
bedrag van 3 ton dat gemoeid is met de taakstelling voor Scoop, te gebruiken als dekking voor de
bezuiniging op Almelo Sociaal. Over Almelo Sociaal zijn vragen gesteld door verschillende fracties. De
bezuiniging van € 50.000,-- in 2012 kan gerealiseerd worden. Ik zeg toe dat wij bij de voorjaarsnota
2012 zullen komen met voorstellen over de bezuinigingen in 2013 en 2014. Dat hadden wij trouwens
al afgesproken bij de laatste voorjaarsnotabehandeling, maar het onderzoeksrapport ligt er nu. Daar
komen wij nog over te spreken. Gelet daarop, ontraad ik amendement N.
          Amendement AA van de VVD-fractie gaat over de overschrijding van budgetten voor
openeinderegelingen. Dat is een lastige. Wij moeten het eerst maar eens met elkaar hebben over wat
wij verstaan onder het in het amendement genoemde sociale domein. In het niet al te lange verleden
hebben wij volgens mij met elkaar afgesproken dat daar waar er overschrijdingen zijn, ze in principe
opgelost worden binnen de koker zelf (zo noem ik dat maar even) als dat mogelijk is. Het sociale
domein is voor mij veel breder. Tot het sociale domein behoren zaken als het leerlingenvervoer, theater,
welzijn en sport. Het sociale domein is dus heel breed. Ik weet niet of het de bedoeling van de VVD-
fractie is dat wanneer er een overschrijding is op de uitkeringen, het daarmee gemoeide geld
teruggehaald wordt bij de voetbalclubs of zo. Daar gaan wij het over hebben met elkaar. Kortom, ik
meen dat wat in amendement AA gesteld wordt, heel lastig is. Volgens mij hebben wij daar afspraken
over gemaakt met elkaar. Ik wil het amendement dan ook ontraden.
          In motie A, van de CDA-fractie, wordt het college verzocht om in 2012 te komen tot een
afgebakend welzijnsbeleid. Wij zijn druk bezig, op een aantal fronten, met de Sociale Agenda 2020.
Binnenkort wordt die agenda ter besluitvorming voorgelegd aan de raad. De koers die wij willen varen
in het sociale domein, betreft de vier stijgingsroutes. Wij willen het dan hebben over werken, leren,
vrije tijd en het vangnet/de ondersteuning. Deze zaken moeten dan verder uitgewerkt worden. Het
afgebakende welzijnsbeleid waar in motie A om gevraagd wordt, volgt volgens mij hieruit. Daar komt
nog bij dat wij eigenlijk juist minder beleidsnota's willen (ik heb het daar eerder al eens over gehad). Wij
willen in ieder geval niet meer beleidsnota's, gelet ook op het korten op het ambtelijk apparaat. Ook
moet je je afvragen of je voor elk beleidsonderdeel een aparte nota wilt, dan wel of je de onderdelen
wilt bundelen in het totaal. Conclusie: ook motie A ontraden wij.
          Met betrekking tot inkomensondersteuning heeft de PvdA-fractie motie H ingediend. In deze
motie wordt gevraagd om op dit gebied actie te ondernemen, bijvoorbeeld een brief schrijven, richting
de VNG. Wij gaan niet over inkomensbeleid; daar gaat het rijk over. De motie op zich is sympathiek, en
ook het college vindt wat daarin staat. Dat willen wij wel overnemen, maar wij zullen eerst moeten
afwachten wat de Eerste Kamer besluit over de inkomensgrens die het kabinet heeft vastgesteld voor
het gemeentelijke minimabeleid, namelijk 110% van het sociaal minimum. De minimaregelingen die
betrekking hebben op de grens van 115%, kunnen wij als gemeente natuurlijk gewoon in stand
houden.
          Ik kom nu bij de motie (I) van de ChristenUnie-fractie met betrekking tot het Werkplein. In 2010
hebben wij met het UWV afspraken gemaakt (een programma van eisen). Daarbij is het UWV de
verplichting aangegaan om voor tien jaar ruimte te huren en om vervolgens telkens te bekijken of de
huurperiode met vijf jaar verlengd kan worden. Inmiddels gebeurt er het nodige bij het UWV. Wij zijn
daar dan ook druk mee in gesprek. Toen ik daar onlangs (in april) mee aan tafel zat, heeft het UWV
gezegd: Wij houden ons aan onze verplichtingen. Verder hebben wij nog deze maand een volgend
gesprek met het UWV (daarbij zal dan ook mijn collega de heer Andela aanwezig zijn). Dan willen wij
met elkaar bekijken wat er in Almelo blijft. Wat er ook gebeurt, Almelo moet er natuurlijk wel voor
zorgen dat de mensen zonder baan die hier wonen, weer aan het werk komen. Ook moet de gemeente
Almelo bekijken wat de gevolgen van een en ander zijn. Maar nogmaals, het UWV houdt zich aan de
gemaakte afspraken. Dat moet trouwens ook. Gelet op wat ik gezegd heb, lijkt motie I mij overbodig.




INT - 1201417                                                                                             6
</p><p>                                                     (De heer Hammink komt, te 10.33 uur, ter
                                                     vergadering.)

        De SP-fractie heeft een motie (X) ingediend met betrekking tot internetaansluitingen voor een-
en meerpersoonshuishoudens die voor hun levensonderhoud zijn aangewezen op een bijstandsuitkering.
Ik ontraad deze motie, hoe sympathiek die ook is. Wij hebben gesteld dat de grote overschrijdingen
teruggebracht moeten worden. Wanneer wij de huidige regeling voor internetaansluitingen laten
bestaan, dan leidt dat tot grote overschrijdingen. Wij hebben daarom gekeken naar de beweging Almelo
Doet Mee. Dat initiatief komt vanuit de betrokken groep zelf. Wij gaan daarvoor werken met vouchers,
waardebonnen of hoe je het noemen wilt. Overigens ben ik over dit onderwerp nog wel in gesprek met
zowel Cogas als Ziggo (ik moet zeggen dat Ziggo een heel lastige partij is om mee te onderhandelen).
Wij overleggen met deze partijen over dit onderwerp en de vraag of er andere mogelijkheden zijn.
Verder kunnen mensen een korting van 40% krijgen- en dat zullen wij nog laten weten - op de
Twentsche Courant Tubantia wanneer zij een (wat ik maar noem) midweek- of weekendabonnement
hebben. Daarnaast hebben de woningcorporaties en de bibliotheek de afspraak gemaakt dat huurders
van de corporaties een gratis abonnement op de bibliotheek kunnen krijgen.
        Volgens motie PA van de fractie van ALA-AOV moet het college opgedragen worden de
Stadsbank minder taken te geven en Almelo Sociaal te laten assisteren. Deze motie wil ik ontraden. Ik
meen dat iedere keer door elkaar wordt gehaald wie waarvoor verantwoordelijk is. Almelo Sociaal is
een belangenbehartiger. Het zou een beetje vreemd zijn als een belangenbehartiger voor zijn eigen
cliënten taken overneemt ter beoordeling (ik zie nu dat de heer Stork nee schudt; dan zal hij hier vast
op terugkomen). Ik wijs in dit verband op de uitzending van Rumoer die ging over de Stadsbank en
waarbij ik aanwezig was. Toen hebben zowel Almelo Sociaal als de Servicedienst Almelo en Humanitas
het administratieproject gepresenteerd, omdat zij iets willen doen ter ondersteuning van mensen. Ik heb
ze toen hartelijk uitgenodigd daarvoor, want volgens mij moeten wij gebruikmaken van vrijwilligers die
kennis van zaken hebben en iets willen betekenen voor de gemeenschap. Dat geldt ook voor Almelo
Sociaal.

       De heer NIJHUIS (VVD): Voorzitter! Ik wil een verhelderende vraag stellen aan de wethouder. Ik
wil namelijk weten wat haar advies is over motie PA.

        Mevrouw KUIK-VERWEG (wethouder): Ik wil die ontraden, omdat al een aantal dingen gebeurt
met betrekking tot de Stadsbank en Almelo Sociaal. Verder is de Stadsbank een uitvoeringsorgaan voor
ons. Wij als gemeente doen niets anders dan de bedragen overmaken die betaald moeten worden.
Overigens hebben wij inmiddels een aantal zaken op papier gezet (die zijn ook bij de raad bekend, als
het goed is) waarover wij zeggen: Die moeten anders met ingang van 1 januari 2012. Dat is gebeurd in
overleg met de Stadsbank.

         De heer SCHOUTEN (wethouder): Mevrouw de voorzitter! In de eerste plaats ga ik in op de
Burgemeester Schneiderssingel. De VVD-fractie heeft een vraag gesteld over het doorschuiven van een
financieel probleem - een bedrag van € 1,5 miljoen - naar de komende jaren. Het deel van de
Burgemeester Schneiderssingel tussen de Kolthofsingel en de Grenzen wordt in drie delen aangelegd:
1. de aansluiting met de Kolthofsingel ter hoogte van het bedrijventerrein De Velden;
2. vervolgens, in 2013, komt de aansluiting met De Grenzen, als ontsluiting van het woongebied;
3. ten slotte het middenstuk in 2014.
         Dit staat overigens allemaal uitvoerig beschreven in de voorjaarsnota.
         Met betrekking tot de vraag of wij geen lasten naar de toekomst schuiven, antwoord ik dat wij
als uitgangspunt hanteren dat de lasten worden gespreid over de komende begrotingsjaren. In de
begroting 2012 hebben wij nu de kosten meegenomen voor die singeldelen die het eerst worden
aangelegd. In de begroting 2013 gaan wij de kosten meenemen voor het middendeel, dat later wordt
aangelegd. Op deze manier worden de rentelasten beter beheerst. Een alternatief zou zijn geweest om
alles al in de begroting 2012 mee te nemen, zoals volgens de voorjaarsnota de bedoeling was, maar
dat is begrotingtechnisch niet haalbaar gebleken. Wij hopen dat met dit antwoord een motie hierover
niet meer nodig wordt geoordeeld door de VVD-fractie.
         Richting de fracties van GroenLinks en LKA merk ik op dat het gedeelte van de Burgemeester
Schneiderssingel waar ik zojuist over sprak (dat wil zeggen: tussen de Kolthofsingel en de Sluiskade),
allesbehalve overbodig is. Dat vormt namelijk een onderdeel van de noordelijke radiaal naar de
binnenstad en is ook los van Waterrijk nodig, ter verbetering van de bereikbaarheid van de binnenstad
vanuit het noorden van de stad. Ook is dat nodig voor de ontsluiting van het Indiëterrein.



INT - 1201417                                                                                        7
</p><p>         Dan kom ik nu bij motie FA van de fractie van ALA-AOV. In deze motie wordt gevraagd de
Burgemeester Schneiderssingel niet aan te leggen. Deze motie wordt ontraden, dat zal men begrijpen
uit wat ik zojuist daarover heb gezegd.
         Ik ga nu over naar het volgende onderwerp, en dat is Waterrijk. De VVD-fractie heeft een
amendement (DA) ingediend waarin gesteld wordt dat de fasering zo ingericht moet worden, dat de
raad telkens een go-/no go-besluit kan nemen. Met dit amendement loopt de VVD-fractie feitelijk
vooruit op de behandeling van het bestemmingsplan voor Waterrijk. Het ontwerpbestemmingsplan is
onlangs toegelicht in een informatieve raadsbijeenkomst. De bedoeling is dat de raad voor het komende
kerstreces het definitieve bestemmingsplan Waterrijk ontvangt en dat begin volgend jaar het
bestemmingsplan aan de orde komt, samen met alle stukken die daarbij behoren. De fasering van de
zaken, bijvoorbeeld de aanvang van de bebouwing, maakt daar deel van uit en zal wat ons betreft ook
expliciet onderdeel uitmaken van de besluitvorming. Volgens de stukken zullen de volgorde van de
ontwikkelingen en de fasering ten minste eenmaal per jaar aan de raad worden voorgelegd, namelijk bij
de actualisering van het exploitatieplan dat bij het bestemmingsplan hoort. In die zin voorzien wij dus al
in de fasering die de VVD-fractie vraagt. Amendement DA is wat ons betreft dan ook overbodig.
         De fractie van ALA-AOV heeft bij motie EA gevraagd om het plan Waterrijk niet in exploitatie te
nemen, een alternatief te zoeken en grond te verkopen. Mijn antwoord daarop is dat het
Waterrijkgebied al in exploitatie is op dit moment (dit voor de helderheid). In de motie wordt
waarschijnlijk bedoeld dat niet gestart moet worden met de woningbouw in het gebied. Welnu, al
eerder is de raad geïnformeerd dat Waterrijk gefaseerd wordt ingevuld en ontwikkeld. Het
bestemmingsplan betreft de helft van de in bezit zijnde gronden. Tevens wordt al verkennend
onderzoek gedaan naar alternatieve invullingen van het exploitatiegebied. Daar zal collega Van
Woudenbergh straks nader op ingaan. Motie EA ontraden wij.
         Motie S van de fractie van D66 stelt dat de ontwikkeling van Waterrijk geparkeerd moet
worden tot de begroting 2013. Ook deze motie loopt wat ons betreft vooruit op de behandeling van
het bestemmingsplan (ik heb daar zojuist al het nodige over gezegd). Derhalve ontraden wij ook motie
S.
         Ik ga nu over naar het volgende onderwerp, namelijk de kinderboerderij, oftewel Stadsboerderij
Beeklust. De SP-fractie heeft daarover amendement W ingediend, waarin gesteld wordt dat niet
bezuinigd moet worden op Beeklust. Ook is daarover een motie aangekondigd door de fractie van
GroenLinks. En de heer Schuur heeft aangegeven dat sluiting van de Stadsboerderij Beeklust voor de
PvdA-fractie geen optie is en dat de openstelling gegarandeerd zal moeten worden. Ik wil hier graag
aangeven dat ook voor het college sluiting van de kinderboerderij geen optie is. In het kader van de
voorjaarsnota hebben wij de raad het voorstel gedaan om de jaarlijkse exploitatielast voor de gemeente
te halveren. Volgens dat voorstel zouden wij in overleg gaan met de huidige exploitant, namelijk De
Twentse Zorgcentra, om te bekijken of die binnen de nieuwe voorwaarden de exploitatie kan
voortzetten. Dit overleg is gaande. Ook heeft de raad in de vorige informatieve raadsbijeenkomst de
inbreng van de kant van De Twentse Zorgcentra gehoord. Toen was even de vraag aan de orde waar
het nu om gaat, om een bezuiniging per 1 januari 2012 of een bezuiniging die later ingaat. Ik heb dit
later ook nog eens toegelicht voor de heer Buitenweg. De bezuiniging is pas aan de orde (dat staat in
de voorjaarsnota) per 1 januari 2013. Wij hebben de komende tijd dus nog de gelegenheid om er met
De Twentse Zorgcentra over te praten. Komen wij er uit met De Twentse Zorgcentra, dan zullen wij bij
de raad komen met een voorstel over Stadsboerderij Beeklust. Voor het geval dat wij er niet met die
partij uitkomen, kan ik melden dat er inmiddels al twee andere partijen zijn die mij benaderd hebben
omdat zij belangstelling hebben voor het eventueel exploiteren van de kinderboerderij binnen de
voorwaarden zoals de raad die bij de voorjaarsnota heeft vastgesteld. Wat ik nu heb aangegeven, geeft
voldoende aanleiding om te kunnen stellen dat er echt een goede kans is dat de exploitatie van de
kinderboerderij wordt voortgezet. Het amendement (W) van de SP-fractie met betrekking tot de
kinderboerderij wil ik ontraden. De aangekondigde motie echter van de fractie van GroenLinks vinden
wij sympathiek, want daar staat in wezen hetzelfde in als wat wij beogen.
         Motie BA van de VVD-fractie betreft het planmatig onderhoud van kunstwerken. Enkele jaren
geleden is een beheersysteem voor de kunstwerken aangeschaft. Sindsdien wordt gewerkt aan de
vulling van dat systeem met gegevens. Wij verwachten de raad bij de voorjaarsnota 2012 een goed
inzicht te kunnen geven in de huidige staat van de kunstwerken, de onderhoudsbehoefte en de
resterende levensduur. Ook de financiële consequenties daarvan zullen betrokken worden bij de
voorjaarsnota. Overigens is er nu al een structurele post van ongeveer € 140.000,-- voor onderhoud en
vervanging van kunstwerken. In de praktijk wordt deze post volledig besteed aan
grootonderhoudsmaatregelen. Dat betekent dat er voor vervanging van kunstwerken op dit moment
geen structureel budget beschikbaar is. Voor noodzakelijke vervangingen zijn tot nu toe telkens op



INT - 1201417                                                                                            8
</p><p>incidentele basis middelen vrijgemaakt. Dat is een keuze die de raad steeds gemaakt heeft. Een vrij
recent voorbeeld daarvan is de Vriezebrug, die twee jaar geleden is vernieuwd. Zoals gezegd, zullen wij
bij de voorjaarsnota meer duidelijkheid geven over de vervangingsbehoefte. Met andere woorden: wij
zijn al bezig datgene te doen wat in motie BA gevraagd wordt. Hieruit kan de VVD-fractie zelf
concluderen wat ze met de motie zou kunnen doen.
          De fractie van GroenLinks heeft een amendement aangekondigd met betrekking tot het gebruik
van chemische gewasbeschermingsmiddelen. Zoals de heer Buitenweg zei, is in de Tweede Kamer
inderdaad een motie aangenomen die oproept tot een verbod op het gebruik van
gewasbeschermingsmiddelen, zoals Roundup. De minister van infrastructuur en milieu heeft
aangekondigd die motie te zullen uitvoeren. De inzet van de minister richt zich op een verbod binnen
enkele jaren. Daarbij wordt ook rekening gehouden met de overgang naar het gebruiken van andere
technieken voor onkruidbestrijding en afschrijving van machines en apparatuur. Wij als gemeente
Almelo zullen die landelijke ontwikkeling nadrukkelijk volgen. Zodra het gebruik van chemische
gewasbeschermingsmiddelen landelijk wordt verboden of beperkt, zullen wij onze onkruidbestrijding
vanzelfsprekend daaraan aanpassen. Met andere woorden: het aangekondigde amendement begrijpen
wij, maar is feitelijk overbodig.
          Dan wil ik nu een aantal zaken toelichten naar aanleiding van vragen die gesteld zijn in enkele
bijdragen. De PVA-fractie heeft een vraag gesteld over de wegverhardingen en heeft daarbij
aangegeven dat 12% van de wegverhardingen in slechte staat verkeert. Wij hebben in de begroting
(waarin de cijfers over de wegverhardingen staan) inderdaad aangegeven dat dit aan de orde is, maar
recentelijk zijn er nieuwe cijfers beschikbaar gekomen ten aanzien van de wegverhardingen. Die cijfers
tonen een verbetering. In 2008 was 13% van de wegverhardingen matig, terwijl 12% slecht was. Nu
zijn die cijfers: 5% matig en 7% slecht. Dit betekent een aanzienlijke verbetering ten opzichte van de
stand van zaken in 2008. Het komt er op neer dat 88% van de wegverhardingen nu een voldoende
scoort, terwijl in 2008 dat percentage 75 was. Wij zijn dus bezig met een verbeterslag.
          De fractie van LKA heeft een opmerking gemaakt over de verhoging van het parkeertarief. Die
verhoging zou de mensen afschrikken, waardoor zij elders zouden gaan winkelen. Het is in ons land,
ook in dit deel van het land, volstrekt normaal dat in binnensteden moet worden betaald voor parkeren.
Dat houdt natuurlijk ook verband met het grotere aanbod van winkelvoorzieningen in de binnensteden.
Kleinere plaatsen hebben een kleiner winkelaanbod. Daardoor is het veel moeilijker om daar betaald
parkeren te realiseren (daar zijn legio voorbeelden van hier in de regio). In verhouding tot Hengelo en
Enschede kent Almelo ook na doorvoering van de voorgestelde tariefsverhoging, namelijk met € 0,20
tot € 1,50, nog steeds een concurrerend uurtarief voor het parkeren, want Hengelo rekent € 1,90 per
uur en Enschede rekent € 2,20. Wanneer het parkeertarief in Almelo € 1,50 per uur wordt, dan is het
tarief relatief nog steeds aan de goedkope kant. Het college ziet dan ook geen reden om daar anders
mee om te gaan dan het heeft voorgesteld.
          Voorts heeft de heer Kamphuis een vraag gesteld over de regionale samenwerking. Hij stelde:
De opbrengsten van die samenwerking zijn nihil terwijl de samenwerking ons € 2 miljoen kost. Hoe zit
dat, vroeg hij, kan de wethouder uitleggen wat er met dat geld wordt gedaan? Dat kan ik wel
uitleggen. De totale bijdrage van de gemeente Almelo aan de Regio Twente bedraagt ruim € 2,7 miljoen
per jaar. Verreweg het grootste deel van dat bedrag bestaat uit een bijdrage aan de GGD, namelijk
€ 1,9 miljoen per jaar. De gemeente krijgt dit bedrag van het rijk en sluist dat door naar de GGD Regio
Twente. Die GGD voert namens de 14 Twentse gemeenten de basisgezondheidszorg en de
jeugdgezondheidszorg uit. Daar wordt het geld aan besteed. De rest van het bedrag van € 2,7 miljoen
wordt besteed aan de samenwerking in de Netwerkstad, verkeer en vervoer, recreatieve voorzieningen
enz. Overigens heeft de raad bij de voorjaarsnotabehandeling het voorstel aangenomen om met ingang
van 2015 een bedrag van € 165.000,-- te besparen op de recreatieparken. Dit komt in de loop van de
tijd dus terug.
          Tegenover de bijdrage van Almelo aan de Regio Twente staan wel degelijk ook opbrengsten.
Het deelnemen aan de regio zorgt er bijvoorbeeld voor dat Almelo meebeslist over de aanwending van
de brede doeluitkering verkeer en vervoer. Dat is een jaarlijkse bijdrage van € 40 miljoen van de
rijksoverheid aan de Regio Twente (zolang die nog WGR+-regio is, zo zeg ik erbij). Almelo ontvangt
voor verkeers- en vervoersprojecten geregeld cofinancieringsmiddelen uit deze regionale middelen. Ons
college heeft overigens medio dit jaar een raadsbrief hierover gestuurd naar aanleiding van vragen die
gesteld waren door de heer Pauwels. Bij die vragen zat een heel mooi staatje (dat heb ik bij mij). Daar
blijkt uit dat wat betreft de regionale uitgaven per inwoner, Almelo het beste scoort. Het is dus
absoluut niet zo dat wij alleen maar zouden betalen aan de regio en daar geen opbrengsten voor terug
zouden krijgen.




INT - 1201417                                                                                          9
</p><p>         De heer VAN WOUDENBERGH (wethouder): Mevrouw de voorzitter! Ik zal nu eerst een
algemene reactie geven op een aantal punten uit de verschillende algemene beschouwingen. Daarbij
begin ik met de opmerking van de PvdA-fractie over het evenementenfonds in relatie tot cultuur. Voor
de ontwikkeling van het evenementenfonds is nadrukkelijk meegegeven dat er aandacht moet komen
voor een evenwichtige verhouding tussen de recreatieve en culturele activiteiten. Het college is niet
voornemens om een heel nieuwe cultuurnota te schrijven. Aan het Beleidskader cultuur 2008-2011,
dat in 2008 door de raad is vastgesteld, is destijds ook een werkprogramma verbonden. Dit
werkprogramma is nagenoeg gerealiseerd, en zeer binnenkort zal het college een nieuw
werkprogramma cultuur vaststellen waarin de prioriteiten op cultuurgebied voor de periode 2011 tot
2015 worden vastgesteld. Het nieuwe werkprogramma komt naar ons idee tegemoet aan de vraag over
de wijze waarop wij in de komende periode de middelen voor cultuur willen inzetten. Uiteraard zullen
wij het werkprogramma met de raad delen.
         De fractie van Leefbaar Almelo vroeg om ruimte voor persoonlijke initiatieven van burgers en
groepen burgers. Met het overhevelen van het budget voor culturele activiteiten naar het
evenementenfonds, heeft de raad besloten om de besteding van het geld voor culturele activiteiten
zoveel mogelijk aan het initiatief van de bevolking over te laten. Daarmee wordt naar ons idee tegemoet
gekomen aan de wens om zoveel mogelijk ruimte te geven aan initiatieven van de burgers. In het door
mij reeds aangehaalde werkprogramma cultuur 2011-2015 dat nu ontwikkeld wordt, hebben wij het
voornemen opgenomen (ik zeg dit ook naar aanleiding van een opmerking van de CDA-fractie) om de
subsidieregeling amateurkunst zodanig te herzien, dat die een minder automatische subsidieverstrekking
inhoudt en meer recht doet aan initiatieven vanuit de bevolking en de dynamiek van de sector. Ook het
concept van de talentenfabriek en het bijbehorende jongerenproductiefonds heeft de intentie zoveel
mogelijk ondersteuning aan en ruimte te geven voor eigen culturele initiatieven zonder onnodige
bemoeienis van de overheid.
         Voorts vroeg de fractie van Leefbaar Almelo in de schriftelijke versie van haar algemene
beschouwing: Is het juist dat 22 interne cultuurcoördinatoren via bemiddeling van het KEPA zijn
opgeleid? Dat is inderdaad juist.
         Ook over het Theaterhotel heeft de fractie van Leefbaar Almelo schriftelijk een opmerking
gemaakt. Het verstrekken van de subsidie aan het Theaterhotel is een contractuele verplichting die de
gemeente tot 2017 is aangegaan. Omdat wij geen contractbreuk willen of kunnen plegen, kan er tot
2017 niets afgebouwd worden van deze subsidie. In het nieuwe werkprogramma cultuur dat wij op dit
moment in ontwikkeling hebben, is opgenomen dat wij in de periode 2011 tot 2015 willen werken aan
een nieuwe visie op de podiuminfrastructuur van onze stad. Daarbij willen wij onderzoeken op welke
manier wij de podia in Almelo moeten ontwikkelen, gelet op en in overeenstemming met het nu
geldende ontwikkelingsprogramma van onze stad.
         De VVD-fractie vroeg: Wil het college de vele particuliere initiatieven, zoals de Kunsthal Almelo,
krachtig ondersteunen? Het college ziet het als het ware als een missie om particuliere initiatieven in de
kunst- en cultuursector krachtig te ondersteunen. Wat dit betreft, is de ondersteuning van de
initiatieven voor de kunsthal een sprekend voorbeeld.
         Is het mogelijk te onderzoeken, zo vroeg de VVD-fractie, of door krachtenbundeling op het
gebied van cultuur tot kostenbeperking gekomen kan worden? Ja, wij onderzoeken die mogelijkheid op
dit moment inderdaad. Ik geef hierbij als voorbeelden de kunsteducatie-instelling, de muziekschool en
het creatief centrum. Daarmee hebben wij inmiddels de afspraak gemaakt om in het komende jaar te
bekijken in hoeverre nauwere samenwerking, shared serviceconcepten of zelfs fusie kan bijdragen tot
een beperking van de kosten. Voor 2013 hopen wij de raad daarover te rapporteren.
         Ook heeft de VVD-fractie gevraagd naar de mogelijkheid om kleinere c.q. grotere kavels te
verkopen in Almelo Noordoost. Het is bijna vanzelfsprekend dat de vraag vanuit de markt wordt
meegewogen in de ontwikkeling van Almelo Noordoost. Echter, deze afweging zal moeten worden
gemaakt in het kader van de totale woningbouwprogrammering, waarin het aanbod van verschillende
woonmilieus een belangrijk element vormt. Gelet op het feit dat er op dit moment 28 kavels in optie
zijn gegeven, is er nog steeds voldoende vraag naar kavels in Almelo Noordoost. Er zijn op dit moment
zelfs 17 optiehouders voor kavels kleiner dan 700 m2. In de praktijk blijkt echter dat het niet kunnen
verkopen van de eigen woning de grootste belemmering is voor de aankoop van een kavel.
         De fractie van Leefbaar Almelo vroeg in de schriftelijke versie van haar algemene beschouwing
of een eventueel paardendorp in Waterrijk wordt afgestemd met het Twents Hippisch Centrum Almelo.
Inmiddels hebben wij daar gesprekken over gehad met zowel de gehandicaptensport als het hippisch
centrum. Ook nemen wij dit onderwerp mee in de transitiestrategie voor Waterrijk.
         De fractie van de ChristenUnie heeft voorstellen gedaan ten aanzien van het kernwinkelgebied
en de exploitatie daarvan. Op de vragen hierover wordt antwoord gegeven in de verdere uitwerking van



INT - 1201417                                                                                           10
</p><p>de stedenbouwkundige visie voor de binnenstad. Daar komen wij op 22 november a.s. uitgebreid op
terug. De beslissing over de stedenbouwkundige visie voor de binnenstad heeft echter geen directe
gevolgen voor de begroting van 2012.
         De PvdA-fractie vroeg wanneer in de binnenstad de schop in de grond gaat. Inderdaad gaat dit
jaar de schop in de grond voor de Waterboulevard en de verbindende pleinen.
         Ik kom even terug op wat wethouder Schouten heeft gezegd ten aanzien van de motie (EA) van
de fractie van ALA-AOV waarin verzocht wordt om het plan Waterrijk niet in exploitatie te nemen,
alternatieven te zoeken en de grond te verkopen. Wij zijn op dit moment volop bezig met een
verkennend onderzoek naar de alternatieven en al dan niet tijdelijke invullingen van het
exploitatiegebied. Eerder hebben wij al toegezegd dat de raad op korte termijn geïnformeerd zal worden
over de Kansrijkheid van deze alternatieven. Daarbij nemen wij ook de opmerking van de VVD-fractie
mee dat tevens gekeken moet worden naar de juridische mogelijkheden om de bestaande contracten
met agrariërs open te breken. Daar komen wij zeer binnenkort op terug.
         Ik ga nu in op enkele amendementen, allereerst amendement Z van de fractie van de VVD,
waarvan de strekking is dat er gespaard moet worden voor de ontwikkeling van de binnenstad. Ook de
fractie van D66 heeft dat voorgesteld. In het kader van de ontwikkelstrategie die wij samen met de
provincie opstellen, wordt de financieringsstructuur verder uitgewerkt. Die zal bij de voorjaarsnota
2012 aan de raad worden voorgelegd. Fasering is daar een belangrijk onderdeel van, maar de suggestie
om te sparen, zullen wij zeker meenemen. Wanneer de indieners van amendement Z het hiermee eens
zijn, dan is het amendement volgens mij overbodig.
         Motie LA, ingediend door de fractie van ALA-AOV, stelt dat het binnenstadsplan
getemporiseerd moet worden en dat het bestaande winkelgebied aantrekkelijker gemaakt moet worden.
Daarnaast heeft de fractie van D66 gevraagd om meer aandacht te besteden aan de Grotestraat en
gedurende vijf jaar jaarlijks een bedrag van € 0,5 miljoen te investeren in de kwaliteit van de huidige
winkelstraten. Uiteraard hebben ook wij zorg voor het bestaande winkelgebied. Wat is opgenomen in
het nieuwe binnenstadsplan, is niet bedoeld als concurrentie voor het bestaande winkelbestand, maar
als een toevoeging om een complete binnenstad te maken, dus als een versterking daarvan. Het gaat
om het totale winkelgebied. Wij gooien absoluut niet het kind met het badwater weg. Wij ontraden dan
ook motie LA; ik meen dat mijn antwoord daar duidelijk over is.
         Ook over het water in de binnenstad is door de fractie van ALA-AOV een motie (NA) ingediend.
De discussie over het bevaarbare water is eerder gevoerd. De raad is eerder tot de conclusie gekomen
dat het water in de binnenstad bevaarbaar moet zijn. Het water terug in de stad is één van de
identiteitsdragers voor de stad. Als je het water terugbrengt in de stad, dan moet dat ook een echte
kwaliteitsslag zijn. Het wordt ook een kwaliteitsslag en een duurzame oplossing voor het openbaar
gebied. Om dit mogelijk te maken, maken wij ook heel graag gebruik van de bijdrage die wij van de
provincie ontvangen. Omgekeerd is deze kwaliteitsslag ook de voorwaarde voor de provincie om bij te
dragen. Overigens: ook als wij zouden kiezen voor ondiep water tot aan het nieuwe stadsplein, zouden
wij daarvoor praktisch gezien dezelfde panden moeten slopen.
         Motie MA gaat over de Kloosterhofflat. De fractie van ALA-AOV is van mening dat deze flat
niet gesloopt moet worden, maar dat wij moeten proberen deze te verkopen aan Beter Wonen of een
andere partij. Hoe het verder moet met de Kloosterhofflat, is afhankelijk van hoe het verder gaat met de
ontwikkeling van de binnenstad. Het vraagt nog even tijd om een definitieve koers in te slaan. Tot die
tijd wordt de flat in ieder geval niet gesloopt.
         De fractie van GroenLinks vroeg: Wanneer gaat de gemeente de Kloosterhofflat betalen? Dat is
geen kwestie van betalen (mijnheer Buitenweg), maar een kwestie van terugleveren van nieuw voor
oud en het vergoeden van de kosten van Beter Wonen. Hoe sneller voortgang kan worden gemaakt,
hoe lager de kosten zullen uitvallen.
         Motie KA van de ALA-AOV-fractie gaat over het project verbetering kruispunt Gravenallee-Van
Rechteren Limpurgsingel. Dit project is een onderdeel van de Visie Stadsranden Netwerkstad Twente.
Deze visie is het resultaat van het clusteren van alle beleidsdocumenten van de Netwerksteden op dit
terrein, een samenhangend overzicht derhalve van alle plannen op het gebied van de stadsranden. In de
informatieve raadsbijeenkomst van volgende week (15 november) zal de raad uitgebreid worden
geïnformeerd over deze visie, inclusief het uitvoeringsprogramma dat hiervan deel uitmaakt. Het laten
vervallen van dit project heeft geen enkel positief effect voor de begroting van 2012. Het is met het
oog op externe financieringsmogelijkheden juist wenselijk om tijdig over gemeentelijke projecten te
beschikken. Zodra deze financieringsmogelijkheden zich voordoen, zal het college zich nader op de
voortgang van dit project beraden. Wij ontraden motie KA dan ook.




INT - 1201417                                                                                        11
</p><p>        De heer ANDELA (wethouder): Mevrouw de voorzitter! Ik zal eerst een aantal opmerkingen
maken over het personeelsbeleid en het personeel in zijn algemeenheid. De CDA-fractie heeft in haar
algemene beschouwingen opmerkingen gemaakt over Lean Six Sigma. Wij stellen voor om het proces-
en zaakgericht werken in te richten volgens de principes van Lean Management. Dat betekent:
terugdringen van de kosten, verkorten van de doorlooptijden en het verbeteren van de kwaliteit. Wij
willen daarbij uitgaan van het klantperspectief en het proces op basis daarvan analyseren en inrichten.
Ook willen wij daarbij alle onnodige activiteiten uit het proces verwijderen. Dat willen wij samen met
onze medewerkers doen. Wij schromen niet om daarvoor onorthodoxe middelen te gebruiken. Dat kan
bijvoorbeeld door processen te volgen met een lint, dwars door dit gebouw, om te zien waar een
proces overal langskomt en of het allemaal even handig is ingericht.

                                                             (De heer Kozijn komt, te 11.04 uur, ter
                                                             vergadering.)

         Vervolgens de vraag hoe wij de reductie van het personeel met 100 fte's willen realiseren. Die
reductie willen wij met name realiseren door taken te schrappen, zaak- en procesgericht te werken,
digitaal te gaan werken en te dereguleren. Dit jaar zijn de taken van een kleine 20 medewerkers
geschrapt. Die medewerkers hebben dat ook medegedeeld gekregen. De betrokken taken gaan wij niet
meer uitvoeren.
         De fractie van D66 heeft gevraagd om een grotere reductie van het personeelsbestand dan de
100 fte’s die wij hebben voorgenomen. Wij willen erop wijzen dat wij nu met de reductie met 100 fte's
een uiterste inspanning doen om te voorkomen dat er gedwongen ontslagen vallen. Als wij overgaan
tot meer reductie, dan ontkomen wij niet aan gedwongen ontslagen. Wanneer wij bovenop de
voorgenomen reductie nog eens een reductie met 100 fte's realiseren, dan leidt dat tot minimaal
€ 250.000,-- kosten per fte, maal 100. Het laat zich uitrekenen dat je dan niet komt aan een
bezuiniging van € 5 miljoen. Ook weet het college op dit moment niet welke taken het daarnaast nog
zou moeten schrappen. Dit is ook de reden waarom wij de motie hierover (Q) ontraden.
         Ook is gevraagd waar wij dereguleren. Welnu, in het kader van het zaak- en procesgericht
werken willen wij dereguleren en processen stroomlijnen. Een voorbeeld daarvan is de subsidieverlening
voor energiebesparing. Daarvoor is digitaal een heel korte vragenlijst aangereikt die ook digitaal wordt
afgehandeld. Zo konden de burgers met het invullen van tien vragen de subsidie aanvragen. Daarnaast
denken wij ook te dereguleren als het gaat om kapvergunningen, brandvergunningen,
parkeervergunningen, exploitatievergunningen en wat dies meer zij.
         De 10%-norm voor de inhuur van externen is aan de orde gesteld door de SP-fractie. Van 2007
tot 2010 was de inhuur ongeveer 15%. In 2010 zaten wij op de norm van 10%. Nu sturen wij op
reductie. Inhuur mag alleen plaatsvinden via het directieteam; lager in de organisatie mag daar niet over
besloten worden. Voorts rapporteren wij de raad maandelijks over de externe inhuur, maar wij willen
wel graag flexibiliteit houden. Als wij bijvoorbeeld een aantal medewerkers elders kunnen plaatsen en
wij daarvoor tijdelijke inhuur van personeel nodig hebben om die plaatsing te kunnen realiseren, dan
willen wij de flexibiliteit daarvoor wel graag hebben. Daarom hebben wij geen behoefte aan de strikte
norm van 10%, alhoewel wij de norm op zich delen. Daarom ontraden wij de motie (Y) hierover.
         Nog dit jaar zullen wij een raadsvoorstel doen over outsourcing. Outsourcen betekent niet op
voorhand bezuinigen. Het betekent namelijk dat je een bepaalde activiteit outsourct, maar die activiteit
gebeurt dan nog wel, en dat kost geld.
         Door de PvdA-fractie is gewezen op het gevaar van het uitstromen van goede werknemers. Dat
gevaar bestaat altijd en wij onderkennen dat. Wij proberen de medewerkers vroegtijdig duidelijkheid te
verschaffen over wie wel en wie niet vallen onder de personeelsreductie. Daarnaast proberen wij een
aantrekkelijke werkgever te zijn. Wij gaan over op Het Nieuwe Werken, dat wil zeggen: niet gebonden
aan plaats en tijd. Ook gaan wij meer digitaal werken en proberen wij daarvoor met name jongere
werknemers aan te trekken. Daarvoor willen wij ook aantrekkelijk blijven
         Dan maak ik nu een aantal opmerkingen over de woningbouw. Daarbij ga ik eerst in op het
migratieonderzoek, waar de VVD-fractie naar gevraagd heeft. Wij hebben toegezegd dat dit onderzoek
begin volgend jaar naar de raad zal gaan. Ons is verzocht om te onderzoeken wat er precies gebeurt op
dit gebied, met name of bewoners met een dikke portemonnee onze stad verlaten en mensen met een
smalle portemonnee naar onze stad toe komen? Dit is een moeizaam onderzoek, omdat wij daarbij
botsen met de privacywetgeving. Daardoor is het moeilijk bepaalde gegevens boven water te krijgen.
Ook blijkt dat in de omliggende gemeenten een aantal zaken niet geregistreerd wordt, althans niet zo
zorgvuldig en uitgebreid als bij ons gebeurt. Toch komen wij begin volgend jaar met de resultaten van
het migratieonderzoek.



INT - 1201417                                                                                          12
</p><p>         De fractie van Leefbaar Almelo heeft een motie aangekondigd met betrekking tot de woonvisie.
Mijn reactie daarop is dat wij de woonvisie samen met de provincie opstellen. Ook worden de
omliggende gemeenten daarover geïnformeerd. Zij krijgen onze woonvisie ter inzage en kunnen daarop
reageren. Dan kunnen zij de woonvisie ook toetsen aan bijvoorbeeld de prestatieafspraken. Volgende
week komt de woonvisie in een informatieve raadsbijeenkomst ter bespreking aan de orde. Dan
bespreken wij met name het aandeel van de sociale woningbouw in deze stad. Op dit moment is er een
hoog percentage huurwoningen in Almelo. De vraag is hoe wij daarmee moeten omgaan. Het college
wil in ieder geval graag draagvlak hebben voor de woonvisie. Wij zijn daarvoor in gesprek met de
corporaties. Kortom, wij doen een uiterste inspanning om uiteindelijk een gedragen woonvisie te
presenteren.
         Voorts is gevraagd of de koppeling tussen wonen en water niet beperkt is tot Waterrijk. Die
koppeling is inderdaad niet beperkt tot Waterrijk. Ik wijs bijvoorbeeld op Kerkelanden, waar wij water
teruggebracht hebben in de wijk. Daardoor snijdt het mes daar aan twee kanten: niet alleen is daardoor
de woonomgeving verfraaid, maar ook hebben wij daar wat gedaan aan de problematiek van het
grondwater.
         Verder zijn er, na de discussie die wij een aantal weken geleden daarover gehad hebben, vragen
gesteld over de afvalinzameling. Ik vraag de raad om de evaluatie te laten plaatsvinden een jaar nadat
de nieuwe werkwijze is ingevoerd. Dan hebben wij zicht daarop. Overigens heb ik toegezegd dat wij de
raad een brief zullen sturen over de parameters op basis waarvan wij evalueren. Daarover wordt de
raad dus geïnformeerd.
         Voorts is aandacht gevraagd voor het inzamelen van afval bij ouderen en mindervaliden. Daarbij
gaat het om het eens in de vier weken ledigen van de grijze bak. Dat is met name een probleem voor
de grote gezinnen en speelt wat minder voor ouderen en mindervaliden. Waar wij wel scherp op zullen
letten, is het zwerfvuil/-afval. Oplossingen voor plastic afval en luiers hebben wij al aangeboden, maar
wij letten ook hier scherp op.
         Wij zijn gekomen tot een tijdelijke invulling van de functie NME, namelijk 0,5 fte tot 2014. Wat
wordt er gedaan met die 0,5 fte? Om te beginnen het werk voor het programma WET (Water Energie
Twente). Dit is een samenwerkingsverband van een aantal Twentse steden waarvoor wij subsidie
krijgen van het rijk. Een belangrijk onderdeel daarvan is het uitlenen van lesmaterialen. Dat gebeurt op
het NME-centrum en is tot 2014 gegarandeerd. Ook wordt de functie van adviseur en aanspreekpunt
voor het te ontwikkelen Natuurhus vervuld. Met die ontwikkeling zijn wij bezig. Overigens zijn de
betrokken organisaties daarvoor de kartrekkers. Dat zijn wij dus niet als gemeente, want wij zijn
aanspreekpunt en adviseur, zoals gezegd. Het proces om te komen tot het Natuurhus verloopt goed. De
natuurorganisaties zoeken, naast een gezamenlijk onderkomen, ook een commerciële drager om het
Natuurhus niet tijdelijk maar structureel te laten zijn.
         Met betrekking tot het leerlingenvervoer heeft de CDA-fractie amendement C ingediend. Wij
kenden een overschrijding op deze post van € 130.000,--. Later was de overschrijding € 180.000,--, bij
elkaar opgeteld wel te verstaan. Het ging om € 130.000,-- plus € 180.000,--, dus ruim 3 ton. Er zijn
maatregelen getroffen om de overschrijding van € 180.000,-- terug te dringen. Wij hebben toegezegd
dat wij de raad tweemaandelijks zullen rapporteren over de ontwikkeling. De maanden september en
oktober zijn nu achter de rug, en eind november zullen wij de raad van de eerste resultaten van de
maatregelen op de hoogte stellen. Daarom vinden wij amendement C prematuur. Wij verzoeken de raad
te wachten op de presentatie van de effecten van de maatregelen. Dan kan de raad zien wat de
effecten zijn en beoordelen of aanvullende maatregelen nodig zijn. Als de effecten voldoende blijken te
zijn, kunnen wij alsnog conform amendement C handelen en de daarin bedoelde middelen toevoegen
aan de risicovoorziening. Op dit moment ontraden wij het amendement.
         Tot slot het stadhuis. Over het huidige stadhuis is een opmerking gemaakt in motie U van de
D66-fractie. In het vroege voorjaar van 2012 komen wij met voorstellen bij de raad met betrekking tot
het huidige stadhuis.
         Over de omvang van het nieuwe stadhuis is motie OA van de fractie van ALA-AOV ingediend.
Over dat onderwerp hebben wij gediscussieerd in verband met de personeelsreductie met 100 fte's.
Die reductie vindt lang niet alleen binnen dit huis plaats. Bovendien zal er een personeelsuitbreiding
plaatsvinden in verband met de decentralisatie van de jeugdzorg en de AWBZ-uitvoering. Wat wel
speelt, is de norm die wij hebben gehanteerd voor Het Nieuwe Werken (ook bij de presentatie van het
voorlopig ontwerp heb ik dat gezegd). Wij kijken nog eens naar die norm, want wij krijgen signalen dat
de norm van 0,9 wat ruim is voor het succesvol invoeren van Het Nieuwe Werken. Daarom kijken wij
naar een lagere norm. Als er een lagere norm komt, dan komt er ook een kleiner stadhuis. Daarvoor
komen wij in december aanstaande bij de raad. Als wij zouden besluiten tot het hanteren van een




INT - 1201417                                                                                         13
</p><p>lagere norm - en dus tot een kleiner stadhuis - dan zullen wij meteen ook met de financiële gevolgen
daarvan bij de raad komen. De motie die hierover is ingediend (OA), ontraden wij derhalve.
        Als wij komen met het definitieve ontwerp van het nieuwe stadhuis, dan komen wij ook met
het plan voor de parkeerlift (waar de fractie van Leefbaar Almelo iets over gesteld heeft in de
schriftelijke versie van haar algemene beschouwing). Wij gaan er nu van uit dat de parkeerlift de meest
efficiënte variant is om te gebruiken.
        Het besluit om een nieuw stadhuis te bouwen, is een jaar geleden genomen, hier in de raad. Het
is dan ook aan het college om dat besluit uit te voeren. Wij doen dat ook. Wij hebben vorig jaar
gekozen voor de goedkoopste variant. Weliswaar verschillen de kosten daarvan niet veel met de kosten
van de renovatievariant, maar toch is de nieuwbouwvariant - de sobere nieuwbouwvariant, zoals wij
die genoemd hebben - goedkoper. Het opknappen van dit stadhuis bleek veel duurder te zijn. Ik wijs
erop dat in de kostenopzet voor de renovatievariant niet de € 12 miljoen boekwaarde van het
Fortezzaterrein was meegenomen. In de kostenopzet voor de sobere nieuwbouwvariant is dat bedrag
wel meegenomen. Kortom, wij hebben gekozen voor de goedkoopste variant.
        Sommigen hebben de suggestie gedaan om te stoppen met de plannen voor de nieuwbouw van
het stadhuis. Huisvesten kost wel geld. Alle bewoners van deze stad kunnen bevestigen dat
huisvestingslasten spelen, ook voor hen. Ook voor ons spelen de huisvestingslasten. Als de raad geen
huisvestingslasten wil, dan zal hij in de buitenlucht moeten opereren. De raad zal dan in de buitenlucht
moeten vergaderen. Dan krijgen wij een soort hagenpreken! Ik ben het ermee eens dat daar in Waterrijk
nog ruimte voor is, maar ik adviseer om dan bij slecht weer een paraplu mee te nemen, want van
overheidswege worden paraplu´s niet verschaft!

        De heer VAN MARLE (wethouder): Mevrouw de voorzitter! Het college bestuurt de stad. Het
college neemt zijn verantwoordelijkheid en zorgt voor een sluitende begroting. Die ligt nu voor.
        De economische ontwikkelingen zijn zorgelijk. Deze ontwikkelingen gaan ook Almelo niet
voorbij. De fracties hebben daar in hun bijdragen hun zorg over uitgesproken. Het college deelt die
zorg, zoals door mij al is medegedeeld in een vorige zitting. Almelo verkeert financieel in zwaar weer.
Het is niet uitgesloten dat het zware weer blijft aanhouden. Dat is de reden waarom het college
voortdurend en nauwgezet kijkt naar de financiële ontwikkelingen. Rekening houdend met de positie
van het grondbedrijf, kan dit betekenen dat aanvullende voorzieningen noodzakelijk zijn. Als dat nodig
mocht zijn, dan gaan wij daarover in gesprek met de raad.
        Tegelijkertijd willen wij de stad niet op slot zetten en daarmee alle ontwikkelingen op voorhand
onmogelijk maken. Helemaal niets meer doen, levert wellicht op korte termijn financieel enige lucht op,
maar is funest voor de concurrentiepositie van Almelo op de lange termijn. Men proeft het al: daar zit
spanning. Die vraagt om weloverwogen keuzes en geen ad-hocmaatregelen.
        Vandaag gaat het om de begroting 2012. Dat is, ik heb het al gezegd, een solide en sluitende
begroting. Over die begroting beslist de raad vandaag, en bij de behandeling van de voorjaarsnota 2012
geven wij de richting aan voor 2013 en verder.
        Ik ga nu in op een aantal punten die afgelopen dinsdag in de bijdragen van de fracties zijn
genoemd. Ik zal daarbij meteen een aantal amendementen en moties behandelen.
        De fractie van het CDA heeft een vraag gesteld over de doelmatigheid, doeltreffendheid en
rechtmatigheid van de subsidies. Daarop wil ik antwoorden dat het door de CDA-fractie bedoelde
proces voor de subsidieverlening al sinds 2005, op basis van de vastgestelde nota subsidiebeleid en de
algemene subsidieverordening, operationeel is. In dit proces wordt getoetst op doelmatigheid,
doeltreffendheid en rechtmatigheid als het gaat om de formulering van de maatschappelijke vraag, de
samenstelling van de uitvoeringsovereenkomst, de kwartaalgesprekken, de kwartaalverslagen en de
vaststelling van de jaarrekening. De planning van dit proces is opgenomen in de
uitvoeringsovereenkomsten met de instellingen en sluit aan op de P&C-cyclus van de gemeente. Deze
werkwijze is voor alle gesubsidieerde instellingen in ontwikkeling.
        Dan kom ik nu bij amendement B, dat gaat over Scoop. In dit amendement wordt gesteld dat
een deel van de taakstelling voor Scoop toegevoegd moet worden aan de risicovoorziening. Het college
heeft voorgesteld om de risicovoorziening als een structurele risicovoorziening beschikbaar te houden.
Hieruit kunnen dan ook bij de voorjaarsnota structurele tegenvallers worden bestreden, zoals bedoeld in
amendement B. Incidentele mee- en tegenvallers bij bezuinigingen worden gestort in de reserve
takenanalyse. Dit is ook in lijn met de doelstelling van het instellen van de reserve takenanalyse, begin
2010. Daarbij is aangegeven dat incidentele opbrengsten van extra bezuinigingen aan de reserve
worden toegevoegd en dat incidentele extra kosten als gevolg van faseringsverschillen ten laste van
deze reserve worden gebracht. Over de besteding van de reserve wordt de raad in het kader van de
P&C-cyclus geïnformeerd.



INT - 1201417                                                                                         14
</p><p>         Met betrekking tot amendement N (Almelo Sociaal) merk ik op dat wij voor de algemene mee-
en tegenvallers de algemene reserve hebben. Deze is bedoeld als een algemene risicoreserve. Het moge
duidelijk zijn dat het college dan ook wenst vast te houden aan zijn voorstel met betrekking tot de
inwerkingtreding van de reserve risicovoorziening, de structurele reserve en de reserve takenanalyse.
         Ik wil nu wat zeggen over de toeristenbelasting, waar een aantal fracties opmerkingen over
gemaakt heeft. Allereerst wil ik duidelijk stellen dat de toeristenbelasting geen doelbelasting is, geen
bestemmingsheffing, maar een algemeen dekkingsmiddel en dus een algemene heffing. Met de
invoering van de toeristenbelasting beogen wij niet om een bijdrage te leveren aan de
bezuinigingsopgave, maar om met de opbrengst nieuw beleid te financieren, zoals afgesproken is in de
najaarsnota van 2010. Nogmaals: het gaat niet om een bezuinigingsmaatregel. Het gaat wel om de
inzet van middelen waarover de raad bij de voorjaarsnotabehandeling van 2011 heeft besloten en welk
besluit is verwerkt in de begroting voor 2012. Hieruit blijkt dat de opbrengst wordt besteed aan diverse
intensiveringen, waar ook de toerist direct of indirect baat bij heeft.
         Wat betreft de lastendruk voor de burgers, is een vraag gesteld over de WOZ-waarden. De
WOZ-waarden worden inderdaad gebruikt als grondslag voor de aanslagen ozb, dat is correct. Met de
raad is echter ook afgesproken dat het bedrag van de ozb-aanslag gelijk dient te blijven. In het geval
dat de WOZ-waarden in Almelo stijgen, zal het tarief voor de ozb naar beneden worden bijgesteld, zoals
ook in de afgelopen jaren is gebeurd. Nu er sprake is van een waardedaling, zal het tarief dan ook
stijgen om het bedrag aan ozb gelijk te laten blijven.
         Als ik het heb over de totale lastendruk, heb ik het over de ozb, de afvalstoffenheffing en het
rioolrecht. Ik kan de raad mededelen dat wij een nivellerend beleid volgen op dit gebied. De financieel
zwakkeren in onze samenleving worden daarbij zoveel mogelijk ontzien. De totale gemiddelde
woonlasten per huishouden met een koopwoning, worden in 2012 verhoogd ten opzichte van 2011.
De gemiddelde woonlasten stijgen met 2,9% (zoals men heeft kunnen zien in het staatje op bladzijde
201 van de begroting). Deze stijging van de lastendruk is voornamelijk het gevolg van de aanpassing
aan de inflatie (2,25%). Daarnaast is rekening gehouden - daar had de raad toe besloten - met de
stijging van het rioolrecht. Dat is gedaan om toekomstige investeringen te kunnen doen. Hierover is
motie T van de fractie van D66 ingediend. Daarin staat dat de ozb met ingang van 2013 met 4,8%
verhoogd moet worden. 2013 is nu niet aan de orde. Ik stel voor om dit onderwerp mee te nemen bij
de behandeling van de voorjaarsnota 2012.
         Vervolgens is een vraag gesteld over de voortgang rond het evenementenfonds. Ik kan
mededelen dat naar verwachting (dat zal ook gebeuren) het evenementenfonds per 1 januari 2012
wordt overgedragen aan een nieuwe stichting. De heer Egbert Willems heeft zich opgeworpen als
kartrekker en is geïnstalleerd als kwartiermaker. Hij heeft een nieuw bestuur geformeerd. Op 15
november a.s. komt het nieuwe bestuur voor het eerst bij elkaar. In grote lijnen is er overeenstemming
over de over te dragen taken en middelen. In de nieuwe organisatie worden delen van het huidige PECT
en de huidige Stichting Stadspromotie Almelo ondergebracht. De taken van de nieuwe stichting zijn:
coördinatie en ondersteuning van sport (geen topsport), cultuur, publieksevenementen, stadspromotie
(onder andere de VVV) en citymarketing.
         Met betrekking tot zwembad Het Sportpark zijn de moties G en JA (niet te verwarren met ja!)
ingediend. De raad is al eerder toegezegd dat een onderzoek zal worden uitgevoerd naar de exploitatie-
mogelijkheden van het binnen- en buitenbad hier in Almelo. Zodra de resultaten van dit onderzoek
bekend zijn, komen wij daar uiteraard op terug bij de raad.
         De PvdA-fractie heeft een opmerking gemaakt over de vangnetregeling schoolzwemmen. Met
betrekking tot deze regeling kan ik mededelen dat kinderen van ouders met een minimuminkomen een
vergoeding kunnen krijgen van maximaal € 225,-- per jaar uit het jeugdsportfonds in Almelo. Voor
zwemlessen met diplomagarantie in zwembad Het Sportpark zijn ouders € 10,-- per maand
verschuldigd. Voorwaarde hiervoor is wel dat zij aan de criteria voor het minimabeleid voldoen. Dat wil
zeggen dat zij een inkomen hebben dat minder is dan 115% van het wettelijk sociaal minimum. Het
jeugdsportfonds biedt de mogelijkheid voor alle kinderen van minima om zwemles vergoed te krijgen.
Daarmee is in mijn ogen een vangnetregeling schoolzwemmen niet meer nodig, zoals wij ook
geconstateerd hebben bij de voorjaarsnotabehandeling.
         Met betrekking tot de bezuinigingen op de sport heeft de VVD-fractie een vraag gesteld met het
oog op de mogelijkheid van privatisering. Bij de uitwerking van de bezuinigingstaakstelling voor de sport
wordt ook gekeken naar de mogelijkheden voor het overdragen van taken aan verenigingen. Hierbij
wordt ook nadrukkelijk bekeken of en, zo ja, in hoeverre privatisering van de complexen mogelijk is. Ter
toevoeging: het hockeycomplex is al geprivatiseerd.
         Tegen de fractie van Leefbaar Almelo zeg ik dat naar onze mening verschraling van het aanbod
niet aan de orde is bij ons voorstel met betrekking tot de combinatiefunctionarissen. De



INT - 1201417                                                                                         15
</p><p>combinatiefunctionarissen zijn werkzaam voor zowel onderwijs als sport en vormen de verbinding
tussen de scholen en de sportorganisaties. Door de inzet van de combinatiefunctionarissen wordt het
mogelijk activiteiten af te stemmen op zowel de behoeften van de jeugd als de wensen van de
sportverenigingen.
         Er zijn door een aantal fracties opmerkingen gemaakt over de schuldpositie van de gemeente
Almelo. Die positie werd uitgedrukt in een percentage van het begrotingstotaal, allemaal heel leuk en
aardig! Hierbij werd aangegeven dat onze schuld ten opzichte van de begrotingsomvang fors is - te fors
is, werd te verstaan gegeven - en dat de schuldpositie in de afgelopen jaren flink is toegenomen. Het
klopt dat de schuldpositie in de afgelopen jaren is toegenomen. Wij hebben ook fors geïnvesteerd in
bezit. Dat heeft te maken met de aankopen voor de binnenstad en Waterrijk die voortvloeien uit het
vestigen van het voorkeursrecht van de gemeente. Voor Waterrijk heeft de gemeenteraad daartoe
besloten in juli 2004, voor de binnenstad is daartoe besloten in 2005 en tot een uitbreiding is besloten
in 2006. Kortom, dat heeft plaatsgevonden in de raadsperiode 2002-2006. Voor het verwerven van
bezit zijn uiteraard leningen aangegaan. Voor deze schuld hebben wij echter wel iets teruggekregen,
namelijk het bezit. De bezittingen staan voor een bedrag van ruim € 300 miljoen in de boeken (ten
opzichte van de schuldpositie). Dit bedrag alleen zegt niet zoveel. Om een indruk te geven, noem ik
enkele vergelijkende cijfers die wij hebben over de nettoschuld ten opzichte van de exploitatie. Over
2009 waren die cijfers:
- Hengelo - 183%;
- Enschede - 72% (niveau Almelo);
- Amersfoort - 165%.
         Ik wil hier alleen maar mee aangeven dat deze percentages op zich niet zoveel zeggen.
Daarnaast is niet alleen de schuldpositie van belang, want het gaat er ook om of wij de rente over de
schuld kunnen dragen binnen de begroting. Het college zal een aantal parameters ontwikkelen welke
inzicht geven in onze financieringspositie en onze schuldpositie. Het zal echter duidelijk zijn uit mijn
betoog dat het college de motie hierover (R) van de fractie van D66 ontraadt.
         Motie M van de fractie van de ChristenUnie gaat over vrijval van reserves en voorzieningen.
Jaarlijks vindt een heroverweging van de reserves en voorzieningen plaats. Dat gebeurt bij het opstellen
van de voorjaarsnota. Hierover vindt in juni van elk jaar besluitvorming plaats in de raad. Motie M
vraagt om de heroverweging van de reserves en voorzieningen al in maart 2012 aan de raad
beschikbaar te stellen en om vrijvallende middelen al op voorhand te bestemmen voor het grondbedrijf.
Het college stelt voor deze zaken integraal tegen elkaar af te wegen en om dat te doen bij de
voorjaarsnota 2012. Zoals aangegeven, zullen deze zaken dan samen met de raad worden opgepakt.
Conclusie: de bedoeling van motie M kunnen wij onderschrijven - dat is immers bestaand beleid - maar
de gestelde termijn onderschrijven wij niet.
         Wat betreft overschrijding van budgetten, is het standpunt van het college: taakstellend is
taakstellend. Echter, bij een begroting van ruim € 280 miljoen vinden er wel eens onderschrijdingen en
overschrijdingen plaats. In de P&C-cyclus is er in voorzien dat substantiële afwijkingen ten opzichte van
vastgestelde budgetten (zowel begrotingen als kredieten) worden gerapporteerd aan de raad. Het
begrip "substantiële afwijking" is niet geconcretiseerd, omdat het om diverse aspecten en grootheden
kan gaan. Ik noem: financiële afwijkingen, afwijkingen in de planning (doorlooptijd) en afwijkingen in
kwaliteit. Ten aanzien van het financiële aspect is door de CDA-fractie gesteld dat de raad
geïnformeerd zou moeten worden bij afwijkingen van 10% op een krediet van € 50.000,--. Wij vinden
die grens in dit voorbeeld te laag. Van een afwijking van € 5000,--(zijnde 10% van € 50.000,--) mag
verondersteld worden dat deze door bijsturing elders binnen het programma kan worden opgevangen.
Een dermate "lage" overschrijding in absolute zin is te gering om op het sturingsniveau van de raad te
brengen, naar het inzicht van het college. Dit is ook in lijn met het autorisatieniveau van de raad. Wij
stellen derhalve voor om de volgende interne richtsnoer te hanteren: er moet gerapporteerd worden
over financiële afwijkingen van meer dan 10%, met een bodem (oftewel een minimum) van € 25.000,--
. Dat is een norm die wij ook al gehanteerd hebben bij de berap. Die geldt dan voor kredieten en
dreigende overschrijdingen van programma's. Wij zullen dit in de interne richtlijn voor de P&C-cyclus
verwerken.
         In motie IA van de fractie van ALA-AOV wordt gesteld dat het kantoor van het Sportbedrijf
moet verhuizen naar de IISPA. De kaders voor het Sportbedrijf worden vastgesteld door de raad.
Tijdens de behandeling van de kaderstelling, die overigens nog zal plaatsvinden, kan de raad
discussiëren over de inhoud. Overigens is de huisvesting een onderdeel van de bedrijfsvoering en dus
een aangelegenheid van het Sportbedrijf zelf.
         Over de werkgelegenheidsreserve is door de CDA-fractie amendement E ingediend. Dit betreft
de binnen de algemene reserve geoormerkte werkgelegenheidsreserve van € 1.362.000,--. Het aandeel



INT - 1201417                                                                                         16
</p><p>van de gemeente Almelo in het exploitatietekort van Soweco over 2012 is begroot op € 1,4 miljoen.
Dit aandeel is in de gemeentelijke begroting volledig afgedekt: € 1,2 miljoen komt ten laste van de
reserve participatie en 2 ton komt ten laste van het positieve resultaat op het begeleid werken WSW
dat door de gemeente wordt uitgevoerd (ik verwijs hierbij naar pagina 208 van de begroting). Zoals
men weet, staat de uitvoering van de huidige WSW onder grote druk. Er loopt een onderzoek naar hoe
het verder moet gaan met Soweco. Nog dit jaar kunnen wij daar de resultaten van tegemoet zien.
Duidelijk is al wel dat het anders moet en dat wij een weg moeten inslaan die niet eenvoudig is en geld
gaat kosten. Te denken valt aan kosten voor ontvlechting en transitie. Binnen de reeds geoormerkte
algemene middelen voor werkgelegenheid willen wij een bedrag van € 1.362.000,-- reserveren voor
afdekking van de financiële risico's ten aanzien van de omvorming van Soweco. Dit zeg ik als reactie
op amendement E.

        De heer NIJHUIS (VVD): Voorzitter! Wat is dan het advies van de wethouder in dezen?

        De heer VAN MARLE (wethouder): Dat wij amendement E kunnen omarmen.

       De heer BRAND (CDA): Voorzitter! Ik wil de wethouder nog een vraag stellen over motie D, die
handelt over de taakstellingen en dreigende overschrijdingen van 10%. De wethouder stelde voor om
de bodem te bepalen op een bedrag van € 25.000,--. Betekent dit dat hij voor het overige motie D
aanvaardt?

         De heer VAN MARLE (wethouder): Ja. Wij zijn uitgegaan van een afwijking van 10%, met een
minimum van € 25.000,--.
         Voorzitter! De heer Veenstra heeft amendement L ingediend, waarin staat dat een
besparingstaakstelling per maatschappelijke opgave geformuleerd moet worden. Het amendement stelt
in feite voor om de besluiten die genomen zijn bij de behandeling van de voorjaarsnota 2011, terug te
draaien en om te zetten in een taakstelling voor verdere gesprekken met de maatschappelijke partners.
Een dergelijk besluit doet geen recht aan de bezuinigingen die nu ook al worden behaald, bijvoorbeeld in
het eigen apparaat, en leidt naar ons idee tot maatschappelijke onrust, omdat maatschappelijke
organisaties dan niet meer weten waar zij aan toe zijn. Het leidt bovendien tot een begroting met
taakstellingen. Een dergelijke begroting wordt niet goedgekeurd door de provincie.
         Motie O van de fractie van D66 gaat over extra bezuinigingen op alle beleidsterreinen. De
komende jaren, tot eind 2014, bezuinigt de gemeente Almelo € 9,5 miljoen. Dat is wat afgesproken is
bij de voorjaarsnota. In die bezuiniging zit een krimp van het ambtelijk apparaat met minstens 110
fte's. Het gaat dus om een enorme operatie met grote maatschappelijke effecten. Die operatie vraagt
veel van de stad en de organisaties. De fractie van D66 stelt voor (en nu combineer ik enkele moties
van die fractie) om nog eens € 9,5 miljoen extra te bezuinigen. Daarmee gaat die fractie volledig voorbij
aan alle onderliggende vraagstukken. Welke taken moeten komen te vervallen? En welke taken kunnen
volgens de D66-fractie efficiënter? Moeten we helemaal stoppen op het terrein van sport? Of cultuur?
Geen huisvestingsplan voor het onderwijs? Geen leerlingenvervoer? Het is al te gemakkelijk om te
roepen dat er, conform motie O, moet worden bezuinigd op alle beleidsterreinen zonder daar ook maar
enige inhoudelijke sturing aan te geven. Graag zouden wij van de fractie van D66 horen welke
inhoudelijke koers zij vaart. Dat Almelo in zwaar weer zit, moge duidelijk zijn. Tegelijkertijd zijn wij van
mening dat nu de begroting voor de jaarschijf 2012 wordt vastgesteld. Motie O en ons gesprek
daarover horen thuis bij de voorjaarsnota 2012. Dan moeten wij met elkaar in debat gaan. Geef dan
vooral aan welke keuzes, met name welke inhoudelijke keuzes, de fractie van D66 wil maken. Het mag
duidelijk zijn dat wij motie O met grote klem ontraden.
         Ik wil nog wat zeggen over de septembercirculaire, waar door een aantal fracties naar gevraagd
is. De raad is via een raadsbrief al geïnformeerd over de uitkomsten van de septembercirculaire.
Conform de beleidslijn worden de gevolgen van de meicirculaire verwerkt in de begroting (dat is ook
gebeurd in de begroting voor 2012) en worden de gevolgen van de septembercirculaire meegenomen
bij het opstellen van de voorjaarsnota 2012.
         Ten aanzien van risicobeheersing en risicomanagement is gevraagd wat in de begroting bedoeld
wordt met de categorieën laag, middel en hoog risico en hoe wij daarbij komen. De risicotaxatie is
ingegeven door: de kans dat een bedreiging zich daadwerkelijk voor zou kunnen doen, de onzekerheid
hierover, de hiermee gemoeide financiële belangen (dus de omvang van de bedreiging) en, secundair,
de daarmee samenhangende verwachte bijsturingsmogelijkheden en mogelijke alternatieven. De
kwaliteit van de taxatie van het risico wordt procesmatig geborgd door voorafgaande afstemming




INT - 1201417                                                                                            17
</p><p>tussen verschillende disciplines, met name control, materiedeskundigen en verantwoordelijk ambtelijk
management. Aan de taxatie ligt geen rekenmodel ten grondslag.
         Het laatste onderwerp waar ik op in ga, is het eventueel meedoen aan de zogenaamde
stresstest. Een dergelijke test is onlangs ook bij banken uitgevoerd, en men heeft kunnen zien dat een
bank die mee had gedaan, toch in de problemen kwam. Recentelijk is in diverse publicaties aandacht
gevraagd voor een gemeentelijke stresstest. Dit analoog aan de stresstest voor de banken (ik heb het al
gezegd). Het idee van een stresstest voor gemeenten is niet nieuw; Deloitte heeft er in het afgelopen
jaar al een aantal uitgevoerd. De 100.000+- gemeenten hebben aangegeven ook een stresstest te
willen ontwikkelen. Dat wil men gaan doen in overleg met de provincies, uiteraard omdat de provincies
een toezichthoudende taak hebben. Op dit moment is een gemeentelijke stresstest nog niet
beschikbaar. De komende jaren lopen wij (ik heb het al eerder gezegd) op diverse terreinen financiële
risico's. De grondexploitatie blijft en houdt onze aandacht, en is dus een aandachtspunt. Ook met de
komende decentralisaties, die gepaard gaan met efficiencykortingen, nemen de financiële risico's toe.
Daarnaast zijn op diverse terreinen bezuinigingen vastgesteld welke de komende jaren doorgevoerd
moeten worden. Al met al zijn er dus voldoende uitdagingen voor de komende jaren. Hierbij is het van
belang te zorgen voor een deugdelijk huishoudboekje. Voor diverse onderdelen zijn inmiddels
maatregelen getroffen. Hierbij kan men denken aan extra bezuinigingen en het vormen van reserves (ik
doel nu op het bedrag van € 1 miljoen dat extra is bezuinigd en dat onderdeel uitmaakt van de € 9,5
miljoen). Ook hebben wij het afgelopen jaar extra aandacht gegeven aan instrumenten voor
risicomanagement (ik heb daar zojuist al wat over gezegd). Dat zullen wij de komende periode verder
uitwerken en doorontwikkelen. Mocht de te ontwikkelen stresstest bijdragen aan het beter in beeld
krijgen van financiële risico's, dan zullen wij dat uiteraard meenemen.

        De VOORZITTER: Dames en heren! Hiermee zijn wij aan het eind gekomen van de
beantwoording door het college in de eerste termijn. Die beantwoording heeft langer geduurd dan wij
gepland hadden. Vooraf hadden wij echter vastgesteld dat u behoefte heeft aan een goede
beantwoording door het college van uw vragen, amendementen en moties. Voorts is gevraagd om na
de beantwoording door het college de vergadering te schorsen. Het lijkt mij goed om dat te doen. Ik
stel voor om nu tot 12.00 uur te schorsen.

        De heer SCHUUR (PvdA): Voorzitter! Ik stel voor om tot de lunch te schorsen, daarna te
lunchen en vervolgens de vergadering voort te zetten.
        Ook heb ik een aanvullend ordevoorstel. Ik wil namelijk graag dat het college ook in een tweede
termijn reageert, want ik wil de raad graag de mogelijkheid geven om met de diverse wethouders in
debat te gaan.

       De VOORZITTER: Wij zullen straks even bekijken hoe dit laatste voorstel past in het
programma.
       Dames en heren! Ik stel voor om nu over te gaan tot een schorsing en om tijdens de schorsing
tevens te lunchen. Dan kunnen wij elkaar hier weer om 13.00 uur treffen. Ik stel vast dat u daarmee
instemt.

        Hierna schorst de VOORZITTER de vergadering.

        Na hervatting der vergadering spreekt de VOORZITTER als volgt:
        Dames en heren! Ik geef nu graag het woord aan de heer Stapel, voorzitter van het presidium.
Tijdens de pauze heeft hij samen met het presidium een agendavoorstel gemaakt voor de rest van deze
dag.

        De heer STAPEL (PvdA): Mevrouw de voorzitter! Zoals iedereen nu kan zien op de schermen in
deze zaal, hebben wij het ordevoorstel gedaan om, in plaats van om 15.00 uur, pas om 17.00 uur te
schorsen tot de besluitvorming. Wij willen nu eerst twee uur uittrekken voor het debat in de raad en
vervolgens een halfuurtje schorsen. Daarna kan anderhalf uur worden uitgetrokken voor de tweede
termijn voor het college. Overigens heeft de raad het verloop van de vergadering uiteraard zelf in de
hand. Wel vraag ik om de interrupties kort te houden en de debatten zo kort mogelijk te voeren, zodat
er zoveel mogelijk tijd overblijft voor de fracties om na te denken over de amendementen en moties.

       De VOORZITTER: Dames en heren! Stemt u in met het ordevoorstel van het presidium? Ik stel
vast dat dat inderdaad het geval is. Dan stel ik nu aan de orde:



INT - 1201417                                                                                       18
</p><p>2e termijn raad/raadsdebat.

        Mevrouw TEN SELDAM-EVEN (CDA): Mevrouw de voorzitter! Met veel belangstelling heeft de
CDA-fractie afgelopen dinsdag naar de commentaren en visies op de begroting 2012 van de andere
fracties geluisterd. De rode draad in alle bijdragen was de constatering dat de financiële positie van de
gemeente zorgelijk is. De visies op hoe deze positie opgelost zou moeten worden, zijn niet eensluidend.
Nee, ze lopen uiteen, maar dat is natuurlijk inherent aan de politieke achtergrond.
        Om de stad in balans te brengen, moeten wij durven kiezen voor bezuinigingen, maar ook met
betrekking tot de beleidskeuzes dienen wij ons kritisch op te stellen. De keuzes moeten verantwoord
zijn en er moet een gedegen plan aan ten grondslag liggen. "Gedegen" is voor de CDA-fractie dat het
ook realistisch is, zowel in de uitvoering als vanuit financieel oogpunt. Wij stellen voor om externe
deskundigen te vragen om grote projecten op hoofdlijnen door te lichten. De vraag die daarbij in wezen
beantwoord moet worden, is dan: Zijn het realistische plannen vanuit financieel perspectief gezien?

       De heer KOZIJN (PvdA): Voorzitter! Bedoelt mevrouw Ten Seldam te zeggen dat het haar
voorkeur heeft om projecten te laten doorlichten op hun financiële haalbaarheid?

         Mevrouw TEN SELDAM-EVEN (CDA): Dat bedoelde ik inderdaad te zeggen. Wel gaat het mij om
een doorlichting op hoofdlijnen. Ik wil graag dat bekeken wordt of een project realistisch of niet
realistisch is. Ik stel dit voor, omdat wanneer mensen de ambitie hebben om iets te realiseren, ze de
neiging hebben om de zaken in hun eigen voordeel te bekijken. Wat dat betreft, meen ik dat een
externe doorlichting buitengewoon goed is.

       De heer KOZIJN (PvdA): Maar mijn vervolgvraag is natuurlijk: Heeft u niet de indruk dat
voldoende routines zijn opgedaan met de vaste ritmes die wij in dit huis hebben bij zowel het ambtelijk
apparaat als de gemeenteraad (via de commissie planning en control)? Kan daarmee niet het eventuele
ongemak ondervangen worden dat u probeert te duiden?

       Mevrouw TEN SELDAM-EVEN (CDA): Voor een groot aantal projecten ben ik dat met u eens,
maar ik meen dat het altijd goed is om echt grote projecten, majeure projecten, te laten doorlichten.
Zoals gezegd, moet dat dan niet in detail maar op hoofdlijnen gebeuren.

        De heer STORK (ALA-AOV): Voorzitter! Ook ik heb een vraagje aan mevrouw Ten Seldam over
wat zij nu voorstelt. Zou zij daar bijvoorbeeld ook Waterrijk bij willen betrekken, of de opdracht van
Deloitte om de grond te taxeren op marktwaarde? Zou zij zo ver willen gaan?

        Mevrouw TEN SELDAM-EVEN (CDA): U gaat nu in op één specifiek project.

        De heer STORK (ALA-AOV): Dat mag ook voor alle projecten van kracht zijn, hoor.

        Mevrouw TEN SELDAM-EVEN (CDA): Wat ik wil aangeven, is dat de financiële haalbaarheid
van de majeure projecten, ook gezien in de context van de tijd, heel belangrijk is. Wij stellen voor om
dat te laten beoordelen. De verdere uitwerking volgt dan vanzelf.

       De heer GERRITSEN (D66): Voorzitter! Ik ben toch wel benieuwd wat mevrouw Ten Seldam
bedoelt. Welke projecten zou zij willen laten doorlichten of op hoofdlijnen willen laten beoordelen door
externe deskundigen? Noem eens een voorbeeld, zodat onze fractie daar een beeld bij krijgt.

        Mevrouw TEN SELDAM-EVEN (CDA): Volgens mij denkt u altijd in beelden! Als ik het over de
grote, majeure projecten heb, heb ik het over bijvoorbeeld het binnenstadsplan. Dat noem ik een groot
en majeur project.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik vraag mij af of de CDA-fractie de door haar
voorgestelde toets aan het begin van projecten wil, dus voordat wij daar als raad zelf een besluit over
genomen hebben. Of wil zij die toets daarna? Overigens kan ik mij ook voorstellen dat de raad in
principe projecten wil steunen, maar dat hij alleen over grote projecten behoefte heeft om te zeggen:
Laat ons daarvoor nog een externe toets inbouwen. Wij moeten toch eerst zelf de discussie kunnen
voeren over de vraag of wij überhaupt willen dat bepaalde plannen doorgezet worden ja of nee?



INT - 1201417                                                                                             19
</p><p>           Mevrouw TEN SELDAM-EVEN (CDA): Je voert eerst de interne discussie. Mijn fractie hecht er
echter aan dat voordat het finale besluit genomen wordt over grote projecten, die extern getoetst
worden op de financiële haalbaarheid. In die zin is de door ons voorgestelde toets realistisch. Het gaat
erom welke alternatieven worden aangereikt en of die realistisch zijn, met name ook in de geest van de
tijd.
          Voorzitter! Er is eergisteren veel gesproken over de voorjaarsnota 2012. Wij willen daar graag
nog een punt aan toevoegen: Kunnen wij als Almelo een gooi doen naar de titel: Groenste stad van
Nederland? Het zou erg mooi zijn als wij deze titel nog in deze zittingsperiode kunnen behalen. Wil het
college hiervoor een voorstel uitwerken voor de voorjaarsnota 2012, om na te gaan of hetgeen wij
ambiëren, ook daadwerkelijk realiseerbaar is?
          Wat betreft Lean Six Sigma, is de CDA-fractie ontzettend blij dat ook het college dit omarmt. Ik
zou zeggen tegen de wethouder: Het is geen enkel probleem om onorthodox te werk te gaan op dit
gebied, want hoe onorthodoxer, hoe beter het resultaat vaak is. Of de wethouder daarbij kiest voor
lintjes, lijntjes, streepjes of wat dan ook, onze steun heeft hij daarbij.
          De SP-fractie stelde dat de raad bij de behandeling van de voorjaarsnota het stomme besluit
heeft genomen om zonder overleg met Scoop te bezuinigen op Scoop. Wij zien dat toch echt heel
anders. Immers, de gemeente is opdrachtgever en Scoop is opdrachtnemer. Het gaat daarbij om een
productsubsidie. Die dien je niet automatisch te verlengen. Nee, dat doe je op basis van
noodzakelijkheid, al dan niet behaalde resultaten en tegen marktconforme prijzen. Wij zijn dan ook blij
met het antwoord dat de werkwijze met betrekking tot de subsidies geen automatisme meer is, maar
dat daadwerkelijk wordt bekeken: Wat is de doelstelling, is het resultaat naar tevredenheid, moeten wij
daarin bijsturen of kunnen wij zeggen dat de doelstelling is gehaald, prima, streep eronder, klaar?

        De heer CORNELISSEN (SP): Voorzitter! De reden waarom de SP-fractie heeft gezegd dat een
dom besluit is genomen met betrekking tot Scoop, is het feit dat dat besluit niet aansluit bij "Het
verschil maken". In het kader van "Het verschil maken" hebben wij gezegd dat wij eerst zouden gaan
praten over hoe wij de doelstellingen bij zouden gaan stellen en hoe wij de zaken anders willen inrichten
in de stad. Daarna pas zouden wij gaan praten over het geld. Wat Scoop betreft, heeft de raad echter
tijdens de voorjaarsnotabehandeling besloten om eerst € 300.000,-- te gaan bezuinigen en vervolgens
te gaan nadenken over de invulling daarvan. Dat is de strekking van onze opmerking geweest.

        Mevrouw TEN SELDAM-EVEN (CDA): Wij zien Scoop, met alle respect voor de kennis en kunde
daarvan, als een zakelijke partner, een zelfstandige entiteit. Wij moeten echter zelf keuzes gaan maken.
Dat je daar betrokkenen bij betrekt, is prima, maar dat zijn niet degenen waaraan je aanbesteedt. Daar
kijken wij heel zakelijk naar, zonder afbreuk te doen aan de kwaliteit van Scoop.

       De heer VEENSTRA (ChristenUnie): Voorzitter! Ik begrijp niet zo goed waarom de CDA-fractie
het bedrag dat aan Scoop betaald wordt, een subsidie noemt. Daarmee wordt de discussie volgens mij
verward. Mevrouw Ten Seldam zei terecht dat taken worden aanbesteed aan Scoop. Dat is inderdaad
aanbesteden, waarbij gezegd wordt: Voor de uitvoering van die taken hebben wij zoveel geld over.
Toch? Ik meen dat het goed is om ook in die termen te blijven praten en om het niet iedere keer te
hebben over subsidie voor Scoop. In mijn beleving is dat een onjuiste term voor dit soort vergoedingen.

         Mevrouw TEN SELDAM-EVEN (CDA): Nu hebben wij het over de vraag welk etiket opgeplakt
moet worden en wat de naamgeving moet zijn. In mijn perceptie noem je de bijdrage aan Scoop een
productsubsidie. Als u wilt spreken over een som waartegen iets aangenomen wordt, dan vind ik dat
prima. De essentie voor mij is dat in ieder geval bekeken wordt wat de doelstelling is en met welk
middel dat bereikt wordt tegen welke prijs. Dat moet je elke keer als een periode afgelopen is, opnieuw
tegen het licht houden.
         Voorzitter! De PvdA-fractie stelde in relatie tot Scoop dat er de komende jaren geen
vermindering van het aantal activiteiten mag plaatsvinden. Zo ver wil de CDA-fractie niet gaan. Immers,
er zijn veel activiteiten die uitgevoerd worden in het kader van projecten. Een kenmerk van een project
is dat het een begin en een eind heeft. Waar het ons om gaat, is dat wij komen tot een afgebakend
welzijnsbeleid waarin doelstellingen en beoogde activiteiten zijn geformuleerd en aangegeven wordt
welke gelden waarvoor beschikbaar zijn. Het gaat ons veel meer om de kwaliteit dan om de kwantiteit.
Overigens denk ik dat de PvdA-fractie dat ook bedoeld heeft te zeggen, namelijk dat er een goed
aanbod moet zijn van welzijnswerk.




INT - 1201417                                                                                          20
</p><p>       De heer SCHUUR (PvdA): Voorzitter! Ik zal even zeggen wat wij bedoeld hebben te zeggen. Wij
hebben willen zeggen dat wij de bezuiniging waartoe is besloten bij de voorjaarsnotabehandeling,
hebben gezien als een efficiencyslag binnen Scoop. Dat is volgens mij ook de strekking van het besluit
dat toen genomen is, niet meer en niet minder. Daarmee zeg ik dus ook dat er wat ons betreft niet
gesneden moet gaan worden in de activiteiten van Scoop, noch kwalitatief, noch kwantitatief.

         Mevrouw TEN SELDAM-EVEN (CDA): Uw standpunt is mij duidelijk.
         Voorzitter! De fractie van D66 deed veel constateringen maar ging vervolgens niet in op de hoe-
vraag. Hoe kan het dan wel? De D66-fractie stelt een aantal forse bezuinigingen voor, maar daarin kun
je ook doorschieten wat de CDA-fractie betreft. Sterker nog: het lijkt er sterk op dat de fysieke
maatregelen volgens de fractie van D66 moeten worden betaald uit forse bezuinigingen op
leefbaarheid, sociaal beleid en veiligheid. Daar help je de stad echt niet mee.
         Wat betreft de moties en amendementen, ga ik eerst in op de moties en amendementen van de
CDA-fractie zelf en op de reacties die daarop zijn gegeven door het college.
         Het door ons ingediende amendement B gaat over Scoop. Wethouder Van Marle heeft
aangegeven dat het daarin genoemde geld incidenteel geld is, maar wat de CDA-fractie betreft, heeft
dat een structureel karakter. De taakstelling van Scoop wordt ingevoerd voor de komende drie jaren.
Het kan toch niet zo zijn dat wij vervolgens, in 2015, het budget automatisch weer opplussen met 3
ton? Dat is niet wat wij beogen. Nee, de taakstelling heeft een structureel karakter, waarvoor wij nu de
periode 2012 tot en met 2014 vaststellen. Vandaar dat wij tot de conclusie komen dat een deel van
het geld toegevoegd moet worden aan de risicovoorziening en niet aan de reserve takenanalyse.
         Ons amendement C gaat over het leerlingenvervoer. In de perceptie van de CDA-fractie gaat
het daarbij om een additioneel bedrag van € 130.000,--. Het bedrag van € 180.000,-- stellen wij niet
ter discussie. Ook stellen wij niet voor om het bedrag van € 130.000,-- nu niet ter beschikking te
stellen. Wel vinden wij het college een beetje pessimistisch als het gaat om 2013 en verder. Wij zien de
situatie veel optimistischer. Met wijzigingen in het beleid kunnen op dat moment de financiën voor het
leerlingenvervoer waarschijnlijk toereikend zijn. Mocht dat niet het geval blijken te zijn, dan kan het
college gewoon met een voorstel daarover komen. Op dit moment willen wij het geld niet volledig ter
beschikking stellen voor 2013 en verder. Vandaar ons amendement hierover.
         Dan amendement E, dat handelt over het reserveren van de geoormerkte
werkgelegenheidsreserve. Dank voor het feit dat het college dit amendement, zeg maar, omarmt. Nu
nog de steun van de raad, zou ik zeggen, maar dat hoor ik wel van de sprekers die na mij volgen.
         Ik kom nu bij onze moties, om te beginnen motie A over afgebakend welzijnsbeleid. Gehoord de
beantwoording door de wethouder, is het volgens mij van belang dat ik aangeef dat het ons niet om
een nieuwe beleidsnota gaat. De CDA-fractie heeft onvoldoende zicht op wat de gemeente doet aan
welzijnsbeleid. Dat bedoel ik echt heel positief. Ik bedoel niet te zeggen dat er te weinig welzijnsbeleid
is of dat het niet goed is. Nee, maar mijn vraag is wel: Wat doen wij met betrekking tot welzijnsbeleid?
Kan het college mij daar op enigerlei wijze inzicht in geven? Als ik dat inzicht heb, kan ik er ook op
sturen. Dat is wat ik vraag in motie A.

         De heer VEENSTRA (ChristenUnie): Voorzitter! Kan de CDA-fractie ook aangeven hoe zij de
relatie ziet tussen hetgeen in motie A gevraagd wordt en "Het verschil maken"? Zoals de wethouder
ook min of meer zei, is het formuleren van het welzijnsbeleid onderdeel van de Sociale Agenda 2020,
en dat is juist een project in het kader van "Het verschil maken". Zou wat de CDA-fractie verzoekt in
motie A, niet heel goed daarin kunnen passen?

       Mevrouw TEN SELDAM-EVEN (CDA): Dat zou er mogelijk in kunnen passen, maar wat ik
gezegd heb in de eerste termijn, is dat het welzijnsbeleid nu wat fragmentarisch is. Hier staat wat, daar
staat wat en daar staat wat over het welzijnsbeleid. Ik vraag om meer overzicht. Wat betreft de vraag
in welke vorm dat overzicht gegoten moet worden, moet volgens mij bekeken worden of aangesloten
kan worden bij bestaande systemen/overzichten/notities. Probeer daarvoor dus vooral niet iets totaal
nieuws te bedenken, maar probeer wel iets meer overzicht te geven. Dat vragen wij in motie A.

        De heer VEENSTRA (ChristenUnie): Voorzitter! De aanpak voor "Het verschil maken" en de
Sociale Agenda 2020 is toch bedoeld om dat overzicht te verschaffen en de onderlinge relaties aan te
brengen? De bedoeling is voorts om de doelstellingen daarvoor gezamenlijk met partners te formuleren.

       Mevrouw TEN SELDAM-EVEN (CDA): De Sociale Agenda 2020 kan zeker een alternatief zijn.
Dat hoor ik dan graag ook van de wethouder.



INT - 1201417                                                                                          21
</p><p>        De heer VEENSTRA (ChristenUnie): Dank u wel.

         Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Met betrekking tot onze motie D
(Taakstellingen) gaf de wethouder aan: Op zich vind ik de daarin verwoorde gedachtegang goed, maar
de situatie moet wel werkbaar blijven als er overschrijdingen zijn (ik vertaal zijn woorden nu even vrij).
De wethouder heeft het voorstel gedaan om te bepalen dat er gerapporteerd zal worden wanneer
overschrijdingen hoger zijn dan € 25.000,--, oftewel wanneer een krediet van € 250.000,-- met 10%
of meer overschreden dreigt te worden. Daar kunnen wij in meegaan, ervan uitgaande dat ik goed
verstaan heb dat kleinere overschrijdingen worden opgevangen binnen de eigen programma's. Daar
hoeft dan ook geen ander geld voor beschikbaar gesteld te worden, juist omdat de overschrijdingen
worden opgevangen binnen de programma's. Ook wil ik graag afspreken met de wethouder dat hij met
het rapporteren over overschrijdingen niet wacht tot het moment dat de berap verschijnt, maar dat hij
eerder komt met de rapportage. Als er daadwerkelijk overschrijdingen zijn, kom daarmee dan naar de
raad. Dan zitten wij er met elkaar bovenop. Dat is sturen. Naar aanleiding van wat de wethouder zei,
zal ik een gewijzigde versie van motie D indienen.

        De heer ISENDOORN (D66): Voorzitter! Ik wil iets vragen aan mevrouw Ten Seldam. Ik hoor
haar nu zeggen dat zij iets dat in mijn ogen bij het college thuishoort, naar de raad wil trekken. Ik vind
het namelijk de verantwoordelijkheid van het college om de begroting uit te voeren zoals wij dat
afgesproken hebben. Als zich daarbij dan overschrijdingen voordoen, dan moet het college gewoon
ingrijpen. Dat is zijn verantwoordelijkheid en rol. Wij als raad moeten controleren of het college dat
goed doet. Daar hebben wij iets over te zeggen. Ik wil waarschuwen niet alles naar de raad te trekken.
Dat gebeurt in mijn ogen toch al te veel. Laat ons de zaken bij het college houden, want dat zit er niet
voor niets. Het college moet gewoon goed zijn werk doen (daarvoor heeft het ook adviseurs in dienst).
Vervolgens horen wij het wel. Als het college zijn werk niet goed doet, dan leggen wij de vinger op de
zere plek. Of ziet mevrouw Ten Seldam de wijze waarop met overschrijdingen omgegaan moet worden,
heel anders? Vindt zij dat dit bij het college niet goed gebeurt en dat het daarom naar de raad
getrokken moet worden? Ik ben het overigens wel met de conclusie eens dat er geen overschrijdingen
getolereerd moeten worden, maar dat is wat anders.

        Mevrouw TEN SELDAM-EVEN (CDA): Wat u nu schetst, is niet datgene wat ik in gedachten
heb hierbij. Waar het mij om gaat, is dat ik mijn controlerende taak wil kunnen uitvoeren. Om die taak
te kunnen uitvoeren, wil ik graag tijdig geïnformeerd worden, gewoon op basis van feiten, wanneer
men fors uit de pas dreigt te lopen. Dat is wat ik wil weten, niet meer en niet minder. Volgens motie D
moet het college daarmee naar de raad komen. Forse overschrijdingen wil ik niet pas vernemen bij de
berap. Oftewel: ik wil mijn controlerende taak op elk gewenst tijdstip kunnen uitvoeren. Daarvoor wil ik
absoluut niet op de stoel van het college gaan zitten; de raad bevalt mij uitstekend.
        Voorzitter! De PvdA-fractie heeft een amendement (F) ingediend over "Het verschil maken" en
het bedrag van € 125.000,-- daarvoor. Over dat onderwerp hebben wij bij de voorjaarsnotabehandeling
uitvoerig gesproken. Ik kan de in de motie verwoorde gedachtegang heel goed volgen, maar de CDA-
fractie heeft bij de voorjaarsnotabehandeling haar keuze gemaakt. Daar willen wij vooralsnog zeker aan
vasthouden, maar wellicht weet de PvdA-fractie ons op een later moment nog te overtuigen.
        Amendement J heeft als titel: Kiezen voor sociale stijging. Dit amendement betreft de discussie:
Wat is de huidige balans, hoe constateer je dat en wat wil je verbeteren? Wat wij delen met elkaar, is
dat wij graag sociale stijging willen en dat wij daarvoor blijven gaan. Wij omarmen het amendement
echter niet.
        In amendement K wordt gesteld dat per programma een agenda 2020 moet worden opgesteld.
In deze economisch en financieel roerige tijd is het wel heel erg moeilijk om een realistische agenda
2020 op te stellen. Wij vinden dat niet realistisch en het roept het beeld op van een papieren tijgertje.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Wat wij beogen, heb ik alleen maar agenda
2020 genoemd omdat de sociale agenda ook zo heet. Dat mag wat mij betreft ook agenda 2015 of
agenda 2012 heten, dat zal mij worst zijn. Ik heb willen aansluiten bij de bestaande terminologie. De
reden daarvoor is dat ik meen dat de aanpak die gevolgd wordt voor het opstellen van de sociale
agenda 2020, juist de aanpak is die wij met ons allen hebben afgesproken in het kader van "Het
verschil maken". In dat kader is de aanpak heel fundamenteel aangegeven. Ik stel voor om diezelfde
aanpak voor alle programma's te volgen, want dan pas krijgen wij een inhoudelijke discussie met de
stad en met elkaar over de inhoud van de programma's.



INT - 1201417                                                                                           22
</p><p>         Mevrouw TEN SELDAM-EVEN (CDA): Goed, ik heb mijn zorg uitgesproken over het papieren
tijgertje. Mijn grotere zorg is dat dat een tijger wordt.
         Voorzitter! Ik ben aangekomen bij amendement N van de fractie van D66, dat handelt over
Almelo Sociaal. In onze beleving wordt volgens dit amendement geld toegevoegd. De CDA-fractie pleit
voor herallocatie van welzijnsactiviteiten c.q. activiteiten die een gunstig effect kunnen hebben op de
bedrijfsvoering van - en dat is echt een voorbeeld - Almelo Sociaal. Voor de rest verwijs ik graag naar
amendement B (over Scoop) van de CDA-fractie.
         Amendement W stelt dat Stadsboerderij Beeklust opengehouden moet worden. Gehoord de
wethouder, hebben wij er alle vertrouwen in dat dit gaat lukken. Daarom zullen wij amendement W niet
steunen.
         Volgens amendement Z van de VVD-fractie moet er gespaard worden voor de ontwikkeling van
de binnenstad. Gelet op de beantwoording door het college, wachten wij graag de integrale afweging
af. Wij hebben de indruk dat dit amendement net iets te vroeg is ingediend, maar wellicht weet de
VVD-fractie ons zo dadelijk nog te overtuigen dat het juist heel goed zou zijn om het te omarmen.
         Over overschrijding van de budgetten voor openeinderegelingen heeft de VVD-fractie
amendement AA ingediend. Naar aanleiding daarvan hebben wij de teksten van de najaarsnota, de
handelingen en het raadsbesluit erbij genomen. Volgens de CDA-fractie moeten oplossingen voor de
overschrijdingen in principe gevonden worden binnen het eigen programma (dat hadden wij destijds ook
voor ogen). Ik ga ervan uit dat als dat niet mogelijk is, daarover gecommuniceerd wordt met de raad.
De meest voor de hand liggende stap is dan dat de oplossing gevonden wordt binnen de algemene
reserve.

        De VOORZITTER: U heeft nog een halve minuut spreektijd, mevrouw Ten Seldam.

        Mevrouw TEN SELDAM-EVEN (CDA): Ik praat mijn tijd gewoon vol, voorzitter.

       De heer SCHUUR (PvdA): Voorzitter! Ik heb een vraag aan mevrouw Ten Seldam. Als van
tevoren al bekend is dat de overschrijding van het budget voor een openeinderegeling niet opgelost kan
worden binnen het eigen programma, dan is het toch raar om daar in principe wel een uitspraak over te
doen?

        Mevrouw TEN SELDAM-EVEN (CDA): Hoe bedoelt u dit?

        De heer SCHUUR (PvdA): Wij weten met ons allen dat de openeinderegelingen zullen leiden tot
forse overschrijdingen. Ook weten wij met ons allen dat wanneer wij die ten laste laten komen van het
betrokken programma - en hierbij kijk ik vooral naar bijvoorbeeld de openeinderegeling inzake de
bijstand - wij een heleboel van de ambities om sociale stijging te creëren, niet meer waar kunnen
maken, want die worden daardoor grosso modo wegbezuinigd. Dat is namelijk wat er dan gebeurt. Wij
weten per definitie dat de overschrijdingen van de budgetten voor de openeinderegelingen niet op te
lossen zijn binnen het sociale domein. Dan moet je daar op dit moment toch geen principe-uitspraak
over doen? Dat is toch niet handig?

      Mevrouw TEN SELDAM-EVEN (CDA): Het raadsbesluit dat wij destijds hierover hebben
aangenomen, luidt:
           "Kennis te nemen van de mogelijkheden om te komen tot kostenbesparingen op de
           gebieden Wmo en Sociale Zekerheid. Deze besparingen verder uit te werken en voor een
           deel in te zetten om de overschrijdingen van openeinderegelingen terug te dringen en voor
           een deel te gebruiken als mogelijke bezuiniging;"
      Dit is heel optimistisch geformuleerd, en daar refereer ik nu aan.

          De heer SCHUUR (PvdA): Wij hebben dit debatje al een keer gevoerd. Toen stond de wethouder
van financiën, de heer Van Marle, op de plek waar u nu staat. Hij heeft toen op mijn vraag of de
overschrijdingen op de openeinderegelingen ten laste worden gebracht van de algemene reserve,
letterlijk gezegd: Ja, mijnheer Schuur, die gaan wij ten laste brengen van de algemene reserve. Dat wil
de fractie van de PvdA graag zo houden.

        Mevrouw TEN SELDAM-EVEN (CDA): Dat is wat de heer Van Marle gezegd heeft, maar ik sta
hier natuurlijk als fractievoorzitter in een duaal stelsel en geef aan wat de CDA-fractie daarvan vindt.



INT - 1201417                                                                                          23
</p><p>Dan kan de indruk ontstaan dat er licht tussen zit, maar volgens mij zit er niet zo vreselijk veel licht
tussen. Alleen willen wij wel graag sturen.

         De heer NIJHUIS (VVD): Voorzitter! Volgens mij zijn wij het in dezen aardig eens met de CDA-
fractie. Verder heb ik met de wethouder afgesproken dat ik graag zal luisteren naar haar reactie in de
tweede termijn, omdat ik niet helemaal goed begrepen heb wat zij bedoelde met "de koker". Zij
gebruikte namelijk die term, terwijl wij het hadden over het sociaal domein. Dat is inderdaad heel breed.
Het is niet onze bedoeling om allerlei dingen die niet bij elkaar horen, tegen elkaar weg te strepen. Wel
willen wij zorgvuldig omgaan met de openeinderegelingen. Wij weten ook wel dat als iets absoluut niet
kan, wij uiteindelijk bij de reserve terechtkomen. Dat moet dan echter wel in die volgorde gebeuren.
Daar gaat het om, en daarom staat in ons amendement AA dat overschrijdingen op de
openeinderegelingen "in principe" binnen het sociale domein moeten worden opgevangen.

       De heer VEENSTRA (ChristenUnie): Voorzitter! Zou de CDA-fractie kunnen aangeven wat voor
haar de grenzen zijn waarbinnen de compensatie binnen de openeinderegelingen nog mag plaatsvinden?
Heeft de CDA-fractie zelf grenzen waarover zij zegt: Dan accepteren wij maar dat het budget wordt
overschreden?

        Mevrouw TEN SELDAM-EVEN (CDA): Nee. U richt zich nu alleen op het financiële aspect. Het
gaat ons niet alleen om het financiële aspect. Het gaat erom dat je met het geld doet wat nodig is.
Vandaar dat ik niet op voorhand mijn financiële grenzen wil aangeven, want daarmee zou ik het sociale
beleid tekortdoen.

         De heer VEENSTRA (ChristenUnie): Voorzitter! Als je zegt dat de openeinderegelingen
taakstellend zijn, dan stel je toch een financieel kader? Dat heeft toch niets te maken met de criteria?
Mevrouw Ten Seldam stelde dat zij geen financieel kader wil stellen in dezen, maar voor mijn gevoel
doet zij dat juist wel door alleen maar te zeggen: Het geld bepaalt uiteindelijk wat wij gaan doen in het
kader van de openeinderegelingen.

         De heer SCHUUR (PvdA): Voorzitter! Ik wil graag inzichtelijk maken wat het voorstel van de
VVD-fractie met betrekking tot de openeinderegelingen behelst. Dat houdt in dat daar waar
overschrijdingen plaatsvinden op de budgetten voor openeinderegelingen binnen het sociale domein,
eigenlijk gezegd wordt: Ga eerst maar eens kijken of de voorzieningen niet wat minder kunnen en of wij
niet wat minder mensen tegemoet kunnen komen die het al vreselijk moeilijk hebben. Dat is wat er
gezegd wordt, en dat is voor de fractie van de PvdA onaanvaardbaar.

       De heer NIJHUIS (VVD): Voorzitter! Wij zijn inderdaad van mening dat je eerst binnen het veld
moet bekijken wat kan. Uiteraard is daar dan een ondergrens aan verbonden en ga je niet beneden die
grens opereren. Op enig moment moet je die grens misschien met elkaar vaststellen. Ik ben het echter
eens met de CDA-fractie dat wanneer wij als raad goed ingelicht worden op het moment dat enorme
overschrijdingen dreigen, wij moeten proberen mee te denken over hoe daarmee omgegaan moet
worden. Daarbij is de algemene reserve uiteraard altijd (ik zeg het nogmaals) de escape die uiteindelijk
van toepassing kan zijn. Dat moet dan wel gebeuren in de volgorde die ik heb aangegeven.

         Mevrouw TEN SELDAM-EVEN (CDA): Ik zie dat de heren Schuur en Veenstra nog willen
interrumperen, maar helaas ga ik een ander punt aansnijden! Volgens mij heb ik over het vorige
onderwerp gewisseld wat ik wilde wisselen.
         Ik kom nu bij het voorstel van de fractie van D66, verwoord in motie O, om extra te bezuinigen
op alle beleidsterreinen. Ik koppel mijn opmerkingen daarover aan motie P (Extra bezuinigen door focus
op samenwerking). Ik ben heel erg benieuwd hoe groot het huidige budget is dat gemoeid is met de
regionale samenwerking. Ik vraag mij namelijk af hoeveel € 1,5 miljoen is ten opzichte van het totaal?
En hoe denkt de fractie van D66 de extra bezuinigingen concreet te bereiken? Ik wil heel graag dat dit
SMART geformuleerd wordt. Overigens heeft de fractie van D66 nog even tijd, dus die hoeft dat nu
niet bij interruptie aan te geven; ik hoor dat graag in haar bijdrage straks.
         Hetzelfde geldt voor het voorstel van de fractie van D66 inzake de extra bezuiniging op de
gemeentelijke organisatie.

        De VOORZITTER: Dit is het laatste onderwerp waarover u nu nog wat kunt zeggen.




INT - 1201417                                                                                              24
</p><p>        Mevrouw TEN SELDAM-EVEN (CDA): Prima.
        Vindt de fractie van D66 haar voorstel inzake de extra bezuinigingen op de gemeentelijke
organisatie (motie Q) realistisch? Zo ja, waar denkt die fractie dan echt aan? Toen de heer Gerritsen mij
zojuist vroeg om een concreet voorbeeld te noemen, heb ik dat gegeven. Ik vraag de heer Gerritsen nu
om twee voorbeelden te noemen. Ik hoor ze graag.
        Tot slot dien ik, zoals aangekondigd, namens mijn fractie een gewijzigde versie in van motie D.
Die luidt:

                "Motie D1.
                Onderwerp: taakstellingen
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011
                Constateert dat,
                - de gemeenteraad van Almelo bij diverse gelegenheden budgetten beschikbaar stelt
                    ten behoeve van kapitaalsinvesteringen, subsidies, exploitatieuitgaven e.d.;
                - dat deze budgetten in principe taakstellend zijn;
                - dat het bij uitzondering kan voorkomen dat de beschikbare budgetten niet toereikend
                    zijn.
                Is van mening dat,
                - de gemeenteraad vooraf dient te worden geïnformeerd over dreigende substantiële
                    overschrijdingen;
                - dat er sprake is van een substantiële overschrijding wanneer een krediet van ten
                    minste € 250.000,-- met 10% of meer dreigt te worden overschreden.
                Besluit:
                het college te verzoeken de raad vooraf te informeren en met aanvullende
                kredietvoorstellen te komen wanneer een budget van ten minste € 250.000,-- met 10%
                of meer dreigt te worden overschreden
                en gaat over tot de orde van de dag."

        De VOORZITTER: Dames en heren! Ik stel vast dat motie D is vervangen door motie D1.

          De heer DE OLDE (PVA): Mevrouw de voorzitter! Ik geef een korte samenvatting van de
algemene beschouwing van afgelopen dinsdag:
- Het is een treurige constatering dat ondanks de dalende huizenmarkt er toch nog meer ozb (5,16%)
   bij de huiseigenaren in Almelo geïnd wordt.
- Het is een treurige constatering dat het tarief voor de hondenbelasting omhooggaat met 2,25%,
   hetgeen een verkeerd signaal is vanwege de inbraakpreventie en het toenemende sociaal isolement,
   vooral bij ouderen.
- Het is een treurige constatering dat de parkeerbelasting voor vergunninghouders onrechtmatig is,
   omdat deze slechts in een beperkt aantal wijken geheven wordt, dus rechtsongelijkheid oplevert. Wij
   hebben het dan nog niet over het feit dat het tarief voor een parkeervergunning in het
   bewonersgebied in 2012 buitensporig verhoogd wordt, namelijk van € 19,30 naar € 42.--.
- Het is een treurige constatering dat de toeristenbelasting volstrekt oneigenlijk is omdat (en ik citeer
   nu rechtstreeks uit de begroting): "de opbrengst geheel ten goede komt aan de algemene middelen
   van de gemeente", dus letterlijk in haar geheel niet ten goede komt aan toerisme en recreatie.
          Verder heb ik nog wat op- en aanmerkingen over de moties en amendementen, te beginnen met
de moties.
          Motie EA van de fractie van ALA-AOV houdt in dat er geen grootschalige woningbouw mag
plaatsvinden in Waterrijk. Een dubbele motie met dezelfde strekking (motie S) is ingediend door de
fractie van D66. Die is o.i. overbodig.
          Motie O van de fractie van D66 heeft als titel: Extra bezuinigen op alle beleidsterreinen. Een
goed plan. Vraag extern advies aan Ernst & Young. Wij zijn voor deze motie.
          Motie JA is van de fractie van ALA-AOV en stelt dat het buitenbad van zwembad Het
Sportpark gesloten moet worden vanwege de bezuinigingen. Sport is belangrijk voor de gezondheid en
vitaliteit. 50% van de Almeloërs heeft reeds chronische kwalen, handicaps en overgewicht. Wij zijn dan
ook tegen motie JA.
          Motie I is ingediend door de fractie van de ChristenUnie en gaat over de vestiging van het
Werkplein in het Twenthe Centrum. Volgens deze motie gaat de voorkeur uit naar vestiging in het (veel
te dure) gemeentehuis alvorens te besluiten over het onderzoek naar de financiële consequenties. Ik
ben dan ook voor deze motie.



INT - 1201417                                                                                         25
</p><p>        Motie HA van de fractie van ALA-AOV stelt dat gemeentelijke publicaties beperkt moeten
worden tot de wettelijk noodzakelijke publicaties. De gemeente loopt zichzelf, ondanks wanbeleid, al
veel te veel te promoten. We zijn daarom voor motie HA.
        Motie X van de SP-fractie stelt dat het geven van een vergoeding voor internetaansluitingen
mogelijk moet blijven voor de minima in verband met het tegengaan van sociaal isolement. Daar zijn wij
voor.

        De heer SCHUUR (PvdA): Mevrouw de voorzitter! De heer De Olde zegt nu dat hij voor motie X
is. Daar kan ik mij iets bij voorstellen, maar hij zei ook dat hij € 3 miljoen wil bezuinigen her en der in
alle programma's. Beseft hij wel dat hij dit soort zaken weg bezuinigt op het moment dat hij akkoord
gaat met het voorstel van de fractie van D66 om € 3 miljoen extra te bezuinigen?

        De heer DE OLDE (PVA): Er zijn toch nog wel andere dingen om op te bezuinigen?

        De heer SCHUUR (PvdA): Welke dan?

        De heer DE OLDE (PVA): Bijvoorbeeld stoppen met de plannen voor het nieuwe gemeentehuis,
Waterrijk en Leemslagen.

         De heer SCHUUR (PvdA): Voorzitter! Volgens mij heeft wethouder Andela vanmorgen een heel
goede verhandeling gehouden over de bouw van het nieuwe gemeentehuis. Daarbij heeft hij
aangegeven dat wanneer wij geen nieuw gemeentehuis laten bouwen, dat niet betekent dat wij
daarmee bezuinigingsgeld binnenhalen. Absoluut niet. Verder stelde de heer De Olde voor om te
stoppen met het ontwikkelen van Waterrijk en Leemslagen. Volgens mij moet het nu iedereen toch wel
duidelijk zijn dat als wij dat zonder meer en rücksichtslos zouden doen, wij een nog veel groter
financieel gat zouden hebben. Dat zijn dus geen opties, mijnheer De Olde.

        De heer NIJHUIS (VVD): Voorzitter! Ik sluit mij aan bij deze opmerking van de heer Schuur. Als
wij de ontwikkeling van Waterrijk niet laten doorgaan, dan moeten wij afboeken. Dat betekent dat wij
in één keer ergens € 50 miljoen vandaan moeten toveren. Dat lijkt mij een heel lastige. Wat dat betreft,
is de begroting heel evenwichtig opgebouwd. Ik verwacht dat wij de volgende voorjaarsnota zullen
gebruiken om te bekijken of een aantal premissen wel uitkomt. Dan gaan wij een aantal besluiten
nemen waarover wij zeggen: Die moeten echt genomen worden, wil de gemeente overeind blijven. Zo
is de zaak nu opgezet. Ik begrijp uit het betoog van de heer De Olde absoluut niet waar hij denkt
bezuinigingen vandaan te halen, temeer daar bij het schrappen van het plan om een nieuw stadhuis te
bouwen, ook nog een risico van € 12 miljoen voor het Fortezzagebied boven de markt blijft hangen.
Dan zijn wij wel heel veel geld kwijt volgens mij.

        De heer DE OLDE (PVA): Dus u denkt dat er gebouwd gaat worden in Waterrijk?

         De heer GERRITSEN (D66): Voorzitter! Ik meen dat wij de woorden van de heer De Olde een
beetje moeten ondersteunen. Wanneer je wilt bezuinigen, zul je altijd keuzes moeten maken. Er wordt
nu geroepen, vooral door de coalitiefracties: Wat wilt u dan, en wat gaat u dan doen? Ik kom daar in de
tweede instantie op terug, maar merk nu alvast op dat heel veel moties en amendementen zijn
ingediend door de coalitiefracties waar ook maar wat in wordt geroepen en waarin geen dekking wordt
aangegeven. Wat wij aangeven richting het college en de ambtelijke organisatie, is dat er keuzes
gemaakt moeten worden. Wij hebben daarvoor mensen in dienst (ook de heer Isendoorn noemde dit
zojuist) en wij hebben een goed bestuur (daar gaan wij van uit). Die moeten meedenken.

        De VOORZITTER: Bent u nu al bezig met uw bijdrage in de tweede termijn?

        De heer GERRITSEN (D66): Nee, maar ik probeer het wel!

        De VOORZITTER: Ik meen dat u al wel bezig was met uw bijdrage in de tweede termijn.

       De heer GERRITSEN (D66): Voorzitter! Ik wil de tijd en ruimte hebben om te reageren, want die
gelegenheid krijgen anderen ook. Laat ons daar wel even heel reëel in zijn.




INT - 1201417                                                                                            26
</p><p>       De VOORZITTER: Dat kan, maar u kunt alleen een vraag stellen aan degene die het woord heeft
en achter het spreekgestoelte staat, in dit geval de heer De Olde, want daar zijn interrupties voor
bedoeld.

        De heer GERRITSEN (D66): Ik ondersteun wat de heer De Olde zojuist zei. Laat mij zeggen dat
ik het met hem eens ben dat hij lef durft te tonen. En het bestuur van deze stad moet maar eens laten
zien dat het ook keuzes durft te maken.

       Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Het nadeel van het houden van een lang
betoog is vaak dat je ergens iets net niet goed zegt qua feitelijkheid. De heer Gerritsen zei dat wij geen
dekking zouden hebben aangegeven bij onze voorstellen. Bij elk amendement dat wij ingediend
hebben…

        De heer GERRITSEN (D66): Voorzitter…

        Mevrouw TEN SELDAM-EVEN (CDA): …bij de behandeling van de voorjaarsnota, hebben wij
keurig dekking…

        De heer GERRITSEN (D66): Voorzitter…

        De VOORZITTER: Mijnheer Gerritsen! U hebt niet het woord van mij gekregen.

        De heer GERRITSEN (D66): Sorry.

       Mevrouw TEN SELDAM-EVEN (CDA): Mijnheer Gerritsen! U heeft toch ook enige
fatsoensregels?

       De VOORZITTER: Dames en heren raadsleden! U heeft mij - en niet uzelf -ingehuurd om deze
vergadering te leiden. Laat u mij dan ook de ruimte om u te helpen.
       Mevrouw Ten Seldam heeft het woord.

       Mevrouw TEN SELDAM-EVEN (CDA): Ik gaf aan dat de heer Gerritsen de feiten niet correct
weergeeft. De CDA-fractie heeft namelijk bij elk amendement keurig een dekking aangegeven. Ik vind
dan ook dat hij dat correct moet benoemen.
       Ten tweede wil ik opmerken, mijnheer Gerritsen, dat u zojuist de gelegenheid kreeg om te
reageren, maar toen bleef u zitten. Ik had verwacht dat u atletisch in de benen was gesprongen.

        De VOORZITTER: De heer De Olde kan zijn betoog vervolgen.

        De heer DE OLDE (PVA): Voorzitter! Ik ga nu in op enkele amendementen.
        Naar aanleiding van amendement AA (Overschrijding budgetten openeinderegelingen) merk ik
op dat de veroorzakers - de wethouder e.d. - van de overschrijdingen zich moeten verantwoorden en
consequenties moeten trekken. De overschrijdingen mogen niet ten koste gaan van andere nobele
doelen. Verder heb ik nog geen mening over amendement AA.

       De heer NIJHUIS (VVD): Voorzitter! Even voor alle duidelijkheid: als de heer De Olde met de
veroorzakers de wethouders bedoelt, dan is dat niet onze tekst.

         De heer DE OLDE (PVA): Voorzitter! Amendement DA van de VVD-fractie heeft als titel: Go or
no go Waterrijk. Door woningbouwplanreducties wordt de terugverdiencapaciteit beperkt. Financiële
risico's dienen uiteraard, indien mogelijk, te worden beperkt. Dat lijkt mij een logisch verhaal, dus ik
ben voor dit amendement.
         Dan amendement J van de fractie van de ChristenUnie: Kiezen voor sociale stijging. Dit klinkt
leuk, maar het geld is op en Almelo stevent af op een artikel 12-status. Ik heb nog geen mening over
dit amendement.

       De heer VEENSTRA (ChristenUnie): Ik dacht dat u steeds in al uw betogen zei dat wij de armen
en kwetsbaren moeten ontzien. Dat is juist sociale stijging. Hoe kunt u dan zeggen dat u niet voor de
focus op sociale stijging bent?



INT - 1201417                                                                                           27
</p><p>        De heer DE OLDE (PVA): Het gaat er in eerste instantie om hoe dat betaald moet worden.

       De heer VEENSTRA (ChristenUnie): Nee. U wijst een heleboel bezuinigingsvoorstellen af, omdat
u van mening bent dat er ruimte moet komen voor de kwetsbare groepen. U zegt dus: Sociale stijging
gaat voor grote projecten. Dat zegt u toch steeds? Daarom begrijp ik niet waarom u nu zegt…

        De heer DE OLDE (PVA): Nee, ik heb er nog geen mening over. Dat staat ook in mijn tekst (ik
heb die nog eens gelezen) en ik heb dat zojuist gezegd.

       De heer VEENSTRA (ChristenUnie): Dan hoop ik dat mijn interruptie uw mening heeft
veranderd.

        De heer DE OLDE (PVA): Oké, mijnheer Veenstra.
        Ik kom nu bij het volgende amendement van de heer Veenstra van de ChristenUnie-fractie,
namelijk amendement L (Formuleren van een besparingstaakstelling per maatschappelijke opgave). Ik
heb daar een halfuur naar zitten kijken, maar ik snap er echt niets van!
        Amendement W van de SP-fractie gaat over het openhouden van Stadsboerderij Beeklust en het
dekken van de kosten daarvan uit de vergoeding voor raadsleden. Er komt reeds in 2014 een
wetsvoorstel over bezuinigingen op de gelden voor raadsleden. Dit is dus een voorbarig amendement;
het raadsgeld is reeds het minimumloon.

       De VOORZITTER: Ik meen dat u dat fout hebt (ik grijp nu even in als voorzitter). Er is geen
sprake van een relatie tussen de raadswedde en enige salariëring. Het is een raadswedde, geen
minimumloon of zo. De wedde kan er gemakkelijk ook onder zitten.

        De heer DE OLDE (PVA): Oké. Dank u wel, voorzitter.

        De VOORZITTER: Graag gedaan.

        De heer DE OLDE (PVA): Bij dezen sluit ik mijn betoog. Ik ben erdoorheen.

        De heer SCHUUR (PvdA): Mevrouw de voorzitter! Ik dank het college voor de beantwoording in
de eerste termijn. Er zijn echter nog wel wat open eindjes blijven hangen. Vandaar dat ik er aan het
eind van de beantwoording op aangedrongen heb het college ook nog in de tweede termijn terug te
kunnen zien (ik kom hier in mijn verhaal nog op terug).
        Ik wil nu eerst een kleine aanvulling geven op onze bijdrage in de eerste termijn. Die betreft de
bezuiniging op internationaal beleid. Wij begrijpen dat ook dit beleidsveld niet aan bezuiniging ontkomt,
maar wij vinden het wel wrang dat het in zijn geheel wegbezuinigd wordt. De reden daarvoor is dat wij
denken dat Almelo in het verleden best veel te danken heeft gehad aan bijvoorbeeld de relaties met de
partnersteden etc. etc. Ik ben benieuwd hoe het college hier tegenaan kijkt.
        Vervolgens wil ik wethouder Van Woudenbergh danken voor zijn betoog over het
werkprogramma cultuur. Wat hij zei, was precies wat wij bedoelen. Of het nu visie of werkprogramma
heet, de zaken die daarin genoemd moeten worden, komen in ieder geval aan bod, en daar zijn wij blij
mee.
        In de eerste termijn heb ik uitgebreid stilgestaan bij de sociale opgaven waar Almelo voor staat.
Ik heb toen heel expliciet gevraagd aan het college: Kom bij de voorjaarsnota 2012 met een integrale
visie op de sociale opgaven. Ik heb het dan over bijvoorbeeld de openeinderegelingen, maar ook over de
Wet werk en bijstand, de Wet werken naar vermogen, de AWBZ en alle andere nieuwe uitdagingen die
op onze gemeente af komen. Ik krijg graag helder het antwoord van de wethouder dat zij inderdaad met
een integrale visie op de sociale opgaven komt bij de voorjaarsnota.
        Hetzelfde heb ik gevraagd met betrekking tot het grondbeleid en het grondbedrijf. Kom nu eens
met een visie op het grondbedrijf. Ik heb dit gevraagd naar aanleiding van de voorjaarsnota 2011 en de
evaluatie bij de voorjaarsnota 2012. Wees niet bang om afscheid te nemen van ambities en om
inzichtelijk te maken wat de financiële consequenties zijn. Ik krijg graag de uitspraak van de heer Van
Marle dat het college inderdaad komt met een visie op het grondbedrijf. In mijn ogen maakt dat het
amendement (CA) van de VVD-fractie overbodig, want volgens mij heeft dat dezelfde strekking als wat
ik nu vraag. Als de wethouder toezegt dat hij met een visie op het grondbedrijf komt, zullen wij dat
amendement niet steunen, omdat wij denken dat het dan overbodig is.



INT - 1201417                                                                                         28
</p><p>       De heer ISENDOORN (D66): Voorzitter! Het betoog van de heer Schuur is opgehangen aan, zeg
maar, het sociale domein. Mijn vraag is of wat hij daarover heeft gezegd, ook voor andere domeinen
geldt.

        De heer SCHUUR (PvdA): Uiteraard, mijnheer Isendoorn. Wij zijn altijd geïnteresseerd in
integrale visies op welk domein dan ook. Wij verwachten echter - en daarom heb ik zojuist twee
beleidsvelden specifiek genoemd - dat het zware weer vooral die kant op gaat. Daarom menen wij dat
het goed is dat het college bij de voorjaarsnota 2012 met integrale visies daarop komt. Als er echter
ook voor andere domeinen problemen ontstaan, dan zijn wij natuurlijk ook zeer geïnteresseerd in visies
daarop.

         De heer ISENDOORN (D66): Ik ben dit met u eens, want ik heb daar zelf ook zorgen over. De
grote druk ligt met name op het sociale domein. Wij weten dat die druk niet is gekwantificeerd in de
begroting, en dat betreur ik. Voor een belangrijk deel kan dat natuurlijk wel, al zou die maar uitgedrukt
worden in een percentage, bijvoorbeeld 50. Dat missen wij nu, maar als dat wel aangegeven wordt,
dan krijgen wij een beetje een indruk in welke orde van grootte wij moeten denken als het gaat om het
sociale domein. Dat wij dat nu missen, is volgens mij ernstig. Iedereen praat vol liefde over sociale
stijging, maar ik vind dat een beetje merkwaardig, vervelend en raar woord. Als wij geen helderheid
hebben over wat ik aangaf, leidt dat tot (wat ik zou willen noemen) sociale ontwrichting. Bent u dat
met mij eens, of niet?

        De heer SCHUUR (PvdA): Dat is exact de reden waarom wij daar om vragen. Het is ook exact
de reden waarom ik zojuist in discussie ging over de overschrijdingen van de budgetten voor de
openeinderegelingen, die binnen het sociale domein opgelost zouden moeten worden. Als je goed gaat
bekijken wat je daarmee veroorzaakt, dan zul je per definitie tot de slotsom komen dat je dat niet wilt.
Daarover zijn wij het dus eens.

        De heer ISENDOORN (D66): Deelt u dan ook mijn mening dat het beter zou zijn geweest dat het
college inzichtelijk gemaakt had wat het risico is, of in ieder geval een poging had gedaan om dat te
kwantificeren?

        De heer SCHUUR (PvdA): Nee. Ik meen namelijk dat dat op dit moment nog erg lastig is.
Volgens mij is de voorjaarsnota 2012 het moment voor het college om dat te kwantificeren. Volgens
mij gaat het college straks ook antwoorden dat het dat gaat doen! (Ik hoor nu iemand zeggen, zonder
microfoon, dat dat een gevaarlijke opmerking is. Dat is niet zozeer een gevaarlijke opmerking, maar ik
weet wie het vraagt!)
        Voorzitter! Ik kom nu bij de stadhuisdiscussie. Hulde aan wethouder Andela, omdat hij nog een
keer duidelijk en gestructureerd op een rijtje heeft gezet hoe het financiële plaatje eruitziet. Wat ik
overigens apart vind, is wat ik hoorde toen gezegd werd dat de Belastingdienst eventueel uit Almelo
verhuist (ik had dat al gelezen op Twitter, via een collega-raadslid). Ik hoorde toen namelijk direct al
mensen roepen: Oh, dan hoeft het nieuwe stadhuis niet gebouwd te worden, want dat kan dan
ondergebracht worden in het belastingkantoor. Ik vind dat politiek ongelooflijk onhandig, en nu druk ik
het nog zacht uit. Daarmee zeg je namelijk eigenlijk: Oké, laat de Belastingdienst maar uit Almelo
verdwijnen, want dan hebben wij mooi een nieuwe locatie voor het stadhuis. Volgens mij is het van
niemand hier de bedoeling dat de Belastingdienst uit Almelo verdwijnt. Gelukkig is ook minister Donner
inmiddels wakker geworden en heeft hij gezegd: Ik heb helemaal niet gesteld dat de Belastingdienst uit
Almelo gaat verdwijnen. Daar blijkt uit dat dat helemaal nog geen gelopen race is. Ik vind dat wij met
elkaar moeten oppassen dat wij het kind niet met het badwater weggooien door te zeggen: Hé, dan
hebben wij een oplossing voor het stadhuis. Vervolgens verdwijnt dan de Belastingdienst uit Almelo.
Dat zou erg jammer zijn.
        In de eerste instantie hebben wij met betrekking tot het binnenstadsplan gevraagd: Wanneer
gaat de schop nu eens de grond in? Wethouder Van Woudenbergh antwoordde daarop dat de schop
nog dit najaar de grond in zal gaan voor de eerste fase van het binnenstadsplan. Wat wij voorts
gevraagd hebben, is om ook te kijken naar laaghangend fruit. Wat wij daarmee bedoelden, is dat er nog
een aantal lege plekken in de binnenstad is. Volgens ons kan het niet zo heel veel moeite kosten om
alvast te beginnen om daar een invulling aan te geven. Ik ben benieuwd of er inmiddels plannen zijn om
die plekken op korte termijn in te vullen.




INT - 1201417                                                                                          29
</p><p>        Wij zijn content met de uitleg van de wethouder over het evenementenfonds. Wij begrijpen
daaruit dat dat in de steigers staat en op 1 januari a.s. van start gaat.
        Verder hebben wij nog een vraag met betrekking tot de schuldpositie van de gemeente Almelo.
Een aantal oppositiefracties heeft daar nogal zwaar op ingezet. Daarbij heb ik ook horen zeggen dat de
schuld van de gemeente Almelo dit jaar met meer dan € 30 miljoen toegenomen zou zijn. Onze fractie
heeft daar wat vraagtekens bij. Ik weet dat de gemeente nog een aankoop in Waterrijk moest
financieren van om en nabij € 9 miljoen. Dat vloeide nog voort uit collegebesluiten uit de periode 2002-
2006 inzake het vestigen van het voorkeursrecht door de gemeente. Daar zitten immers ook
verplichtingen aan vast. Volgens onze fractie is de andere gekwantificeerde toename van de schuld het
gevolg van het omzetten van kortlopende leningen in langlopende leningen. Dat is de facto dus geen
vergroting van de schuld geweest. Ik hoor graag dat wethouder Van Marle nog even op dit punt ingaat.

        Mevrouw TEN SELDAM-EVEN (CDA): Heb ik het goed dat u, net als ik, de wethouder heeft
horen zeggen dat de schuldenlast van de gemeente met name opgelopen is in de periode 2002-2006?
En is mijn geheugen goed als ik constateer dat het college toen bestond uit wethouders van de PvdA,
het CDA en D66?

        De heer SCHUUR (PvdA): Volgens mij is er met uw historisch besef niets aan de hand!

        Mevrouw TEN SELDAM-EVEN (CDA): Dank u wel.

         De heer ISENDOORN (D66): Het is toch eigenlijk een mooi en merkwaardig verschijnsel van het
menselijk lichaam en de menselijke geest dat het geheugen zo selectief kan zijn. Dat is fantastisch! D66
was inderdaad vertegenwoordigd in het college gedurende de periode 2002-2006. Dat was een andere
tijd. In 2007 begon de financiële crisis (u weet dat, mijnheer Schuur). Die werd gevolgd door een
economische crisis in het najaar van 2007 en in 2008. Deze crisis zette door in 2009. Daarna volgden
de schuldencrisissen in Europa. Nu krijgen wij het hele debat over de eurozone; daar zitten wij nog
steeds in. Daarbij is Griekenland aan de orde geweest, en nu zijn wij bij Italië. Wat al die crisissen
gemeenschappelijk hebben, is dat er elke keer sprake is van een enorm oplopende schuldenlast, waarbij
de inkomsten dalen en de schulden door het plafond stijgen. Dat is precies de trend die wij ook hier in
de gemeentelijke begroting zien. De inkomsten dalen, met uitzondering van 2010, want toen waren er
nog de Essentgelden die voor extra inkomsten zorgden en nog wat extra bijdragen van het rijk om
opgelopen tekorten uit de periode daarvoor wat te herstellen. Je ziet nu een piek…

        De VOORZITTER: Dit wordt wel een heel lange interruptie, mijnheer Isendoorn.

        De heer ISENDOORN (D66): Ja, maar ik vind dit wel belangrijk, want hier wordt zwaar de
suggestie gewekt dat D66 de boel zit te vernachelen. Dat is absoluut niet waar. Voor onze uitspraken
over de schuldenlast van deze gemeente, gebruiken wij de cijfers die staan in de begroting voor 2012
en die door het huidige college zijn gepresenteerd. Daarbij wordt gesproken over de vaste schuldenlast
zoals wij die genoemd hebben in ons pamflet Morgen is Nu! De inkomsten staan daar precies
tegenover. Ons betoog is (mijnheer Schuur) dat de schuldenlast drastisch moet verminderen. Dat stemt
precies overeen met het rijtje dat ik zojuist heb genoemd. De schuldenlast zorgt voor een failliet.
Uiteindelijk is dat het gevolg.

        Mevrouw TEN SELDAM-EVEN (CDA): Ik heb de wethouder ook expliciet horen zeggen dat je de
schuldenlast mede in relatie moet zien met de inkomsten en de vraag of je de rentelast kunt dragen.
Dat is dus een wat bredere context dan geschetst is door de heer Isendoorn. Ik ben het eens met hem
dat ik hem op zijn minst als medeverantwoordelijk beschouw voor het gegeven. Hij wekte namelijk bij
mij de indruk dat hij er part noch deel aan heeft gehad, en dat vind ik geen terechte weergave.

        De VOORZITTER: Dames en heren! Ik wil dat interrupties alleen gericht zijn tot de spreker
achter het spreekgestoelte, niet tot elkaar. Dat moet dan maar op een andere manier.

        De heer ISENDOORN (D66): Mag ik de heer Schuur dan een vraag stellen?

        De VOORZITTER: Dat mag.




INT - 1201417                                                                                        30
</p><p>      De heer ISENDOORN (D66): Mijnheer Schuur! Bent u dat met mevrouw Ten Seldam van
mening?!

         De heer SCHUUR (PvdA): Ik meen dat het duidelijk is dat de Wet voorkeursrecht gemeenten
(die wet kent u, mijnheer Isendoorn) niet alleen rechten maar ook plichten geeft. De laatste aankoop die
in dat kader nog dit jaar gedaan is, vloeide voort uit een verplichting die de wet oplegt. Indertijd is door
het college waarin ook D66 participeerde, het besluit genomen om het voorkeursrecht te vestigen. In
die zin kunt u uw complete verantwoordelijkheid daarvoor niet ontkennen. Maar, mijnheer Isendoorn, ik
vind ook dat u het recht heeft om nu een andere mening te hebben gezien de veranderde situatie (dat is
duidelijk, denk ik). Daar kan ik mij in vinden, maar de verantwoordelijkheid blijft.

        De heer ISENDOORN (D66): Daar ben ik u zeer dankbaar voor, mijnheer Schuur, maar kent u
ook ons staatsrecht, waarin wij het principe kennen van continuïteit van besturen. Dat betekent
gewoon dat colleges eigenlijk ook de lasten van de vorige periodes dragen. Daar dragen zij ook
verantwoordelijkheid voor. Dat geeft het staatsrechtelijk bestel aan waarin wij zitten. Het gaat dan ook
niet aan, mevrouw Ten Seldam, dat u mij verwijt dat ik destijds deel heb genomen aan het college. U
zat ook in dat kabinet, zal ik maar zeggen!

        De VOORZITTER: Vervolgt u uw betoog, mijnheer Schuur.

       Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Mag ik toch de gelegenheid krijgen om aan te
geven dat ik mijn vorige opmerking niet aan het adres van de heer Isendoorn maar wel aan het adres
van D66 had moeten maken?

        De heer SCHUUR (PvdA): Voorzitter! Ik wil nog één ding zeggen over het debat dat wij zojuist
voerden. Dat is dat ik nooit een andere fractie in deze zaal zal betichten van het vernachelen van zaken.
Volgens mij heb ik dat woord ook niet in de mond genomen. Hierbij wens ik daar ook afstand van te
nemen.
        Ik ga niet in op alle moties en amendementen, want dat zijn er gewoon echt te veel. Wel ga ik
in op de moties en amendementen die er voor ons uitspringen. Daarbij begin ik met onze motie (G) over
zwembad Het Sportpark. Wij hechten er heel veel waarde aan dat Almelo gewoon de beschikking
houdt over een zwembad met zowel een binnen- als een buitenbad. Hiermee heb ik ook iets gezegd
over de motie (JA) van de fractie van ALA-AOV over het sluiten van het buitenbad. Dit onderwerp
kunnen wij niet los zien van het Sportbedrijf. Wat dat betreft, is er in onze fractie wat ongerustheid,
want wij krijgen daarover geen helderheid. Niettemin heeft het Sportbedrijf een directe relatie met de
mogelijke handhaving van het zwembad. Daarom wil ik de wethouder daar graag nog over horen.

        De heer STORK (ALA-AOV): Mijnheer Schuur! Wij willen het binnenbad handhaven. Om dat te
kunnen handhaven, moeten wij oppassen met de manier waarop de centen besteed worden. Het
buitenbad wordt slechts selectief gebruikt. In een goede zomer wordt het behoorlijk gebruikt, maar in
een slechte zomer heel weinig. Daarbij komt dat het buitenbad niet meer van deze tijd is en enorme
investeringen vraagt. Daarom stellen wij voor om het buitenbad op dit moment te sluiten, om het
binnenbad te kunnen behouden. Wanneer het buitenbad gesloten wordt, kan dat over een paar jaar
rustig weer geopend worden. Als er dan geld is, kan dat geïnvesteerd worden in het buitenbad om het
aan de eisen van de tijd te laten voldoen. Dat is onze insteek.

        Mevrouw TEN SELDAM-EVEN (CDA): Wij hebben bij de behandeling van de berap gevraagd om
een onderzoek naar de situatie van het zwembad. Mag ik uit hetgeen de heer Schuur hierover gezegd
heeft, opmaken dat het resultaat van het onderzoek voor hem niet meetelt bij de besluitvorming over
hoe verdergegaan moet worden met het zwembad (al dan niet met een buitenbad)?

        De heer SCHUUR (PvdA): Mevrouw Ten Seldam! Natuurlijk telt het resultaat van het onderzoek
mee, maar wij vinden per definitie dat een stad als Almelo, met 72.000 inwoners, een voorziening als
een zwembad moet hebben met een binnen- en een buitenbad.
        Tegen u, mijnheer Stork, zeg ik dat ik uw voorstel met betrekking tot het zwembad prima heb
begrepen. Ik ben blij dat u het nog een keer uitlegde, maar ik zei slechts dat ik uw visie niet deel.
        Voorzitter! Het door ons ingediende amendement F gaat over "Het verschil maken". Wij zijn met
ons allen van mening dat wij minder externen binnen de organisatie moeten hebben en dat wij
voorzichtig moeten zijn met inhuren. Gelet daarop, vinden wij het echt niet meer passen dat er toch



INT - 1201417                                                                                            31
</p><p>nog € 125.000,-- gereserveerd wordt om het proces van "Het verschil maken" te begeleiden. Wij
stellen dat dat gewoon intern opgelost moet kunnen worden. Bovendien - ik zeg dit omdat wij als
coalitiefracties het verwijt kregen dat wij geen dekking aangeven - willen wij met het geld dat daarmee
gemoeid is, gedurende vijf jaar het schoolzwemmen garanderen.        Overigens hebben wij een vraag
over het schoolzwemmen. In de begroting hebben wij ongelooflijk gezocht in verband met de
bezuiniging op het schoolzwemmen, maar wij konden daarin de post schoolzwemmen nergens vinden.
Vandaar dat het voorstel dat wij hierover indienen, samen met de fracties van GroenLinks en de SP,
geen amendement geworden is maar een motie. Verder wil ik graag wat duidelijkheid over deze zaak
krijgen van de wethouder. Waar wordt het geld voor het schoolzwemmen precies genoemd? Waar valt
dit onderwerp onder? Of is deze post misschien vergeten en is het schoolzwemmen überhaupt niet
wegbezuinigd? Dan is de motie niet meer nodig en zijn wij het gauw met elkaar eens.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Zouden de fracties van de PvdA, GroenLinks en
de SP hun voorstel over het schoolzwemmen niet moeten indienen in de vorm van een amendement,
juist omdat zij daarmee de begroting willen beïnvloeden? Het lastige van een motie is volgens mij dat
eerst de begroting wordt vastgesteld en dat vervolgens eventueel moties worden aangenomen. De heer
Schuur zou in overleg kunnen zoeken naar een mogelijkheid om het voorstel inzake het
schoolzwemmen in te dienen in de vorm van een amendement.

        De heer SCHUUR (PvdA): Voorzitter! Dat ben ik met de heer Veenstra eens. De eerste insteek
was ook om een amendement in te dienen met betrekking tot het schoolzwemmen, maar dan moet dat
onderwerp wel ergens genoemd worden in het te nemen besluit. Ik kan in een amendement immers niet
refereren aan een bezuiniging die ongedaan gemaakt moet worden als die bezuiniging niet genoemd
wordt.

        De heer VEENSTRA (ChristenUnie): Maar je kunt wel zeggen: De in de voorjaarsnota ingeboekte
en ergens in de begroting ondergebrachte bezuiniging willen wij schrappen. Als die bezuiniging er niet
in staat, dan hoeft die ook niet benoemd te worden.

      De heer SCHUUR (PvdA): Voorzitter! Dit wordt een technische discussie. Dat vind ik een beetje
jammer. Ik wil het liever over de inhoud hebben.

       Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Ik wil de heer Schuur een tip geven: gewoon
toevoegen. Dan is hij er al.

         De heer SCHUUR (PvdA): Dank u wel.
         Voorzitter! Voor onze motie H (Inkomensondersteuning) heb ik inmiddels behoorlijk wat
steunbetuigingen mogen ontvangen, waarvoor dank.
         Motie I en amendement J van de fractie van de ChristenUnie met betrekking tot respectievelijk
het Werkplein en kiezen voor sociale stijging, zullen wij steunen.
         Dat geldt ook voor motie Y van de SP-fractie over het inhuren van externen. Volgens deze
motie mag maximaal 10% van het totale budget voor salarissen en vergoedingen worden uitgegeven
aan ingehuurde medewerkers. Daarmee wordt niet expliciet gezegd dat dit nooit mag. Wel dwingt deze
norm het college om verantwoording af te leggen aan de raad over de externe inhuur. Volgens mij is
dat ook de intentie van de heer Cornelissen. Wij steunen motie Y dan ook.
         Wij steunen eveneens de motie (BA) van de VVD-fractie met betrekking tot het onderhoud van
kunstwerken. Deze motie steunen wij, omdat ook wij het belang hiervan inzien. Sterker nog: in een
aantal begrotingsbehandelingen in voorgaande jaren is dit voorstel ook al eens door onze fractie op tafel
gelegd. Goed dat motie BA er ligt.
         Over het realiteitsgehalte van Marsroute 2, waarover de VVD-fractie amendement CA heeft
ingediend, heb ik het al gehad. Wij wachten op dit punt het antwoord van het college af.
         Wij zijn van mening dat de wethouder adequaat geantwoord heeft op de moties die zijn
ingediend met betrekking tot Waterrijk. Wij zijn van mening dat de besluitvorming hierover inderdaad
aan de orde is bij de vaststelling van het bestemmingsplan. Daarbij komen dan ook de faseringen aan
de orde. Met dit antwoord van de wethouder kunnen wij prima leven.
         Ik heb het zojuist al gehad over de overschrijdingen van budgetten voor de
openeinderegelingen, waarover de VVD-fractie amendement AA heeft ingediend. Dit amendement gaat
onze fractie echt aan het hart, want volgens het amendement wordt de rekening van de economische
crisis heel eenzijdig neergelegd bij de groep die het al heel erg zwaar heeft. Dan kan men wel zeggen



INT - 1201417                                                                                         32
</p><p>dat in het amendement staat dat de consequenties "in principe" binnen het sociale domein opgevangen
moeten worden, maar zelfs al het uitspreken van die intentie doet mij zeer. Dat meen ik echt. De
overschrijdingen van de openeinderegelingen zijn het gevolg van de economische crisis, want daardoor
komen er meer mensen in de bijstand en worden de budgetten overschreden. Ik vind niet dat wij
kunnen zeggen: Dan laten wij de groepen die dat treft, daarvoor boeten door de voorzieningen die wij
voor hen hadden, weg te bezuinigen. Volgens onze fractie kan dat niet aan de orde zijn. Ik vraag de
fractie van de VVD dan ook om amendement AA in te trekken.

        De heer NIJHUIS (VVD): Voorzitter! Dat zullen wij niet doen. Ik heb zojuist al aangegeven - en
volgens mij is dat ook de intentie van onze collega's van de CDA-fractie, als ik het goed begrepen heb -
 dat wanneer door de wethouder gemeld wordt aan de raad dat een substantiële overschrijding dreigt,
wij daarover met elkaar kunnen spreken. Dan kunnen wij bekijken wat er mogelijk is om de gevolgen op
te lossen binnen de eigen koker (ik gebruik nu de term van de wethouder). Indien de gevolgen daar niet
opgelost kunnen worden, kan bekeken worden welke andere mogelijkheden er zijn, met als uiterste
mogelijkheid putten uit de algemene reserve. Dat is de intentie van amendement AA. Het is ons iets te
gemakkelijk om te zeggen: Omdat het om rijksbeleid gaat, is geen enkele overschrijding beïnvloedbaar
en kun je er niets aan doen. Ik meen dat daar nuances in zitten. Die willen wij graag met elkaar
bespreken. Natuurlijk is er ook voor de VVD-fractie een ondergrens. Mensen hebben recht op een
fatsoenlijk bestaan, ook in onze stad.

        De heer SCHUUR (PvdA): Voorzitter! Volgens mij verschillen wij niet van mening over de
intentie dat wij zorgvuldig moeten bekijken hoe wij zelf de overschrijdingen kunnen beïnvloeden. Dat is
heel duidelijk; je kunt niet alleen maar zeggen dat het om een rijksbijdrage gaat waar wij geen invloed
op hebben. Daar zijn wij het wel over eens, maar waar leg je vervolgens de rekening neer? Daar gaat
het om. Wat ik de heer Nijhuis nu hoor zeggen - ook daar ben ik het mee eens - is dat hij graag wil dat
de raad invloed kan uitoefenen als het gaat om de vraag waar de rekening van de overschrijdingen
neergelegd moet worden. Daar willen wij graag over meepraten, maar daarvoor kan de heer Nijhuis wat
mij betreft een heel ander amendement indienen dan het amendement (AA) dat nu voorligt.

        De heer NIJHUIS (VVD): Voorzitter! Ik heb de volgorde aangegeven waarin wij het proces
zouden willen lopen. Die volgorde is:
1. Nagaan waar de overschrijdingen zitten en of daar nog iets aan te doen is.
2. Als dat niet lukt, bekijken waar wij dan de overschrijdingen kunnen compenseren. Kan dat binnen de
   koker? Ik gebruik nu de term van de wethouder, hoewel ik nog niet helemaal goed in beeld heb wat
   dat is, maar dat hoop ik straks van haar te horen.
3. Als compensatie niet mogelijk is, bespreken hoe de overschrijding dan opgelost moet worden.
        Het gaat mij om deze volgorde.

        Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! De wethouder heeft meerdere keren
aangegeven dat zij zorgvuldig zal handelen en zal bekijken hoe zoveel mogelijk bereikt kan worden met
het toegekende geld. Met andere woorden: zij zal bekijken of overschrijdingen voorkomen kunnen
worden. De CDA-fractie heeft alle vertrouwen dat de wethouder (maar ook het hele college en de
ambtelijke organisatie) daar goed mee omgaat. Dat de gelden op enig moment, zeker in deze
economisch slechte tijden, niet toereikend zullen zijn, kunnen wij ons voorstellen. Sterker nog: die
mogelijkheid baart ook ons zorgen. Ik meen echter dat het goed is om daar dan over van gedachten te
wisselen met elkaar en te bekijken hoe de problematiek opgelost kan worden. Daarbij gaat het dan om
de oplossing en niet om het al op voorhand snijden in het voorzieningenniveau. Ik zie dit punt niet als
een bezuiniging, maar veel meer als het maken van afspraken met elkaar over hoe wij omgaan met de
openeinderegelingen om een goed voorzieningenniveau overeind te kunnen houden.

        De heer SCHUUR (PvdA): Voorzitter! Het vertrouwen in de wethouder delen wij,
vanzelfsprekend. Het gaat mij vooral om de intentie die spreekt uit het amendement (AA) over de
openeinderegelingen. Die intentie is dat de rekening daar neergelegd moet worden waar die
overschreden wordt, dus bij de groepen die het al ongelooflijk moeilijk hebben. Dat stuit mij tegen de
borst.

        De VOORZITTER: Het lijkt mij dat wij dit punt nu rond hebben.

        De heer SCHUUR (PvdA): Ja, dat lijkt mij ook.



INT - 1201417                                                                                            33
</p><p>        Voorzitter! Ik kom nu bij de moties en het amendement van de fractie van D66 (ik heb daar
zojuist al even aan gerefereerd). Ik vind het altijd goed dat een fractie met een tegenbegroting komt.
Volgens mij moet iedere democraat in hart en nieren dat toejuichen, maar ik verwacht dan wel iets
meer dan wat nu gepresenteerd is door de D66-fractie. Ik las een tweet van ik dacht de Stichting NAT
(volgens mij heeft de heer Gerritsen die zelf doorgezonden). Daar stond in: "Heb net begroting D66
doorgerekend". Toen dacht ik: Ja, daar heb je maar twee minuten voor nodig, want meer is het niet en
meer staat er niet. Ik vind het ongelooflijk jammer dat daar waar het college het verwijt krijgt geen
keuzes te maken, de fractie van D66 zegt: "Wij gaan € 3 miljoen bezuinigen", maar de consequenties
daarvan vervolgens niet in beeld brengt. Dat vind ik jammer.

       De VOORZITTER: U heeft nog een minuut spreektijd, mijnheer Schuur, maar ik geef nu eerst
het woord aan de heer Gerritsen.

         De heer GERRITSEN (D66): Voorzitter! Wat wij proberen met onze tegenbegroting, is gewoon
een richting aangeven. Wat wij vandaag weer constateren - en dat doet de heer Schuur zelf ook met
zijn eigen fractie - is het probleem. U kunt heel goed, mijnheer Schuur, het probleem weergeven dat nu
in Almelo gaande is. Als fractie van D66 hebben wij gezegd: Wij moeten nu keuzes zien te maken. Ja,
dat klopt, en de keuzes liggen er. Als u dan vraagt waar de consequenties van de keuzes opgevangen
moeten worden, kan ik zeggen dat wij dat geformuleerd hebben. Ik zei zojuist al in een andere
interruptie dat het ook de taak van het college is om ervoor te zorgen dat er op verschillende plekken
mogelijkheden zijn om te bezuinigen. Eén ding is heel duidelijk: na het vaststellen van de voorstellen en
de begroting zoals die er nu liggen, zullen zaken weer ter discussie komen bij de behandeling van de
volgende voorjaarsnota. Dan staan wij weer voor dezelfde situatie. U zult het met mij eens zijn dat wij
van de heer Van Marle hebben gehoord dat de schuld van de gemeente is gegroeid. Ik zal het nu niet
over de details hebben, maar merk wel op dat de cijfers duidelijk aangeven dat de schuld groeit.
Wanneer wij op deze manier doorgaan, blijft die groeien. Wij zullen dus wat moeten. Nu is er nog de
mogelijkheid…

        De VOORZITTER: Oké.

        De heer GERRITSEN (D66): Nee, voorzitter, ik wil dit even afmaken.

        De VOORZITTER: Nee, u moet afronden, want u maakt erg lange interrupties.

        De heer GERRITSEN (D66): Ja, soms heb je dat even nodig, voorzitter.

        De VOORZITTER: U hebt straks ook nog de mogelijkheid om in uw eigen spreektijd in de
tweede termijn te reageren. Dan zou ik dit punt nog even terugpakken, als ik u was. Graag nu de
afronding van uw interruptie.

        De heer GERRITSEN (D66): Dan laat ik het hierbij en kom ik er straks nog even op terug.
        Mijnheer Schuur! Het gaat erom dat ik best veel dingen met u deel, maar u moet ook begrijpen
dat de intentie die wij hebben met de tegenbegroting, het oplossen van structurele problemen is.

        De heer SCHUUR (PvdA): Voorzitter! Ik twijfel niet aan de intenties van de fractie van D66, laat
dat duidelijk zijn. Ik vind echter wel dat wanneer je iets een tegenbegroting noemt, dat meer moet zijn
dan het verhaal dat er nu ligt.
        Ik kom aan het eind van mijn bijdrage in de tweede termijn. Ik ben van mening dat wij een
goede begroting voor ons hebben liggen. Het is geen vrolijke begroting, want dat is wat anders. Ik
word er namelijk zelf ook niet vrolijk van. Ook vragen wij niet voor niets naar integrale visies op het
grondbedrijf en de sociale opgaven. Ik meen echter dat het college goed werk heeft verricht met de
manier waarop het de begroting heeft opgesteld. Conform hetgeen wij bij de behandeling van de
voorjaarsnota 2011 hebben afgesproken, is de behandeling van de voorjaarsnota 2012 het moment om
te bekijken of het beleid daadwerkelijk aanslaat. Volgens mij heb ik meerdere fracties in deze raadzaal
dat horen zeggen.

      De heer HÜMMELS (Leefbaar Almelo): Mevrouw de voorzitter! Ik ga nu niet alle moties en
amendementen doornemen. U merkt wel hoe ik daarover denk als wij gaan stemmen.




INT - 1201417                                                                                          34
</p><p>         Ik heb geprobeerd mijn eigen bijdrage van afgelopen dinsdag nog eens tegen het licht te
houden. Of het er minder donker van is geworden, weet ik niet! De heer Schuur zei zojuist dat er een
goede begroting ligt, maar dat vind ik al haast te ver gaan. Ik ben het wel met hem eens dat het geen
vrolijke begroting is, maar of het een goede is? Waarschijnlijk ben ik niet deskundig genoeg om dat te
beoordelen. Er zitten voor mij in ieder geval nog heel wat haken en ogen aan. Die zullen er ook wel aan
blijven zitten en die krijg ik er waarschijnlijk niet uit.
         Afgelopen dinsdag heb ik acht amendementen en enkele moties aangekondigd. Ik zal de raad nu
iets vrolijker proberen te stemmen met de mededeling dat ik uiteindelijk slechts drie amendementen zal
indienen. Dat scheelt alweer iets. Misschien blijven er uiteindelijk slechts twee over, want de fractie
van GroenLinks laat zo af en toe zien dat ze op onderdelen net iets beter functioneert dan wij. Daarom
is het mogelijk dat ik het amendement met betrekking tot de ozb zal intrekken - want ik dien het wel
in - als blijkt dat het nog in te dienen amendement van de fractie van GroenLinks over dat onderwerp
beter is.
         Misschien doe ik een en ander weer in de verkeerde volgorde, voorzitter, maar u zegt het maar
als ik hier weg moet. Overigens geloof ik dat er nooit een verkeerde volgorde is. Nee, dit klopt ook niet
helemaal.

       De VOORZITTER: Dan geef ik u een tip, mijnheer Hümmels. Als u wilt dat uw moties en/of
amendementen ook meegenomen worden in het debat en dat ze afgewogen worden, dan adviseer ik u
om ze hier echt te benoemen en in te dienen.

        De heer HÜMMELS (Leefbaar Almelo): Dat ga ik ook doen, maar ik vroeg mij even af of ik eerst
mijn verhaal moet vertellen en dan de moties en amendementen zal noemen.

        De VOORZITTER: Het is handig om eerst uw verhaal te vertellen.

      De heer HÜMMELS (Leefbaar Almelo): Ja, voor u misschien, maar het is de vraag of ik mij
daarmee red. Daarom stond ik even te twijfelen.

        De VOORZITTER: U gaat over uw eigen keuzes.

        De heer HÜMMELS (Leefbaar Almelo): Voorzitter! Ons amendement met betrekking tot de ozb,
dat ik bij dezen indien, luidt als volgt:

                "Amendement RA.
                Betreft het voorstel begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Besluit:
                Tot verbetering van de voorgestelde ozb-plannen de tekst als volgt te wijzigen:
                Over te gaan tot het verhogen van de ozb met 5% i.p.v. de voorgestelde 2,25% + 1,5%
                (= 3,75 %).
                Toelichting:
                Daar waar er mogelijkheden zijn de maatschappij naar draagkracht te belasten moeten die
                mogelijkheden optimaal benut worden."

         In dit amendement wordt gesproken over een ozb-verhoging van 3,75%, maar ik heb inmiddels
begrepen dat dat een heel ander percentage is. Vandaar dat ik mogelijk steun geef aan het
amendement dat de heer Buitenweg van de fractie van GroenLinks zal indienen met betrekking tot de
ozb.
         Ik behandel een en ander toch een beetje door elkaar. Er is gezegd door de heer De Olde dat het
bestuur van de stad een dikke onvoldoende zou krijgen. Daar werden ook cijfers aan gekoppeld,
namelijk 5,5 tot 5,7. Weliswaar ben ik nooit een echt rekenwonder of schoolintellect geweest, maar
volgens mij is dat helemaal geen dikke onvoldoende!
         Voorzitter! U noemde de begroting een logische uitwerking van de voorjaarsnota. Ja, het ligt er
maar aan hoe je de discussie bekijkt die over de voorjaarsnota gevoerd is. Als je kijkt naar wat
uiteindelijk bij meerderheid besloten is, dan lijkt dat inderdaad het geval te zijn. Als ik echter kijk naar
de kritische kanttekeningen die op onderdelen geplaatst zijn, vind ik het toch wel jammer dat er te
weinig alternatieven aangeboden zijn. Daar wil ik het nu over hebben (ik dacht dat ik dit ook in de
eerste instantie al gemeld heb). Hoeveel ruimte is er voor het college om ervoor te zorgen dat een



INT - 1201417                                                                                            35
</p><p>begroting gepresenteerd wordt waarin keuzemogelijkheden zijn opgenomen voor de raad, zodat een
betere afweging gemaakt kan worden? Misschien moet het college daarvoor nog harder werken dan
het nu al doet, maar mogelijkerwijs komen wij dan uiteindelijk wel tot een betere besluitvorming, als u
begrijpt wat ik bedoel.
         Een aangekondigd amendement dat ik niet zal indienen, gaat over Stadstoezicht. Eigenlijk wil ik
dat amendement wel heel graag indienen, maar waarschijnlijk bereik ik daar niets mee. Daarom wil ik
zo meteen het signaal over Stadstoezicht hier nog een keer afgeven. Eerst wil ik echter melden dat ik
een aantal van de door mij aangekondigde amendementen niet zal indienen, omdat ik mij afvraag of ze
bij de begrotingsbehandeling thuishoren. Dat zijn namelijk meer voorjaarsnotaverhalen. De afgelopen
drie jaren ben ik er echter haast aan gewend geraakt dat die dingen zo verschrikkelijk door elkaar
gegooid werden, dat je helemaal niet meer wist waar je het over had. Om te voorkomen dat iedereen
denkt dat ik niet weet waar ik het over heb: Dat valt wel mee hoor!
         Als je vindt dat veiligheid een belangrijk aspect is, moet je er naar mijn idee juist voor zorgen -
 ondanks hetgeen door het rijk aangegeven wordt of wat er in andere gemeentes gebeurt - dat je de
expertise die je in huis hebt, niet overboord gooit. Ook moet je niet zomaar allerlei nieuwe initiatieven
starten, maar moet je bekijken of je met datgene wat je hebt, niet beter inhoud kunt geven aan nieuwe
initiatieven. Daarbij zou wat mij betreft een grotere rol voor Stadstoezicht weggelegd moeten zijn.
         Ik wil de fractie van de ChristenUnie graag een schouderklopje geven. Ik vond de bijdrage van
die fractie namelijk geweldig, zij het af en toe wat gedetailleerd, te gedetailleerd zelfs. Dan wordt het
zelfs voor zo'n deskundige als ik ben, nog wat ingewikkeld! Ik zeg dan ook tegen de fractie van de
ChristenUnie: Sterkte bij het verder verbeteren van uw bijdragen. Ik heb wel het idee dat wij op
onderdelen nog veel van u kunnen leren. Schieten jullie hier wat mee op?
         Voorzitter! Een amendement dat ik zeker wil indienen - ik weet niet of dat wat heeft te maken
met hoe iedereen denkt over hoe een overheid moet functioneren - gaat over het verlagen van de
algemene reserve. Er wordt nu weer gestreefd naar een algemene reserve van 10% van de algemene
uitkering. Volgens mij is dat helemaal niet handig, zeker niet wanneer je als overheid ruimte moet zien
te creëren voor nieuwe ontwikkelingen, voor experimenten en voor het verkrijgen van steun in de
maatschappij. Iedereen heeft gezegd dat er voor alle voorstellen een dekking moet zijn. Ik meen dat
wanneer mijn voorstellen worden aangenomen, de gemeente geld overhoudt. De gemeente kan dan een
aantal zaken beter ondersteunen in de stad. Misschien dat ook de PvdA-fractie daar heel erg blij mee
zou kunnen zijn, omdat het sociale domein dan misschien een extra impuls kan krijgen. Ik heb
geprobeerd om voor mijn amendement over het verlagen van de algemene reserve allerlei
handtekeningen binnen te halen, weliswaar nog niet in de stad, maar tot nu toe alleen hier in de raad.
Die van de PVA-fractie ontbreekt echter nog. Dat is jammer.

        De heer SCHUUR (PvdA): Voorzitter! In het verleden hebben ook wij wel eens gepleit voor het
verlagen van de algemene reserve naar 8% (want daar gaat het nu over). Dat hebben wij toen gedaan -
 en dat hebben wij ook gezegd - omdat wij de andere 2% wilden laten gebruiken om de
openeinderegelingen mee te financieren (ik doelde daar zojuist al op in het debatje daarover). Toen
bleek dat de overschrijdingen op de openeinderegelingen gewoon uit de algemene reserve zouden
worden gefinancierd, hebben wij gezegd: Laat de algemene reserve dan maar op 10% staan.

       De heer HÜMMELS (Leefbaar Almelo): Wilt u nu zeggen dat wat ik voorstel met betrekking tot
de algemene reserve, op hetzelfde neerkomt als wat er nu in de praktijk gebeurt?

        De heer SCHUUR (PvdA): Dat wil ik niet helemaal zeggen, maar…

        De heer HÜMMELS (Leefbaar Almelo): Nee, dat probeer ik…

        De heer SCHUUR (PvdA): …het was wel een belangrijk statement richting het sociale domein
dat wij dat toen afgesproken hebben.

        De VOORZITTER: Mijnheer Hümmels! Vervolgt u uw betoog.

       De heer HÜMMELS (Leefbaar Almelo): Jazeker.
       Een ander amendement dat wij indienen, gaat over de toeristenbelasting. Daar staan
onverwacht veel handtekeningen onder, want het is ondertekend namens de fracties van ALA-AOV,
GroenLinks, PVA, LKA, de VVD, D66, de SP en Leefbaar Almelo. Het luidt als volgt:




INT - 1201417                                                                                            36
</p><p>                "Amendement SA.
                Betreft het voorstel begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Besluit:
                Toe te voegen aan de tekst met betrekking tot de toeristenbelasting:
                - Een vergelijkbaar bedrag als de opbrengst toeristenbelasting te reserveren voor de
                   sector die het opbrengt, door het onder te brengen in een daarvoor bedoelde reserve als
                   bijvoorbeeld doelreserve toerisme en recreatie.
                Toelichting:
                - gezien het door menigeen aangegeven belang van toerisme en recreatie als boven te
                   besluiten."

        In de eerste instantie was de reactie van het college dat de opbrengsten uit de
toeristenbelasting niet geoormerkt mogen worden, omdat deze belasting een algemene heffing is. Door
welingelichte bronnen ben ik echter geïnformeerd dat het met de in het amendement opgenomen
constructie wel mogelijk is om deze opbrengsten te reserveren voor de sector die het geld opbrengt.
Voor het geval men zich daarin wil verdiepen: dat mag.

       De heer SCHUUR (PvdA): Voorzitter! Ik vind de discussie over de toeristenbelasting altijd wel
grappig. Nu wordt voorgesteld om de opbrengsten uit de toeristenbelasting ten goede te laten komen
aan de sector, maar wij doen al ongelooflijk veel voor het toerisme, ook voordat de toeristenbelasting
geheven wordt. Alles wat wij doen aan verfraaiing van de stad (het aanleggen van fietsroutes en noem
maar op), komt ook ten goede aan het toerisme. Nu vragen wij daar wat voor terug, maar nu wordt er
opeens gezegd: Dan moeten wij de opbrengst wel aan toerisme besteden. Dat vind ik een beetje raar,
want dat doen wij eigenlijk al jaren.

        De heer HÜMMELS (Leefbaar Almelo): Als wij dat al jaren doen, dan zou u dat helemaal niet zo
heel erg hoeven te vinden, want het is juist de bedoeling dat wij meer aan toerisme en recreatie gaan
doen en dat dat uiteindelijk een breder effect heeft. Volgens mij is dat één van de prioriteiten van het
beleid, namelijk aandacht voor toerisme en recreatie. Dat daar wat meer voor gaat gebeuren, is niet zo
eigenaardig, dacht ik, maar u vindt dat blijkbaar wel vreemd.

        De heer SCHUUR (PvdA): Dat wij als gemeente daar wat meer aan gaan doen, vind ik niet
vreemd, maar u zegt specifiek: De opbrengsten uit de toeristenbelasting gaan wij daarvoor reserveren.
Misschien beperkt u het dan wel te veel, want dan zijn alleen maar die opbrengsten beschikbaar voor
het toerisme. Ik zou dan ook zeggen…

        De heer HÜMMELS (Leefbaar Almelo): Nee, nee, nee.

       De heer SCHUUR (PvdA): Voeg de inkomsten uit toeristenbelasting toe aan de algemene
middelen, want er gebeurt al heel veel op het gebied van toerisme. Ook gaat er nog meer gebeuren.

        De heer HÜMMELS (Leefbaar Almelo): U doet nu net alsof ik gezegd zou hebben dat door de
opbrengsten uit de toeristenbelasting te reserveren, andere zaken ten behoeve van het toerisme
beperkt moeten worden. Dat maakt u ervan, want dat heb ik geen moment gezegd. Ik heb gezegd dat
het geld dat opgebracht wordt door de sector, ook voor die sector gereserveerd moet worden, althans
een vergelijkbaar bedrag. Dat houdt volgens mij niet in dat dan ook minder gedaan moet worden voor
de sector, zoals uw opmerking haast impliceert.

       De heer SCHUUR (PvdA): Voorzitter! Wat de heer Hümmels zei, probeerde ik niet impliciet te
zeggen.

       De heer HÜMMELS (Leefbaar Almelo): Dat deed u wel. Het was juist niet de bedoeling dat het
ergens anders van afgehaald wordt.

        De VOORZITTER: Gaat u verder met uw betoog, mijnheer Hümmels.

       De heer HÜMMELS (Leefbaar Almelo): Voorzitter! Ik ga verder. Ik wilde zeggen dat ik niet zo
heel veel meer te melden heb, maar dan doe ik mijzelf haast tekort. Ik kijk nog even mijn notities na, als



INT - 1201417                                                                                          37
</p><p>dat mag van u. Ik denk dat ik nog een minuutje nodig heb. De kern van mijn verhaal heb ik in ieder
geval aangegeven. Verder wil ik een paar zaken terug laten komen bij de voorjaarsnotabehandeling.
        Ik zou het nu best nog even kunnen hebben over de maatregelen bij de Stadsbank, met name
over het feit dat die geen nadelige effecten mogen hebben. Ik vertrouw echter op het college dat dat zo
goed mogelijk verloopt.
        Wat Almelo Sociaal betreft, heb ik geen idee of ik er goed aan doe om aan mijn voorstel
daarover (het ongedaan maken van de bezuiniging) vast te houden als ik besef dat dat toch geen steun
krijgt.
        Ik wil in ieder geval genoemd hebben dat er veel te veel zaken blijven liggen. Ik hoop dat wij bij
de behandeling van de volgende voorjaarsnota nog een aantal zaken een andere richting in kunnen
sturen. En ik hoop dat wij op tijd met de voorjaarsnota beginnen, want zaken als de Stadsbank, Almelo
Sociaal en Stadstoezicht, maar ook de dingen die wij eerder aangekaart hebben, zoals sport en de jeugd
betrekken bij de raad, mogen wij niet laten liggen.

       De heer STORK (ALA-AOV): Voorzitter! Ik wil graag iets vragen aan de heer Hümmels en
terugkomen op het punt van de toeristenbelasting.

        De heer HÜMMELS (Leefbaar Almelo): Slome!

         De heer STORK (ALA-AOV): Ja, slome. Je hoeft niet altijd snel te reageren. Je kunt ook een
tijdje gebruiken om na te denken.

        De VOORZITTER: Ik sta daarover slechts een heel kort interruptiedebat toe, want de toeristen-
belasting is al aan de orde geweest.

        De heer STORK (ALA-AOV): Is de heer Hümmels bereid om de opbrengsten uit de
toeristenbelasting te laten besteden aan bijvoorbeeld het opknappen en up-to-date maken van het
buitenbad van zwembad Het Sportpark, zodat het voor toeristen aantrekkelijk is?

         De heer HÜMMELS (Leefbaar Almelo): Ik vind het wat overdreven om daar zo gedetailleerd op
te gaan reageren. Ik zei zojuist al dat ik vertrouw op het college. Ik ga ervan uit dat het college samen
met de sector recreatie en toerisme, die het geld opbrengt, de juiste wegen weet te vinden om het geld
te besteden. Ik zou hier nu wel kunnen zeggen dat het besteed moet worden aan het zwembad, maar
dat lijkt mij een beetje onzinnig.
         Ik krijg een natte neus, wacht even.

       De heer STORK (ALA-AOV): Dat is een teken dat een hond gezond is! Maar dat is een
constatering van een dierenarts.

       De heer HÜMMELS (Leefbaar Almelo): Ik denk niet dat mijn natte neus betekent dat een hond
gezond is. Dan zou ik ook nog een glanzende vacht moeten hebben, geloof ik!

       De heer STORK (ALA-AOV): Ik wil u wel een zakdoek aanbieden.
       Het gaat er mij alleen om of u in principe bereid zou zijn de toeristenbelasting te laten besteden
aan het opknappen van het buitenbad en dat soort dingen?

         De heer HÜMMELS (Leefbaar Almelo): Van principekwesties in de raad begrijp ik niet zo heel
veel. Daarom ga ik niet zeggen dat ik daar in principe toe bereid ben. Ook op dit gebied vertrouw ik het
het college en de sector toe om de juiste wegen te vinden.
         Ik had ook nog enkele opmerkingen gemaakt over cultuur en de woonvisie. Ik weet niet wat ik
heb met het college, maar een deel van het college heeft in elk geval een groot deel van mijn bijdrage
wonderbaarlijk goed beantwoord. Dank u wel.
         Tot slot dien ik het aangekondigde amendement over de algemene reserve in. Dat is
ondertekend namens de fracties van GroenLinks, PVA, LKA, de SP, Lijst Çete en Leefbaar Almelo. Het
luidt als volgt:

                "Amendement TA.
                Betreft het voorstel begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.



INT - 1201417                                                                                          38
</p><p>                Besluit:
                Het voornemen de algemene reserve op 10% te stellen te vervangen door:
                Het verlagen van de algemene reserve met 2% naar 8% i.p.v. de voorgestelde 10%.
                Toelichting:
                Daar waar weinig ruimte is voor nieuwe ontwikkelingen moeten we als overheid het lef
                tonen tot het creëren van mogelijkheden voor de benodigde experimenten en creatieve
                ideeën."

         De heer ÇETE (Lijst Çete): Mevrouw de voorzitter! Ik heb nog een paar aanvullende vragen aan
het college over onderwerpen waarover ik volgens mij geen antwoord heb gekregen.
         Voorzitter! U hebt het een en ander gezegd over de artikel 12-status. De bedoeling van wat ik
daarover heb gezegd, was niet zozeer dat de gemeente nu al of de komende dagen, maanden of jaren
die status moet krijgen. Mijn vraag is wel welk moment daarvoor gekozen moet worden. Nu wij een
registeraccountant als wethouder van financiën hebben, is het niet zo heel erg moeilijk om de begroting
rond te krijgen, want dat lukt hem wel. Mijn vraag is echter wanneer beleidsmatig het eind is bereikt.
Wanneer zeggen wij: Nu is het wel leuk geweest, nu kan het niet meer?
         Verder heb ik geen antwoord gekregen op mijn vraag hoe wij bij de provincie extra steun
kunnen krijgen. Ik ben van mening dat wij daar nog steeds genoeg geld vandaan kunnen halen. Rond de
€ 80 miljoen, zei iemand tegen mij (maar ik zal geen namen noemen). Verder is er nog een budget voor
de Netwerkstad Twente. Weliswaar krijgen wij voor de aanleg van de Waterboulevard al geld, maar ik
meen dat er bij de provincie nog steeds geld ligt waarmee die ons tegemoet kan komen (€ 78 miljoen,
fluistert iemand mij nu in).
         In de eerste termijn heb ik ook gesproken over het programma Aantrekkelijke stad, kansarmen,
kansrijken en stijging van de kansarmen allochtone groeperingen. Wat is de verwachting daarvoor, wat
denkt de gemeente daarvoor te gaan doen en hoe wordt geprobeerd ongewenste zaken te voorkomen?
Ook daarop heb ik geen antwoord gekregen van het college.
         Ik wil reageren op wat collega's hebben gezegd in de eerste termijn. Ten eerste vond ik het heel
erg leuk dat onze raadsnestor, de heer Stork, er uiteindelijk van overtuigd is dat er een nieuw stadhuis
moet komen, zij het dat hij wel een kleiner stadhuis wil. Dit wil ik even zeggen, want dit vind ik heel
erg leuk.

      De heer STORK (ALA-AOV): Ik ben er niet van overtuigd dat er een nieuw stadhuis moet
komen, laat dat even duidelijk zijn.

        De heer ÇETE (Lijst Çete): Als het ervan komt.

       De heer STORK (ALA-AOV): Nee, nee. Er is een democratisch besluit genomen over de bouw
van een nieuw stadhuis, en daar hebben wij ons aan te houden. Wel kunnen wij dat amenderen. Dat
kan wel.

        De heer ÇETE (Lijst Çete): Ik was er blij mee, persoonlijk. Dat wilde ik even overdragen aan de
raadsnestor.
        Voorzitter! De vorige spreker heeft de verschillende moties en amendementen niet willen
noemen, maar ik wil daar kort en bondig het volgende over zeggen:
- Motie H (Inkomensondersteuning) van de PvdA-fractie ondersteunen wij.
- Het amendement (TA) van de fractie van Leefbaar Almelo over een algemene reserve van 8% van
   de algemene uitkering heb ik al ondertekend.
- De voorstellen van de SP-fractie over de Stadsboerderij Beeklust (amendement W), schoolzwemmen
   en internetaansluiting voor minima (motie X) steunen wij.
- Ook steunen wij het voorstel van de CDA-fractie (motie D1) inzake het melden van overschrijdingen
   van 10% of meer op bedragen boven € 250.000,--.
- Hetzelfde geldt voor het voorstel van de VVD-fractie (motie BA) inzake het onderhoud van
   kunstwerken.
        De behandeling van de moties en amendementen wil ik niet langer maken. Wel wil ik nog een
paar opmerkingen maken over de inbreng van de fracties van de SP en D66.
        De heer Cornelissen begint bij elke voorjaarsnotabehandeling en begrotingsbespreking over het
verlagen van onze raadsvergoeding. Ik ben lid van een plaatselijke politieke partij, maar als ik in zijn
schoenen zou staan, zou ik daarvoor gepleit hebben in Den Haag. Daar zou zijn partij ervoor kunnen
gaan pleiten dat het raadslidmaatschap vrijwilligerswerk moet zijn; dan zijn wij tenminste een keer af



INT - 1201417                                                                                         39
</p><p>van zijn opmerkingen hierover. Het is geen taak voor ons om elke keer te komen met opmerkingen of
ideeën over onze raadsvergoeding. Dat is namelijk een taak van Den Haag. Ik hoop dat hij het in Den
Haag dan wel haalt, met de SP.

         De heer CORNELISSEN (SP): Voorzitter! Onze raadsvergoeding moet vallen binnen een bepaalde
bandbreedte. Die bandbreedte heeft een ondergrens en een bovengrens. In Almelo zitten wij aan de
bovengrens. Wij mogen dus van Den Haag de raadsvergoeding iets verlagen. De SP-fractie is van
mening - en ik hoop dat de heer Çete die deelt - dat als de raad diep snijdt in de gelden voor
maatschappelijke partners en daarbij bij wijze spreken de laatste € 1500,-- subsidie afpakt van de
klompendansvereniging, de raadsleden ook een beroep mogen doen op hun eigen morele besef. Dan
kunnen wij zeggen: Als wij daar zoveel in snijden, dan mogen wij ook een beetje bij onszelf weghalen.
In dit geval gaat het om een bezuiniging van 6% op de vergoeding voor raadsleden om de
stadsboerderij open te kunnen houden. (Overigens kan ik nu al melden dat ons amendement daarover
(W) zal worden ingetrokken, omdat dat niet kan. Bij de voorjaarsnotabehandeling kom ik daar misschien
weer op terug.) Dat is onze opmerking over de raadsvergoeding geweest. Die heeft niets te maken met
het lid zijn van een landelijke politieke partij of een plaatselijke politieke partij. Wij zitten hier als
gekozen raadsleden, allemaal hoofdelijk gekozen. Wij hebben allemaal onze eigen afweging te maken
met betrekking tot de raadsvergoeding.

         De heer ÇETE (Lijst Çete): Mijnheer Cornelissen! Ik weet welke moeite ik eind 2009 had om
mensen te vinden die gestudeerd hadden, die deelnamen in Almelo en een beetje politiek konden
bedrijven. Ook heb ik van collega's gehoord dat dat niet gemakkelijk is. Mensen van een bepaald niveau
die politiek kunnen voeren, maar al een heel belangrijke baan hebben, doen het raadswerk niet voor het
geld. Daarom is het ook een vergoeding. Maar de SP schetst altijd het beeld dat het raadslidmaatschap
geldtrekkerij is voor niets doen. Er zijn echter mensen die uren en uren met het raadswerk bezig zijn. Ik
heb tot nu toe niet gereageerd op de voorstellen over de raadsvergoeding, omdat het politiek gezien
niet echt verantwoord is om over je eigen vergoeding te praten. U geeft de bevolking echter een extra
reden om ons te beschuldigen van niets doen en geld trekken. Daar erger ik mij een beetje aan, want zo
is het niet.

         De heer CORNELISSEN (SP): Voorzitter! De heer Çete geeft zelf aan dat mensen het raadswerk
niet voor het geld doen, dus dat mag de discussie niet zijn. Verder is het volgens mij niet meer dan
billijk om ook bij onszelf te rade te gaan als wij andere mensen vragen geld in te leveren. 100
medewerkers gaan er uiteindelijk uit bij de gemeente. Die komen voor een deel op straat te staan of
verdwijnen via natuurlijk verloop…

        De heer ÇETE (Lijst Çete): Nou, nou, nou!

        De heer CORNELISSEN (SP): Er worden elf mensen ontslagen bij Stadstoezicht. Dat is allemaal
heel normaal, maar dan moeten wij ook een keer bij onszelf te rade gaan.

        De heer ÇETE (Lijst Çete): Of u heeft naar een andere wethouder geluisterd, of ik heb dat
gedaan. Wat ik van de wethouder heb begrepen, zullen de betrokken medewerkers allemaal via
natuurlijk verloop verdwijnen. Er zullen bijna geen gedwongen ontslagen vallen. Daarom neem ik
afstand van de uitspraak dat straks 100 medewerkers op straat komen te staan.

          De heer SCHUUR (PvdA): Voorzitter! Ik wil dit onderwerp graag in perspectief zetten. Volgens
mij moet je niet alleen naar de raadsvergoedingen kijken, maar moet je ook kijken naar bijvoorbeeld de
fractievergoedingen. Wij zijn op excursie geweest in Utrecht en Amersfoort. Daar hebben wij gehoord
welke bedragen daar uitgegeven worden aan fractievergoedingen. Die zijn meer dan het tienvoudige
van de bedragen die hier in Almelo aan fractievergoedingen worden besteed. Ik ben van mening dat ook
dit feit in deze discussie betrokken moet worden.

        De VOORZITTER: Mijnheer Çete, ik vraag u om verder te gaan met uw betoog.

        De heer ÇETE (Lijst Çete): Ja, voorzitter.
        Mijn emoties dwingen mij heel erg om te reageren op de inbreng van de fractie van D66. Hoe
moet ik dat politiek verantwoord doen? Daar heb ik afgelopen week al aan zitten denken. De D66-
fractie heeft een tegenbegroting ingediend. Ja, prachtig, maar mijn collega-fractievoorzitter de heer



INT - 1201417                                                                                           40
</p><p>Gerritsen suggereert hier dat zijn fractie binnen drie maanden het werk kan opknappen dat ons met ons
vijfendertigen plus een college van zes leden met een ambtelijk apparaat van 1000 mensen niet lukt. Ik
ben dan ook zeer benieuwd hoe de fractie van D66 durft te zeggen dat zij binnen drie maanden kan
wat wij met ons allen niet kunnen.
        Ook ben ik heel benieuwd wat betreft de motie (V) van D66 over de peiling onder Almeloërs.
Als er in het begin van de jaren tweeduizend peilingen waren gehouden over Waterrijk, Leemslagen,
Fortezza en alle andere dossiers waar wij nu mee zitten, dan hadden wij nu geen peiling hoeven te
houden.
        De heer Isendoorn zei zojuist dat een bestuur ook voor het verleden verantwoordelijk is. Daar
heb ik alle respect voor. Hij heeft helemaal gelijk daarmee, maar als mens en als politicus denk ik het
volgende. Wanneer ik als politicus tussen 2002 en 2006 in het college en in de coalitie had gezeten,
had ook ik mijn bijdrage geleverd, met een goed hart, niet slecht bedoeld. Maar om het beleid van toen
nu zo te bekritiseren alsof het uit de lucht is komen vallen, dat zou voor mij een stap te ver gaan.

       De heer ISENDOORN (D66): Mevrouw de voorzitter! Ik geloof niet dat de heer Çete goed
begrepen heeft wat er allemaal aan de hand is in deze stad.

        De heer ÇETE (Lijst Çete): Voorzitter…

        De VOORZITTER: De heer Isendoorn heeft het woord.

         De heer ISENDOORN (D66): Mijnheer Çete! U refereert aan de periode 2002-2006. Dat is
inmiddels tien jaar geleden, maar vooruit. Wij hadden toen de opvatting dat er een integraal plan moest
komen voor de binnenstad. Vervolgens hebben wij een integrale visie gemaakt voor de hele stad,
namelijk het Masterplan. Eén van de kernprojecten van de uitwerking van dat plan is het
binnenstadsplan. Dat plan is in de collegeperiode daarna vormgegeven. Aan het eind van die periode
was de situatie in de wereld drastisch veranderd, ook op financieel en economisch gebied. Die
veranderde situatie beïnvloedt de keuzen die het bestuur maakt. Je moet dan als bestuur dingen
bijstellen, omdat de werkelijkheid niet meer hetzelfde is als vier of vijf jaar daarvoor. Het is evident dat
je dat doet, want dat is ook een kenmerk van goed besturen. Ik verwijt het huidige bestuur dan ook
niet dat het niets doet. We hebben ook gezegd - u moet niet half luisteren - dat wij niet bestrijden dat
wethouder Van Woudenbergh nu met het binnenstadsplan bezig is en dat hij dat op een kleinere schaal
doet dan destijds bedacht werd. Het is nu 10% van wat wij toen bedachten. Het binnenstadsplan is
aangepast, en dat is een kwestie van goed besturen. Nu doen wij het op een andere schaal die meer
past in deze tijd. Nu zitten wij echter met de volgende kwestie, en dat hebben wij willen betogen. Wij
hebben niet gezegd dat de begroting verkeerd is. De begroting sluit, en daar kunnen wij het mee eens
zijn.

        De heer ÇETE (Lijst Çete): Is dit nog steeds…

        De heer ISENDOORN (D66): Mijnheer Çete, luister nu even naar wat ik zeg.

        De heer ÇETE (Lijst Çete): Ik bepaal zelf…

       De heer ISENDOORN (D66): Dat laat onverlet dat er onderliggende problemen zijn die aangepakt
moeten worden. Dat hebben wij in ons pamflet aangegeven, want daar moet je wat aan doen. Ons
bezwaar tegen de voorliggende begroting is dat daarvoor geen structurele oplossingen worden
aangedragen.

       De heer ÇETE (Lijst Çete): Voorzitter! Een gemiddelde politicus is ijdel, zei laatst een collega van
mij. Maar de ijdelheid van de heer Isendoorn en de heer Gerritsen gaat soms echt zo ver…

        De VOORZITTER: Nee, ik…

        De heer ÇETE (Lijst Çete): … zover dat ik…

        De VOORZITTER: Nee.

        De heer ÇETE (Lijst Çete): Ik moet, ik begrijp niet, ik luister niet. Iedere…



INT - 1201417                                                                                            41
</p><p>        De VOORZITTER: Mijnheer Çete! Ik grijp nu in. Ik vind echt dat wij het debat in de raad op de
inhoud moeten voeren, met inhoudelijke argumenten. Laat ons dat alstublieft niet persoonlijk maken.
Dat is nergens voor nodig.

       De heer ÇETE (Lijst Çete): Als de fractie van D66 ooit met inhoud komt, dan zal ik dat debat
ook voeren. Dank u wel.

        De heer BUITENWEG (GroenLinks): Voorzitter! Als ik de tijd daarvoor had gehad, zou ik nu met
een integraal verhaal gekomen zijn, maar dat gaat niet lukken. Daarom zal ik nu reageren op de
bijdragen van de collega-fracties en de antwoorden van het college. Vervolgens zal ik enkele moties en
amendementen indienen. Tot slot wandel ik nog even langs de amendementen en moties die ingediend
zijn door andere fracties, maar dat zal ik in vogelvlucht doen, verwacht ik.
        Ik begin met een reactie op de bijdrage van de CDA-fractie in de eerste termijn. Het bevreemdt
mij dat de CDA-fractie pleit voor een notitie over het welzijnsbeleid, maar daar vooraf bij heeft gezegd:
Wij gaan bezuinigen op Scoop. Mijn logica is (daar is zojuist ook al iets over gezegd) dat je eerst beleid
gaat maken en vervolgens zegt wat je daarvoor over hebt.

        Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! In de tweede termijn heb ik aangegeven dat ik
niet vraag om een beleidsnotitie over welzijn waarin allerlei nieuwe dingen beschreven worden. Nee, ik
heb met name gevraagd om een overzicht te geven van het welzijnsbeleid dat wij als gemeente thans
voeren. Dat is iets heel anders.

        De heer BUITENWEG (GroenLinks): Wat wilt u daar dan mee?

        Mevrouw TEN SELDAM-EVEN (CDA): Ik wil graag een integraal overzicht krijgen van het
welzijnsbeleid. Op basis daarvan kan bekeken worden wat gecontinueerd dient te worden en welke
doelstellingen worden gehaald. Dat moet men zien in lijn met datgene wat wij zeggen over hoe met
beleid moet worden omgegaan. Je stelt een doel, je stelt de middelen daarvoor beschikbaar en bekijkt
vervolgens of je dat moet blijven continueren dan wel of je wijzigingen moet aanbrengen. Misschien
moet je wel opplussen.

        De heer BUITENWEG (GroenLinks): Ik blijf bij mijn standpunt dat ik de volgorde vreemd vind.
        Voorzitter! Ik kom nu bij de bijdrage van de PvdA-fractie. Ik sta volledig achter haar standpunt
dat het zwembad gehandhaafd moet blijven voor een stad als Almelo.
        Ik vind het prima dat ook de fractie van D66 nu ineens pleit voor Energierijk. Wij hebben daar al
heel vaak voor gepleit. Zoek alternatieven voor het gebied van Waterrijk.
        Verder ben ik benieuwd wat de D66-fractie zal doen wanneer de door haar ingediende motie (T)
inzake het verhogen van de ozb met 4,8% het niet haalt. Zal die fractie dan wel het voorstel van de
fractie van GroenLinks steunen om de ozb te verhogen met 1,5%?
        Ik heb een aantal dingen genoteerd over de inbreng van de VVD-fractie. Dat geldt met name
voor hetgeen die fractie gezegd heeft over overschrijdingen van de budgetten voor de
openeinderegelingen (WWB, Wet werk en bijstand en WMO). Hierover is zojuist al gedebatteerd. Ik ben
van mening dat je de consequenties van de overschrijdingen niet alleen maar kunt neerleggen binnen
het sociale domein. Daar moeten grenzen voor aangegeven worden

         De heer NIJHUIS (VVD): Voorzitter! Ik blijf in de benen komen, omdat iedere keer het beeld
geschetst wordt dat wij alleen maar willen wat de heer Buitenweg nu aangeeft. Dat is niet het geval.
Wij willen de zaken in een volgorde zetten. Dat hebben wij een- en andermaal uitgelegd en wij hebben
daarover gediscussieerd met de heer Schuur. Nu komt de heer Buitenweg daar weer op terug. Iedere
keer wordt het beeld geschetst dat wij willen dat wanneer een regeling wordt overschreden, de
consequenties daarvan bij de doelgroep neergelegd moeten worden. Nee, wij zeggen: Ga een trap af en
bekijk in welke volgorde je de problemen kunt analyseren. Als er mogelijkheden zijn om binnen het
eigen gebied iets te doen, doe dat dan, maar als dat niet kan, dan kunnen de middelen elders gehaald
worden. Als ook dat niet kan, dan hebben wij altijd nog de algemene reserve waaruit geput kan
worden. Doe dat dan wel in de volgorde die ik heb aangegeven. Het beeld dat wij de consequenties
enkel en alleen bij de doelgroep neerleggen, is absoluut onjuist.




INT - 1201417                                                                                           42
</p><p>        De heer BUITENWEG (GroenLinks): Maar u heeft in uw betoog ook gezegd dat de VVD-fractie
wil dat er versoberd wordt en dat de uitvoering van de regelingen goedkoper moet worden of dat
bepaalde middelen niet toegewezen moeten worden.

         De heer NIJHUIS (VVD): Wij hebben een aantal mogelijkheden genoemd die wellicht uitgevoerd
kunnen worden. De wethouder heeft daar ook op gereageerd (overigens hebben wij over die
mogelijkheden geen moties of amendementen ingediend; dat zullen wij ook niet doen). Verder heeft de
wethouder aangegeven dat al veel zaken gebeurd zijn. Dan hoeven ze niet alsnog te gebeuren, maar ga
je bekijken welke mogelijkheden er nog meer zijn. Zijn die er niet, dan ga je na wat de volgende fase is.
Zo probeer je fatsoenlijk beleid te voeren en daarbij zoveel mogelijk financieel degelijk en verantwoord
te handelen, want je moet het een met het ander combineren. Dat wil zeggen: én sociaal verantwoord
én financieel verantwoord.

       De heer SCHUUR (PvdA): Voorzitter! De heer Nijhuis zei zojuist dat telkens een bepaalde indruk
wordt gewekt. Die indruk wekt de VVD-fractie zelf met de tekst in het amendement (AA) over de
overschrijdingen van budgetten voor de openeinderegelingen. De tekst van het amendement zou zo
veranderd moeten worden, dat je wel bereikt wat je wilt bereiken, maar daarbij tevens de indruk
wegneemt dat je de zwaksten in de samenleving wilt laten bloeden.

        De VOORZITTER: Dames en heren! Misschien kan de heer Nijhuis hier straks in zijn eigen
bijdrage op reageren. Anders krijgen wij nu een debatje tussen interruptieplegers.
        Ik geef het woord aan de heer Buitenweg.

        De heer BUITENWEG (GroenLinks): Voorzitter! Het verhaal van de VVD-fractie over Waterrijk
vind ik grappig. Dat verhaal kwam neer op: eerst zeker weten of je wat wilt bouwen en dan pas
investeren ten behoeve van de infrastructuur. Daar ga ik zo meteen nog heel goed over nadenken.
Weliswaar hebben wij al tegen het voorstel gestemd om überhaupt te gaan bouwen in Waterrijk, maar
wat de VVD-fractie hierover heeft gezegd, komt al aardig in de richting die wij wensen. In mijn visie zal
er de komende tijd namelijk weinig of helemaal niet worden gebouwd.
        Dan heb ik nog enkele reacties op de antwoorden van het college.
        Wethouder Schouten zei in zijn reactie dat de Burgemeester Schneiderssingel niet overbodig is,
maar zorgt voor ontsluiting van de wijk Indië. Dat begrijp ik, maar ik doelde meer op het vervolg,
richting Waterrijk (misschien was ik daar niet duidelijk in). Dat vervolg kent m.i. nog geen enkele
noodzaak.
        Voorts gaf wethouder Schouten aan dat er goede kansen zijn op het openhouden van
Stadsboerderij Beeklust. Hij stelde dat mogelijk stichting De Twentse Zorgcentra zelf de boerderij blijft
exploiteren en dat er anders nog twee gegadigden zijn. De motie die ik hierover zal indienen, is gericht
op de situatie dat dat niet het geval is en dat er geen enkel alternatief voorhanden is. Dan zou de
gemeente genoodzaakt zijn de kinderboerderij te sluiten, en dat wil ik graag voorkomen. Dat is de
strekking van de motie die ik zo meteen hierover zal indienen.
        Verder heeft wethouder Schouten gereageerd op het door mij aangekondigde amendement
inzake gewasbescherming (ik zeg dat woord nu zelf, maar het blijft een raar woord,
gewasbescherming, terwijl je er gif op spuit). De wethouder zei: Wij gaan de maatregelen van het rijk
met betrekking tot het gebruiken van gewasbeschermingsmiddelen bekijken en gaan daar de komende
jaren mee aan het werk. Het is nu duidelijk dat de Tweede Kamer heeft gezegd, middels het aannemen
van een motie van de GroenLinks-Kamerfractie, dat gemeenten en particulieren moeten stoppen met
het gebruiken van Roundup. Mijn oproep is om dat vanaf gisteren te doen en voortaan alleen nog
alternatieven te gebruiken, niet pas over twee jaar.
        Wethouder Van Woudenbergh heeft antwoord gegeven op mijn vraag wanneer de
Kloosterhofflat wordt betaald. Dat antwoord was op zich helder, maar ik had daarbij in relatie tot het
binnenstadsplan ook een vraag gesteld over Achmea. Is Achmea nog een partner in dezen? Daar heb ik
geen antwoord op gekregen.
        Wethouder Van Marle is ingegaan op wat ik heb gezegd over de stresstest voor gemeenten.
Daarbij heeft hij niet duidelijk aangegeven of hij een dergelijk middel ziet zitten en of hij bereid is om
mee te doen aan de ontwikkeling daarvan. Volgens mij heeft de wethouder namelijk alleen gezegd dat
hij ervan op de hoogte is dat een stresstest voor gemeenten wordt ontwikkeld door gemeenten met
meer dan 100.000 inwoners.
        Dan dien ik nu de door mij aangekondigde moties en amendementen in. Die luiden:




INT - 1201417                                                                                          43
</p><p>                "Amendement UA.
                Chemische 'gewasbescherming'
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Besluit:
                Het voorstel betreffende programmabegroting 2012 als volgt aan te passen: De tabel op
                pagina 111 als volgt te wijzigen: De tekst Efficiency bevorderen door meer gebruik te
                maken van chemische gewasbeschermingsmiddelen, te laten vervallen en te wijzigen in:
                Alternatieven te zoeken voor het gebruik van gewasbeschermingsmiddelen (Roundup).
                Tevens wordt het verzamelbedrag van € 107.000,-- (pag. 111 begroting 2012) met het
                hiervoor te berekenen bedrag naar beneden bijgesteld.
                Toelichting:
                De Tweede Kamer heeft op 16 september 2011 een motie aangenomen waarmee het niet-
                economisch gebruik van glyfosaat in Nederland wordt verboden. Rik Grashoff van
                Groenlinks diende de motie in.
                Met de motie wil Grashoff het gebruik van glyfosaat, waaronder Roundup, verbieden voor
                particulieren en gemeenten. Veel gemeenten gebruiken het middel om onkruid te
                verwijderen. 'Hiervoor is geen economisch belang. Alleen misschien omdat andere
                methoden om onkruid te verwijderen mogelijk duurder zijn.'
                Uit onderzoek blijkt dat gebruik van het middel leidt tot verhoogde concentraties glyfosaat
                in het oppervlaktewater, wat daarmee schadelijk is voor het milieu."

       Dit amendement is ondertekend namens de fracties van GroenLinks en Leefbaar Almelo.
       De door mij aangekondigde motie met betrekking tot de Stadsboerderij Beeklust dien ik in,
omdat ik toch graag enige zekerheid heb dat deze openblijft, wat er in 2012 ook onderzocht wordt en
gebeurt. Deze motie luidt:

                "Motie VA.
                Stadsboerderij Beeklust
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Constateert dat:
                - In de jaren negentig besloten is de exploitatie van de Stadsboerderij over te doen aan
                   St. De Twentse Zorgcentra. Dit destijds gebeurde in het kader van bezuinigingen op de
                   gemeentelijke begroting.
                - Het voortbestaan van Stadsboerderij Beeklust nu in het geding is door de geplande
                   bezuinigingen. Het College van B. en W. het exploitatiebudget van € 55.000,-- wil
                   halveren. De Twentse Zorgcentra, die nu de exploitatie op zich neemt, denkt het niet
                   voor dat bedrag te kunnen doen. Als de instelling afhaakt, de gemeente op zoek moet
                   naar een andere exploitant. Wordt die niet gevonden, dan gaat de kinderboerderij dicht.
                Overweegt dat:
                - Sluiting te allen tijde moet worden voorkomen omdat de Stadsboerderij ca. 30.000 be-
                   zoekers per jaar trekt en dit aantal alleen maar zal gaan stijgen door de
                   nieuwbouwprojecten zoals: de Titus Brandsmahof en de nieuwe appartementen.
                Draagt het College van B. en W. op:
                In 2012 te zoeken naar mogelijkheden om de stadsboerderij vanaf 2013 open te houden of
                anders de exploitatie in eigen beheer te nemen.
                En gaat over tot de orde van de dag."

       Deze motie is ondertekend namens de fracties van GroenLinks, de SP, Leefbaar Almelo, Lijst
Çete en LKA.

                "Motie WA.
                Schoolzwemmen
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Constateert dat:
                - Voor een speciale groep kinderen, m.n. van allochtonen en asielzoekers, is het school-
                   zwemmen erg belangrijk.
                - Alleen kinderen die in de bovenbouw van de basisschool nog niet kunnen zwemmen,
                   maken gebruik van deze regeling.
                - Het gaat om 150 kinderen per schooljaar. Een deel van deze kinderen komt uit het AZC.



INT - 1201417                                                                                            44
</p><p>                Overweegt dat:
                - Van veel van deze kinderen de verwachting is dat zij zonder dit aanbod schoolzwemmen
                    nooit zullen leren zwemmen, mede doordat vaak hun ouders ook niet kunnen zwemmen.
                Draagt het college van B. en W. op:
                Het voorstel schoolzwemmen in stand te houden, voor de komende vijf jaar, en te dekken
                uit de € 125.000,-- t.b.v. 'Het verschil maken'.

       Deze motie is ondertekend namens de fracties van GroenLinks, de SP, Leefbaar Almelo, de
PvdA, Lijst Çete en LKA.

                "Amendement XA.
                Verhoging ozb
                De raad van de gemeente Almelo, bijeen op 8 en 10 november 2011.
                Besluit:
                Aan het raadsbesluit bij punt 6 toe te voegen:
                - de ozb van 2012 met 1,5% extra te verhogen.
                Toelichting:
                De gemeente moet fors bezuinigen en de inkomsten blijven achter.
                De burgers niet te veel mogen worden belast maar Groenlinks vindt het acceptabel de ozb
                met 1,5% extra te verhogen en zo ca. € 210.000,-- extra te genereren (per 1%
                € 140.000,--).
                De feitelijke tarieven ozb zijn nu nog niet volledig beschikbaar maar worden in ieder geval
                met 1,5% verhoogd (pag. 224, bijlage 4 van de programmabegroting 2012)."

        Dit amendement is ondertekend namens de fracties van GroenLinks en de SP.
        Ik heb in de begroting gelezen dat nu nog niet helemaal bekend is hoe hoog de ozb zal zijn in
2012. In het door mij ingediende amendement gaat het om een extra verhoging van 1,5% bovenop wat
de ozb überhaupt zal zijn.
        Zoals ik aangekondigd heb, wil ik nu ingaan op een aantal moties en amendementen van andere
fracties. Ik neem aan dat ik daarvoor nog tijd heb.

       De VOORZITTER: Ja, dat kan. Wel vraag ik, in navolging van wat de vicevoorzitter heeft
gevraagd, om wat tempo te maken.

        De heer BUITENWEG (GroenLinks): Volgens mij doe ik niet anders.

        De VOORZITTER: Ik vind dat het in een traag tempo gaat.

        De heer BUITENWEG (GroenLinks): Voorzitter! De fractie van D66 heeft een motie (S) ingediend
met als titel: Parkeren Waterrijk. Deze motie kunnen wij steunen.
        De fractie van ALA-AOV stelt in motie GA dat Leemslagen niet ontwikkeld moet worden. Dat
steunen wij.
        Dat geldt ook voor de motie (G) van de PvdA over het zwembad.
        Ook steunen wij de motie (BA) van de VVD-fractie over het onderhoud van kunstwerken.
        Wat betreft de fractie van Leefbaar Almelo, steunen wij de motie (SA) die de heer Hümmels
mede namens een aantal fracties heeft ingediend over de toeristenbelasting.
        Tot slot steunen wij motie Y van de SP-fractie over inhuur van personeel. Ik had nog iets willen
zeggen over een aantal andere moties en amendementen, maar die heb ik niet op mijn lijstje staan.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik zal nog een keer een betoog houden aan de
hand van een Powerpointpresentatie. De heer Hümmels zei dat mijn bijdragen misschien wat minder
gedetailleerd zouden kunnen zijn. Ik zal nu proberen het anders te doen.
        Ik wil in deze bijdrage een paar dingen aan de orde stellen, om te beginnen de vraag hoe wij
aankijken tegen de toekomst van het grondbedrijf en hoe wij die kunnen verbeteren. Ik meen dat het
ongelooflijk belangrijk is dat wij samen komen tot een visie op de toekomst van de stad. Ik heb gemerkt
dat er al behoorlijk wat twijfel is over de haalbaarheid van het woningbouwprogramma:
- ik hoorde de fracties van de PvdA en de VVD vragen om een studie daarnaar die bij de voorjaarsnota
   aan de orde moet komen;
- andere fracties zeiden: Maak van Waterrijk Energierijk;



INT - 1201417                                                                                            45
</p><p>- en een heleboel fracties zeggen dat projecten afgestoten moeten worden.
        Ik ga ervan uit dat deze zaken allemaal in de woonvisie aan de orde komen en dat wij bij de
voorjaarsnota 2012 definitief besluiten over het al dan niet aanpassen van het woningbouwprogramma.
        Als het woningbouwprogramma wordt aangepast, dan is natuurlijk wel het geweldige probleem
aan de orde waar het geld vandaan gehaald moet worden. Dat is de reden waarom wij gezocht hebben
naar mogelijkheden om geld te verkrijgen. Het college zegt dat dat jaarlijks bekeken wordt bij de
voorjaarsnota. Dan komt er een totaalsom die de uitkomst aangeeft. Wij stellen echter voor om echt
fundamenteel aan te geven welke reserves en voorzieningen niet vrij kunnen vallen om te bestemmen
voor iets anders en om daar in de raad op een fatsoenlijke manier over te discussiëren. Wat mij betreft,
betekent dit een heel andere focus bij het kijken naar de reserves en voorzieningen dan de focus die er
tot nu toe is geweest. Dat is de reden waarom ik voorstel om dat vooruitlopend op de voorjaarsnota te
doen. Overigens is dit voor mij niet een halszaak. Het gaat er mij namelijk om dat wij echt heel
fundamenteel bekijken of er geld te halen is uit de reserves en voorzieningen en of het verantwoord is
om dat vrij te laten vallen om het verlies te nemen op de verkoop van grond.
        Het volgende onderwerp waar ik op inga, is hoe wij kunnen voorkomen dat de bezuinigings- en
besparingsmaatregelen vooral de kwetsbare inwoners van Almelo treffen. Ik ondersteun voor een
belangrijk deel het verhaal dat de PvdA-fractie (maar ook andere fracties) daarover naar voren heeft
gebracht. Wij staan voor een gigantische besparingstaakstelling. Het college heeft daar een
totaalsommetje van gemaakt. Daaruit blijkt dat het budget voor participatie met 74% vermindert. Ook
moeten wij in totaal € 18 miljoen besparen op de uitkeringen. Ik durf de stelling aan dat die
besparingen vooral de kwetsbare inwoners treffen. Er is immers bewust gekozen, in alle voorstellen,
voor het verminderen van het gebruik en het soberder maken van de voorzieningen.
        Wat is het gevolg wanneer je de overschrijdingen op de budgetten voor de openeinderegelingen
binnen het sociale domein opvangt (ik kom hier toch op terug)? Het gaat er om wat je verantwoord
vindt in dezen. De VVD-fractie heeft aangegeven wat volgens haar het criterium is om te bepalen wat
wel of niet verantwoord is. Die heeft gezegd (ik heb dat woordelijk opgeschreven):
             "Alleen diegenen die door ernstige ziekte, handicap of anderszins absoluut niet kunnen
             werken, hebben recht op een uitkering. De uitdaging wordt dus om het aantal uitkeringen
             te beperken tot de laatste categorie en daarmee deze openeinderegelingen beheersbaar te
             houden."
        Als dit de lijn is die wij met ons allen kiezen, dan is dat voor onze fractie absoluut een
onbegaanbare weg. Daarom zullen wij niet het voorstel steunen (amendement AA) dat de VVD-fractie
heeft gedaan met betrekking tot de openeinderegelingen.

        De heer NIJHUIS (VVD): Voorzitter! Wat de heer Veenstra nu zegt, intrigeert mij enorm.
Volgens mij zijn namelijk de betrokken voorzieningen juist voor de door ons genoemde mensen bedoeld.
Verder heb ik al eens betoogd (ik dacht in een heel kort debat met mevrouw Heite in de vorige
raadsperiode) dat ook de VVD-fractie vindt dat een fatsoenlijke samenleving goede voorzieningen moet
treffen voor mensen die absoluut niet kunnen werken, maar dan ook echt goede voorzieningen. Wij
komen nu nog te vaak situaties tegen waarin mensen niet meedoen terwijl zij dat in principe wel
kunnen (wat dit betreft, heb ik wel eens gewezen op het boek van Mirjam Pool). Dat is waar wij op
hameren. Als je goed regelt dat zij wel meedoen, blijft er voldoende geld over om degenen die echt niet
kunnen werken, goed te faciliteren. Ik weet niet waar de heer Veenstra het vandaan haalt dat wat hij
citeerde uit onze algemene beschouwing, asociaal zou zijn, want zo doet hij het een beetje voorkomen.
Daar verzet ik mij heftig tegen.

         De heer SCHUUR (PvdA): Voorzitter! Als aanvulling op wat de heer Veenstra zei, merk ik op dat
ik begrijp wat de heer Nijhuis zegt. Ik begrijp ook intentie van de heer Nijhuis. Daar ken ik hem goed
genoeg voor; ik weet dat hij het zo bedoelt. Er bestaat echter gewoon frictie tussen wat mensen
kunnen en wat ze gevraagd wordt. Daar wordt in zijn verhaal geen rekening mee gehouden, hoewel dat
wel heel erg belangrijk is.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Het gaat er nu om wie recht heeft op een
uitkering, maar de discussie gaat ook over de vraag: Hoe zorg je ervoor dat mensen aan de
maatschappij kunnen deelnemen? Ook daarbij gaat het om geld van de overheid. Het is geld dat
besteed moet worden om ervoor te zorgen dat mensen uiteindelijk misschien weer aan het werk
komen. Mijn grote bezwaar tegen alle kortingen en maatregelen die nu voorgesteld zijn, is dat ze
allemaal gericht zijn op het verminderen van het gebruik en niets doen aan het oplossen van de




INT - 1201417                                                                                        46
</p><p>problemen. Die maatregelen zorgen er niet voor dat mensen gaan participeren, want in feite behelzen
ze dat er minder mensen aan deelnemen.

        De heer NIJHUIS (VVD): Voorzitter! In dezelfde bijdrage waaruit de heer Veenstra zojuist
citeerde, heb ik het college opgeroepen er alles aan te doen om mensen naar de arbeidsmarkt toe te
leiden. Wat dat betreft, praat ik uit eigen ervaring. Ik weet dat het in heel veel gevallen mogelijk is om
mensen met een grote afstand tot de arbeidsmarkt, daar naartoe te leiden. Vaak kan dat beter dan wij
tot op heden denken. Daar verdienen wij uiteindelijk aan. Wanneer je inzet op het toe leiden van
mensen naar de arbeidsmarkt, dan is het zeker mogelijk om middelen daarvoor te gebruiken. Die
renderen namelijk uiteindelijk vanzelf. Ook hebben wij in onze algemene beschouwing gezegd: Mensen
aan de kant laten staan en eeuwig van een uitkering voorzien, is asociaal. Ik constateer een zeer
selectieve waarneming door de heer Veenstra.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik vind niet dat wij op deze manier moeten
discussiëren. Ik probeer inhoudelijk te discussiëren. Ik ben het met de heer Nijhuis eens dat ingezet
moet worden op het begeleiden van mensen naar de arbeidsmarkt. Wij moeten ons echter realiseren -
 en dat heb ik aangegeven - dat er nauwelijks geld meer is voor re-integratietrajecten. Als de heer
Nijhuis dan stelt dat het toch mogelijk is om binnen het budget mensen naar de arbeidsmarkt te
begeleiden, dan spreekt hij zichzelf tegen, vind ik.

        De heer NIJHUIS (VVD): Voorzitter! Juist op het gebied van re-integratie is in het verleden
ontzettend veel geld uitgegeven aan particuliere bureaus die er uiteindelijk niets van bakten (laat mij dat
maar heel erg plat en kort door de bocht zeggen). Als de re-integratie op een andere manier
georganiseerd wordt, kunnen de effecten veel groter zijn. Je moet heel erg goed opletten dat de zaken
waarin je geld investeert, rendement opleveren. Natuurlijk zijn wij dan de eersten die zeggen: Jongens,
dat moeten wij doen, want uiteindelijk verdienen wij het geld terug.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik vervolg mijn betoog en ga in op het volgende
onderwerp, namelijk "Het verschil maken" (overigens sluit wat ik daarover wil zeggen, aan bij de
opmerkingen die de heer Nijhuis maakte). Fundamenteel in de discussie over "Het verschil maken" is
volgens mij hoe gekomen kan worden tot een andere verdeling van taken en verantwoordelijkheden,
hoe wij dan de noodzakelijke bezuinigingen realiseren en hoe wij voorkomen dat de effecten vooral de
inwoners treffen die het al zo moeilijk hebben. Dit is de reden waarom wij hebben voorgesteld om de
voorstellen voor besparingen om te zetten in besparingstaakstellingen per maatschappelijke opgave.
Dan kun je namelijk ook nadenken over wat een andere verdeling van taken en verantwoordelijkheden
opbrengt. Dat hoeft overigens niet altijd meteen een besparing te betekenen op de uitkeringen of
voorzieningen, want de oplossing kan ook zitten in een andere manier van werken. Tevens is er dan de
mogelijkheid om een relatie te leggen tussen de bezuinigingen en het effect op de ambtelijke organisatie
(hier kom ik zo meteen op terug). Eén van de grote problemen is dat wij nu op geen enkele manier
kunnen zien wat de relatie is tussen alle besparingsvoorstellen en de organisatie.
        Het college zei over mijn voorstel om een besparingstaakstelling te formuleren per
maatschappelijke opgave: Dat voorstel komt terug op eerdere besluiten en het is onduidelijk, terwijl de
provincie dan bovendien geen goedkeuring zal geven aan de begroting. Mijn vraag is wat het zegt over
de bezuinigingsvoorstellen met betrekking tot de eigen organisatie.
        Als ik het goed begrepen heb, is wat ik voorgesteld heb over het formuleren van een
besparingstaakstelling per maatschappelijke opgave, al de werkwijze die gevolgd wordt voor het
opstellen van de Sociale Agenda 2020. Daarbij gaat het er immers om hoeveel geld er beschikbaar is
en wat de beste oplossingsrichtingen zijn. Dat zijn niet direct de oplossingsrichtingen die in de
bezuinigingsvoorstellen staan. Ik heb bedoeld de voorstellen voor horizontaal werken, en niet voor de
eigen organisatie.
        De wethouder zei dat de provincie niet akkoord gaat met een begroting waarin
besparingstaakstellingen zijn opgenomen. Ik stel bewust dat het totaal van de besparingen hetzelfde
moet zijn als het totaal waar het college van uitgaat (dat is ook mijn uitgangspunt). Verder ben ik van
mening dat veel voorstellen die er nu liggen, ook niet hard zijn. Ze worden wel genoemd, maar de vraag
is in hoeverre ze gerealiseerd worden.
        Naar aanleiding van de Sociale Agenda 2020 hebben wij voorgesteld om voor meer
programma's agenda's op te stellen. Dit voorstel is door het college omarmd. Ik ben van mening dat
een deel van de voorstellen die de andere fracties hebben gedaan, eigenlijk in dergelijke agenda’s
passen. Voor alles wat wij gaan doen, moeten wij m.i. de vorm kiezen die gevolgd is voor de Sociale



INT - 1201417                                                                                           47
</p><p>Agenda 2020. U vroeg zich terecht af, voorzitter, of er ook agenda's moeten worden opgesteld voor
deelprogramma's. Ik ben van mening dat wij met de raad moeten bespreken of dat nodig is. Sommige
fracties hebben verzocht om deelprogramma's, althans zo heb ik dat geïnterpreteerd.
         Ik vind het heel moeilijk dat ik totaal niet kan zeggen wat de relatie is tussen de bezuinigingen
en de krimp van de organisatie. Daarom meen ik dat wij ons moeten focussen op een andere verdeling
van taken en verantwoordelijkheden. Dat is ook één van de aspecten van mijn voorstel (amendement L)
om de bezuinigingstaakstellingen te formuleren als besparingstaakstellingen per maatschappelijke
opgave. Daarmee wordt namelijk meteen de relatie gelegd met de ambtelijke organisatie. Als er dan
verschuivingen komen in taken en verantwoordelijkheden, dan leidt dat automatisch tot inzicht in de
mogelijkheden waar verder bespaard zou kunnen worden.
         In het vorige punt zit volgens mij wel een geweldig dilemma, namelijk of het nog op een ver-
antwoorde manier mogelijk is om de organisatie verder te laten krimpen. Natuurlijk begrijp ik dat dat
problemen kan opleveren. Daarom meen ik dat het beter is om afspraken te maken over het tempo
waarin de organisatie krimpt, in plaats van dat niet zichtbaar te maken. Ik pleit er dan ook absoluut
voor om de relaties zichtbaar te maken tussen alle bezuinigingsvoorstellen en de ambtelijke organisatie.
         Dan de vraag op wie wij ons richten. Dat is een heel bijzondere vraag. U zei, voorzitter: Die
vraag is eigenlijk overbodig; wat schieten wij op met een discussie over de stad in balans en sociale
stijging? Ik heb dit onderwerp bewust aan de orde gesteld op de manier waarop ik dat gedaan heb. Ook
wil ik dit nog een keer nadrukkelijk aan de orde stellen. Ik constateer namelijk dat er heel veel
misverstanden zijn over sociale stijging. In dit verband citeer ik een tekst die ik al eens eerder heb
voorgelezen, namelijk de visie van Almelo op wat een aantrekkelijke stad is. In de begroting staat
daarover:
              "Almelo wil een aantrekkelijke stad zijn voor kansrijke bevolkingsgroepen. Almelo weet
              deze groepen vast te houden en aan te trekken door het stimuleren van de economie"
              (enz.).
         Dit is de missie van het programma Aantrekkelijke stad, en daarom heb ik die genoemd. U zei
terecht, voorzitter, dat het college er voor iedereen wil zijn, zowel voor de eigen bevolking als voor
anderen. Dan moet je wat ik zojuist citeerde, niet als missie formuleren. Ook moet je dan niet steeds
roepen dat Almelo een stad in balans moet zijn. Ik meen namelijk dat wij dan ook eens moeten zeggen
wat die balans is.
         Voorts is gezegd dat de gemeente moet gaan voor sociale stijging. De VVD-fractie koppelt het
aantrekken van kansrijke mensen aan sociale stijging. In het programma worden deze twee aspecten
uitdrukkelijk uit elkaar getrokken.

       De heer NIJHUIS (VVD): Voorzitter! Ik ben heel benieuwd naar de tekstanalyse van de heer
Veenstra, want dat is absoluut niet wat wij zeggen. Het is én én.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Het is mogelijk dat ik het verkeerd
geïnterpreteerd heb, maar de VVD-fractie heeft letterlijk gezegd:
             "Onze fractie vraagt zich dan ook af hoe het komt dat we er nog onvoldoende in slagen om
             kansrijke inwoners aan te trekken c.q. vast te houden en de zo gewenste sociale stijging in
             voldoende mate op gang te brengen."
        Ik heb daarin een koppeling gezien. Volgens mij gaat het met name om hoe wij sociale stijging
bewerkstelligen en hoe wij dat volgen.

       De heer NIJHUIS (VVD): Voorzitter! In zijn antwoord geeft de heer Veenstra al aan dat wij drie
elementen zien die noodzakelijk zijn om op sociaal gebied de stad in balans te krijgen. Dat zijn de drie
elementen die hij genoemd heeft.

         De heer VEENSTRA (ChristenUnie): Voorzitter! Ik meen dat het goed is om te zeggen: Sociale
stijging is datgene wat wij kunnen beïnvloeden, en daarvoor kiezen wij.

       De heer NIJHUIS (VVD): Voorzitter! Daar zijn wij het uitdrukkelijk niet mee eens. Wij zien ze
graag alle drie, in combinatie. Wij denken namelijk dat dat werkt. Eén eruit pikken, werkt niet.

      De heer VEENSTRA (ChristenUnie): Voorzitter! Ik zeg ook niet: Eén eruit pikken. Wel vraag ik
waar wij op moeten focussen. Wat is de richting van ons beleid?

        De VOORZITTER: Mijnheer Veenstra! U heeft nog ruim een minuut spreektijd.



INT - 1201417                                                                                          48
</p><p>         De heer VEENSTRA (ChristenUnie): Voorzitter! Dan wil ik nog iets zeggen over een zaak waar
wij verder op zouden kunnen besparen. Weliswaar zijn er al heel veel dingen genoemd waarop bespaard
zou kunnen worden, maar ik wijs nog een keer op de motie (I) die ik in de eerste instantie heb
ingediend met betrekking tot het Werkplein. Ik heb die motie aangepast. Ik zal straks aangeven wat de
motivatie daarvoor is.
         Vervolgens een opmerking over de gemeentelijke heffingen. De ChristenUnie-fractie vindt dat de
gemeente met name over de heffingen heel transparant moet zijn richting de burgers. Dit betekent dat
de gemeente zichtbaar moet maken dat ze een verlaging van de afvalstoffenheffing en een verhoging
van de ozb doorvoert. Wij moeten niet roepen dat de lasten gelijk blijven voor de burgers, maar moeten
eerlijk en transparant zijn. Wat ons betreft, is een ozb-stijging alleen verantwoord als de opbrengst
daarvan ingezet wordt voor de onderkant van de samenleving, dus voor de sociale stijgers, zoals ik ze
heb genoemd.
         Verder is ons idee dat de toeristenbelasting samengevoegd moet worden met andere
belastingen. In Almelo kennen wij een reclamebelasting en een evenementenfonds. De verschillende
zaken zijn allemaal gericht op dezelfde activiteiten. Waarom proberen wij niet die bij elkaar te brengen,
in plaats van zoiets gekunstelds als een toeristenbelasting in te voeren?
         Dan dien ik nu, zoals ik al aangekondigd heb, een aangepaste versie in van mijn motie (I) over
het Werkplein.

        De VOORZITTER: Uw 12 minuten spreektijd zijn inmiddels voorbij.

        De heer VEENSTRA (ChristenUnie): Ik ben klaar met mijn betoog, voorzitter.
        Motie I had ik in eerste instantie gekoppeld aan de vraag of het UWV wel of niet zou intrekken
bij het Werkplein. Bij nader inzien constateer ik dat het participatiebudget gigantisch naar beneden gaat.
Volgens mij heeft dat consequenties voor zowel de hoeveelheid mensen die nog beschikbaar is voor het
UWV in het Werkplein, als de hoeveelheid mensen die voor de gemeente nog beschikbaar is. De
aangepaste versie van motie I luidt:

                "Motie I1.
                Werkplein
                De raad van de gemeente Almelo, in vergadering bijeen op 8 en 10 november 2011,
                Constateert:
                - Dat de raad ermee heeft ingestemd om het Werkplein te vestigen in het Twenthe
                   Centrum;
                - Dat de huisvestingslasten voor de gemeente hierdoor aanzienlijk hoger worden;
                - Dat het rijk de middelen voor participatie de komende jaren met zo'n 75%
                   vermindert;
                - Dat de raad heeft besloten om een nieuw stadhuis te bouwen vlak naast het
                   Twenthe Centrum.
                Overweegt:
                - Dat het UWV heeft aangedrongen op vestiging van het Werkplein in het Twenthe
                   Centrum;
                - Dat de risico's voor de gemeente als hoofdhuurder zijn;
                - Dat, vanwege het lagere participatiebudget, het aantal mensen dat zich in het
                   Werkplein zal vestigen mogelijk aanzienlijk lager is dan de 120 (UWV: 48, gemeente:
                   62, anderen: 10), waarvan nu wordt uitgegaan;
                - Dat niet zeker is dat dit ook leidt tot lagere huisvestingskosten;
                - Dat het mogelijk (aanzienlijk) goedkoper is om het Werkplein in het nieuwe stadhuis
                   onder te brengen;
                - Dat het onlogisch is dat een gemeentelijke dienst zich pal naast het nieuwe stadhuis
                   vestigt;
                - Dat het daarom gewenst is dat het college de vestiging van het Werkplein in het
                   Twenthe Centrum heroverweegt.
                Besluit:
                - Het college op te dragen om de vestiging van het werkplein in het Twenthe Centrum
                   te heroverwegen en daartoe uiterlijk in maart 2012 met een voorstel te komen;
                - In dit voorstel aan te geven:




INT - 1201417                                                                                          49
</p><p>                   -  Het aantal mensen, zowel van de gemeente, het UWV als van derden, dat zich
                      daadwerkelijk in het Werkplein zal vestigen;
                   - Een vergelijking van de kosten tussen vestiging van het Werkplein in het Twenthe
                      Centrum en in het nieuwe stadhuis;
                   - De (financiële) consequenties van het opzeggen van het contract met het
                      Twenthe Centrum.
                En gaat over tot de orde van de dag."

        Ik wijs er nog op dat het in de motie gaat om forse kosten, namelijk € 600.000,--
huisvestingskosten per jaar, alleen voor de gemeente, terwijl er aan dekking slechts ongeveer
€ 290.000,-- vanuit de gemeentelijke organisatie in de eigen huisvesting zit. Gelet hierop, meen ik dat
het duidelijk is waarom ik in de motie vraag om met een voorstel hierover te komen.

        De VOORZITTER: Dank u zeer, mijnheer Veenstra.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik vind dit … (Zonder microfoon.)

        De VOORZITTER: Ik maakte een gebaar, mijnheer Veenstra, waarmee ik u uitnodigde om uw
verhaal af te ronden. Volgens mij is het goed als de voorzitter dat doet.
        Dames en heren! Ik stel vast dat motie I is vervangen door motie I1.

        De heer GERRITSEN (D66): Mevrouw de voorzitter! Onze fractie kwam afgelopen dinsdag tot
de conclusie dat alle fracties in de gemeenteraad van Almelo hebben aangegeven dat de gemeente in
financieel zwaar weer zit. Een mooie conclusie. Maar wat dan? Ik richt mij nu tot de collegefracties,
want zoals ik het proef, zal de begroting zoals die voorligt, ook op deze manier worden vastgesteld, op
een aantal details na.
        Wij kiezen ervoor om extra bezuinigingen door te voeren, maar wij kiezen er ook voor om de
inkomsten te verhogen. Met andere woorden: de fractie van D66 probeert het probleem - de schuld en
de risico's - aan te pakken. Wij hebben ideeën aangedragen die het college de komende tijd zou kunnen
ontwikkelen en bij de voorjaarsnota kan presenteren. Dat is de bedoeling van het amendement en de
moties die wij hebben ingediend.
        De collegefracties hebben zich, zoals ik zojuist zei, gericht op details. Maar details zetten op dit
moment geen zoden aan de dijk. Ik had graag willen zien dat er concrete verzoeken waren gedaan aan
het college om doelgericht de nu bestaande schuld te verlagen. Want laten we eerlijk zijn: wij komen
elkaar bij de voorjaarsnota weer tegen. Op dat moment zullen wij de keuzes moeten maken.

        De heer NIJHUIS (VVD): Voorzitter! Volgens mij heeft de raad bezuinigingsvoorstellen van het
college aangenomen ter waarde van € 9 miljoen. Ik meen zelfs dat de fractie van D66 toen mede
voorgesteld heeft om daar € 0,5 miljoen van af te halen. De oorspronkelijke collegevoorstellen waren
gericht op een bezuiniging van € 12 miljoen, als ik het mij goed herinner. Volgens mij wordt er
behoorlijk bezuinigd, binnen datgene wat politiek haalbaar is. De heer Gerritsen moet niet het beeld
schetsen dat er niets wordt gedaan, alsof er alleen maar sprake is van pappen en nathouden.

        De heer GERRITSEN (D66): Dat hoort u mij niet zeggen.

        De heer NIJHUIS (VVD): Nee, maar dat heeft u in eerdere bijdragen wel iedere keer betoogd.

        De heer GERRITSEN (D66): Mijnheer Nijhuis! Dan heeft u het verkeerd begrepen of heeft u niet
goed geluisterd. Wij geven aan dat er nu een bepaalde situatie is. Wat dat betreft, spreken de getallen
in de begroting voor zich zelf. Uit de begroting blijkt dat de schuld van de gemeente op dit moment
groeiende is. Die schuld blijft groeien als wij nu geen concrete keuzes maken om de stijging een halt
toe te roepen. Dat heeft niets te maken met de voorjaarsnota 2011, waarbij wij keuzes hebben
gemaakt voor bezuinigingen ter waarde van € 9,5 miljoen. Wij waren er toen zelfs voorstander van om
te zeggen: Moet je luisteren, er is ons meer geboden door het college; waarom pakken wij dat niet aan?
Wij wisten namelijk dat wij nu weer voor het dilemma zouden komen te staan.

        De heer NIJHUIS (VVD): Waarom heeft u toen dan een bezuiniging van € 0,5 miljoen mee
afgestemd? Los daarvan, hebben wij bij de voorjaarsnotabehandeling samen geconstateerd dat de
financiële situatie van de gemeente niet denderend was en dat de bezuinigingen die toen voorlagen, in



INT - 1201417                                                                                            50
</p><p>eerste instantie haalbaar waren. Laat ons eerlijk zijn: het is niet zo moeilijk om puur boekhoudkundig te
bezuinigen. Overigens waardeer ik het zeer dat de fractie van D66 met een tegenbegroting is gekomen,
laat dat helder zijn. Maar nogmaals, het is niet zo moeilijk om puur boekhoudkundig te bezuinigen. U
hebt, dacht ik, het college ooit verweten dat het boekhoudkundig bezig was (dat is iets dat wij in het
verleden ook al eens genoemd hebben). Dat is voor een deel juist, en ik neem aan dat iedereen dat
erkent. Puur boekhoudkundig is het best gemakkelijk om € 10 miljoen op te hoesten. Dat is niet zo
moeilijk en dat zouden wij ook kunnen, maar als wij vervolgens dat bedrag invullen, komen wij echt aan
de harde kant van het gemeentelijk bedrijf en aan de bezuinigingen.

        De heer GERRITSEN (D66): Mijnheer Nijhuis! De situatie waar wij nu voor staan, komen wij bij
de voorjaarsnota weer tegen. Immers, wij lossen het probleem van het woningbouwprogramma niet op
en wij lossen het probleem van de grondvoorraad niet op. Wij zullen geld moeten reserveren of
genereren om de schuld naar beneden te brengen. Hoe wilt u het anders doen? Bent u met mij van
mening dat wij anders straks weer moeten bezuinigen en dat wij het dan niet redden met bedragen van
5 ton of € 1 miljoen? Wij zullen dan meer moeten bezuinigen. Bent u dat met mij eens? Of heeft u - en
dat is de vraag die ik in mijn bijdrage wilde stellen - andere oplossingen? Zo ja, dan ben ik daar heel
benieuwd naar. Geef nu alvast aan wat het college bij de voorjaarsnota moet doen

        De heer NIJHUIS (VVD): U draait nu de zaak om. Wij hebben geen begroting ingediend met
€ 10 miljoen minder. Ook hebben wij niet gezegd: Ga € 3 miljoen bezuinigen, college, en zoek het
lekker zelf uit.

        De heer GERRITSEN (D66): Ik stelde een andere vraag…

         De heer NIJHUIS (VVD): U noemt voor personeel een bezuinigingsbedrag van € 5 miljoen.
Daarover moet ik zeggen dat dat niet realistisch is. Kortweg, voorzitter, de fractie van D66 draait nu de
rollen om. Ik vind dat wij dat maar niet moeten doen. Ik heb al gezegd dat het voorstel van de D66-
fractie een boekhoudkundig voorstel is. Ik waardeer het dat die fractie de moeite genomen heeft om
daarmee te komen, maar tegelijkertijd constateer ik (dat heb ik eerder al eens gezegd) dat er echt
onmogelijke bezuinigingen in zitten, waaronder die van € 5 miljoen op personeel. Daarvan weet ook de
fractie van D66 dat die bezuiniging niet zomaar realiseerbaar is. Boekhoudkundig klopt het, maar er zit
een hele wereld achter.

         De heer GERRITSEN (D66): Mijnheer Nijhuis! Ik kom even terug op de vraag die ik u zojuist
stelde. Bent u het met mij eens dat wij straks, bij de voorjaarsnota 2012, wat moeten doen aan de
schuld die er staat, het oplopen van de schuld, de rentekosten, de woningbouwprogramma's en alle
risico's en open einden die er nog zijn? Hoe denkt de VVD-fractie dit op te lossen? U zegt nu tegen het
college: Ga er maar eens over nadenken en kom met voorstellen. Dat is hetzelfde als wat u onze fractie
verwijt.

         De heer NIJHUIS (VVD): Nee. Het is prachtig - dat blijkt ook nu weer - om met u te
discussiëren. U probeert altijd beelden te schetsen die niet juist zijn. Wij hebben gezegd: Het moment
waarop wij de voorjaarsnota behandelen, is een belangrijk ijkmoment om te constateren of datgene wat
wij met elkaar hebben afgesproken, ook gerealiseerd wordt. Natuurlijk weten wij dat daar een aantal
enorme risico's in zit. Vandaar dat wij gezegd hebben dat er sprake moet zijn van risicobeheersing,
maar ook risicoberekening. Risicomanagement is in dezen vreselijk belangrijk. Ik heb ook gezegd dat ik
niet uitsluit dat verdere bezuinigingen nodig zijn. Dan moeten wij echter wel heel goed gaan bekijken
waar. Dan kom ik er niet mee weg door te zeggen: Wij gaan € 3 miljoen bezuinigen op alle
beleidsvelden en wij gaan € 5 miljoen bezuinigen op personeel. Dat kan dan niet meer, want dan
moeten wij heel specifiek met elkaar aangeven in deze raad waar de bezuinigingen gezocht moeten
worden. Dat er mogelijk verdere bezuinigingen komen, ja. Dat heb ik ook steeds beweerd.

         De heer VEENSTRA (ChristenUnie): Voorzitter! Ik heb nog steeds niet begrepen hoe volgens het
voorstel van de fractie van D66 de schuld van de gemeente naar beneden gaat. Volgens mij zegt de
fractie van D66: Wij gaan eerst een aantal jaren sparen, en dan pas zijn wij in staat eventueel verliezen
op Waterrijk te nemen. Hoe gaat dan tot die tijd de schuld naar beneden? Ik zie dat niet.

        De heer GERRITSEN (D66): Op dit moment is de situatie zo, mijnheer Veenstra - dat weet u zelf
ook - dat de schuld die er is, wordt opgeplust met de rente daarover. Er wordt niet afgeschreven, dus



INT - 1201417                                                                                         51
</p><p>de totale grondwaarde groeit. De fractie van D66 zegt dat wij als gemeente ervoor moeten zorgen dat
wij de rentekosten kunnen beheersen. Wij moeten er dus voor zorgen dat wij die in de pocket hebben,
zodat de schuld niet oploopt. Ook moeten wij een X-bedrag sparen om in de komende tijd Waterrijk
(het voorbeeld dat u noemde) af te waarderen. Overigens zegt de fractie van D66 daar het volgende
bij. Stel dat wij over 10 of 15 jaar het geluk hebben dat de ontwikkelingen weer aantrekken (tenslotte
wordt hier wel eens gesuggereerd dat het vanzelf goed komt), dan hebben wij een boekwaardewinst.
Maar wat op dit moment gebeurt, is dat de schuld alleen maar oploopt. Laat ons eerlijk zijn, mijnheer
Veenstra. Onze fractie kiest ergens voor, waardoor de schuld naar beneden zal gaan en de kosten niet
omhoog gaan.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik vind dit een lastige discussie. De heer
Gerritsen stelt dat de rentekosten volgens zijn voorstel omlaag gaan.

        De heer GERRITSEN (D66): Nee.

        De heer VEENSTRA (ChristenUnie): Het gaat mij om de schuld. Hoe gaat volgens uw voorstel
de schuld van de gemeente echt naar beneden? Ik begrijp dat niet. Volgens mij zegt u met uw voorstel
dat de gemeente geld moet sparen de komende jaren, want u wilt dat eerst het geld verdiend wordt.
Daarmee zegt u dat zodra wij het bedrag van € 45 miljoen hebben - dan zijn wij 15 jaar verder - de
grond in Waterrijk verkocht kan worden. Dan pas kan het verlies genomen worden. Volgens mij gaat
daardoor volgens uw voorstel de schuld pas over 15 jaar naar beneden.

        De heer GERRITSEN (D66): Voorzitter! In onze motie hierover (S) kan men zien dat wij ervoor
willen zorgen dat op dat moment afwaardering mogelijk is. In een andere motie hebben wij gezegd:
Luister, zorg ook dat je naar een alternatief gaat kijken. Dat betekent dat je de grondwaarde afbouwt
en dat je een stuk gaat gebruiken voor een alternatief. Dat zou Energierijk kunnen zijn, maar als er
betere voorstellen zijn, dan hoor ik die graag. Met andere woorden (en laat mij daarvoor de term van de
VVD-fractie gebruiken): er kan ook verdiencapaciteit gecreëerd worden. Dat bedoelen wij te zeggen.
Daarmee loopt de schuld niet op. De kosten van de schuld zullen straks bij de voorjaarsnota, mijnheer
Veenstra (uw fractie is dan één van de fracties die daarbij zit), ergens vandaan gehaald moeten
worden.

      De VOORZITTER: Dames en heren! Ik constateer dat de heer Veenstra wil reageren op deze
woorden, maar ik vraag de heer Gerritsen om verder te gaan met zijn betoog.

         De heer GERRITSEN (D66): Voorzitter! Wat iedereen ervan vindt, kan men vinden, maar onze
fractie is van mening dat ze duidelijk heeft aangegeven waar het probleem zit en dat ze met concrete
voorstellen daarvoor is gekomen. Maar goed, iedereen wil natuurlijk proberen die af te schieten, maar
dat lukt vandaag niet!
         De bestaande schuld van de gemeente zal verminderen volgens de keuzes die de fractie van
D66 op dit moment maakt. Daarbij zorgen die keuzes ook voor nieuwe inkomsten. Wat ik vandaag
echter merkte, natuurlijk ook in de wandelgangen, is dat gezegd wordt: De fractie van D66 wil de ozb
verhogen. Onze fractie wil dat de ozb verhoogd wordt, mits ook haar andere voorstellen worden
aangenomen.

       Mevrouw TEN SELDAM-EVEN (CDA): De heer Gerritsen gaf aan dat de fractie van D66 met
concrete voorstellen is gekomen. Ik had daar ook om gevraagd. Het ligt wellicht aan mij, maar ik heb
geen concrete voorstellen gehoord. Vandaar dat ik vraag: Mijnheer Gerritsen, kunt u ze even herhalen
voor mij?

       De heer GERRITSEN (D66): Voorzitter! Geeft u mij de gelegenheid om alle door ons ingediende
moties voor te lezen en te kijken of mevrouw Ten Seldam ze dan snapt?!
       Mevrouw Ten Seldam, onze voorstellen staan in onze moties. Die zijn concreet. Als u ze niet op
die manier kunt uitleggen, tja, dan kan ik daar ook niets aan doen.

       Mevrouw TEN SELDAM-EVEN (CDA): Ik begrijp dat u daar niets aan kunt doen. Als uw moties
in uw perceptie concreet zijn, dan hebben wij duidelijk een verschil van mening over wat concreet is.
Maar ik zal het verschil vasthouden.




INT - 1201417                                                                                       52
</p><p>        De heer GERRITSEN (D66): Het verschil vasthouden? Het verschil maken, wilt u toch altijd?!

       De VOORZITTER: Dames en heren! Ik vraag om de heer Gerritsen verder te laten gaan met zijn
inbreng.

        De heer SCHUUR (PvdA): Voorzitter!

        De VOORZITTER: Wilt u nog op dit laatste punt reageren?

        De heer SCHUUR (PvdA): Ja, voorzitter.

        De VOORZITTER: Kan dat kort?

        De heer SCHUUR (PvdA): Ja. Ik ben altijd kort!

        De VOORZITTER: Oké.

       De heer SCHUUR (PvdA): Voorzitter! Wat mij tegen de borst stuit, ondanks de ongetwijfeld
goede intenties van de D66-fractie, is dat door die fractie heel simpel gezegd wordt: Gaan wij sparen of
gaan wij lenen? Daarbij wordt in feite niet tegen de bevolking gezegd: Houd er even rekening mee,
maar als de D66-plannen uitgevoerd worden, dan is dat geen kwestie meer van de broekriem aanhalen,
want die wordt bijna doormidden gezaagd door deze fractie, omdat er dan geen dubbeltje meer te
besteden blijft. € 3 miljoen wilt u bezuinigen op alle programma's, mijnheer Gerritsen, € 3 miljoen.

        De heer GERRITSEN (D66): Er zijn zes programma's. Dat zou € 500.000,-- per programma zijn.
Ik vertel u dat wij volgend jaar bij de behandeling van de voorjaarsnota weer dezelfde discussie zullen
voeren als nu. Dan zal de broekriem die wij nu naar uw mening misschien half doorsnijden, afgeknipt
worden. Dat zou veel erger zijn.

        De VOORZITTER: Ik stel voor om nu echt verder te gaan.

       De heer GERRITSEN (D66): Voorzitter! Ik wil graag een reactie geven op de antwoorden van
een aantal collegeleden.

        De heer SCHUUR (PvdA): Voorzitter! Ik heb het reeds eerder…

       De VOORZITTER: Mijnheer Schuur! Ik had u niet het woord gegeven. U kunt nu met één zin
reageren op die laatste opmerking van de heer Gerritsen. Ik snap dat u dat wilt, maar daarna gaan wij
echt verder.

        De heer SCHUUR (PvdA): Oké, dank u wel. Wat ik wil zeggen, is dat de fractie van de PvdA - ik
herhaal dat - niet zal meewerken aan het kapot bezuinigen van deze stad.

         De heer GERRITSEN (D66): Voorzitter! U ging in uw antwoord in op onze motie V over het
houden van een peiling onder Almeloërs. U gaf aan: Dat wordt veel te ingewikkeld. De burgers moeten
echter wel alle gevolgen dragen van de begroting - oftewel van de toekomstige bezuinigingen en extra
bezuinigingen - maar het is heel ingewikkeld voor de Almeloërs om daar wat van te vinden. Dat gaf u
eigenlijk aan. Moet ik daaruit concluderen dat u zegt: Wij vragen niets aan de burgers van Almelo. Of
zegt u: Nee, ik vind het houden van een peiling wel een goed idee, maar ik zou daar graag een andere
vorm aan willen geven dan in de motie wordt geschetst, want ik vind het op dit soort beslismomenten
belangrijk dat de burgers er wat van kunnen vinden. Immers, bij de keuze van een nieuwe
burgemeester werden de burgers ook gevraagd om hun mening. Dat geldt ook voor het nieuwe
stadhuis. Wij lopen met de raad op straat, en ook dan vragen wij de burgers om hun mening. Maar op
het moment dat de burgers in hun portemonnee worden geraakt, gaan wij hun mening niet meer
vragen, want dan wordt het ingewikkeld. Het huishoudboekje behoort op orde te zijn, en dat kunnen de
burgers thuis ook.




INT - 1201417                                                                                        53
</p><p>       De heer SCHUUR (PvdA): Voorzitter! Ik vind het wel heel apart, hoor, wat de fractie van D66
nu vraagt. Die zegt: Wij willen dat € 3 miljoen bezuinigd wordt; wij zeggen niet waarop, maar vragen
wel aan de burgers of ze dat een goed idee vinden. Volgens mij is dat niet erg handig.

        De heer VEENSTRA (ChristenUnie): De heer Gerritsen heeft een aantal concrete voorbeelden
genoemd. Dat is volgens mij wat anders dan vragen: Wilt u sparen of lenen? Dan zou je de vraag
moeten stellen: Wilt u voor het kopen van een huis sparen of lenen? Dan weet ik het antwoord wel,
want dan zegt iedereen: Wij gaan lenen. Dit is dus een heel simpele discussie. Het gaat echter om de
grootte van de bedragen en de consequenties daarvan. Volgens mij heeft de heer Gerritsen ook niet
gespaard voor zijn huis, maar heeft hij daarvoor geleend!

        De heer GERRITSEN (D66): Daar wil ik met u nog wel even over praten!

        De VOORZITTER: Volgens mij is het een beetje lastig om dit soort voorbeelden te gebruiken.

        Mevrouw TEN SELDAM-EVEN (CDA): De indruk wordt gewekt dat er maar één medium is om
meningen te peilen en te vernemen wat de burger van iets vindt. Er zijn echter vele manieren om de
burgers te laten aangeven wat zij van iets vinden. Die manieren zijn ook ingebouwd in de procedures.
Wij hebben hier niet voor niets inspraakmomenten en inspraakavonden. Ook gaat, zoals de heer
Gerritsen zelf zei, de raad de straat op. Ik ben dan ook van mening dat er volop gelegenheden zijn om
de mening van de burgers te vernemen. Het middel dat de fractie van D66 nu voorstelt, heeft in ieder
geval niet onze voorkeur.

        De heer GERRITSEN (D66): Voorzitter! Misschien bent u bereid om straks antwoord te geven op
onze opmerkingen over dit onderwerp. Wellicht heeft u nog goede suggesties of ideeën.
        De heer Schouten gaf met betrekking tot de ontwikkeling van Waterrijk aan dat de raad bij de
behandeling van het bestemmingsplan voor dat gebied kan besluiten tot go or not go. Dat kan de raad
inderdaad besluiten, maar dan is het probleem nog niet opgelost, laat ons dat even goed realiseren met
elkaar. Daarom proberen wij om door middel van onze motie S te bereiken dat de ontwikkeling van
Waterrijk geparkeerd wordt voor een jaar en dat naar alternatieven gekeken wordt. Ik denk dat de
wethouder dit heel sympathiek vindt, want ik ga ervan uit dat wanneer hij bij de behandeling van het
bestemmingsplan een no go krijgt, de heer Van Marle niet meer op de deur van wethouder Schouten
klopt, maar wel heel hard naar binnen zal lopen! Dan hebben wij namelijk echt een heel groot probleem
met ons allen. Dan zitten wij hier weer.

        De heer BRAND (CDA): Mijnheer Gerritsen! U spreekt nu voor de tweede keer over het vinden
van alternatieven voor Waterrijk, alsof dat de uitvinding van de D66-fractie is.

        De heer GERRITSEN (D66): Dat zeg ik niet.

        De heer BRAND (CDA): Ik heb de wethouder vanmorgen horen zeggen dat bij de voorjaarsnota
een integraal stuk aan de orde zal komen. Daarvoor wordt ook gekeken naar alternatieven voor de
ontwikkeling van Waterrijk. Ik begrijp uw opmerkingen over dit onderwerp niet. Of u zegt dat u geen
enkele fiducie heeft in de uitspraak van de wethouder, of u hebt het hem niet horen zeggen.

        De heer GERRITSEN (D66): Daarvoor zei ik nog wat anders, namelijk dat wij een probleem
hebben bij een no go bij de behandeling van het bestemmingsplan Waterrijk. Dan kunnen wij wel naar
alternatieven gaan kijken, maar als er helemaal niets gebeurt, hebben wij een probleem. Daarom zeggen
wij dat het komende jaar gebruikt moet worden voor wat in motie S omschreven staat over Waterrijk.
        Voorzitter! Dan de kinderboerderij. Daarover wil ik heel kort en duidelijk zijn. Ik ga ervan uit dat
de heer Schouten het voor elkaar gaat krijgen de kinderboerderij open te houden en dat hij de
gesprekken daarover blijft voeren die hij nu al voert, met een goed resultaat. Daarbij wil ik nog wel
zeggen dat wij bij de afgelopen voorjaarsnotabehandeling een motie met betrekking tot de
stadsboerderij hebben ingediend die vergelijkbaar is met de motie daarover (VA) die nu op tafel ligt,
maar die werd toen afgestemd. Ik ben benieuwd wat straks het resultaat zal zijn.
        De heer Van Woudenbergh zei het perfect: Sparen in plaats van lenen, en een gefaseerde
uitvoering van het binnenstadsplan. Chapeau! Dat is gewoon een heel goede beantwoording. Hij weet
wat hij wil en wil de keuzes daarin maken.




INT - 1201417                                                                                            54
</p><p>        Dan de heer Andela. Nu slaak ik een diepe zucht! Met betrekking tot de bezuinigingen op de
gemeentelijke organisatie roept hij gelijk: 100 fte's. Waarom? Ik heb geen idee. Bezuinigen op de
gemeentelijke organisatie betreft niet alleen het personeel. Dat neemt overigens niet weg dat ten
aanzien daarvan wel keuzes gemaakt kunnen worden, laat dat even duidelijk zijn. Ook kunnen wij in
sommige situaties niet weglopen voor gedwongen ontslag. Echter, de heer Andela weet dat wanneer
de druk binnen een organisatie wordt verhoogd, er voldoende mogelijkheden zijn voor de
personeelsleden zelf om maatregelen te nemen. In het bedrijfsleven werkt het ook zo wanneer de druk
hoog wordt. Daarbij kijk ik naar Philips, want dat is een goed voorbeeld. Philips heeft aangekondigd dat
in Nederland 1400 mensen ontslagen worden. Meer dan de helft van hen heeft al een nieuwe plek.
Onthoud dat, meer dan de helft heeft al nieuw werk.
        Nogmaals: het gaat bij de bezuinigingen op de gemeentelijke organisatie niet alleen om het
personeel. Dat vind ik een te-kort-door-de-bochtredenatie. Het gaat namelijk ook om de wijze waarop
omgegaan wordt met de centrale inkoop, het maaien in de buitengebieden, de kwaliteitseisen die
daaraan gesteld worden, wat wij daarvoor willen betalen en of wij als gemeente een en ander nog zelf
willen doen. Dit betekent dat er keuzes gemaakt kunnen worden.

        Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Ik heb de heer Gerritsen nu een aantal
voorbeelden horen aanstippen. Hij noemt zaken vanuit zowel de programma's, waar volgens hem voor
€ 3 miljoen uit geschrapt moet worden, als de ambtelijke organisatie, met volgens hem een taakstelling
van € 5 miljoen die vooral in de bedrijfsvoering gezocht moet worden. Als de heer Gerritsen de
bezuinigingsmogelijkheden niet zoekt in de formatie, dan is mijn reactie: Tja, wij weten allebei dat wij
geen € 5 miljoen bij elkaar sprokkelen door wat minder paperclips en elastiekjes te gaan gebruiken.

        De heer GERRITSEN (D66): Goed, u praat nu over paperclips en elastiekjes. Laat mij er echter
van uitgaan dat u een beetje kennis van zaken heeft. Dan weet u dat bepaalde dingen wel gerealiseerd
kunnen worden en dat de genoemde bedragen wel te vinden zijn. Mijn opmerkingen daarover zijn niet
zomaar uit de lucht gegrepen. Waar ik op doelde, is de opmerking van de wethouder dat het bij de
bezuiniging op de gemeentelijke organisatie alleen gaat om fte's. Die suggestie wil ik compleet
wegnemen, want er zijn veel meer mogelijkheden.

        Mevrouw TEN SELDAM-EVEN (CDA): Heeft u dan een bepaalde verhouding in uw hoofd wat
betreft de bezuinigingen op de bedrijfsvoering versus de bezuinigingen op de formatie, bijvoorbeeld de
verhouding 3 tot 5?

        De heer GERRITSEN (D66): Nee.

         Mevrouw TEN SELDAM-EVEN (CDA): De invulling daarvan is dus geheel vrij aan het college,
begrijp ik.

        De heer SCHUUR (PvdA): Ik kan mevrouw Ten Seldam daar wel een antwoord op geven. Ik
weet uit ervaring dat als je heel efficiënt te werk gaat, je misschien nog ongeveer 1 ton en 2 fte's kunt
bezuinigen op zaken als elastiekjes, paperclips, het restaurant en noem maar op. Dit betekent dat de
fractie van D66 grosso modo nog steeds 98 fte's extra wil wegbezuinigen.

        De heer GERRITSEN (D66): Ik geloof dat u niet goed geluisterd heeft naar het antwoord dat ik
zojuist gaf. U mist weer een gedeelte, dacht ik.

        De heer SCHUUR (PvdA): Nee.

       De heer GERRITSEN (D66): Volgens mij wel. Ik heb namelijk nog een andere opmerking
gemaakt, want ik heb gezegd dat ook bezuinigingen gevonden kunnen worden door anders te denken
en anders te werken. Ook daarmee kan geld gevonden worden. Het gaat niet alleen om de fte's (want
daar doelt u op).

        De heer NIJHUIS (VVD): Voorzitter! Ik heb twee dingen gehoord:
1. In de bijdrage van de CDA-fractie werd gesproken over de methode van Lean Six Sigma. De
   wethouder heeft uitdrukkelijk toegezegd dat hij die methode zal toepassen. Daarbij wil hij eventueel
   met linten uitbeelden welke weg bepaalde stukken gaan. Anders gezegd: er zal straks kritisch naar




INT - 1201417                                                                                          55
</p><p>   de organisatie worden gekeken, zoals ook een bedrijf als Philips en andere bedrijven doen, om na te
   gaan wat anders gedaan zou kunnen worden.
2. Wat betreft het uitbesteden, noemde de heer Gerritsen zojuist het groenonderhoud. Ik kan mij
   herinneren dat eerder geprobeerd is dat onderhoud uit te besteden, ik dacht aan Soweco. Dat bleek
   toen een hell of a job te zijn. Die uitbesteding is uiteindelijk zelfs niet gelukt, omdat andere
   organisaties die daar ook mee in moesten stemmen, dat niet wilden.
        Verder ben ik heel benieuwd welk deel/percentage van de totale kosten van de gemeentelijke
organisatie (afgezien van subsidies e.d.) bestemd is voor personeelskosten. Misschien kan de
wethouder dat in zijn antwoord aangeven. Volgens mij kunnen wij dan de fractie van D66 meteen
helder maken wat precies de impact van haar voorstel op dit punt is.

         De heer GERRITSEN (D66): Dank u wel, mijnheer Nijhuis, voor de steun richting de wethouder!
         Voorzitter! De heer Van Marle wil ik de volgende vraag stellen: Het is toch bestaand beleid dat
geld dat overblijft op bepaalde posten, naar de algemene reserve gaat? Vandaar dat wij in ons
amendement (N) over Almelo Sociaal en de Scoop-gelden het restant gelijk verwijzen naar de algemene
reserve. Of heb ik dat verkeerd? Zo ja, dan hoor ik dat graag van de wethouder.
         Voorts gaf de heer Van Marle aan een verhoging van de ozb wel sympathiek te vinden. Ons
voorstel in motie T is om een dergelijke verhoging te bespreken bij de behandeling van de voorjaarsnota
2012 en op te nemen in de begroting 2013. Kan de wethouder zich vinden in de omvang van de
verhoging die wij hebben voorgesteld (4,8%), of stelt hij wat anders voor?
         Vervolgens gaf de heer Van Marle aan dat de in de begroting opgenomen schuld groot is. Ja,
dat klopt, maar dat hebben wij allemaal geconstateerd. Wat de schulden betreft, maakte hij
vergelijkingen met andere gemeenten en noemde hij percentages als 183, 72 en 165. Die percentages
kloppen, maar dat zijn gemiddelden. En daar lossen wij ons probleem niet mee op. De gemeente Almelo
heeft een grote schuld. Ik ben ervan overtuigd dat de wethouder het met mij eens is dat wij daar niet
voor weg moeten lopen en dat wij straks bij de voorjaarsnota radicale keuzes moeten maken. Kan de
wethouder mij misschien vertellen wat hij verwacht bij de voorjaarsnota te moeten bezuinigen om de
kosten die hij nu al in beeld heeft voor de openeinderegelingen, te kunnen afdekken zonder - want daar
gaat het om - de algemene reserve daarmee tekort te doen? Wij hebben als raad bij de begroting 2010
aangegeven dat de algemene reserve 10% van de algemene uitkering moet zijn. Wij zien nu dat gelden
voor Soweco al worden geoormerkt in de algemene reserve. Is de wethouder het met mij eens dat de
algemene reserve dan geen 10% van de algemene uitkering meer is?
         Onze motie (O) met betrekking tot extra bezuinigen op alle beleidsterreinen is door wethouder
Van Marle met klem afgewezen. De wethouder kan dat wel doen, maar wij ontkomen daar niet aan bij
de behandeling van de voorjaarsnota. Wij zullen het geld toch ergens vandaan moeten halen? Is de
wethouder bereid om het bedrag dat de fractie van D66 heeft geopperd als extra bezuiniging op alle
beleidsterreinen, mee te nemen en te bekijken of dat te realiseren is?
         Wij hebben in de eerste termijn een motie (P) ingediend met als titel: Extra bezuinigen door
focus op samenwerking. Het dictum van deze motie hebben wij iets aangepast door daarin te stellen
dat gestreefd moet worden naar een structurele besparing van € 1,5 miljoen (in plaats van te stellen
dat die besparing gerealiseerd moet worden). Ook hebben wij in de gewijzigde versie van de motie
aangegeven dat de bevindingen besproken moeten worden bij de behandeling van de voorjaarsnota
2012. Wij gaan ervan uit dat wij nu nog meer steun voor onze motie zullen krijgen vanuit de raad! De
gewijzigde versie van de motie luidt:

                "Motie P1.
                Extra bezuinigen door focus op samenwerking
                De raad van de gemeente Almelo in vergadering bijeen op 8 en 10 november 2011;
                Constateert:
                a. dat de schuldenlast van de gemeente toeneemt;
                b. dat de inkomsten van de gemeente afnemen;
                c. dat de schuldenlast bij ongewijzigd beleid het begrotingstotaal in 2013 zal
                   overtreffen;
                d. dat het college niet voornemens is extra maatregelen te treffen;
                Overweegt:
                a. dat extra maatregelen nodig zijn om aan de verplichtingen te kunnen blijven voldoen;
                b. dat extra maatregelen nodig zijn om nieuwe ontwikkelingen mogelijk te maken;
                c. dat extra bezuinigingen, het verhogen van de inkomsten of een mix van beide
                   onontkoombaar zijn;



INT - 1201417                                                                                        56
</p><p>                Besluit:
                a. het college op te dragen een sterkere focus op samenwerking en uitbesteding van
                   werk in de regio te realiseren en daarbij te streven naar een structurele besparing van
                   € 1,5 miljoen;
                b. de bevindingen te bespreken bij de behandeling van de voorjaarsnota 2012;
                c. dit bedrag aan te wenden voor fase 2 van het binnenstadsplan;
                En gaat over tot de orde van de dag."

         De VOORZITTER: Dames en heren! Ik stel vast dat motie P is vervangen door motie P1.
         U heeft inmiddels gezien dat het tijdschema dat tijdens de schorsing was bedacht, niet haalbaar
is. Ik stel voor om verder te gaan met de vergadering. Ik vraag u om echt korte en bondige bijdragen te
leveren en om selectief en scherp te zijn op interrupties. Mijn verzoek is om alleen interrupties te
plaatsen waardoor het debat verder komt. De in het schema genoemde tijden schuiven een uur op,
maar dat begrijpt u.

         De heer CORNELISSEN (SP): Voorzitter! Het betoog dat wij in de eerste termijn hebben
gehouden, blijft voor ons voor een groot deel overeind staan. Ik zal dat betoog niet herhalen, maar wil
zo snel mogelijk doorgaan naar onze eigen moties en ons amendement, te beginnen met motie X. Die
handelt over internetaansluitingen voor minimahuishoudens. De wethouder heeft daar opnieuw over
gezegd, net zoals de vorige keer, dat zij zoekt naar oplossingen daarvoor. Ook heeft zij aangegeven dat
zij daarbij steeds een stapje verder komt. Niettemin maakt de SP-fractie zich heel veel zorgen over het
feit dat kinderen in armoede of een achterstandssituatie op moeten groeien terwijl er thuis geen
internetaansluiting is. Ik heb daar nog steeds geen concrete oplossing voor gevonden. Die heb ik ook
niet gehoord van de wethouder. Wij zullen onze motie over dit onderwerp aanhouden en de wethouder
nog een kans geven om tot een verdere uitwerking te komen. Dan kunnen wij als raad bij de
behandeling van de volgende voorjaarsnota gezamenlijk bekijken of wij tot een goede oplossing voor de
internetaansluitingen kunnen komen. Het kan niet zo zijn dat kinderen die in een bijstandsgezin
opgroeien en goed kunnen leren, thuis niet de mogelijkheden hebben om goed te leren omdat er geen
internetaansluiting is. Internet is inmiddels het belangrijkste instrument voor het maken van huiswerk
etc. Om die reden houden wij motie X aan.

      De VOORZITTER: Dames en heren! Ik stel vast dat motie X. wordt aangehouden en daarom
vanavond niet in stemming gebracht hoeft te worden.

        De heer CORNELISSEN (SP): Voorzitter! In de eerste instantie hebben wij ook een amendement
(W) ingediend met betrekking tot de Stadsboerderij Beeklust. Dit amendement trekken wij in, want het
is inmiddels vervangen door de motie (VA) die de fractie van GroenLinks heeft ingediend met
betrekking tot de stadsboerderij. Wij hopen en gaan ervan uit dat tot een goede oplossing voor de
stadsboerderij gekomen zal worden. Mocht dat niet lukken, dan horen wij dat graag voor de
behandeling van de volgende voorjaarsnota, want dan kunnen wij alsnog een soortgelijk amendement
indienen.

        De VOORZITTER: Dames en heren! Ik stel vast dat amendement W is ingetrokken.

         De heer CORNELISSEN (SP): Voorzitter! Ik kom nu bij onze motie Y, die handelt over het
inhuren van personeel. De opmerking die de wethouder hierover maakte, vond ik heel mooi. Eigenlijk zei
hij dat deze motie een ondersteuning is van het beleid, omdat de gemeente het door hard werken
inmiddels voor elkaar heeft gekregen om met de inhuur van personeel onder de 10%-norm te komen,
terwijl dat nog geen norm is. Ik ben van mening dat wij als raad op dit gebied een kader moeten stellen
en moeten zeggen tegen het college: 10% is voor ons de norm; gaat u daar overheen, dan komt u
daarmee naar de raad. Dan beslissen wij als raad of wij het daarmee eens zijn, dan wel of wij een
andere oplossing kunnen vinden. Wat wij voorstellen, hoeft dus geen beperking te zijn voor het college
of de directie. Wel scherpt het alleen maar aan dat er goed nagedacht moet worden voordat iemand
ingehuurd wordt.
         Ik kom nu bij "Het verschil maken". In de eerste termijn heb ik gezegd dat wij ertoe neigen om
een motie van afkeuring in te dienen met betrekking tot dit onderwerp. De reden daarvoor is dat in
onze beleving "Het verschil maken" inhoudt: Bekijken wat je wilt, in welke mate je dat wilt, daarover
praten met de maatschappelijke partners en nagaan tot welke bezuinigingen je ongeveer kunt komen.
Bij de voorjaarsnotabehandeling zijn wij er achter gekomen dat dat niet gelukt is. Toen hebben wij als



INT - 1201417                                                                                          57
</p><p>raad bij amendement bepaald dat voor de behandeling van de begroting 2012 overeenstemming moest
zijn bereikt met de maatschappelijke partners. Thans zijn er nog geen uitwerkingen daarvan. Die zullen
komen in 2012. Daarom houden wij de aangekondigde motie van afkeuring nog in onze achterzak.
Zodra de eerste uitwerkingen in het nieuwe jaar komen, zullen wij bekijken hoe die tot stand zijn
gekomen. Als wij daar dan niet tevreden mee zijn, dan zullen wij alsnog met de motie komen.
         Ik stel vast dat het proces van "Het verschil maken" niet gelukt is in de mate waarin wij dat in
eerste instantie voor ogen hadden en met het geld dat wij daarvoor hadden gereserveerd. Dit vind ik
een heel goede reden om het bedrag van € 125.000,-- voor het proces van "Het verschil maken" te
schrappen en te stoppen in het schoolzwemmen, zodat dat de komende jaren overeind kan blijven.
         Ik schrok vanmorgen een beetje toen ik in de krant las dat ondanks dat de gemeente vorig jaar
een heel leuke bijdrage - een flinke meevaller - heeft gekregen door het efficiënte werk van de
Stadswacht, er elf mensen uit moeten daar. Ik kan dat niet helemaal rijmen met elkaar. Wel ben ik
benieuwd wat de eerste reactie van de wethouder daarop is. Op dit moment zal ik niets doen met dit
gegeven, maar in het komende jaar zullen wij daar zeker scherp op toezien. Dit temeer omdat, zoals
ook de fractie van Leefbaar Almelo heeft aangegeven, de veiligheid belangrijk is. Inmiddels wordt die
voor een groot deel afgewaardeerd. Dat blijft onze aandacht houden.
         Ik ga nu in op enkele amendementen en moties van andere fracties. Ik zal ze omwille van de tijd
niet allemaal langslopen.
         De motie (I1) over het Werkplein en het amendement over Soweco (E) ondersteunen wij. Dat is
in lijn met wat wij in de eerste instantie hebben ingebracht. Wij vinden het goed dat geld geoormerkt
wordt voor bijvoorbeeld Soweco, maar nog niet op voorhand al voor een speciaal doel. Wat betreft de
middelen voor het Werkplein, merk ik op dat wij dat heel veel geld vinden voor steen. Wij moeten
bekijken of wij dat geld op een andere manier voor re-integratie kunnen inzetten. Daarom ondersteunen
wij motie I1.
         Met betrekking tot het gebruik van Roundup heeft de fractie van GroenLinks amendement UA
ingediend. Ik ben het helemaal met die fractie eens over dit onderwerp. In de Tweede Kamer is gezegd
dat dit spul niet deugt en dat het gebruik daarvan gestopt moet worden. Dan kunnen wij als gemeente
onszelf wel een heel lange termijn geven om het gebruik af te bouwen, maar ik ben van mening dat wij
als de wiedeweerga moeten zorgen dat wij Roundup niet meer gebruiken en andere oplossingen
toepassen. Er wordt al heel veel onkruid verwijderd door middel van verbranding en andere zaken.
Overigens is het woord "gewasbescherming" helemaal fout als het gaat om het toepassen van
Roundup, zoals de fractie van GroenLinks zei. Daar sta ik helemaal achter, want het gaat om
gewasbestrijding. Roundup is een bestrijdingsmiddel, beslist geen beschermingsmiddel.
         Dan een lastig onderwerp: Waterrijk. Wij hebben altijd problemen gehad met de ontwikkeling
daarvan. Nu zijn er verschillende moties en amendementen ingediend over de vraag hoe verder gegaan
moet worden met Waterrijk. Wat dat betreft, neigen wij heel erg naar wat in het VVD-amendement
(DA) is verwoord. De reden daarvoor is dat dit het minst verstrekkend is en het meeste ruimte geeft om
er bij de voorjaarsnotabehandeling op terug te komen. Eigenlijk is dat ook een ondersteuning van het
beleid dat de wethouder heeft aangegeven. Weliswaar moeten wij nog even heel goed nadenken over
amendement DA en er nog eens heel goed naar kijken, maar volgens mij sluit dat voor een groot deel
aan bij de beantwoording door de wethouder.
         Nogmaals: de ontwikkeling van Waterrijk is voor ons altijd een heel groot probleem geweest.
Wij vinden het zonde dat de gemeente daar zo belachelijk veel geld in heeft gepompt. Het is van belang
dat de gemeente dat geld op de een of andere manier terug kan krijgen, maar dat is het probleem, ook
als amendement DA wordt aangenomen. Dan zitten wij straks namelijk bij de behandeling van de
volgende voorjaarsnota en begroting alsnog met de vraag: Wij hebben zoveel grond; wat doen wij
daarmee? Alternatieven zullen waarschijnlijk weinig geld opleveren. Alternatieven zijn ook al vaak
aangedragen, bijvoorbeeld door de fractie van GroenLinks. Het blijft gewoon een lastig verhaal, dat hele
Waterrijk. Het kost de gemeente klauwen met geld. De gemeente had daar nooit aan moeten beginnen,
maar goed, de realiteit is anders. De droom gaat door.

        De heer KAMPHUIS (LKA): Mevrouw de voorzitter! Ik heb mijn verhaal zo gemaakt, dat ik denk
dat interrupties niet nodig zijn. Ik zal dan ook binnen twee minuten klaar zijn met het verhaal.
        Bezuinigen op subsidies is te zien als een vorm van diefstal. Hoe komt hij daar nu bij, zult u
denken. Ik zal dat verduidelijken. In Nederland, ook in Almelo, is het een feit dat wij in verhouding erg
veel belastingen betalen. Subsidies die landelijk en lokaal worden gegeven, zijn dus eigenlijk betaald
met onze eigen belastingcenten. Met andere woorden: een sigaar uit eigen doos. Het saneren van
subsidies zou dus eigenlijk gepaard moeten gaan met het verlagen van belastingen, omdat deze, zoals




INT - 1201417                                                                                          58
</p><p>beschreven, onlosmakelijk met elkaar verbonden zijn. Die onlosmakelijkheid is kennelijk toch te kraken,
waardoor oneerlijkheid boven water komt. De doos met eigen sigaren wordt leger en leger.
        Daar komt nog bij dat via de commercialisering van energie er oneigenlijke belastingen worden
geheven bij de burgers, namelijk het te veel betaalde aan gas en licht. Dat wordt tegenwoordig aan
gemeenten uitgekeerd als winst of dividend. De gemeentelijke energiebedrijven van weleer zijn
verworden tot geldslurpende organisaties.
        Het college blijft maar volhouden dat investeren in de binnenstad, in stenen, in het belang is
van alle huidige inwoners. Maar investeren in de mensen zelf lijkt de fractie van LKA toch veel
belangrijker, want er zijn steeds meer Almeloërs die van een minimum moeten rondkomen of moeten
aankloppen bij de Voedselbank. Er zijn mensen genoeg die met de kerst geen kalkoen op tafel kunnen
zetten. Zolang deze armoede blijft bestaan, is het investeren in stenen en bouwplannen voor ons uit
den boze. Het college wil een gerichte uitstraling van de stad. Wij willen voldoende eten en
voorzieningen voor iedere Almeloër. Maar goed, ziehier: ieder heeft zijn of haar eigen prioriteiten.
        Volgens de fractie van het CDA (dat werd zojuist gezegd) is de schuldenlast tevens een oorzaak
van dalende inkomsten, maar volgens mij spreekt die fractie zichzelf tegen op dit punt. Ik herinner mij
namelijk de volgende woorden van die fractie: Hoe meer wij binnenkrijgen, hoe meer wij kunnen
uitgeven.
        De PvdA-fractie is het oneens met de bezuinigingen op de zwakste groepen, en terecht. Maar
ondertussen is die fractie nog steeds voorstander van de grote en kansloze megaprojecten van deze
stad. Ik trek hieruit de conclusie dat de PvdA-fractie gewoon keuzes moet maken, maar dit door haar
eigen tegenstrijdigheid niet kan.
        De LKA-fractie is tegen toeristenbelasting, zeker omdat het geld niet naar toerisme gaat.
Aangezien de meeste fracties wel voor het invoeren van toeristenbelasting zullen stemmen, steunen wij
het amendement (SA) van de fractie van Leefbaar Almelo inzake de toeristenbelasting. Volgens dat
plan komt het geld in ieder geval ten goede aan het toerisme, in plaats van dat het in de algemene
middelen verdwijnt.
        Zojuist gaf de PvdA-fractie in een interruptie van de bijdrage van de fractie van Leefbaar Almelo
aan dat wij in Almelo al zoveel hebben gedaan voor het toerisme. Als voorbeeld daarvan werd de
aanleg van fietspaden genoemd. Vergeten is volgens mij de totale afbraak van monumentale panden in
Almelo in het verleden. Er is weinig moois meer over dat toerisme zou kunnen opleveren.
        Ook gaf de PvdA-fractie zojuist aan niet mee te willen werken aan het kapot bezuinigen van
deze stad. Kapot bezuinigen is natuurlijk een relatief begrip. In de ogen van de LKA-fractie is men met
de voorliggende begroting al bezig met kapot bezuinigen, als gevolg van de eerdere plannen en het
beleid van de fracties van de PvdA, het CDA en mogelijk D66.
        De PVA-fractie wil over de begroting een second opinion van Ernst & Young. Waarom zou je die
vragen, als je zelf al weet waar de schoen wringt? Waarom dan een bedrijf, waar Balkenende 7,5 ton
per jaar verdient, spekken met nog eens enkele duizenden euro's van de Almelose belastingbetaler?
        Ik kom nu bij de bijdrage van de VVD-fractie. Die stelde:
             "Alleen diegenen die door ernstige ziekte, handicap of anderszins absoluut niet kunnen
             werken, hebben recht op een uitkering."
        Dit standpunt van de VVD-fractie is helemaal mijn gedachte, maar de VVD-fractie vergeet één
ding, of eigenlijk twee dingen:
1. De mensen die door het UWV onterecht niet ziek genoeg worden verklaard en eruit worden
   geschopt. Zij komen in de bijstand, tenminste na het opeten van hun zuur verdiende bezittingen.
2. De werklozen die wel willen werken maar geen werk kunnen vinden en na verloop van tijd uit de
   WW worden geschopt, de bijstand in, tenminste na het opeten van hun eigen bezittingen.
        Allemaal de bijstand in! Het resultaat van de sociale afbraak in dit land als het gaat om de WW
en WAO. Nu roept een lokale partij iets waarvoor de landelijke politiek eerst maar eens goede kaders en
wetten moet maken.
        Volgens wethouder Schouten is de aanleg van de Burgemeester Schneiderssingel nodig in
verband met de toegankelijkheid van het centrum vanuit het noorden en voor het Indiëterrein. Hij kan
dat wel zeggen, maar toon dat maar eens aan. De fractie van LKA blijft van mening dat de mensen ook
wel via de bestaande wegen op hun bestemming kunnen komen.
        Wethouder Schouten gaf ook aan dat betaald parkeren nu eenmaal normaal is in de grotere
steden en dat het normaal is dat de kleinere steden of dorpen geen betaald parkeren kennen, omdat
daar nu eenmaal minder druk wordt gewinkeld. Dat lijkt mij niet te kloppen, want in verhouding tot het
aantal inwoners lijkt mij er weinig verschil te bestaan. Sterker nog: mensen uit de grote steden gaan
steeds vaker naar de kleinere steden om te winkelen, alleen al om het gemak omdat ze dan gratis




INT - 1201417                                                                                         59
</p><p>kunnen parkeren, maar ook vanwege de gastvrijheid die dat uitstraalt. Nee, er is maar één ding
normaal, en dat is de leegloop van Almelo.
        De gang van zaken omtrent de Kloosterhofflat wordt steeds vreemder. Al meer dan een jaar
geleden zei een medewerker van Beter Wonen dat ze er luxeappartementen van gingen maken. Met
andere woorden: het volk eruit en yuppen erin, voor veel meer huur. Vragen hierover werden
ontwijkend beantwoord. Sterker nog: de wethouder hoefde van de voorzitter niet eens te antwoorden.
Zeer recent kwamen er nieuwe geruchten. Een vrouwelijke ambtenaar van de gemeente had gezegd dat
het betrokken deel van de Kloosterhofflat niet wordt gesloopt, maar dat de woningen worden
opgeknapt en eenzelfde balkon krijgen als de rest van de flatwoningen. Gerucht op gerucht van mensen
die het kunnen weten, maar het college weet het beter. Ons wordt een binnenstadsplan voorgelegd, en
veel mensen zijn onnodig en veel te vroeg hun huis uitgezet.

        De heer NIJHUIS (VVD): Mevrouw de voorzitter! Allereerst dank ik het college voor de
uitgebreide beantwoording van allerlei vragen en opmerkingen. Dat heeft het college zo snel mogelijk
gedaan. Wij snappen dat soms tijdoverschrijding noodzakelijk is, willen wij volledig antwoord krijgen.
Toch wil onze fractie in de tweede termijn graag nog een aantal op- en aanmerkingen maken.
        Die opmerkingen betreffen om te beginnen een aantal aspecten van de grondpolitiek. In de
eerste termijn hebben wij ten aanzien van Almelo Noordoost het voorstel aangedragen om te bekijken
of daar niet "geschoven" kan worden met gronden. Het antwoord van de wethouder daarop heb ik niet
echt duidelijk begrepen. Ik doelde met ons voorstel met name op de mogelijkheden langs de randen van
het gebied, bijvoorbeeld langs de Ootmarsumsestraat. De reden daarvoor is dat het handhaven van
grote kavels daar wel eens minder aantrekkelijk zou kunnen zijn. Wij kennen een partij die daar wil
investeren en die niet eerst haar huizen hoeft te verkopen alvorens daar iets neer te kunnen zetten.
Kortom, wij vragen om flexibiliteit wat betreft de grootte van de kavels. Ik heb de wethouder horen
zeggen dat de marktafwegingen voor de totale programmering gelden. Ik ben benieuwd wat hij daar
precies mee bedoelt.
        Onze fractie heeft geconstateerd dat bijna raadsbreed en bij het college de mening bestaat dat
de behandeling van de voorjaarsnota 2012 een ijkmoment zal zijn. Dat is namelijk het moment waarop
wij met elkaar moeten bepalen of wij met de huidige richtingen (en dan heb ik het over wat in de
begroting is opgenomen) door kunnen blijven gaan, dan wel of wij die moeten aanscherpen. Ik heb
steeds gezegd dat het niet ondenkbaar is dat wij verder moeten bezuinigen, andere maatregelen
moeten nemen of het beleid moeten wijzigen om de zaken uiteindelijk sluitend te houden. Ook voor de
VVD-fractie is de artikel 12-status een doemscenario waar ze niet van uit wil gaan.
        Het nu op voorhand al stellen dat bijvoorbeeld € 10 miljoen bezuinigd moet worden om de
zaken in evenwicht te brengen, gaat ons te ver (ik heb dit eerder benoemd als: boekhoudkundig). Ik
heb dat ook een- en andermaal aangegeven in de discussie met de heer Gerritsen. Dat neemt niet weg
dat ook de fractie van D66 een aantal moties heeft ingediend (ik zal ze straks langslopen) waarvan wij
denken dat de teksten daarvan wellicht gezamenlijk wat aangepast kunnen worden. Overigens zullen
wij ook zelf enkele teksten aanpassen, laat dat helder zijn. Wij willen namelijk even met andere fracties
overleggen over onze teksten. In dit verband refereer ik aan de opmerkingen van de heer Schuur. Ik
denk dat wij er samen wel uit zullen komen. Er zijn echter meer teksten van moties en amendementen
waarvan ik denk dat ze in elkaar geschoven moeten worden. Dan hebben wij straks misschien geen 56
amendementen en moties meer, maar zijn er dat 30. Dat zou heel goed zijn, ook voor de besluitvorming
en voor de helderheid over de kaders voor het college.
        Dan ga ik nu in op de teksten van enkele moties en amendementen.
        Ten aanzien van het leerlingenvervoer, waarover de CDA-fractie amendement C heeft
ingediend, wil ik graag opmerken dat ik niet helemaal goed begrijp wat de wethouder bedoeld heeft. De
redenatie van de CDA-fractie kan ik volgen. Ik ken de sector ook een beetje, geloof ik! Als ik zie wat er
op ons afkomt, dan kan het niet anders dan dat de instroom in het speciaal onderwijs zal afnemen en
dat daardoor ook de behoefte aan leerlingenvervoer zal afnemen. Mij is niet helemaal duidelijk of de
wethouder dat bestrijdt, dan wel of hij van dezelfde gedachte uitgaat. Hoe moet ik in dezen de cijfers
lezen die in de begroting staan voor na 2013/2014? De bezuinigingen gaan immers vanaf 1 augustus
2013 lopen.
        In motie D1 wordt gesproken over het informeren van de raad wanneer budgetten van ten
minste € 250.000,-- met 10% of meer overschreden dreigen te worden. Dat vinden wij prima; daar
kunnen wij mee leven. Wij willen echter wel graag dat de kleine afwijkingen zoveel mogelijk binnen de
betrokken programma's zelf worden opgelost, zodat wij niet voor verrassingen komen te staan.
        Ook met het reserveren van de geoormerkte werkgelegenheidsreserve ten behoeve van Soweco
(amendement E) kunnen wij leven. Wij gaan daar dan ook mee akkoord.



INT - 1201417                                                                                         60
</p><p>        Met betrekking tot "Het verschil maken" is wat verwarring ontstaan. De PvdA-fractie sprak in
dat verband over het inschakelen van externen. Wij hebben wat in de begroting hierover staat, als volgt
vertaald. Er wordt geld vrijgemaakt voor ambtelijke inzet ten behoeve van "Het verschil maken", dus
voor mensen die hier al in huis zijn. De salarissen voor die mensen worden weggeschreven op het
project "Het verschil maken". Kan de portefeuillehouder daar nog even wat duidelijkheid over
verschaffen?
        Met betrekking tot Waterrijk zijn ingediend motie S (D66), amendement DA (VVD) en motie EA
(ALA-AOV). Misschien is het mogelijk om gezamenlijk tot een tekst te komen, een soort amendement
dat past bij wat de wethouder heeft aangegeven maar dat meteen ook al als het ware een richting
aangeeft voor het bestemmingsplan. Ik meen dat dit een ondersteuning is van het streven van de
wethouder.
        Amendement Z (Sparen voor de binnenstad) willen wij vooralsnog handhaven, vooral als een
stimulans voor het college om op die manier verder te gaan. Over de vraag of het amendement
overbodig is, is wat discussie in onze fractie. Enerzijds wordt gezegd: Als wij het amendement
handhaven, dan blijft het krachtig, want dan kunnen wij daar steeds op terugkomen. Anderzijds geldt
dat wanneer een wethouder een belofte doet in de raad, die ook op schrift komt te staan. Ook dan kun
je de wethouder daar op aanspreken.

       De heer SCHUUR (PvdA): Voorzitter! De heer Nijhuis heeft het nu over sparen voor de
ontwikkeling van de binnenstad. Volgens mij kan niemand daar tegen zijn. Waar ik echter benieuwd
naar ben, is waar het spaargeld vandaan gehaald moet worden. Dat moet toch ergens vandaan komen?

         De heer NIJHUIS (VVD): Ja, dat klopt. Dat betekent dat wij, zoals wij ook gezegd hebben,
vragen om in de begroting 2013 een dotatie op te nemen. Als daar verder geen suggesties voor komen,
dan zullen wij die zelf aandragen. Op dit moment heb ik die nog niet scherp, maar het zal een dotatie
zijn die uit iets anders moet komen, dat zal duidelijk zijn.

       De heer SCHUUR (PvdA): Dus u zegt: Wij willen de bezuinigingen opvoeren met het
spaarbedrag. Er wordt dan dus extra bezuinigd om in de binnenstad te kunnen investeren. Is dat uw
plan?

        De heer NIJHUIS (VVD): Dat is niet uit te sluiten, ja.

        De heer SCHUUR (PvdA): Oké, dan is dat helder.

         De heer NIJHUIS (VVD): Voorzitter! In motie R van de D66-fractie staat dat de schuld van de
gemeente maximaal 65% mag zijn van het begrotingstotaal. Wij kunnen die gedachte volgen, ook in
relatie tot onze bijdrage, maar wij stellen voor om de bepaling "maximaal 65%" te wijzigen in: een
substantieel lager percentage. Ook dan wordt de in de motie genoemde trendbreuk ingezet, en dat
willen wij allemaal graag. Verder wordt met de door ons voorgestelde wijziging niet gezegd dat het
moet gaan om 0,5%, maar dat ervoor gezorgd moet worden dat er echt substantieel iets veranderd
moet worden, zij het zonder het resultaat vast te pinnen op 65%. Wat in de ene gemeente kan, kan in
de andere gemeente niet. Op dit gebied moeten wij een stukje flexibiliteit handhaven.
         In motie P1 van de fractie van D66 staat dat ernaar gestreefd moet worden € 1,5 miljoen extra
te bezuinigen door te focussen op samenwerking. Als de D66-fractie dit bedrag schrapt uit de motie,
dan past die ook in het verhaal dat wij hebben verteld. Dan kunnen wij daar elkaar best in vinden.
Kortom, ik nodig de D66-fractie uit om nog eens te kijken naar de teksten van haar moties, om na te
gaan of wij samen tot iets kunnen komen. Een verhoging van de ozb is overigens voor ons absoluut
niet acceptabel.
         Ons amendement CA (Realiteitsgehalte Marsroute 2) is een aanvulling op de besluittekst. De
wethouder heeft daar in zijn bijdrage op gereageerd. Daaruit begrijp ik dat wij dezelfde koers volgen,
maar ik hoor graag nog even van de wethouder wat hij precies bedoelde. Mogen wij inderdaad
aannemen dat de tekst die door de wethouder is uitgesproken, als vaste tekst kan worden gebruikt
waar wij op terug kunnen komen? Zo ja, dan vragen wij ons af of wij amendement CA moeten
handhaven.
         Wij zijn voor onderhoud van de kunstwerken, zoals in onze motie BA wordt aangegeven. Wij
zijn daar ook voor vanwege de vervangingen. Het gaat namelijk niet alleen om onderhoud, want ook
vervanging van kunstwerken speelt daarbij een rol. Daaraan hebben wij de vraag gekoppeld om eens te
onderzoeken of het mogelijk is om de besturing van de beweegbare bruggen wat anders te organiseren.



INT - 1201417                                                                                       61
</p><p>Wij weten dat dat mogelijk extra geld gaat kosten, maar nu horen wij dat het gebruik van de
jachthaven vermindert, omdat mensen zich storen aan de bruggen die gepasseerd moeten worden om
de jachthaven te bereiken.
        Motie HA van de fractie van ALA-AOV gaat over het beperken van gemeentelijke publicaties.
Gemeentelijke publicaties zijn noodzakelijk, zeker de publicaties in bijvoorbeeld het Almelo´s Weekblad.
Dergelijke publicaties zouden wij niet willen beperken. Als de bedoeling is dat het verminderen van
publicaties beperkt wordt tot dure brochures, dan heeft de fractie van ALA-AOV een punt. Dan kunnen
wij daarover meedenken, want ook wij hebben af en toe de neiging om te denken: Het kan misschien
iets soberder.

         De heer STORK (ALA-AOV): Mijnheer Nijhuis! Het is zeker één van de bedoelingen van de
fractie van ALA-AOV om ook daarop te bezuinigen. Verder wil onze fractie dat heel kritisch gekeken
wordt naar de gemeentelijke advertenties. Overigens hebt u, voorzitter, al aangegeven dat daar ook
naar gekeken zal worden. In de gemeentelijke advertenties staan allerlei dingen die niet noodzakelijk
zijn. Ik noem bijvoorbeeld de interviews met raadsleden. Die zijn heel leuk en aardig, maar die hoeven
niet opgenomen te worden. Wij denken aan besparingen op dat soort dingen.

        De heer NIJHUIS (VVD): Oké. Ik denk dat wij daar straks even naar moeten kijken.

        Mevrouw ROZEMA-JUNGJOHANN (D66): Voorzitter! Er is nog een besparing mogelijk op de
gemeentelijke uitgaven, en ik vraag de heer Nijhuis of hij het daarmee eens kan zijn. Het blad Love dat
wij sinds kort hebben, mag dat weg van hem? Ook dat is een dure publicatie.

        De heer NIJHUIS (VVD): Ik kan dat niet helemaal overzien, maar laat mij eerst zeggen dat ik dat
een leuk blad vind om te lezen. Als dat blad met gemeentegeld gefinancierd wordt, dan is het mogelijk
dat ik daarover zeg: Dat is één van de bladen die een grote investering vergen. Die hoeft de gemeente
niet te doen. Aan de andere kant kan ik mij ook voorstellen dat de reclame-inkomsten uit het uitgeven
van dat blad zodanig zijn, dat het zichzelf bedruipt. Dan heb ik er absoluut geen probleem mee want,
zoals ik al zei, ik vind het een heel leuk blad.
        Voorzitter! Over het verhuizen van het kantoor van het Sportbedrijf naar de IISPA zei de
wethouder: Jongens, dat is geen taak van de gemeente. Wij constateren echter dat de gemeente de
enige aandeelhouder is van het Sportbedrijf. Het lijkt ons dat er op basis van het aandeelhouderschap
wel iets over dit fenomeen gezegd kan worden door de gemeente.

        De heer STORK (ALA-AOV): Wij hebben begrepen dat de commerciële ruimtes in de IISPA
onder de gemeente vallen en buiten het Sportbedrijf gehouden zijn. In dat geval is het wel een
bezuiniging wanneer het Sportbedrijf van de locatie aan de Wierdensestraat verhuist naar de IISPA. Dat
is onze intentie, maar ik hoor daar straks graag de wethouder over.

        De heer NIJHUIS (VVD): Ik meen dat de wethouder daar inderdaad op moet reageren. Het enige
dat ik zei, is dat de gemeente wel degelijk beïnvloeder is van de besluitvorming van het Sportbedrijf,
want zij is de enige aandeelhouder.
        Voorzitter! Wat betreft zwembad Het Sportpark, willen wij graag de resultaten van het
onderzoek afwachten. Het is volgens ons voorbarig om nu te zeggen dat het buitenbad gesloten moet
worden. Dat geldt ook voor de uitspraak dat het open moet blijven. Wij willen namelijk graag eerst de
resultaten van het onderzoek afwachten en wij willen de gelegenheid geven om dat onderzoek te doen.

        De heer SCHUUR (PvdA): Mijnheer Nijhuis! Bent u het met mij eens dat het eigenlijk niet kan
dat in een stad als Almelo, met 72.000 inwoners, er geen fatsoenlijk zwembad is met een binnen- en
een buitenbad? Dat kan toch eigenlijk niet?

        De heer NIJHUIS (VVD): In principe zijn wij het daarmee eens, maar tegelijkertijd verwijs ik u
naar een krantenpublicatie van (ik dacht) vorige week. In die publicatie werd het zwembad van Preston
Palace als het beste zwembad van Nederland gekwalificeerd. Daarnaast hebben wij nog een zwembad
in het Theaterhotel. Gelet daarop, wil ik graag kijken naar alle zwemgelegenheden. Verder zeg ik
nogmaals dat wij niet vooruit willen lopen op het zojuist door mij genoemde onderzoek. Laat de
onderzoekers even hun werk doen. Ook wij zijn van mening dat dit soort voorzieningen er wel degelijk
moet zijn in een stad als Almelo. Wat betreft de vraag hoe die precies in elkaar moeten steken, hebben
wij in het verleden al eens gepleit voor een attractieve zwemaccommodatie, desnoods geëxploiteerd



INT - 1201417                                                                                        62
</p><p>door een particulier waar de gemeente subsidie aan geeft om ervoor te zorgen dat de accommodatie
laagdrempelig is voor bepaalde groepen. Kortom, er zijn allerlei vormen denkbaar wat ons betreft.

        De heer SCHUUR (PvdA): De heer Nijhuis zei: Wij hebben toch al het bad van Preston Palace. Ik
weet niet of hij weet hoeveel je moet betalen als je daar een keer gaat zwemmen. Ik laat mijn kinderen
daar niet elke week naartoe gaan, want dat kan echt niet uit. Ik vind gewoon dat er een fatsoenlijk
zwembad moet zijn in Almelo. Het bad van Preston Palace is van een geheel andere orde.

      De heer NIJHUIS (VVD): Goed, wij zijn het ermee eens dat er een fatsoenlijke
zwemaccommodatie moet zijn in Almelo. De vraag is echter of die alleen door de gemeente moet
worden gefinancierd. Nogmaals: wij wachten graag het onderzoek daarnaar af.

        De heer BRAND (CDA): Voorzitter! Het zal duidelijk zijn dat ik graag wil ondersteunen wat de
heer Nijhuis nu zegt. Op verzoek van de CDA-fractie is een onderzoek toegezegd naar het zwembad.
Wij hebben daarbij ook gezegd: Ga peilen. Wij raadsleden vinden dat wij in de raad een statement
moeten maken en stellen dat er een zwembad moet zijn. Daar is onze fractie het mee eens, maar vindt
de Almelose bevolking dat ook? Ik hoop het antwoord op deze vraag terug te vinden in het onderzoek
dat wordt gedaan.

       De heer NIJHUIS (VVD): Voorzitter! Als ik het goed begrijp, heeft de fractie van ALA-AOV
gezegd dat er geen gelijkvloerse kruising Gravenallee-Van Rechteren Limpurgsingel moet komen. Dat is
wel wat de VVD-fractie altijd graag gewild heeft. Wij snappen echter niet helemaal wat de fractie van
ALA-AOV bedoelt in dezen, maar misschien kan de heer Stork daar aanstonds nog een stukje uitleg
over geven, want hij is na mij aan de beurt om het woord te voeren.

       De heer STORK (ALA-AOV): Ik kan u nu de uitleg geven. Als gemeente hebben wij op dit
moment geen geld voor de uitvoering van het plan inzake de kruising Gravenallee-Van Rechteren
Limpurgsingel. Dat is geen absoluut noodzakelijk iets. Vandaar dat wij ons voorstel gedaan hebben.

         De heer NIJHUIS (VVD): Dan is volgens mij uw tekst hierover niet helemaal goed, maar dat
terzijde. Bovendien wordt het plan volgens mij niet uit gemeentemiddelen gefinancierd, maar wordt er
Interreg IVb-subsidie aangevraagd, omdat daarmee de aantrekkelijkheid van onze stad wordt vergroot.
Volgens mij staat dat letterlijk in de begroting.
         Voorzitter! Ik kom nu bij motie LA (Binnenstadsplan temporiseren) van de fractie van ALA-AOV.
Volgens mij heeft de VVD-fractie een heel goed amendement (Z) ingediend met betrekking tot de
binnenstad. Ik nodig de fractie van ALA-AOV uit om dat gezamenlijk met ons op te pakken.
         Ten aanzien van de Burgemeester Schneiderssingel zullen wij geen motie of amendement
indienen. Het antwoord in dezen van de wethouder was duidelijk.
         Tot slot het amendement (SA) van de fractie van Leefbaar Almelo over de toeristenbelasting.
Dit amendement wordt door onze fractie ondersteund, ook naar aanleiding van het antwoord van de
wethouder.

        De heer STORK (ALA-AOV): Voorzitter! In de eerste instantie heb ik een beroep gedaan op de
raad om de handen ineen te slaan om de problemen van de gemeente Almelo op te lossen. Nu ik
beluisterd heb wat de raad naar voren gebracht heeft, stel ik vast dat iedereen daar grote zorgen over
heeft. Daarom roep ik op om na deze vergadering een afspraak te maken met alle fracties (één of twee
figuren per fractie), om een keer een avond met de benen op tafel te praten over hoe wij gezamenlijk
kunnen opereren. Ik meen namelijk dat dat uitermate belangrijk is. Vandaar mijn oproep. Ik kom daarop
terug, maar zal dan de individuele raadsfracties benaderen.
        Ik kom nu bij de beantwoording door het college. Ik ben erg blij met de beantwoording door u,
voorzitter, over de signalen die de raad afgegeven heeft met betrekking tot een artikel 12-situatie. U
stelde dat het college dan minder zelf kan beslissen, maar geholpen wordt bij het nemen van
beslissingen en dat de beslissingen in overleg met de provincie genomen zullen worden. Dat is precies
wat wij willen, namelijk assistentie van de provincie.
        Verder stelde u, voorzitter, dat de gemeente meer inkomsten moet genereren en minder
uitgaven moet doen. Welnu, ook dat is precies wat wij willen. Ik meen dan ook dat de indicaties en
uitleg die u gegeven heeft, precies zijn wat wij willen.
        Ik ga nu in op een paar puntjes zo her en der (misschien een beetje warrig). Wij hebben
gevraagd of er wel voldoende vrijwilligers zijn. Daar hebben wij geen antwoord op gekregen. Het is te



INT - 1201417                                                                                        63
</p><p>gemakkelijk om zomaar te stellen dat er vrijwilligers zijn die de taken van beroepskrachten willen
overnemen. Dat geloven wij niet. Wij horen signalen uit de samenleving dat het steeds moeilijker is om
goede vrijwilligers te krijgen.
       In de eerste instantie hebben wij gesteld dat boekhoudkundig heel leuk is omgegaan met de
grondwaardes, maar dat Deloitte heeft aangegeven dat niet de boekwaardes maar de marktwaardes
moeten worden opgenomen. Wij hebben gezegd dat dat ongeveer € 100 miljoen zal schelen op de
balans van de gemeente Almelo. Daar hebben wij geen reactie op gekregen. Daarom nemen wij aan dat
het bedrag van € 100 miljoen nog te laag ingeschat is door ons.

       De heer NIJHUIS (VVD): Voorzitter! Begrijp ik goed dat wij volgens de heer Stork straks voor
€ 100 miljoen een voorziening moeten treffen? Dat is volgens mij wat dan boekhoudkundig direct moet
gebeuren.

       De heer STORK (ALA-AOV): Dat denk ik ook, ja, maar het is aan het college om daar antwoord
op te geven, want dat is een vraag aan het college. Het gaat daarbij om het advies van Deloitte. Dat
kunnen wij niet zomaar naast ons neerleggen, maar dat gebeurt nu wel. Daar hebben wij bezwaar
tegen.

        De heer NIJHUIS (VVD): Ik heb niet de indruk dat dat advies zomaar naast ons neergelegd
wordt. Ik heb wel de indruk dat daar heel serieus naar gekeken is. Als je handelt volgens het advies van
Deloitte, dan zul je ook de consequentie daarvan moeten aanvaarden. Die consequentie is dat Almelo
een artikel 12-gemeente wordt.

       De heer STORK (ALA-AOV): Dat ben ik helemaal met u eens, maar wij moeten het wel doen.
Wij moeten de problemen oplossen en niet, zoals uw fractie en het college doen, de problemen steeds
weer vooruitschuiven. Wat dat betreft…

        De heer NIJHUIS (VVD): Dat doen wij niet.

        De heer STORK (ALA-AOV): …zijn wij het compleet eens met de fractie van D66.

        De heer NIJHUIS (VVD): Dat is tenminste helder.

        De heer STORK (ALA-AOV): De oplossing hebben wij ook niet, die heeft u niet en die heeft
niemand. Vandaar dat ik voorstel: raad, kom bij elkaar en laat ons daar eens over praten, want de
problematiek moet opgelost worden. Het gaat hierbij om € 100 miljoen.
        Voorzitter! De personeelsreductie. Wij hebben gezegd dat de gemeente een aantal taken moet
schrappen. Ik ben van mening dat aan de raad aangegeven moet worden welke taken geschrapt
worden. De raad moet daar inzicht in hebben. Ik vraag het college om ons dat te geven.
        Wat betreft de verwerking van de (rijkskortingen uit de) septembercirculaire, heeft de
wethouder het grootste gelijk van de wereld met zijn uitspraak: Dat hebben wij altijd zo gedaan, want
wij hebben de gegevens uit de septembercirculaire altijd pas in de voorjaarsnota van het volgende jaar
verwerkt. Maar de tijd is anders. Wij kunnen ons nu niet beroepen op de kreet: Dat hebben wij altijd zo
gedaan. De tijden zijn veranderd. Daarom moeten ook wij veranderen. Wij moeten de consequenties
van de septembercirculaire 2011 wel degelijk opnemen in de begroting voor 2012, hoe ongelooflijk
vervelend dat ook is.
        Wij hebben aangegeven - en dat is door meerdere fracties gezegd - dat de aanleg van de
Burgemeester Schneiderssingel in wezen niet zo nodig is. Er is immers een goede verbinding via de
route Kolthofsingel-Plesmanweg-Bleskolksingel-Schuilenburgsingel. Die verbinding is er. Zit je eenmaal
op de Schuilenburgsingel, dan heb je drie, vier mogelijkheden om naar Waterrijk te gaan, dat er volgens
mij trouwens nooit zal komen. Mocht het er wel ooit komen, dan liggen de verbindingen met Waterrijk
er al. Nota bene: een stuk daarvan heet Burgemeester Schneiderssingel.

        De heer NIJHUIS (VVD): Voorzitter! Volgens mij is de aanleg van de Burgemeester Schneiders-
singel een verplichting. Verder klagen (voor zover mij bekend) de mensen aan de Vissedijk steen en
been over de huidige verkeerssituatie. Volgens mij wordt de verkeersdruk daar alleen maar opgevoerd
met de optie die u noemde.




INT - 1201417                                                                                        64
</p><p>        De heer STORK (ALA-AOV): Nee, die heeft niets met de Vissedijk te maken. Wel heeft de
Vissedijk te maken met de kruising Kolthofsingel. De Vissedijk wordt verkeersluw. De ontsluiting gaat
naar het Indiëterrein.

        De heer NIJHUIS (VVD): Precies, via de Burgemeester Schneiderssingel.

       De heer STORK (ALA-AOV): Nee, tot de Kolthofsingel. Als daar een rotonde ligt, kan het
verkeer rustig doorrijden. Maar goed, ik ga daar graag samen met u kijken.

        De heer NIJHUIS (VVD): Voorzitter! Ik hoor graag de wethouder daar straks over.

        De heer STORK (ALA-AOV): Zullen wij samen gaan kijken? Dan kunt u het zien.
        Voorzitter! Met betrekking tot de grondverkoop ten behoeve van woningbouw in 2012, hebben
wij gevraagd waarop het aantal van 166 te bouwen woningen gebaseerd is. Daar hebben wij geen
antwoord op gekregen. Ook hebben wij geen antwoord gekregen op de vraag wat er in 2010 gebouwd
is. Evenmin hebben wij antwoord gekregen op de vraag hoeveel woningen in 2012 gerealiseerd zullen
worden. Waarom willen wij dat weten? Omdat wij hierbij praten over een totaalbedrag van € 12
miljoen. Wordt dat 20% minder, reken dan maar uit wat het nadeel is. Dit zijn allemaal zorgen die in de
begroting verwerkt hadden moeten worden.
        Wij hebben voorgesteld om de ontwikkeling van Leemslagen maar helemaal te schrappen. Daar
hebben wij geen enkel commentaar op gehoord van het college.
        Zojuist hebben wij het al gehad over de IISPA. Volgens ons gaat de gemeente over de
commerciële ruimtes van de IISPA, niet het Sportbedrijf. Graag krijg ik het commentaar van het college
daarop.
        Over het terugbrengen van het water in de binnenstad, maar dan ondiep water in plaats van
bevaarbaar water, heeft de wethouder gezegd: Wat de kosten betreft, scheelt dat eigenlijk niets, omdat
wij dezelfde kosten moeten maken voor aankopen. Dat is natuurlijk niet zo, en wel om de volgende
redenen:
1. Het water hoeft niet verder te komen dan Kreta.
2. Wanneer er ondiep water komt, dan hoeven de bruggen niet zo geconstrueerd te worden dat ze
    open en dicht kunnen of dat er schepen onderdoor kunnen. Dat is een stuk goedkoper.
        Graag krijg ik het commentaar van het college hierop.
        Hoewel de wethouder adequaat gereageerd heeft op onze opmerkingen over de Stadsbank,
denk ik dat zij die iets verkeerd begrepen heeft. Onze bedoeling is nooit geweest om de
beslissingsbevoegdheid bij de Stadsbank weg te halen. De Stadsbank heeft die bevoegdheid, maar in
het voortraject zit een enorme hoop werk. Dat werk zou aan Almelo Sociaal overgedragen kunnen
worden. Die zou daarmee opgezadeld kunnen worden en het werk kunnen doen, maar wel met hulp
van professionals.

        Mevrouw ROZEMA-JUNGJOHANN (D66): Mijnheer Stork! Ik heb nu indirect een vraag aan de
wethouder, maar ik wil het ook graag van u weten. Eén van de bezuinigingsmogelijkheden voor de
Stadsbank is om Almelo Sociaal aan de voorkant veel werk te laten doen, zodat er door de Stadsbank
minder werk gedaan hoeft te worden. Echter, verderop in het traject via de Stadsbank zit ook een deel
van de bezuiniging, waardoor de mensen veel minder lang begeleid kunnen worden. De mensen mogen
nu nog maar ongeveer negen maanden begeleid worden. De hoop is dat de mensen het daarna zelf
kunnen, maar vaak blijkt dat dat niet het geval is. Is het niet een goed idee om Almelo Sociaal zo in te
richten, dat die ook dingen kan doen op het gebied van de nazorg voor mensen die op een bepaald
moment niet meer bij de Stadsbank terechtkunnen?

        De heer STORK (ALA-AOV): Ik zou daar heel veel voor voelen, maar ben daar wel een beetje
huiverig voor. De nazorg is namelijk heel vaak werk voor professionals. Wij moeten oppassen dat wij
het professionele werk van de Stadsbank niet bij de vrijwilligers neerleggen. Dat mogen wij niet doen.

          Mevrouw ROZEMA-JUNGJOHANN (D66): De mensen van Almelo Sociaal zijn professionele
vrijwilligers.

      De heer STORK (ALA-AOV): Ja, professionele vrijwilligers. Precies. Wij kunnen de door u
genoemde mogelijkheid bekijken. Het zou prachtig zijn als dat lukt.




INT - 1201417                                                                                         65
</p><p>        Voorzitter! Van de door ons ingediende moties, willen wij er een aantal handhaven en een paar
intrekken. Gezien de tijd, zal ik het lijstje daarvan overhandigen aan de griffie. Vervolgens merkt men
wel bij de stemmingen welke moties gehandhaafd blijven.
        Wat betreft de moties en amendementen van de andere fracties, hebben wij een overzicht
gemaakt van de moties en amendementen waar de fractie van ALA-AOV voor of tegen is. Iedere fractie
zullen wij een exemplaar van het overzicht geven, zodat alle fracties zich daar op kunnen beraden.
        Ik zal in de tweede termijn slechts één motie indienen. Die stelt dat de gevolgen van de
septembercirculaire moeten worden opgenomen in de begroting voor 2012. Met die motie zal ik u dus
nog vervelen! De motie luidt:

                "Motie YA.
                De raad van de gemeente Almelo in vergadering bijeen op 8 en 10 november 2011.
                Constateert:
                Dat Almelo er in financieel opzicht uitermate slecht voorstaat.
                Dat het weerstandsvermogen veel te gering is.
                Dat de grondaankopen met geleend geld gedaan zijn.
                Dat de grondverkoop stagneert.
                Dat de bevolkingsgroei stagneert.
                Dat de schuldenlast explosief gestegen is.
                Dat in de begroting 2012 een aantal zaken niet opgenomen zijn die de financiële positie
                van Almelo nadelig beïnvloeden.
                Overweegt:
                Dat er zeer ingrijpende maatregelen genomen moeten worden.
                Besluit:
                Het college van B & W op te dragen:
                De gevolgen van de septembercirculaire in de begroting op te nemen.
                En gaat over tot de orde van de dag."

       De VOORZITTER: Dames en heren! Ik stel vast dat wij aan het eind gekomen zijn van de
tweede termijn voor de raad. Ik heb zojuist al gezegd dat de vergadering een uur uitgelopen is. Dit
betekent dat de eerder aangegeven schorsing opschuift. Het college heeft namelijk de behoefte om
even te beraadslagen; ik hoop dat u dat snapt. Bovendien draagt dat ook bij aan een effectieve
beantwoording. Ik stel voor om de vergadering te schorsen tot 16.40 uur.

       De heer STAPEL (PvdA): Voorzitter! Een puntje van orde. Ik heb gezien dat de heer Hümmels
twee amendementen heeft verstuurd via de e-mail, terwijl die amendementen niet in het openbaar zijn
ingediend. Klopt dat?

     De VOORZITTER: Dat kan ik niet checken, want ik heb hier geen internetaansluiting. Mijnheer
Hümmels, kunt u ons uit de droom helpen?

         De heer HÜMMELS (Leefbaar Almelo): Ik weet niet wie er zit te dromen, maar de pure realiteit
is dat ik in de tweede termijn drie amendementen heb ingediend. Die amendementen heeft iedereen ook
via de e-mail ontvangen, om ze rustig te kunnen doorlezen. Dat heb ik ook gedaan, omdat ik van
mening ben dat dat een manier is om de papierstroom te beperken. Weliswaar werk ook ik mee aan die
papierstroom, en dat is natuurlijk niet zo handig, maar dat leer ik mijzelf misschien nog af.

       De heer STAPEL (PvdA): Sorry, voorzitter, hier ga ik toch even op in. Ik heb inderdaad drie
amendementen op schrift van de heer Hümmels ontvangen, maar zojuist heeft hij er nog twee naar
iedereen verstuurd via de e-mail.

        De VOORZITTER: Klopt dat, mijnheer Hümmels?

        De heer HÜMMELS (Leefbaar Almelo): Nee. Ik heb twee stukken verstuurd die nog geen
amendementen zijn die aangekondigd of ingediend zijn. Of dat gebeurt, hangt een beetje af van de
beantwoording door het college in de tweede instantie. Verder doelde de heer Stapel op een voorstel
op papier. Dat is geen officieel amendement, maar een denkstuk van twee fracties in de raad. Ik hoop
dat dit nu duidelijk is. Of het andere stuk wordt ingediend, hangt af van de beantwoording door het
college.



INT - 1201417                                                                                             66
</p><p>        De VOORZITTER: Dames en heren! Voor alle duidelijkheid: ik ga uit van de drie amendementen
die wij hebben gekregen van de fractie van Leefbaar Almelo. Voor het overige gaan wij het nog zien.

        Hierna schorst de VOORZITTER de vergadering.

                (De burgemeester draagt tijdens deze schorsing het voorzitterschap over aan de heer
                Stapel.)

         Na hervatting der vergadering spreekt de VOORZITTER als volgt:
         Dames en heren! U ziet op dit ogenblik een nieuw ordevoorstel geprojecteerd op de schermen in
deze zaal. Dat voorstel is gedaan vanwege de tijd die wij al nodig hebben gehad in deze
raadsvergadering. Ik stel voor dat wij na de beantwoording door het college bepalen hoe de rest
definitief wordt ingevuld.

         Mevrouw ROZEMA-JUNGJOHANN (D66): Voorzitter! Eén punt in het nieuwe ordevoorstel klopt
in ieder geval niet. Daarin staat dat er tot 17.00 uur een schorsing is en dat de tweede termijn voor het
college aanvangt om 16.40 uur. Dat kan niet kloppen.

       De VOORZITTER: Dat tijdstip is inmiddels inderdaad voorbij; dat had 17.10 uur moeten zijn,
maar dat wordt nu aangepast in het schema.

2e termijn college.

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Mijnheer de voorzitter! Ik zal proberen te
reageren in de volgorde waarin de vragen en opmerkingen aan mijn adres zijn gericht.
         De heer Schuur heeft gevraagd wat er overblijft van het internationale beleid, nu het college
heeft voorgesteld om daarop te bezuinigen. Ik snap dat hij die vraag heeft gesteld. Wat in ieder geval
overblijft van het internationale beleid, zijn de subsidies ter ondersteuning van de jumelagecomités. Die
blijven recht overeind. Wat ook overeind blijft - en ik hoop dat men mij goed begrijpt op dit punt - is het
feit dat wij willen dat de medewerkers, met name de beleidsambtenaren, de internationale aspecten
impliciet meenemen in hun werk. Daarvoor worden ze ook ondersteund door de subsidiedeskundige die
wij in huis hebben, want ook het werk daarvan gaat gewoon door. Daarnaast hebben wij ter zake
afspraken op regionaal niveau. Op regionaal niveau werken wij op dit moment met name samen op het
gebied van Eurocities. Daarbij gaat het om de duurzaamheidsagenda. Het is dus niet zo dat het
internationale beleid helemaal geschrapt is. Wel proberen wij dat slimmer te doen en impliciet te maken
in het werk.

       De heer SCHUUR (PvdA): Voorzitter! Ik heb het antwoord van de burgemeester gehoord,
waarvoor dank. Zou wat zij zei, ook moeten leiden tot herijking van de nota internationaal beleid die wij
met elkaar vastgesteld hebben?

        Mevrouw HERMANS-VLOEDBELD (burgemeester): Uiteindelijk is dat goed, denk ik. Er is al een
evaluatie van het internationale beleid geweest. Op basis daarvan heeft het college zijn voorstel op dit
punt naar de raad gebracht, terwijl dit onderwerp ook was meegenomen in de zoekrichtingen. Het lijkt
mij goed om het daar nog eens met elkaar over te hebben.
        Voorzitter! Ik ga nu in op het amendement (F) van de PvdA-fractie met betrekking tot "Het
verschil maken". In de eerste instantie heb ik daar al op gereageerd, maar ik ben blij met de
opmerkingen die daar in de tweede termijn over gemaakt zijn door de raad, omdat die mij de
mogelijkheid geven om nog wat preciezer op dit onderwerp in te gaan. Daarmee wil ik voorkomen dat
de raad het budget van € 125.000,-- voor het project "Het verschil maken" schrapt en dat bedrag
doorsluist naar het schoolzwemmen. Over deze laatste inhoudelijke keuze ga ik overigens niet, en
daarom zal ik daar niets over zeggen. Het college zou er erg verdrietig van worden als het bedrag van
€ 125.000,-- voor "Het verschil maken" wordt geschrapt. Ik zal daar nogmaals op reageren, maar ik zal
dat nu wat uitgebreider doen.

       De VOORZITTER: Maakt u eerst even dit punt af, mevrouw Hermans, dan geef ik daarna het
woord aan de heer Schuur, want ik zie dat hij wil reageren.




INT - 1201417                                                                                           67
</p><p>        De heer SCHUUR (PvdA): Dat is raar.

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Voorzitter! In mijn eerste reactie op
amendement F heb ik aangegeven dat wij twee soorten ambtenaren hebben (ik besef dat dit onaardig
klinkt). De ene soort wordt bekostigd uit het personeelsbudget en de andere soort uit projectgelden. De
medewerkers die worden ingezet voor het project "Het verschil maken", zijn eigen medewerkers. Dat
zijn geen extern ingehuurde mensen, maar mensen van de afdeling BC (ook de heer Nijhuis heeft daar
in zijn bijdrage op gewezen). Wij onderschrijven het belang van "Het verschil maken", maar dat doet de
heer Schuur ook, dacht ik. Het moet echter wel duidelijk zijn dat wij dit project met eigen ambtenaren
uitvoeren. Voor het inzetten van die mensen hebben wij geld nodig, en dat is het bedrag van
€ 125.000,--.

        De VOORZITTER: Dan geef ik nu de heer Schuur de gelegenheid om hierop te reageren.

        De heer SCHUUR (PvdA): Voorzitter! Wat ik zojuist eerst wilde zeggen, is dat het niet zo is dat
als het probleem met het schoolzwemmen er niet was geweest, wij gezegd zouden hebben: Oké, laat
dan het budget voor "Het verschil maken" maar staan. Wij zijn het namelijk per definitie niet eens met
het reserveren van een budget voor dat doel.
        Verder kom ik terug op het vorige punt, namelijk het feit dat wij fors bezuinigen op het
internationale beleid. Daarbij is gezegd: Impliciet blijven wij dat beleid nog wel voor een groot deel
uitvoeren, want dat brengen wij her en der onder bij de ambtenaren in de organisatie. Deze uitspraak is
voor mij een bevestiging dat dat ook prima zou kunnen gebeuren met het project "Het verschil maken".
De taken daarvoor zouden her en der bij ambtenaren in de organisatie neergelegd kunnen worden
zonder er een apart project voor te maken. Ik vind € 125.000,-- voor dit proces in deze tijd echt te veel
geld.

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Voorzitter! Met alle respect … Nee, dat mag
ik niet zeggen, want dat klinkt heel negatief, en zo bedoel ik het niet, maar de heer Schuur vergelijkt
appels met peren. "Het verschil maken" is echt een project op zich dat geïmplementeerd moet worden
in de organisatie, maar ook in het bestuur. Daarbij gaat het om deels het veranderen van
werkprocessen en deels het veranderen van de cultuur bij ons als lokale overheid. Als wij het hebben
over internationaal werken, dan mogen wij ervan uitgaan dat de internationale context wordt
meegenomen door de beleidsmedewerkers van onze gemeente. Dit is een totaal ander vraagstuk dan
het project "Het verschil maken". Met dit antwoord aan het adres van de PvdA-fractie heb ik volgens
mij ook gereageerd op de opmerkingen van de overige fracties die andere ideeën over "Het verschil
maken" hebben.
         De heer Cornelissen heeft het een en ander gezegd over Stadstoezicht. De transitie die wij in
gang hebben gezet van toezicht naar handhaving, werd door de heer Cornelissen gerelateerd aan de
opbrengst die gegenereerd wordt door Stadstoezicht, zoals men in de krant heeft kunnen lezen. Dit zijn
twee zaken die niets met elkaar te maken hebben. Het genereren van opbrengsten door de dienst is
één. Twee is dat wij daarnaast hebben besloten om meer kwaliteit te brengen in deze dienst. Dat
betekent dat wij investeren in opleiding, scholing en een kwalitatieve verandering, namelijk van toezicht
naar handhaving. Voor dit laatste heb je nu eenmaal andere kwaliteiten en competenties nodig. Dat
doet natuurlijk pijn, maar wij hopen wel dat deze verandering effect heeft (mijnheer Hümmels) op de
veiligheid en leefbaarheid in onze stad.

        De heer HÜMMELS (Leefbaar Almelo): Dat hoop ik ook, maar dat neemt niet weg dat wij van
mening zijn dat wel heel mooie woorden gebruikt worden voor wat voorgenomen is met betrekking tot
Stadstoezicht. Daar wordt een heleboel expertise overboord gegooid, ook al zegt u dat die van een te
laag niveau zou zijn en dat een kwaliteitsslag gemaakt moet worden.

        Mevrouw HERMANS-VLOEDBELD (burgemeester): Nee.

        De heer HÜMMELS (Leefbaar Almelo): Ik zal niet zeggen dat u dat eerste zei, maar u stelde wel
dat er een kwaliteitsslag gemaakt moet worden. Ik concludeer daaruit dat het zou lijken dat wat er
was, onvoldoende zou zijn. Volgens mij is aangetoond wat het nut is van Stadstoezicht. Daarom willen
wij heel graag alsnog enkele amendementen indienen met betrekking tot Stadstoezicht. Ik was al bang
dat er geen bevredigend antwoord zou worden gegeven met betrekking tot deze materie. Ik ben
benieuwd of u daar nog wat aan toe hebt te voegen.



INT - 1201417                                                                                         68
</p><p>        Mevrouw HERMANS-VLOEDBELD (burgemeester): Voorzitter! Het nut van Stadstoezicht wordt
niet betwijfeld. Sterker nog: Stadstoezicht vervult een wezenlijke rol in ons hele veiligheidsbeleid.
Verder heb ik het niet gehad over onvoldoende kwaliteit. Ik heb het gehad over…

        De heer HÜMMELS (Leefbaar Almelo): Een kwaliteitsslag maken.

       Mevrouw HERMANS-VLOEDBELD (burgemeester): …andere competenties die nodig zijn. Als wij
het hebben over de totale ploeg van stadstoezichthouders, hebben wij het niet alleen over de groep die
u waarschijnlijk in beeld heeft. Wij hebben het dan ook over mensen die bij ons in dienst zijn, met name
parkeerwachters. Zij hebben ook een handhavende rol, waardoor de totale ploeg van toezichthouders
toch nog behoorlijk groot blijft. Nogmaals: zij vervullen een belangrijke rol. Zij worden door ons, in
samenwerking met de politie, gericht ingezet, ook in de wijken en buurten. Ik vertel dit om aan te
geven hoe belangrijk de rol van Stadstoezicht hier is. Er is echter niets op tegen om af en toe - dat
doen wij ook voor de rest van de organisatie - te bekijken wat de huidige samenleving nodig heeft en
welke competenties dat vraagt van onze ambtenaren. Daarmee willen wij effectief kunnen werken aan
verandering van beleid.

        De heer HÜMMELS (Leefbaar Almelo): Ondanks uw verhaal, dat wel prettig in mijn oren klinkt,
heb ik toch niet het idee dat u daarmee bereikt wat wij wensen te bereiken. Wat wij willen, is dat de
aanwezige deskundigheid gehandhaafd blijft en dat de inzet daarvan gehandhaafd blijft. Zoals ook in
onze (nog in te dienen) amendementen en in onze voorbereidende stukken staat, willen wij dat de
kwaliteit en kwantiteit die er zijn, ingezet worden voor nieuwe initiatieven. Voor die initiatieven is
blijkbaar wel geld en daarvoor kan blijkbaar wel van alles gebeuren, maar daarvoor zouden juist ook de
stadstoezichthouders ingeschakeld kunnen worden. Dat is ook de strekking van onze amendementen
hierover. Die wil ik nu wil indienen, als dat moet. Anders wordt het heel erg ingewikkeld.

      De VOORZITTER: Ik geef nu eerst het woord aan de heer Cornelissen; daarna kunt u de
amendementen indienen.

         De heer CORNELISSEN (SP): Voorzitter! Ik wil de woorden die de burgemeester zojuist sprak,
even toetsen aan het vorige week uitgelekte stuk over hoe de landelijke politie eruit gaat zien. Het gaat
daarbij om wijkgericht werken, maar met sturing vanuit Den Haag. Kortom, een heel ingewikkeld
verhaal waarvan wij nog niet weten hoe dat eruit gaat zien. Wat wij wel weten, is dat de situatie heel
erg gaat veranderen. Wanneer wij als gemeente nu al beginnen met Stadstoezicht danig te veranderen,
zijn wij straks wellicht de daar aanwezige expertise kwijt. Die expertise hebben wij echter wel heel hard
nodig, omdat landelijk zal worden besloten de politie iets anders aan te gaan sturen. Nogmaals: wij
weten nog niet hoe de situatie dan eruit zal zien. Gelet daarop, meen ik dat wij iets te snel lopen met
wat voorgesteld is ten aanzien van Stadstoezicht. Wat is daarover de mening van de burgemeester?

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Voorzitter! Ik ga eerst in op de opmerking die
de heer Hümmels zojuist maakte. Hij is bevreesd dat wij de expertise weggooien die er nu is bij
Stadstoezicht. Dat is niet het geval, want de toezichthouders, die vooral een handhavende functie
krijgen, zullen geschoold worden om te kunnen handhaven, wat een vak apart is. Zij doen ook aan
toezicht. Die competentie gaat dus niet weg, maar wij voegen er wel wat aan toe. Ik ben van mening
dat men dat goed mag begrijpen.
         Toezichthouders inzetten in buurten en wijken, hebben wij tot nu toe ook gedaan, zo zeg ik
tegen de heer Cornelissen. Op zich heeft dat nog niet zoveel te maken met de transitie van de politie
zoals die nu georganiseerd is, naar de landelijke politie. Deze transitie zal uiteindelijk wel meegenomen
worden in de discussies die wij weer verscherpt zullen gaan voeren over het maken van nieuwe
partnerafspraken tussen rijksoverheid, politie en andere partners. De keuzes maken wij dan zelf. Ik zou
de discussie van dit moment niet willen verwarren met een discussie over de landelijke politie. Volgens
mij zullen wij daar nog gesprekken over voeren, althans voor zover wij daarover gaan. Dan zullen wij
het ook hebben over diverse vormen van toezicht. Daarbij kunnen wij dan ook onze eigen
toezichthouders en handhavers betrekken, maar dat heeft volgens mij niets te maken met de transitie
die het college voorstaat, namelijk van toezicht houden naar handhaven.

      De VOORZITTER: Dames en heren! Ik geef de heer Hümmels nu de gelegenheid om de door
hem aangekondigde amendementen in te dienen.



INT - 1201417                                                                                         69
</p><p>         De heer HÜMMELS (Leefbaar Almelo): Ik wil nog wel even een toelichting geven. Ik heb, om
het college tegemoet te komen, twee amendementen gemaakt met betrekking tot Stadstoezicht. Ik wil
eigenlijk graag dat ze allebei in stemming worden gebracht. Het ene amendement gaat wat verder dan
het andere. Wij zien straks wel hoe ver ze komen. De twee amendementen luiden:

                "Amendement ZA.
                Stadstoezicht 1
                Betreft het voorstel begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Besluit:
                - Aan de tekst m.b.t. het stadstoezicht het volgende te wijzigen:
                - de voorgenomen wijziging van de opzet van stadstoezicht schrappen;
                - een combinatie maken van de verschillende initiatieven als burgernet en buurtpreventie,
                    waarbij stadstoezicht een sturende en begeleidende rol kan spelen.
                Toelichting:
                Als je een veilige stad wilt bieden lijken de maatregelen bij Stadstoezicht onlogisch, en
                deze zouden zelfs financieel een tegenovergesteld effect kunnen hebben. LA stelt voor de
                initiatieven als Burgernet en Buurtpreventieteams te combineren met stadstoezicht,
                waardoor de verworven kwaliteiten van deze toezichthouders blijvend ingezet kunnen
                worden. Voor LA is preventie een belangrijke voorwaarde voor het nastreven van een veilig
                woon- en leefmilieu."

       Dit amendement is ondertekend namens de fracties van GroenLinks, PVA, LKA, de SP, de Lijst
Çete en Leefbaar Almelo.

                "Amendement AB.
                Stadstoezicht 2
                Betreft het voorstel begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen d.d. 8 en 10 november 2011.
                Besluit:
                - Aan de tekst m.b.t. het stadstoezicht het volgende aan te vullen:
                - de voorgenomen wijziging van de opzet van Stadstoezicht zoals voorgesteld deels te
                    realiseren;
                - daarnaast een combinatie maken van de verschillende initiatieven als burgernet en
                    buurtpreventie, waarbij de overige stadstoezichthouders een sturende en begeleidende
                    rol kunnen spelen.
                Toelichting:
                Als je een veilige stad wilt bieden lijken de maatregelen bij Stadstoezicht onlogisch, en
                deze zouden zelfs financieel een tegenovergesteld effect kunnen hebben. LA stelt voor de
                initiatieven als Burgernet en Buurtpreventieteams te combineren met stadstoezicht,
                waardoor de verworven kwaliteiten van deze toezichthouders blijvend ingezet kunnen
                worden. Voor LA is preventie een belangrijke voorwaarde voor het nastreven van een veilig
                woon- en leefmilieu."

       Dit amendement is ondertekend namens de fracties van GroenLinks, LKA, de SP, de Lijst Çete
en Leefbaar Almelo.

        Mevrouw HERMANS-VLOEDBELD (burgemeester): Mijnheer de voorzitter! De fractie van D66
heeft ook in de tweede termijn nog wat gezegd over een peiling onder de burgers, met name om de
burgers te bevragen over sparen en lenen (ik zeg het nu even wat huiselijk, met een paar woorden). Ik
snap de zorg van de D66-fractie. Ook snap ik de ambitie van die fractie om de burgers in toenemende
mate te betrekken bij het beleid van de overheid. Ook wij staan dat voor, maar wij zijn van mening dat
de door de fractie van D66 genoemde onderwerpen zich vanwege hun complexiteit en gelaagdheid niet
lenen voor een enkelvoudige, bijna referendumachtige opzet. Ik snap dat dit één van de kroonjuwelen is
van D66, en die mag overal voor vechten, maar ik meen dat de nu door de D66-fractie voorgestelde
peiling niet effectief is.
        En dan nog iets. Wij werken in Almelo succesvol met het Omnibuspanel. Daar doen veel
Almeloërs aan mee, maar daarmee bereiken wij niet alle groepen burgers in onze stad. Het blijft een



INT - 1201417                                                                                         70
</p><p>uitdaging om te zoeken naar de juiste manier om ook andere burgerschapsstijlen (daar hebben wij het al
eens over gehad) te betrekken. Dat is een opdracht aan het college, maar ook aan de raad.
        Tot slot van mijn antwoord over de door de fractie van D66 voorgestelde peiling, merk ik op dat
de raad in mijn ogen nog altijd de hoogste vorm van burgerparticipatie is. Het is prachtig dat de raad
zijn mening en oordeel mag geven over het soort complexe zaken waar wij nu over spreken en tot
besluitvorming daarover mag komen.
        De fractie van D66 heeft een aangepaste motie ingediend, namelijk motie P1, met betrekking
tot het realiseren van extra bezuinigingen door te focussen op samenwerking. De D66-fractie gaat
ervan uit dat daardoor een structurele besparing van € 1,5 miljoen ingeboekt kan worden. Ik wil de
raad ervoor waarschuwen zich niet op voorhand rijk te rekenen. Als college nemen wij de winst - ik
hoop dat ook de raad dat wil doen - op het moment dat die zich voordoet. Ook motie P1 wordt door
het college ontraden.

        De heer GERRITSEN (D66): Voorzitter! Het is mooi dat de burgemeester hier even over begint.
Wij zullen ook motie P1 aanpassen. In de nieuwe versie van de motie zullen wij het bedrag van € 1,5
miljoen weglaten uit punt a van het dictum, zodat daarin komt te staan: "uitbesteding van werk in de
regio te realiseren en daarbij te streven naar een structurele besparing".

         Mevrouw HERMANS-VLOEDBELD (burgemeester): Oké. Dat maakt de opdracht aan het college
met betrekking tot het stimuleren van regionale samenwerking sterker. Dan zou ik de motie namens het
college kunnen omarmen. Weliswaar heb ik dat niet getoetst, maar ik denk wel dat dat het gevolg is.
Anders heb ik straks een probleem, maar goed, dat is dan mijn zaak!
         Voorzitter! De heer Nijhuis heeft in de tweede instantie nog iets gevraagd en geponeerd met
betrekking tot de gemeentelijke publicaties. In zijn algemeenheid ben ik daar in de eerste termijn op
ingegaan. De VVD-fractie heeft gezegd: College, zou u niet eens kunnen kijken naar de dure brochures
die de gemeente uitgeeft. Dat vinden wij een interessante gedachte. Wij willen daar graag naar kijken
en komen daar bij de volgende voorjaarsnotabehandeling op terug.
         De fractie van ALA-AOV was blij met het antwoord dat ik heb gegeven met betrekking tot de
artikel 12-status. Ik moet zeggen dat de heer Stork mijn antwoord onjuist heeft geïnterpreteerd en dat
zelfs een beetje naar zichzelf toe heeft geredeneerd. Overigens denk ik dat hij dat wel weet. Wat ik
over de artikel 12-status heb gezegd, was echt een serieuze waarschuwing. Het hebben van een
dergelijke status moet men niet lichtzinnig zien en men kan daar niet lichtvaardig mee omgaan.
Wanneer de gemeente een artikel 12-status heeft, wordt voorbijgegaan aan de eigen
verantwoordelijkheden van raad en bestuur. Bovendien betekent dat nogal wat, maar het betekent niet
winst of gouden bergen. Wel betekent het dat de gemeente verplicht wordt om voordat ze überhaupt
de artikel 12-status kan krijgen, op zoek te gaan naar mogelijkheden om het weerstandsvermogen te
verruimen en om te bekijken of het weerstandsvermogen wel op de juiste manier gebruikt wordt. Dit
houdt in dat de gemeente gedwongen wordt om de inkomsten te vergroten door de ozb-tarieven op te
plussen enz. en om tegelijkertijd de uitgaven naar beneden te brengen. Wanneer de gemeente een
artikel 12-status heeft, betekent dat niet dat de gemeente geassisteerd wordt door de provincie. Nee,
het betekent dat de beslissingsbevoegdheid over substantiële zaken en over uitgaven, ook al zijn die
voor een doosje punaises, wordt overgenomen door de provincie en het rijk. Ik zou daar niet
lichtvaardig over denken, maar wat ons betreft, is een artikel 12-status ook niet aan de orde.

         De heer STORK (ALA-AOV): Voorzitter! Ik denk daar zeker niet lichtvaardig over. Ik heb zelf die
situatie meegemaakt hier in de gemeente, maar ik heb daar helemaal niet zo'n trauma aan
overgehouden als het college kennelijk heeft. Ik meen dat de voorstelling van zaken die de
burgemeester nu geeft, iets verder gaat dan de wet, tenzij de regels veranderd zijn. In onze beleving is
de artikel 12-status juist nodig om het college te assisteren en min of meer onder druk te zetten (dit
zijn mijn woorden) om heel rigoureuze maatregelen door te drijven. Het college heeft dan voor die
maatregelen een, zeg maar, legitieme verantwoording die het nu niet krijgt van de raad. Daarom zou ik
het aanvragen van een artikel 12-status heel serieus willen overwegen. Wanneer de gemeente die
status heeft, is het heus niet zo dat ze ieder doosje punaises moet vragen. Nee, dat geldt alleen voor
heel belangrijke beslissingen. Wat de vorige keer gebeurde, is dat een toezichthouder werd aangesteld
die echt niet helemaal gek was en die zich niet bemoeide met details. Nee, de toezichthouder hield
gewoon de grote lijnen in de gaten en had heel goed overleg met het college. Het college had toen
helemaal geen nare gevoelens daarbij.




INT - 1201417                                                                                         71
</p><p>        De heer BRAND (CDA): Voorzitter! Ik wil graag de woorden ondersteunen die de burgemeester
zojuist sprak over de artikel 12-status. Ik weet niet of ik er goed aan doe om het te zeggen, maar ik
doe het wel. Als burger van de gemeente Almelo en als werknemer van de gemeente Almelo in de tijd
waar de heer Stork op doelde, weet ik dat het zijn van artikel 12-gemeente wel degelijk invloed heeft
op het functioneren en de mogelijkheden van de gemeente in de zin die de burgemeester heeft
aangegeven.

       De heer STORK (ALA-AOV): Ik ontken niet dat de artikel 12-status wel degelijk invloed heeft,
maar ik ben niet zo bang voor die invloed. Dat is het grote verschil van mening.

        De VOORZITTER: Uw punt is gemaakt. Het woord is aan mevrouw Hermans.

        Mevrouw HERMANS-VLOEDBELD (burgemeester): Voorzitter! Ik vind toch dat de heer Stork te
kort door de bocht gaat. Dat is wat mij betreft ook de gemakkelijkste weg. Het college komt niet eens
toe aan de vraag of Almelo een artikel 12-gemeente moet worden. Er zijn volgens mij nog voldoende
mogelijkheden om bij te sturen. Overigens is de raad daar ook zelf bij. Door de weg naar een artikel 12-
status te kiezen, zou de raad zichzelf tekortdoen. Kortom, op dit moment is er voor het college geen
reden, gezien de sluitende begroting, om richting de artikel 12-status te gaan. Overigens heb ik zelf ook
ervaring met werken in een artikel 12-situatie. Ik kan op basis daarvan melden dat de gemeente dan
zeker ontheven wordt van haar bevoegdheden om zelfstandig besluitvorming te plegen.

                (De heer Stapel draagt daarop het voorzitterschap over aan de burgemeester.)

         Mevrouw KUIK-VERWEG (wethouder): Voorzitter! Mijn beantwoording in de tweede termijn
begin ik met wat de CDA-fractie heeft gezegd. Mevrouw Ten Seldam sprak in de tweede termijn nog
over het welzijnsbeleid. Zij zei dat zij geen aparte nota daarover wenst, maar dat zij daar op dit moment
wel onvoldoende zicht op heeft. Ik koppel dit aan de opmerking van de heer Schuur dat hij graag een
integrale visie wil op de sociale opgaven van de gemeente. Daar zijn wij nu juist mee bezig, namelijk
met het nieuwe sociale offensief, dat wij Sociale Agenda 2020 noemen. Binnenkort worden de
verschillende maatschappelijke opgaven ter vaststelling voorgelegd aan de raad. Op dit moment worden
in samenspraak met partners in de stad de maatregelen benoemd (de heer Veenstra gaf dat al aan). Die
maatregelen moeten straks antwoord geven op de maatschappelijke opgaven. Overigens speelt de
Sociale Agenda 2020 ook in op de bezuinigingen die op ons af komen en op de nieuwe taken die wij
van het rijk krijgen. Bij dit laatste denk ik aan de jeugdzorg, de AWBZ en de nieuwe Wet werken naar
vermogen. Deze zaken lopen via een apart traject, maar passen perfect in de visie die wij voorstaan
voor de Sociale Agenda 2020. Ik denk dat wij een en ander bij de volgende voorjaarsnota als een
geheel kunnen presenteren aan de raad, hoewel ik verwacht dat de Sociale Agenda 2020 op zich
eerder naar de raad toe zal gaan.
         De heer Hümmels hoopt dat de maatregelen ten aanzien van de Stadsbank geen nadelige
effecten zullen hebben. Zoals ik eerder al heb aangegeven, zal iedereen in onze stad wel wat merken
van de bezuinigingen die wij nu moeten realiseren. Wat de Stadsbank betreft, hebben wij bekeken hoe
de zaken anders zouden kunnen. Als bijvoorbeeld iemand drie keer in een jaar komt om een
schuldenregeling of een budgetbeheerregeling, dan geven wij daar geen medewerking meer aan (ik heb
dit eerder al aangegeven). Uiteraard moeten de mensen aan de voorkant goed geïnformeerd worden dat
zij daar rekening mee moeten houden. Ook moeten de mensen een contract ondertekenen. Verder
schrijft de Stadsbank zelf: Ook als je € 80.000,-- verdient, kun je naar de Stadsbank. Daar is op zich
niets mis mee, alleen meen ik dat de gemeente niet meer moet betalen.

        Mevrouw ROZEMA-JUNGJOHANN (D66): Mevrouw Kuik! Bent u bereid om met uw collega
Myra Koomen uit Enschede werk te maken van de situatie rond het innen van achterstallige
zorgpremies (laat mij maar gewoon zeggen: schulden op de zorgpremies). Misschien heeft u dat al
gedaan, maar dat weet ik niet. Wilt u ervoor zorgen dat ook dit onderwerp ondergebracht kan worden
bij de Stadsbank? Ik vraag dit met name, omdat de mensen die een schuld hebben op de zorgpremie,
gigantisch gepakt worden. Ik vind dat eigenlijk buiten verhouding. Ik begrijp niet waarom
zorgverzekeraars niet dezelfde behandeling kunnen krijgen (wanneer het om schulden gaat) als
bijvoorbeeld Cogas of een woningcorporatie.

        Mevrouw KUIK-VERWEG (wethouder): Ik ben daartoe bereid, want ik maak deel uit van
hetzelfde dagelijks bestuur als mevrouw Koomen. De betrokken brief heeft volgens mij ook in het VNG-



INT - 1201417                                                                                         72
</p><p>magazine gestaan. Mevrouw Koomen en ik zijn samen bezig met deze zaak, maar die moet wel breed
gedragen worden in de AB-vergadering die wij binnenkort hebben. Wij zijn van mening (dat hebben wij
ook gezegd tegen de Stadsbank) dat de partijen die geld van de mensen krijgen, de schulden niet te
hoog moeten laten oplopen en eigenlijk mee moeten betalen. Waarom? Omdat zij er ook belang bij
hebben zoveel mogelijk geld te ontvangen. Dat mag ze dan ook wel iets waard zijn.

        De heer HÜMMELS (Leefbaar Almelo): U maakte zojuist een opmerking over mensen die bij de
Stadsbank komen terwijl zij een vrij hoog inkomen hebben en het toch niet redden. Ik vraag of het u
ook lukt om ervoor te zorgen dat de mensen die niet zo'n hoog inkomen hebben - en dus eigenlijk de
hulp echt nodig hebben - meer geholpen kunnen worden dan de mensen die wel een hoog budget
hebben. Dat kan bijvoorbeeld door de mensen met een hoog inkomen een hogere bijdrage te laten
betalen. Ik vraag dit, omdat ik niet weet hoe dit precies in elkaar zit. Gelukkig heb ik er zelf nog niet zo
vaak gelopen.

        Mevrouw KUIK-VERWEG (wethouder): Wij hebben de grens gelegd bij 115% van de
bijstandsnorm (dat staat volgens mij ook in het stuk dat de raad heeft gekregen), de grens die wij ook
hanteren voor de minimaregeling. Mensen met een hoger inkomen moeten een deel van de kosten van
de dienstverlening zelf gaan betalen. Overigens moeten die kosten dan in de aflossing meegenomen
worden, wat betekent dat de crediteuren wat minder krijgen. Het is in ieder geval zo dat diegenen
geholpen worden die echt aan de onderkant zitten, die het echt nodig hebben en vooral die niet door
eigen schuld in de problemen zijn gekomen. Verder haak ik in dit verband aan bij wat is gezegd over het
aan de voorkant inschakelen van vrijwilligers. Dat is nogal lastig. Zoals ik vanmorgen heb gezegd, vind
ik het prima wanneer de mensen van Almelo Sociaal aan de voorkant dingen doen. Overigens zij niet
alleen, maar meerderen. Wij zullen de betrokkenen daarvoor uitnodigen. Het wordt echter wel heel
lastig om hun een taak te geven in de nazorg. Ten eerste zijn daarvoor professionele mensen nodig. En
ten tweede moet je je afvragen of mensen van een belangenorganisatie dat moeten doen. Waar wij wel
over in gesprek zijn met de Stadsbank, is de echte nazorg (dat is andere nazorg dan de zorg waar de
raad het eerder over had). Wij willen dat vroegtijdig bekeken wordt dat mensen in een zorgtraject
terechtkomen. Dat is een plus waar wij vanaf nu voor gaan zorgen.
        De heer Çete zei in de tweede termijn: De wethouder heeft niets gezegd over de kansarme
allochtonen waar ik het over had. Ja, dat klopt, maar hij heeft mij daar ook niets over gevraagd. De
heer Çete heeft wel veel zaken opgesomd die spelen. Voor mij is elke allochtoon die in de ellende zit
hetzelfde als elke autochtoon die in de ellende zit. Daar hebben wij ook dezelfde voorzieningen voor.
Dat zeg ik dan bij dezen, maar hij heeft geen vragen aan mij gesteld.

        De heer ÇETE (Lijst Çete): Wat ik zei, betreft misschien niet uw portefeuille, mevrouw Kuik.
Wat betreft de portefeuille/het programma Aantrekkelijke stad, wilde ik duidelijk maken dat de
kansarmen en de groep allochtone gezinnen onze gepolariseerde samenleving extra onaantrekkelijk
maken. Kortom, mijn opmerking was niet zozeer voor uw portefeuille bedoeld, als wel gericht op het
programma Aantrekkelijke stad. Ik wilde duidelijk maken dat een groei van het aantal allochtonen een
stad helaas niet aantrekkelijker maakt. Dat was mijn bedoeling, niet een aparte behandeling van
allochtonen en autochtonen. Dat was niet mijn vraag.

        Mevrouw KUIK-VERWEG (wethouder): Misschien komt mijn collega daar nog op terug.
        Voorzitter! Ik kom nu bij de inbreng in de tweede termijn van de ChristenUnie-fractie. De heer
Veenstra heeft met betrekking tot het Werkplein een aangepaste motie (I1) ingediend. Ik verzoek hem,
zoals ik ook vanmorgen heb gedaan, om nog even af te wachten in dezen. Wij zijn in gesprek met het
UWV. Er zijn zes gemeenten waar Almelo het Werkplein mee vormt. Die gemeenten moeten straks
verder met elkaar. Het aantal cliënten wordt verdubbeld, gezien alle taken die naar de gemeenten gaan.
Vandaar dat ik nogmaals zeg: Geef ons even de tijd om straks (nog in november) de gesprekken te
voeren. Vervolgens komen wij met de uitslag daarvan bij de raad. Dan kunnen wij bekijken hoe wij de
zaken verder invullen.

        De heer VEENSTRA (ChristenUnie): Voorzitter! De wethouder zegt nu dat zij nog in november
gesprekken zal voeren over de vestiging van het Werkplein in het Twenthe Centrum. Ik vraag om een
heroverweging, om in maart volgend jaar met een voorstel te komen en om inzichtelijk te maken wat
de consequenties zijn. Het college zegt dat als gevolg van de korting op het participatiebudget het
aantal personeelsleden drastisch naar beneden zal gaan.




INT - 1201417                                                                                             73
</p><p>        Mevrouw KUIK-VERWEG (wethouder): Inderdaad.

        De heer VEENSTRA (ChristenUnie): Het is dan ook heel goed om eens op een rij te zetten wat
er volgens de nieuwe inzichten bij het Werkplein ondergebracht wordt. Dat moet toch in maart 2012
kunnen?

        Mevrouw KUIK-VERWEG (wethouder): Laat mij dan zeggen dat wij onze uiterste best daarvoor
zullen doen. Waar u nu naar vraagt, komt volgens mij uit onze gesprekken en uit de afweging die wij
maken met betrekking tot het Twenthe Centrum.

        De heer VEENSTRA (ChristenUnie): Dat zou dus in maart 2012 kunnen, als streven?

       Mevrouw KUIK-VERWEG (wethouder): Als het goed is, dan is al in februari 2012 het Twenthe
Centrum zo ver gereed. Dan zou de zaak zeker in maart helder moeten kunnen zijn.

        De heer VEENSTRA (ChristenUnie): Dank u wel.

        Mevrouw KUIK-VERWEG (wethouder): Voorzitter! Ik maak nu graag een opmerking richting de
SP-fractie met betrekking tot de aansluiting van minimahuishoudens op internet. Het is sympathiek dat
die fractie daarover zei: Wij wachten nog even. Men moet van mij aannemen dat ik mijn uiterste best
doe op dit gebied en dat het gewoon heel ingewikkeld is om binnen te komen bij de betrokken partijen.
Ik zeg toe dat ik naar de raad toe kom als duidelijk is of het wel of niet mogelijk is om een bijdrage te
laten geven voor internetaansluitingen. Daarover zal ik dan in gesprek gaan.
        De VVD-fractie heeft in de tweede termijn geen vragen aan mij gesteld. Ik vraag mij echter af of
de VVD-fractie nog voornemens is om haar amendement (AA) met betrekking tot de overschrijding van
budgetten voor de openeinderegelingen te wijzigen, zoals de heer Nijhuis eigenlijk zei in het debat.

        De heer NIJHUIS (VVD): Voorzitter! Daar zijn wij mee bezig. Wij bekijken namelijk met een
aantal fracties of wij daar gezamenlijk uit kunnen komen. Wat ik nog wel belangrijk vind en wat ik de
wethouder gevraagd heb, betreft de definitie. Wij hebben gezegd dat eventuele overschrijdingen op de
openeinderegelingen in principe binnen het sociale domein moeten worden opgevangen. Naar aanleiding
daarvan noemde de wethouder het weghalen van geld voor zaken waarover ook wij zeggen: Dat is
onze bedoeling niet. Wij willen uitgaan van de drietrapsraket die ik heb geschetst, maar de wethouder
gebruikte de term "kokers binnen het sociale domein". Heb ik dat goed verstaan? Dat is de vraag die ik
nog had.

         Mevrouw KUIK-VERWEG (wethouder): Ik gebruikte inderdaad de term "koker", maar je mag het
ook anders noemen. Wij hebben te maken met de WSW, de WWB, gehandicapten, ouderen en noem
maar op. Voor elk van die onderwerpen hebben wij een budget. De uitvoering van de WWB wordt
sowieso ingewikkeld. Het kan immers zomaar gebeuren dat een bedrijf failliet gaat, waardoor ik weet
niet hoeveel mensen een uitkering moeten krijgen. De financiële consequenties daarvan zijn dan niet op
te vangen binnen de budgetten. Wanneer er budgetoverschrijdingen zijn, dan zijn die er niet zomaar.
Wij zeggen niet: Wij begroten maar wat. In het verleden hebben wij wel eens gedacht (dat heb ik ook
aangegeven bij de voorjaarsnotabehandeling): Het gaat om een openeinderegeling; aan het eind trekken
wij de zaak wel recht. Dat doen wij niet meer. Er vinden nu heel veel zaken plaats, zoals al is
aangegeven in de najaarsnota 2010. Die zijn wij nu aan het uitvoeren. Als zich daarbij overschrijdingen
voordoen, worden die in principe opgevangen binnen de koker. Als dat niet het geval is, dan komen wij
bij de raad om te zeggen dat er een aanvulling moet komen.

         De heer NIJHUIS (VVD): Voorzitter! Dan denk ik dat onze opvatting spoort met wat de
wethouder zegt. Wij willen substantiële overschrijdingen op de openeinderegelingen in drie stappen
laten opvangen:
1. eerst kijken binnen de kokers van het sociale domein;
2. lukt dat niet, dan komt het college daarmee bij de raad om samen te bekijken of de overschrijding
    elders in de begroting kan worden opgelost;
3. lukt ook dat niet, dan kan het geld geput worden uit de algemene reserve.
         Over deze drietrapsraket hebben wij het.




INT - 1201417                                                                                          74
</p><p>        Mevrouw KUIK-VERWEG (wethouder): Ja, maar die tweede trap is ingewikkeld. Dan zou ik u
liever een grote stap laten maken en de tweede trede er uithalen…

        De heer NIJHUIS (VVD): Nee, nee.

        Mevrouw KUIK-VERWEG (wethouder): …want die kan niet. U kunt wel "nee, nee" zeggen,
maar dat is wel wat ik wil. Waar het om gaat, is het volgende. Bij het opstellen van de begroting
schatten wij de zaken in (dat doen wij altijd). Dan gaan wij uit van ongeveer zoveel mensen met een
uitkering en wat dat betekent. Gaandeweg de uitvoering gebeuren er dan dingen. Die kun je niet voor
zijn. Overigens krijg je soms ook meevallers. Straks krijgen wij (en dat kun je een meevaller noemen)
wat extra geld van het ministerie. Dat krijgen wij omdat er zulke grote overschrijdingen zijn, en daarom
doet het ministerie er een beetje bij. Het kan echter niet zo zijn dat wanneer er overschrijdingen zijn van
bijvoorbeeld het budget voor de WWB-uitvoering, die ten koste gaan van de sport of van het geld voor
bejaarden. Dat is de reden waarom ik de tweede trede niet zie zitten die de heer Nijhuis noemde.
Volgens mij bedoelen wij hetzelfde, alleen maakt hij een extra stapje.

        De heer SCHUUR (PvdA): Voorzitter! Ik vind het een beetje vreemd dat voor het opvangen van
overschrijdingen op de openeinderegelingen zozeer geconcentreerd wordt op het sociale domein. Ik
noem bijvoorbeeld de regeling voor het leerlingenvervoer. Ook dat is een openeinderegeling, maar die
heeft volgens mij weinig met het sociale domein te maken. Het leerlingenvervoer zit ook in een heel
ander programma. Volgens mij moet gezegd worden dat je overschrijdingen op de openeinderegelingen
in eerste instantie begrotingstechnisch binnen de programmabegroting probeert op te lossen. Als dat
niet lukt, moet de raad betrokken worden. Dat is m.i. wat ook de VVD-fractie wil. Die wil niet dat
overschrijdingen op de openeinderegelingen klakkeloos ergens weggemoffeld worden (niet dat dit nu
gebeurt, overigens; dat hoort men mij niet zeggen). De VVD-fractie wil dat de raad daar een
bevoegdheid in krijgt. Dan moet de VVD-fractie ook een amendement met die strekking indienen.
Daarin kan dan gesteld worden dat wanneer er overschrijdingen van de budgetten voor de
openeinderegelingen zijn die niet binnen de programmabegroting opgelost kunnen worden, de raad aan
bod komt om te beslissen hoe de overschrijdingen dan wel opgelost moeten worden.

        Mevrouw KUIK-VERWEG (wethouder): Ja, maar dat wordt in de berap en in andere rapportages
al zichtbaar.

      De VOORZITTER: Dames en heren! Ik geef nu het woord aan de heer Veenstra, want ik wil
voorkomen dat er discussie ontstaat tussen twee mensen die interrumperen.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Volgens mij is ook voor de Sociale Agenda 2020
het discussiepunt hoeveel middelen er nodig zijn. Daarom moet volgens mij een heel groot deel van
deze discussie al zichtbaar worden in de Sociale Agenda 2020. Ik bedoel de discussie over de vraag
voor hoeveel middelen de maatschappelijke opgaven uiteindelijk gerealiseerd kunnen worden in
samenspraak met de maatschappelijke partners. Ik wil dit punt nadrukkelijk meegeven in de discussie
over de Sociale Agenda 2020, want anders wordt het heel ingewikkeld om de zaak op te lossen.

        Mevrouw KUIK-VERWEG (wethouder): Maar die discussie voeren wij nu ook. Dat zei u zelf ook.

        Mevrouw TEN SELDAM-EVEN (CDA): Ik wil graag een opmerking maken ter ondersteuning van
wat de heer Schuur zojuist zei, want wij zitten in dezen op dezelfde lijn. De wethouder gaf aan dat het
college al een heleboel maatregelen heeft getroffen, in tegenstelling tot in periodes waarin dat minder
noodzakelijk was. Zij stelde dat wanneer een interventie verricht wordt waarvan mocht blijken dat die
niet toereikend is, de gevolgen dan opgevangen moeten worden binnen de programma's. Lukt dat niet,
dan komt de wethouder naar de raad met een voorstel. Volgens mij kunnen wij dan niets anders doen
dan de middelen uit de algemene reserve halen. Dat is de drietrapsraket die wij voor ogen hebben.

        De heer NIJHUIS (VVD): Ik heb nog een definitievraag: Wat wordt bedoeld met de programma-
begroting? Volgens mij is het stuk dat nu voorligt, bijna het hele document, überhaupt een
programmabegroting. Of gaat het om programma's binnen de begroting?

       Mevrouw TEN SELDAM-EVEN (CDA): Als ik het heb over programma's, bedoel ik de
programma's binnen de begroting. Dat zijn er vier.



INT - 1201417                                                                                           75
</p><p>        De VOORZITTER: Dat klopt.
        Dames en heren! Ik heb van de griffier gehoord dat er op dit moment een warme snack voor u
gereed staat in de ruimte hiernaast. Ik stel voor dat wij daarom nu tien minuten schorsen, om te
voorkomen dat de snack koud wordt of dat mensen met snack en al naar binnen komen. Als u dat niet
wilt, mijnheer Gerritsen, dan blijft u gewoon zitten.

          Hierna wordt gepauzeerd.

          Na hervatting der vergadering verleent de VOORZITTER het woord aan de heer Schouten.

        De heer SCHOUTEN (wethouder): Voorzitter! In de tweede instantie wil ik nog een vijftal
punten toelichten. Het eerste daarvan is het onderwerp: de groenste stad van Nederland. De CDA-
fractie heeft het voorstel gedaan om de gemeente Almelo voor die titel te kandideren en vroeg om
daarvoor een uitgewerkt voorstel aan te leveren bij de voorjaarsnota 2012. Het college vindt dat een
sympathiek voorstel. Ik zeg dan ook graag toe dat wij het door de CDA-fractie gevraagde voorstel
zullen aanleveren in het kader van de voorjaarsnota. Overigens zeg ik daar op voorhand wel bij dat daar
natuurlijk een kostenplaatje bij hoort. In het afgelopen voorjaar hebben wij al een keer intern over dit
onderwerp gesproken, en toen bleek dat dat een behoorlijk kostenplaatje is. Er zijn al meerdere steden
hiermee bezig geweest, zodat wij wat referentiemateriaal hebben. Daar komen wij in het volgende
voorjaar op terug bij de raad. Verder wil ik hier nog bij zeggen dat ik al tijdens het betoog dat mevrouw
Ten Seldam in de tweede termijn hield - en dat is wel heel opmerkelijk - een telefoontje kreeg van de
organisator van het hele gebeuren. Die wilde mij al spreken daarover. Toen dacht ik: De lijnen zijn wel
heel kort vandaag!

          De VOORZITTER: Het woord is aan mevrouw Ten Seldam. Zij gaat nu vertellen dat zij daar blij
mee is!

        Mevrouw TEN SELDAM-EVEN (CDA): Ik ben hier onwijs blij mee, met name met het feit dat de
wethouder zo ontzettend snel handelt. Ik wil nu aangeven waarom wij gevraagd hebben om bij de
voorjaarsnota te komen met een voorstel om Almelo te kandideren als groenste stad van Nederland. De
reden daarvoor is dat juist de financiële paragraaf buitengewoon belangrijk is. Die hoort erbij. Vandaar
dat wij vragen om bij de voorjaarsnota een voorstel te presenteren en om niet nu al een besluit te
nemen over dit onderwerp.

         De heer SCHOUTEN (wethouder): Wij zullen dit meenemen bij het opstellen van de
voorjaarsnota, voorzitter, dus het gaat helemaal goed komen. Dan kan de raad bepalen wat hij daarmee
wil.
         Het tweede onderwerp waar ik op in ga, is de Stadsboerderij Beeklust, oftewel de
kinderboerderij. Ik heb in de eerste instantie al aangegeven in de richting van de heer Buitenweg dat het
college sympathiek staat tegenover zijn gedachte om de stadsboerderij open te houden. Ik heb toen
duidelijk uitgesproken, ook in de richting van de PvdA-fractie, dat wat het college betreft, sluiting geen
optie is. In de motie (VA) die met betrekking tot de stadsboerderij is ingediend, wordt echter
vooruitgelopen op de uitkomsten van gesprekken die wij nog aan het voeren zijn. Daar heb ik moeite
mee. In het dictum van de motie wordt het college opgedragen te zoeken naar mogelijkheden om de
stadsboerderij vanaf 2013 open te houden. Dat is geen enkel probleem, want daarmee zijn wij het
eens. In de motie wordt daar echter bij gesteld dat de gemeente anders de exploitatie in eigen beheer
moet nemen. Dat wil ik niet overnemen, want wij gaan juist bezuinigen op de stadsboerderij. Dat heeft
de raad zelf bepaald bij de vaststelling van de voorjaarsnota. Wij gaan nu gesprekken voeren over de
kinderboerderij, zoals ik vanmorgen zei, in eerste instantie met De Twentse Zorgcentra. Als die
resultaat opleveren, dan is dat prima. Voor het geval dat de gesprekken geen resultaat opleveren, heb
ik inmiddels al twee alternatieve aanbiedingen binnen van partijen die met ons in gesprek willen gaan
over de kinderboerderij. De bedoeling is om één van de partijen de exploitatie te laten voortzetten. Om
dan te zeggen dat de gemeente anders de exploitatie maar in eigen beheer moet nemen, is absoluut
niet mijn conclusie. Dan zou de kwestie eerder zijn: Is men bereid om het voor het betrokken bedrag te
doen, of zit er mogelijk een verschil in? Het is altijd prettig om te kunnen zeggen dat zich meerdere
partijen hebben aangemeld; het marktmechanisme werkt namelijk op die manier. Voorshands ga ik er
dan ook van uit dat het ons gaat lukken om de bezuiniging op de kinderboerderij te halen. Ik ontraad




INT - 1201417                                                                                          76
</p><p>daarom ervan uit te gaan dat de gemeente anders de exploitatie in eigen beheer zou moeten nemen.
Als die bepaling geschrapt wordt, dan heb ik geen enkel probleem met motie VA.

     De heer BUITENWEG (GroenLinks): Voorzitter! Volgens mij zijn de wethouder en ik het voor
95% met elkaar eens, dat is de discussie niet.

        De heer SCHOUTEN (wethouder): Over dit punt wel!

        De heer BUITENWEG (GroenLinks): Ja. Laatst heeft u bij de behandeling van dit onderwerp in
een informatieve raadsbijeenkomst gezegd dat in het uiterste geval, wanneer er geen kandidaat komt
voor de exploitatie, de stadsboerderij gesloten wordt. Dat wil ik heel graag voorkomen. Vandaar dat in
motie VA is opgenomen dat de gemeente anders de exploitatie in eigen beheer moet nemen.

        De heer SCHOUTEN (wethouder): Dat begrijp ik.

        De heer BUITENWEG (GroenLinks): Ik begrijp uw standpunt.

        De heer SCHOUTEN (wethouder): Ik heb vanmorgen al gezegd dat sluiting van de
kinderboerderij voor ons geen optie is. Wij gaan er niet van uit dat dat gebeurt. Wij komen altijd bij de
raad terug met een onderhandelingsresultaat. Dan kan de raad uiteindelijk beoordelen wat hij daarmee
wil. Mijn uitspraak over de kinderboerderij kan niet duidelijker zijn.

       De heer SCHUUR (PvdA): Volgens mij is de intentie van het college met betrekking tot de
stadsboerderij inderdaad duidelijk. Per 1 januari 2013 is deze zaak aan de orde. Daarom zouden wij
kunnen afspreken dat het college met het onderhandelingsresultaat komt bij de volgende voorjaarsnota.
Dan kunnen wij alsnog bekijken of de raad daarmee akkoord is.

        De heer SCHOUTEN (wethouder): Akkoord.

        De heer BUITENWEG (GroenLinks): Daar kan ik mij in vinden. Nog een herkansing dus bij de
voorjaarsnota. Dan pas ik de motie aan.

        De heer SCHOUTEN (wethouder): Oké. Dan is de discussie over de kinderboerderij nu
voldoende gevoerd. U past motie VA kennelijk aan.
        Voorzitter! Ik ga nu over naar het onderwerp chemische gewasbescherming. De heer Buitenweg
wil dat de gemeente stopt met het gebruiken van Roundup, liefst vanaf nu. Wij hebben gezegd (ik heb
dat vanmorgen aangegeven namens het college) dat wij het rijksbeleid volgen op dit gebied. Dat is dan
inclusief de ingangsdatum van het rijksbeleid. Met andere woorden: wij stoppen pas met het gebruiken
van Roundup als de rijksoverheid zegt dat het niet meer gebruikt mag worden. Overigens wil ik best
een stap verdergaan en toezeggen dat wij eerder zullen stoppen met het gebruiken van Roundup als er
een goed alternatief beschikbaar is. Wij zullen de komende tijd bekijken hoe wij willen omgaan met wat
in dezen op ons af komt. Ik wil best toezeggen dat wanneer mocht blijken dat er een goed alternatief
beschikbaar is, wij de bereidheid hebben om dat alternatief eerder in te zetten. De intentie van datgene
wat op rijksniveau is besloten, is helder en onderschrijven wij. In het amendement (UA) dat de heer
Buitenweg hierover heeft ingediend, gaat hij echter nog een stap verder. Volgens dat amendement wil
hij immers de bezuiniging die door de raad bij de voorjaarsnota is vastgesteld, namelijk € 107.000,--
structureel, verlagen met het bedrag dat eventueel noodzakelijk is om te stoppen met het gebruiken
van Roundup. Daar geeft hij helemaal geen dekking voor aan. Dat gaat het college echt te ver.
Bovendien gaat het om twee heel verschillende dingen die in één amendement zijn samengevoegd. Ik
kan dan ook niet anders dan het amendement namens het college ontraden.

        De heer BUITENWEG (GroenLinks): Wat u nu zegt, klopt ten dele. Als echter vanaf morgen
verboden zou worden om Roundup te gebruiken, dan heeft u geen alternatief. Dan zou het bedrag van
€ 107.000,-- naar beneden moeten worden herberekend, zij het dat ik niet weet hoe groot het bedrag
is dat bespaard wordt. Ik heb een brief van staatssecretaris Atsma bij mij waarin over de motie-
Grashoff staat: "Ik zal de motie uitvoeren". Die voert de motie van de Tweede Kamer dus uit.

        De heer SCHOUTEN (wethouder): Wij ook.




INT - 1201417                                                                                           77
</p><p>       De heer BUITENWEG (GroenLinks): De staatssecretaris zegt voorts in zijn brief: "De motie heeft
gevolgen voor gemeenten en andere terreinbeheerders."

        De heer SCHOUTEN (wethouder): Dat klopt.

        De heer BUITENWEG (GroenLinks): Verder staat in de brief: "Het ware juist, als zij ruim voor
het einde van de overgangstermijn in ieder geval de stap naar niet-chemisch onkruidbeheer (…) zouden
overwegen." Ik meen dan ook dat u snel moet gaan rekenen, om na te gaan waar dan de bezuiniging
gehaald moet worden.

       De VOORZITTER: Volgens mij heeft u de wethouder horen zeggen dat hij bereid is om te
zoeken naar een alternatief.

          De heer SCHOUTEN (wethouder): Ik wou zeggen: Ik vraag mij af of de heer Buitenweg
geluisterd heeft naar of begrepen heeft wat ik heb gezegd. Ik heb namelijk gezegd: Wij zijn bereid om,
vooruitlopend op de ingangsdatum van het rijksbeleid inzake het gebruik van Roundup en wanneer er
alternatieven beschikbaar zijn, die alternatieven eventueel eerder in te voeren als dat mogelijk is. Het
lijkt mij dat ik de heer Buitenweg daarmee maximaal tegemoet kom.

        De heer BUITENWEG (GroenLinks): Dat heb ik gehoord.

         De heer SCHOUTEN (wethouder): De bezuinigingstaakstelling staat helemaal los van het
gebruiken van Roundup. De taakstelling blijft gewoon staan. Ik blijf amendement UA dan ook ontraden.
         Voorzitter! Het volgende onderwerp waar ik op in ga, is Waterrijk en het amendement daarover
(DA) van de VVD-fractie. De heer Nijhuis heeft dit gepresenteerd als een amendement, maar qua
strekking is het volgens mij eerder een motie dan een amendement (als ik zo vrij mag zijn om dat op te
merken). Met de strekking van het amendement (zo zal ik het nog maar even blijven noemen) zijn wij
het eens. Ik wil nog wel een stap verder gaan, namelijk door te stellen dat wij niet pas bij de
voorjaarsnota 2012 maar al volgende maand met een voorstel bij de raad komen inzake het
bestemmingsplan voor Waterrijk. Daarin komt aan de orde wat in het amendement gesteld wordt. Zoals
ik vanmorgen al heb aangegeven, zullen volgens dat voorstel ten minste eenmaal per jaar de volgorde
van de ontwikkelingen en de fasering aan de raad voorgelegd worden, namelijk bij de actualisering van
het exploitatieplan. Bij het bestemmingsplan hoort een exploitatieplan. Wij hebben de verplichting om
de actualisering daarvan ieder jaar voor te leggen aan de raad, juist om datgene te doen wat de VVD-
fractie wil, namelijk een heel duidelijke fasering aanbrengen en duidelijk de kosten in beeld brengen. Dat
is nodig om steeds stap voor stap de beslissing te kunnen nemen hoe het grote plan de volgende tien
jaar in uitvoering gebracht wordt. Volgens mij doen wij dus eigenlijk datgene wat de VVD-fractie wil op
dit gebied. Wel vinden wij, zoals ik vanmorgen ook al heb gezegd, amendement DA als zodanig
overbodig. Als de VVD-fractie het gestelde in het amendement wil opnemen in een motie, dan heb ik er
geen bezwaar tegen.

         De heer NIJHUIS (VVD): Voorzitter! Het zal ongetwijfeld uit mijn ondeskundigheid voortvloeien
als ik het nu fout zeg. Volgens mij wordt met een amendement een stuk tekst vastgelegd in een
beslispunt, terwijl met een motie een signaal wordt gegeven aan het college. Volgens mij - en corrigeer
mij als ik het fout heb - ondersteunen wij met het amendement het college in het streven om ten
aanzien van Waterrijk te handelen zoals de wethouder aangaf. Heb ik dat verkeerd?

        De heer SCHOUTEN (wethouder): In zoverre, dat het er niet om gaat dat wij bij de
voorjaarsnota komen met voorstellen inzake Waterrijk. Nee, het gaat erom dat wij al bij de vaststelling
van het bestemmingsplan - en dat is voorzien in januari aanstaande - de discussie voeren met elkaar.
Dan kunt u zeggen: "Dat doen wij bij de voorjaarsnotabehandeling nog een keer", maar volgens mij is
dat overbodig. U krijgt immers al volop de gelegenheid om in januari te discussiëren over Waterrijk.

        De heer SCHUUR (PvdA): Voorzitter! Zou het niet mogelijk zijn om, op basis van amendement
DA, pas bij de voorjaarsnotabehandeling te gaan praten over de plannen voor Waterrijk, in plaats van in
januari 2012 bij de behandeling van het bestemmingsplan?

        De heer SCHOUTEN (wethouder): Daar zou ik geen voorstander van zijn.




INT - 1201417                                                                                          78
</p><p>       De heer SCHUUR (PvdA): Dat begrijp ik. Daarom lijkt mij dat wat u nu zegt, een prima
vervanging is van amendement DA.

        De heer SCHOUTEN (wethouder): Wat wij doen met het vaststellen van het bestemmingsplan
voor Waterrijk, is het planologisch mogelijk maken dat daar gebouwd gaat worden. Wanneer de raad
het bestemmingsplan vaststelt, zijn wij er nog niet, want dan kunnen mensen bezwaar aantekenen en
is het mogelijk dat het bestemmingsplan of delen daarvan worden voorgelegd aan de Raad van State.
Het is dan dus nog lang niet onherroepelijk. Voordat de schop in Waterrijk de grond in gaat, zijn wij
sowieso een jaar verder, ben ik bang. Dat betekent wel dat de raad ruimschoots de gelegenheid krijgt
om daarover te praten, namelijk ook op het moment dat de raad voor de tweede keer een
exploitatieplan voorgelegd krijgt. Ik meen dan ook dat er voldoende waarborgen zijn ingebouwd om
datgene te realiseren wat u wilt.

        De heer MAATHUIS (VVD): Voorzitter! Dan heb ik een verhelderende vraag, namelijk of in de
prognoses voor 2012 al de bouw van woningen in Waterrijk is opgenomen. Ik weet dat namelijk niet
zeker. De wethouder zei zojuist dat wij een jaar verder zijn voordat het bestemmingsplan onherroepelijk
is en voordat de schop de grond in kan. Lopen wij dan niet achter de feiten aan als wij voor komend
jaar de bouw van 80 woningen in Waterrijk inplannen? Klopt dat?

        De heer SCHOUTEN (wethouder): Wij gaan van gemiddelden uit voor een periode van tien jaar
(maar de heer Andela of andere collega's weten dat nog beter dan ik). Dit betekent dat wanneer in het
eerste jaar nadat het bestemmingsplan onherroepelijk is en er gebouwd kan worden in Waterrijk, dat op
een later moment gebeurt dan in de berekeningen is opgenomen. Dan moet een inhaalslag gemaakt
worden in de jaren daarna, anders hebben wij een probleem met de exploitatie. Maar nogmaals, als
amendement DA wordt omgezet in een motie, dan heeft het college daar geen probleem mee.
        Voorzitter! Het laatste onderwerp waar ik nu op in ga, is een klein punt dat ik in de eerste
instantie vergeten ben. De heer Nijhuis had gevraagd om in te gaan op zijn opmerking over de centrale
bediening van beweegbare bruggen. In de huidige situatie gaat het daarbij om de Eilandsbrug , de
Dollegoorbrug en de Javabrug. Als de Waterboulevard wordt gerealiseerd, gaat het straks om meer
bruggen, maar dan zijn wij al even verder. In het kader van het project Waterboulevard zal dit verhaal
worden meegenomen. Dan kunnen wij komen tot een systeem waarmee wij de bediening van de
bruggen op afstand mogelijk maken. Dan kunnen wij datgene gaan doen wat de VVD-fractie feitelijk
heeft voorgesteld in dezen.

        De heer VAN WOUDENBERGH (wethouder): Mevrouw de voorzitter! De CDA-fractie heeft
gevraagd of het mogelijk is om voorstellen over grote projecten op hoofdlijnen te laten doorlichten door
externen. Eigenlijk laten wij dat al doen. Als het gaat om het binnenstadsplan, kijkt namelijk het bureau
Royal Haskoning al mee over onze schouders. Weliswaar is dat bureau in eerste instantie ingehuurd
door de provincie, maar daar maken ook wij heel nuttig gebruik van. En de transitie Waterrijk wordt
helemaal begeleid door het bureau Twijnstra Gudde, ook een externe grootheid. Daarnaast heeft onze
accountant Deloitte via Deloitte Real Estate Advisory een stevige, kritische vinger in de pap als het gaat
om eigenlijk alle plannen die wij willen uitvoeren. Echter, met het oog op het "stapelen" van externen,
zou ik het voorstel van de CDA-fractie om die plannen te laten doorlichten willen ontraden, omdat die
doorlichting al op diverse terreinen gebeurt.

        Mevrouw TEN SELDAM-EVEN (CDA): U zegt nu dat u ons voorstel wilt ontraden. Wij hadden
geen weet van het gegeven dat er al door externen mee gekeken wordt naar de realiseerbaarheid van
plannen vanuit financieel perspectief. Dit vind ik een buitengewoon goed initiatief. Dat pleit er alleen
maar voor om dat voor de komende jaren vooral ook vast te houden, zeker gezien de economisch
roerige tijd. Ik wil geen "stapeling" van externen, maar hecht er wel aan dat wij bij de besluitvorming
kennis kunnen nemen van hun visie. Wat is volgens hen realistisch in financieel perspectief gezien? Ik
begrijp dat het college ons op dit punt al volkomen bedient.

        De heer VAN WOUDENBERGH (wethouder): Vooral voor dit laatste zie ik een goede rol
weggelegd voor Deloitte Real Estate Advisory. Die partij heeft namelijk intern veel contacten, maar
heeft ook contacten via de commissie planning en control. Via die commissie loopt het dus. Ik hoop dat
ik met deze opmerking mevrouw Ten Seldam geruststel.




INT - 1201417                                                                                          79
</p><p>       De heer KOZIJN (PvdA): Voorzitter! Het kan toch niet zo zijn dat in een raadsvoorstel separaat
het advies van een extern bureau wordt verwoord? Hier in de raad praten wij over enkelvoudige
voorstellen van het college. Wij hopen en verwachten dat daar alle adviezen in meegenomen worden.
Daar gaan wij van uit, maar dat de adviezen separaat zichtbaar gemaakt worden, lijkt mij eigenlijk
ondenkbaar. Of bedoelt mevrouw Ten Seldam iets anders?

        Mevrouw TEN SELDAM-EVEN (CDA): Wat ik bedoel, is dat plannen aan externen worden
aangeboden ter toetsing. Die moeten dan nagaan of de plannen realiseerbaar zijn vanuit financieel
perspectief gezien. In het advies dat hieruit voortvloeit, hoeven geen uitvoerige details te staan. Ik wil
alleen maar weten of de plannen haalbaar zijn. Het gaat mij juist om de betrokkenheid van externen
zoals die zojuist al door de wethouder geschetst is. Neem hun advies gewoon mee, hun conclusie. Ik
hoef niet de hele redenering daarachter te hebben, want daar worden de raadsstukken veel te
omvangrijk van.

        De heer NIJHUIS (VVD): Voorzitter! Volgens mij kan dat waar mevrouw Ten Seldam het nu
over heeft, gewoon als bijlage bijgevoegd worden, zonder dat het deel uitmaakt van raadsvoorstellen.
Verder wil ik nog aangeven dat de VVD-fractie al in de vorige raadsperiode (dacht ik) gevraagd heeft
om een externe accountant te laten kijken naar de grondexploitaties, met name voor de grote
projecten. Ik zie nu dat wat dat betreft, de zaak in beweging wordt gezet. Daar zijn wij het erg mee
eens.

        De heer KOZIJN (PvdA): Voorzitter! Stelt u zich eens voor dat er een collegevoorstel komt en
dat een extern bureau daarover zegt: Dat is financieel niet haalbaar. Dat lijkt mij een ongerijmdheid,
toch? Dan is de situatie toch zo dat het college zelf die conclusie al getrokken heeft. Dat is toch de
essentie? Het college is primair verantwoordelijk.

        De heer VAN WOUDENBERGH (wethouder): In dat geval komt het college niet met een
collegevoorstel volgens mij.

        De heer KOZIJN (PvdA): Dank u wel.

       De heer VAN WOUDENBERGH (wethouder): Het is natuurlijk best mogelijk om in een
raadsvoorstel een alinea op te nemen waarin staat dat (in het geval van de binnenstad) Royal
Haskoning meegekeken heeft en akkoord gaat met het voorstel. Dat is dan gewoon één alinea.

        De heer KOZIJN (PvdA): Ik vind dat zeer opmerkelijk.

        De heer PAUWELS (VVD): Ik ben benieuwd of de heer Kozijn projecten kan noemen die in het
verleden in gang zijn gezet en die, als een externe partij ze had beoordeeld, ook dan waren uitgevoerd.
Zijn daar voorbeelden van te noemen?

        De heer KOZIJN (PvdA): Nee, dat kan ik niet, want ik heb nooit in het college gezeten.

        De heer PAUWELS (VVD): Dat is een beetje een flauw antwoord.

       De heer KOZIJN (PvdA): Maar het is wel de realiteit. De realiteit is dat het college aan zet is om
voorstellen aan te leveren.

        De heer VAN WOUDENBERGH (wethouder): Voorzitter! Ik heb gezegd dat wij al gebruikmaken
van externen op hoog niveau. Ik meen dat wij het daar even bij moeten laten.
        De PvdA-fractie dank ik voor haar instemming met de beantwoording van de vragen over
cultuur.
        Voorts wil de PvdA-fractie dat de gemeente wat meer laaghangend fruit plukt als het gaat om
de bouw in de binnenstad. Wat dat betreft, kan ik twee projecten noemen die in het oog springen. Het
eerste is het project Grotestraat 30. Daarvoor hebben wij onlangs een tender georganiseerd. Ik moet
gewoon eerlijk mededelen dat die tender niet gelukt is. Er was vanuit de markt geen belangstelling voor
het project op de voorwaarden die ten grondslag liggen aan de tender. Dat was teleurstellend. Wij
zullen het object nu opnieuw gaan bekijken. Daarbij zullen wij ook nagaan of wij de basis van de
voorwaarden wat kunnen aanpassen. Die voorwaarden waren gericht op taxaties uit 2008, dus voor de



INT - 1201417                                                                                            80
</p><p>crisis. Ik meen dat wij de voorwaarden moeten actualiseren. Vervolgens gaan wij gewoon opnieuw de
boer op met het project.

        De heer GERRITSEN (D66): Wat bedoelt u met actualiseren? De prijsjes bijstellen naar beneden?

       De heer VAN WOUDENBERGH (wethouder): Ik denk dat dat anno 2011 een logisch gevolg is
van actualisering.

       De heer GERRITSEN (D66): Dat zou betekenen dat wij dat voor meer dingen kunnen doen in de
binnenstad, want daar zijn nog meer van dergelijke plekken. Is het college bereid…

        De heer VAN WOUDENBERGH (wethouder): Noemt u er eens één?

        De heer GERRITSEN (D66): Er zijn natuurlijk meer mooie grasvelden in de binnenstad.

        De heer VAN WOUDENBERGH (wethouder): Die zijn niet in ons bezit.

        De heer GERRITSEN (D66): Nee, maar u kunt daarover wel in overleg gaan met de partners, om
te bekijken of daar wat gerealiseerd kan worden. Tenslotte moeten wij het binnenstadsplan vormgeven,
en de betrokken locaties zijn daar onderdeel van.

      De heer VAN WOUDENBERGH (wethouder): Ik beperk het nu even tot de plekken die
gemeentelijk bezit zijn.

       De heer GERRITSEN (D66): Kunt u mij misschien een beeld geven van waaraan u denkt als u
het heeft over bijstellen? Heeft u ideeën van de bedragen?

        De heer VAN WOUDENBERGH (wethouder): Nee, die ga ik nu niet noemen, want dat zou
luchtfietserij zijn, en daar houd ik niet van.

       De heer GERRITSEN (D66): Maar het betekent wel dat het de gemeente per saldo meer geld
gaat kosten, gelet op de investeringen die ze heeft gedaan.

         De heer VAN WOUDENBERGH (wethouder): Ook dat is een conclusie die ik nog niet deel.
         Voorzitter! Wat ik zojuist zei over het project Grotestraat 30, geldt ook voor de Hinnenlocatie.
Die locatie is grotendeels in bezit van één van onze partners in het consortium. Komende week hebben
wij daarover een gesprek met het consortium en de eigenaar van de Hinnenlocatie. De bedoeling
daarvan is om ook de eigenaar wat meer in beweging te krijgen op dit punt. De huidige situatie is
namelijk niet alleen de omwonenden een doorn in het oog, want ook wij als college willen daar graag
wat beweging zien. Ik doe op dit moment dan ook een oproep, zelfs in het openbaar, aan het
consortium om wat vaart te gaan maken in dezen, want de belemmeringen van de zijde van de
gemeente zijn weggenomen.
         De heer Buitenweg vroeg of Syntrus Achmea nog deel uitmaakt van de partners in de
binnenstad. Zoals ik zojuist al zei, hebben wij nog deze week een gesprek met het consortium. Daar
hoort Syntrus Achmea gewoon bij. Die is tot nu toe gewoon partner in het consortium.
         De fractie van D66 heeft zelfs tegen mij gezegd (ik heb het opgeschreven, in het Frans):
Chapeau. Op het moment dat de heer Gerritsen "chapeau" zegt, moet je natuurlijk altijd even
oppassen! Zoals ik vanmorgen al heb gezegd, nemen wij zijn suggestie om te gaan sparen voor het
binnenstadsplan, positief mee in de ontwikkelstrategie. Sparen is echter nooit de enige manier om de
financiering rond te krijgen. Wij zullen ook moeten zorgen dat wij extern financiële onderbouwing
krijgen voor de plannen. Maar nogmaals, de suggestie om te gaan sparen, nemen wij mee.
         De fractie van LKA stelde dat wij investeren in stenen, maar dat wij meer moeten investeren in
mensen. Als je puur naar de binnenstad kijkt, zie je dat wij daar inderdaad investeren in stenen, maar
ook in mensen. Ik meen dat juist de inwoners van onze eigen gemeente er heel blij mee zullen zijn als
alle plannen gerealiseerd zijn.
         De vraag van de LKA-fractie over de Kloosterhofflat heb ik vanmorgen uitputtend beantwoord.
Ook heb ik daarover het een en ander gezegd in een privémailwisseling met de heer Kamphuis.

        De heer KAMPHUIS (LKA): (Zonder microfoon).



INT - 1201417                                                                                          81
</p><p>        De VOORZITTER: Als u wilt interrumperen, mijnheer Kamphuis, dan moet u achter de
interruptiemicrofoon gaan staan.

        De heer KAMPHUIS (LKA): Ik wil tijd besparen.

        De heer VAN WOUDENBERGH (wethouder): Voorzitter! De VVD-fractie heeft in de tweede
instantie nogmaals gesproken over kleinere kavels in Almelo Noordoost. Ik heb vanmorgen gezegd dat
het vanzelfsprekend is dat wij vragen uit de markt positief beoordelen. Daarbij moeten wij echter wel
de filosofie achter Almelo Noordoost vasthouden. Volgens die filosofie is dat een wijk waarvan mensen
de verwachting mogen hebben dat ze daar op een royale manier kunnen wonen. Niettemin zal ik de
ontwikkelaar die geïnteresseerd is in de strip langs de Ootmarsumsestraat, positief ontvangen, zonder
daar nu enige toezeggingen over te doen. Wel zal ik uiteraard vanuit een positieve grondhouding het
gesprek aangaan met de betrokken mensen.
        De heer Stork heeft voorgesteld om het plan voor de gelijkvloerse kruising van de Gravenallee
met de Van Rechteren Limpurgsingel te laten vervallen. Ik heb vanmorgen dit plan/project toegelicht.
Het college komt pas in beweging als de externe financiering geregeld is. Zoals ik vanmorgen ook
gezegd heb, is het goed dat wij dit project, ook een mooi project, op voorraad hebben liggen.

        De heer STORK (ALA-AOV): Voorzitter! De wethouder spreekt nu over externe financiering van
het project gelijkvloerse kruising Gravenallee-Van Rechteren Limpurgsingel. Houdt dat in dat er geen
gemeentelijk geld bij moet?

        De heer VAN WOUDENBERGH (wethouder): Als ik praat over externe financiering, wil ik als
voorbeeld daarvan geven de westelijke stationsomgeving. Die wordt gerealiseerd zonder gemeentelijk
geld. Dit is ook een goed voorbeeld voor de realisering van het project gelijkvloerse kruising.

        De VOORZITTER: Wethouder! Kunt u komen tot een afronding van uw bijdrage?

        De heer VAN WOUDENBERGH (wethouder): Ik kan komen tot een afronding.

        De VOORZITTER: Maar niet pas over een halfuur!

        De heer VAN WOUDENBERGH (wethouder): Nee, nee, heel snel zelfs.

        De VOORZITTER: Helemaal goed.

        De heer VAN WOUDENBERGH (wethouder): Voorzitter! Ik wilde juist ingaan op de laatste
vraag, namelijk van de heer Stork over het in de binnenstad terug laten komen van bevaarbaar water.
Ik meen dat die vraag op een misverstand berust. Wij praten over bevaarbaar water tot het nieuwe
stadsplein, niet tot Kreta. Tot het stadsplein maakt het niet zoveel uit of het water wel of niet
bevaarbaar is, omdat wij voor beide opties dezelfde panden moeten slopen.

        De heer STORK (ALA-AOV): Ik sprak inderdaad over het terugbrengen van bevaarbaar water tot
de locatie Kreta (over verder terugbrengen, zou ik niet willen praten). Het gaat mij vooral om de
bruggen die open moeten kunnen als er boten passeren. Dergelijke bruggen hoeven niet aangelegd te
worden als het om onbevaarbaar water gaat. Die optie is volgens mij aanzienlijk goedkoper.

       De heer VAN WOUDENBERGH (wethouder): Die optie is niet het uitgangspunt van ons
binnenstadsplan.

         De heer ANDELA (wethouder): Mevrouw de voorzitter! Allereerst een opmerking over het zaak-
en procesgericht werken. Wij willen dat doen volgens de principes van het Lean Management. Daarbij
zullen wij (wij zijn daartoe uitgedaagd) het gebruik van onorthodoxe methoden niet schuwen, mocht
dat noodzakelijk zijn.
         Misschien worden wij het net zo snel eens over het leerlingenvervoer. De situatie op dit gebied
is als volgt. Er was een overschrijding van € 130.000,--. Later werd dat een overschrijding van
€ 130.000,-- plus € 180.000,--. Met de maatregelen die nu afgekondigd zijn, proberen wij om de
overschrijding van € 180.000,-- terug te dringen. Het amendement hierover (C) gaat eigenlijk over het



INT - 1201417                                                                                         82
</p><p>bedrag van € 130.000,-- dat wij nu structureel hebben opgenomen, omdat wij er niet zeker van zijn dat
wij met de nu afgekondigde maatregelen dit bedrag kunnen verminderen. Mocht ons dat wel lukken,
dan kan wat in het amendement gesteld wordt, geïmplementeerd worden. Wij doen echter het voorstel
om eerst te bekijken of het lukt om de overschrijding van € 180.000,-- weg te werken. Wij zullen de
raad in de loop van november (deze maand dus) berichten over de eerste signalen daarover.

        De heer SCHUUR (PvdA): Voorzitter! Ik hoor de wethouder nu toch wat zorgelijke woorden
spreken over het leerlingenvervoer. Hij stelde dat het bedrag van € 130.000,-- niet voor niets
structureel gereserveerd is. Daarmee stelt hij ook dat nieuw beleid baseren op meevallers op dit vlak -
 bijvoorbeeld wanneer een stukje van de overschrijdingen tegengegaan kan worden - absoluut niet aan
de orde kan zijn. Dat hoor ik hem zeggen.

        De heer ANDELA (wethouder): De regeling voor het leerlingenvervoer is en blijft een
openeinderegeling. Wat het uiteindelijke resultaat wordt, staat in de sterren geschreven en blijft in de
sterren geschreven staan, elk jaar. Wij hebben nu echter maatregelen afgekondigd die inhouden dat wij
minder geld gaan uitgeven aan het leerlingenvervoer. De eerste, relatief betrouwbare prognose van het
effect van de maatregelen krijgen wij deze maand. Die prognose zal ik delen met de raad. Dan kan de
raad zien hoe de ontwikkeling is. Wij hebben echter een overgangsperiode van twee jaar gekozen. Dit
betekent dat wij het uiteindelijke effect pas over drie jaar zien. Daarbij speelt dan ook het bedrag van
€ 130.000,-- mee. Op dat moment kunnen wij zien wat het uiteindelijke effect is. Wellicht is het groter
dan wij nu verwachten. Dan kunnen wij ook spreken over de besteding van de middelen.

        De heer SCHUUR (PvdA): Ik hoor u nu zeggen dat het uiteindelijke resultaat in de sterren
geschreven staat en dat het nu vooruitlopen op eventuele meevallers, neerkomt op je baseren op
drijfzand. Dat hoor ik u zeggen, of niet?

       De heer ANDELA (wethouder): Nou, ik heb al even gespiekt om te zien wat de cijfers zijn. Op
grond daarvan kan ik zeggen dat het niet helemaal gebaseerd is op drijfzand.

        De heer BRAND (CDA): Voorzitter! Ik weet niet of ik het helemaal goed begrijp, maar ik wil het
wel goed begrijpen, omdat wij amendement C hebben ingediend met betrekking tot het
leerlingenvervoer. De wethouder zei dat aanvankelijk een bezuiniging van € 130.000,-- op het
leerlingenvervoer was ingeboekt. Dat is € 180.000,-- geworden, en daar zijn de bezuinigingen op
gebaseerd die wij hebben afgesproken met elkaar.

        De heer ANDELA (wethouder): Nee, dit moet ik corrigeren. Er was een overschrijding van
€ 130.000,--. Daarvoor zijn destijds incidentele middelen beschikbaar gekomen. Daarbovenop kwam
een overschrijding van € 180.000,--, en daarvoor zijn maatregelen genomen. Wij zijn nu druk bezig met
het terugdringen van de overschrijding van € 180.000,--. De incidentele bijpassing die steeds heeft
plaatsgevonden, heeft nu structureel plaatsgevonden.

        De heer BRAND (CDA): Ja. Nu wordt nieuwe wetgeving ingevoerd die gevolgen heeft voor het
leerlingenvervoer, namelijk de wetgeving inzake passend onderwijs. Deze wetgeving zal, vanwege de
bezuiniging van € 300 miljoen op landelijk niveau, tot gevolg hebben dat er minder leerlingen in het
speciaal onderwijs komen. Alle scholen voor speciaal onderwijs zijn zich daarop aan het voorbereiden.
De nieuwe wetgeving heeft ook gevolgen voor het leerlingenvervoer. Gelet daarop, moeten wij met
elkaar vaststellen dat er minder geraamd hoeft te worden voor het leerlingenvervoer. Dat staat los van
negatieve of positieve verwachtingen, want dat is gewoon een realiteit die op ons afkomt. Die realiteit
is dat minder leerlingen naar het speciaal onderwijs zullen gaan, waardoor wij als gemeente minder
kosten zullen hebben.

       De heer ANDELA (wethouder): Wij willen ons niet rijk rekenen. Als dat blijkt, dan is dat zo. Dan
lopen wij echt niet weg met het geld, maar dan komen wij daarop terug bij de raad.

        Mevrouw ROZEMA-JUNGJOHANN (D66): Voorzitter! Ook ik denk dat wij ons op het gebied van
het leerlingenvervoer nu niet rijk moeten rekenen en strakjes arm moeten tellen. Ik verwacht dat de
rugzakjesfinanciering voor een heel groot deel teruggebracht wordt. De kinderen met een rugzakje
zitten dan op de school waar ze al op zitten. Die kinderen zitten vaak nog niet eens in het speciaal
onderwijs.



INT - 1201417                                                                                        83
</p><p>       De heer NIJHUIS (VVD): Voorzitter! Ik denk dat ik iets weet van deze sector. Op grond daarvan
moet ik mevrouw Rozema teleurstellen, want het betreft de hele sector.

         De heer ANDELA (wethouder): Voorzitter! Wij ontraden amendement C (daar blijf ik bij). Het
onderwerp leerlingenvervoer komt vanzelf terug. Wanneer wij onze doelstelling realiseren middels de
maatregelen, dan kan vervolgens een en ander gebeuren zoals de raad dat wil. Op dat moment kan
men opnieuw een amendement indienen, want dan heeft dat echt realiteitsgehalte. Dan weten wij
namelijk hoe het uitgepakt is.
         De heer Cornelissen kwam in de tweede termijn terug op het inhuren van externen. Wij delen
eigenlijk zijn visie, maar er staat een motie (Y) tussen ons. Ik probeer een brug te slaan naar hem, om
tot hem te komen. Als de heer Cornelissen motie Y zodanig wijzigt, dat die in overeenstemming is met
zijn reactie in de tweede termijn, dan kan ik daarmee instemmen. Dan stelt de motie namelijk dat het
college bij de raad terug moet komen als de inhuur stijgt boven de 10%. Wij kunnen daar op
terugkomen via de maandelijkse rapportage. Dit moet echter niet opgelegd worden als een harde norm
die bepaalt dat wij altijd terug moeten komen, bij iedere overschrijding van de norm, want zoals ik heb
aangegeven, willen wij graag flexibiliteit op dit gebied voor het geval dat zich bepaalde dingen mochten
voordoen.

        De heer CORNELISSEN (SP): Voorzitter! Volgens mij is het zo dat wanneer het college buiten de
kaders gaat die de raad heeft gesteld, het college automatisch daarmee bij de raad komt. Het enige dat
in motie Y staat, is dat als kader gesteld wordt dat maximaal 10% van de totale vergoedingen en
salarissen mag worden besteed aan het inhuren van externen. Wanneer het college voor een bepaald
doel of project de norm van 10% dreigt te raken, dan is het volgens mij heel normaal dat dat
teruggekoppeld wordt naar de raad. Dat is wat wij eigenlijk vragen en wat ik in mijn betoog zei.
Volgens mij willen wij hetzelfde, alleen heeft de wethouder in de eerste termijn aangegeven dat het
directieteam en het college flexibel willen omgaan met de norm van 10% en dat ze daar niet strikt aan
gehouden willen worden. Ik meen echter dat het heel verstandig is wanneer wij als raad gewoon
zeggen tegen het college: De norm is 10%, en gaat u daar overheen, kom dan even naar de raad. Dan
bespreken wij dat.

          De heer ANDELA (wethouder): Ik moet uw woorden toch corrigeren. Ik heb aangegeven hoe het
gaat met het inhuren. Dat kan alleen maar plaatsvinden via het directieteam. Daar ligt de zeef. De
bedoeling is dat de inhuur naar beneden gebracht wordt. Ik heb aangegeven dat zich een situatie kan
voordoen waarin wij een aantal personeelsleden kwijtraken, meer dan wij geprognosticeerd hadden,
terwijl wij dat op dat moment nog niet direct kunnen aanvullen via het loopbaancentrum maar
verwachten dat dat later wel kan. In een dergelijke situatie willen wij de mogelijkheid hebben om
tijdelijk mensen in te huren. Mocht de inhuur daarbij uitkomen boven de 10%-norm, dan is dat in het
belang van onze organisatie, hoe je het ook wendt of keert. Gun ons als college die ruimte, omdat wij
de visie delen dat de externe inhuur naar beneden gebracht moet worden.

        De heer CORNELISSEN (SP): Voorzitter! Het beeld lijkt nu te ontstaan dat de inhuur volgens ons
nooit, op geen enkel moment, boven de 10%-norm uit mag komen. Volgens mij geldt op dit gebied
hetzelfde als voor de woningbouwprogramma's. Dat wil zeggen dat er een bepaald gemiddelde wordt
vastgesteld. Wat het inhuren van personeel betreft, betekent dit dat de inhuur over het hele jaar gezien
gemiddeld niet meer dan 10% mag zijn. Als dat wel het geval is, komen wij op een glijdende schaal en
komen wij uit op een te hoog percentage. Daarom vraag ik om de norm van 10% vast te leggen.
Wanneer dan gedurende een bepaalde periode de inhuur 11% of 12% is, moet vervolgens de inhuur
verminderd worden, maar die overschrijdingen moeten wel teruggekoppeld worden naar de raad. Wij
willen die namelijk graag beoordelen.

        De heer NIJHUIS (VVD): Voorzitter! In motie Y wordt gesteld dat maximaal 10% van het totale
budget voor salarissen en vergoedingen aan ingehuurde medewerkers uitgegeven mag worden. Als
gesteld wordt dat dat in principe het geval moet zijn, dan kunnen ook wij leven met de motie. Wij
willen namelijk graag de flexibiliteit in ogenschouw nemen die de wethouder wil betrachten, omdat wij
weten dat het streven er wel degelijk op gericht is om onder de 10% te blijven.

        De heer ANDELA (wethouder): Ik vraag om die ruimte voor het college.




INT - 1201417                                                                                        84
</p><p>         Voorzitter! De fractie van D66 heeft voorgesteld € 5 miljoen te bezuinigen op de
bedrijfsvoering. Ik had a priori aangenomen dat die bezuiniging betrekking zou hebben op de
personeelslasten. De kosten van de bedrijfsvoering zijn in totaal € 55 miljoen, waarvan € 45 miljoen
voor personeelslasten. De verhouding tussen deze twee is dus grosso modo 80 tot 20. Die 20% van de
kosten wordt gemaakt voor bijvoorbeeld ICT. Wat dat betreft, wijs ik erop dat wij juist meer digitaal
willen gaan werken. Het is dan ook de vraag of je daar verder op moet bezuinigen. Verder neem ik
hierbij in ogenschouw dat wij al twee keer 2 ton, dus 4 ton in totaal, aan bezuinigingen hebben moeten
realiseren op dit gebied. Als het gaat om de aanbestedingen, is er weinig ruimte. Vandaar dat mijn
logische conclusie was: Als je € 5 miljoen wilt bezuinigen op de bedrijfsvoering, dan moet je die wel
vinden in de personeelslasten.
         De fractie van ALA-AOV wil graag een overzicht hebben van de taken die niet meer uitgevoerd
worden. Ik zeg toe dat elk jaar, aan het eind van het jaar, er een overzicht komt waarin de taken
beschreven zijn die wij niet meer uitvoeren en welke functionarissen dat betreft (niet met naam,
natuurlijk, maar wel wat betreft de functies die weg zijn).

        De heer STORK (ALA-AOV): Voorzitter! Ik denk dat het een beetje anders ligt. Ik meen namelijk
dat de raad moet beslissen over de taken die niet meer uitgevoerd worden. De raad hoeft daar niet tot
in de details op in te gaan, maar ik ben wel van mening dat als het om belangrijke taken gaat, de raad
daarover moet besluiten.

        De heer ANDELA (wethouder): De raad heeft de besluiten daarover al genomen, namelijk bij de
begroting voor 2011. Die staan namelijk allemaal in de begroting. Ik dacht dat het een goede service
van het college richting de raad zou zijn om in een overzicht keurig op een rij te zetten welke functies
het precies betreft.

            De heer STORK (ALA-AOV): Kunnen wij dat overzicht voor het eind van dit jaar krijgen?

            De heer ANDELA (wethouder): Wat betreft 2011, kunt u dat overzicht krijgen voor het eind van
dit jaar.

            De heer STORK (ALA-AOV): Oké.

         De heer VAN MARLE (wethouder): Mevrouw de voorzitter! Met betrekking tot het incidentele
geld dat vrijkomt door de bezuinigingsmaatregel die bij Scoop wordt getroffen, heb ik vanmorgen al
gezegd welke afspraken wij gemaakt hebben. Volgens die afspraken vloeien de opbrengsten van
incidentele bezuinigingen naar de reserve takenanalyse en vloeien structurele bezuinigingsopbrengsten
naar de risicovoorziening. Ik heb in eerste instantie begrepen dat dit voor de jaren 2012, 2013 en 2014
is opgenomen (omdat de begrotingscyclus loopt tot en met 2014), maar ik heb de raad ook horen
zeggen dat het de bedoeling is dat dit structureel gebeurt. Als dat inderdaad de bedoeling is, dan zijn
wij het daarover met elkaar eens.
         Tegen de heer De Olde zeg ik dat de ozb en hondenbelasting stijgen met het inflatiepercentage,
zoals wij hebben afgesproken in de afgelopen jaren. Die afspraak geldt ook voor het jaar 2012. Voor de
volledigheid: het inflatiepercentage is 2,25.
         Verschillende fracties hebben ook in de tweede termijn gesproken over de positie van het
grondbedrijf en hoe wij daarmee willen omgaan (dat slaat ook op een heleboel amendementen en
moties). Wat dat betreft, wil ik een nadere toelichting geven op mijn bijdrage van vanmorgen. Ik heb
toen bedoeld te zeggen dat wij bij de voorjaarsnotabehandeling in 2012 bij de raad komen met een
integrale visie op het grondbedrijf, maar ook op de grote projecten, zijnde de binnenstad en Waterrijk.
Wij zullen daarvoor een inschatting maken van de aantallen, maar kijken dan ook naar het
realiteitsgehalte van Marsroute 2. Ook zullen wij daar uiteraard financieel naar kijken. Een en ander
zullen wij dan meenemen in de voorjaarsnota 2012, niet eerder.

        De heer SCHUUR (PvdA): Voorzitter! Dit is een prachtig antwoord, want dat is waar wij om
gevraagd hadden (volgens mij hadden daar inderdaad meer fracties om gevraagd). Dit antwoord scheelt
volgens mij straks weer een paar amendementen! Wat mij nu triggert, is of het college ook bereid is om
te bekijken welke ambities wellicht niet meer realiseerbaar zijn en om daarvoor scenario's op te stellen.




INT - 1201417                                                                                         85
</p><p>         De heer VAN MARLE (wethouder): Of ambities niet meer realiseerbaar zijn, zal blijken tijdens de
rit op weg naar de voorjaarsnota. Daarvan hebben wij dan de conclusies en consequenties te
aanvaarden.
         Over de schuldpositie van de gemeente Almelo zijn vragen gesteld door de heren Schuur en
Gerritsen. De stijging van de schuldpositie in 2011 is voor een belangrijk deel veroorzaakt door:
- De herfinanciering/omzetting van kortlopende schulden in langlopende schulden. Dit betreft een
    bedrag van € 20 miljoen.
- Ook zijn de kosten van de bouw van de IISPA erbij gekomen.
- Hetzelfde geldt voor de aankoop van grond die wij nog in Waterrijk moesten afnemen vanwege het
    op basis van de WVG gevestigde voorkeursrecht. Dit betreft een bedrag van ruim € 30 miljoen.
         Ik noem nu even de grootste posten (maar er zijn zowel plus- als minposten). Hiermee heb ik de
stijging van de schuldpositie al te pakken. Ik kan vertellen dat bij ongewijzigd beleid de schuldpositie
ook in 2013 zal stijgen, gezien de beslissing die wij voor de afgelopen zomervakantie hebben genomen
inzake Heracles. Als Heracles nieuwbouw gaat plegen, zullen wij een financiering aan moeten trekken,
waardoor de schuldpositie stijgt. Ik wijs erop dat de schuldpositie vooral niet gefixeerd moet worden op
een bepaald percentage, want dan loop je jezelf (zowel college als raad) aardig voor de voeten. Daarom
heb ik vanmorgen ook de percentages van de schuldpositie van diverse andere gemeentes gegeven. Die
percentages variëren nogal. Het is helemaal afhankelijk van je programma en ambities hoe je
schuldpositie eruit ziet.

        De heer SCHUUR (PvdA): Voorzitter! Hoor ik de wethouder nu zeggen dat als wij streven naar
een schuld die maximaal 65% is van het begrotingstotaal, zoals de fractie van D66 vraagt, wij de
lening van € 15 miljoen niet kunnen verstrekken aan Heracles? Is dat wat hij zegt?

        De heer VAN MARLE (wethouder): Als de schuld gefixeerd wordt op die 65%, dan is er
inderdaad geen ruimte meer. Dan loop je jezelf voor de voeten en kun je gemaakte afspraken niet
uitvoeren. Dat klopt.

        De heer BRAND (CDA): Voorzitter! Ik wijs erop dat de gemeente aan de ene kant € 15 miljoen
leent aan Heracles, maar aan de andere kant de kosten terugkrijgt, plus nog een renteopslag. Aan de
ene kant lijkt dit een verhoging van de schuld, maar aan de andere kant nemen de baten toe. Is dat
juist?

       De heer VAN MARLE (wethouder): Het lijkt niet alleen een verhoging van de schuld, het is een
verhoging van de schuld. Aan de andere kant hebben wij op de balans - wij doen aan dubbel
boekhouden - ook een vordering op Heracles. De balans is in evenwicht, maar de schuldpositie stijgt
wel.

       De heer BRAND (CDA): En er is een voordeel, omdat de gemeente meer rente vraagt aan
Heracles dan de rente waartegen ze zelf moet lenen.

        De heer VAN MARLE (wethouder): Ja, in dit geval wel.

        De heer GERRITSEN (D66): Voorzitter! Precies, daar staat wat tegenover van Heracles.
        De opmerking die de heer Schuur zojuist maakte, is natuurlijk heel leuk, maar waar wij ons
specifiek op richten, is de situatie waarin de gemeente nu verkeert. De gemeente heeft nu een schuld,
en die betreft heel veel grond. Waar wij met de schuldpositie van 65% op doelen (en dat zullen wij ook
aangeven in een gewijzigde versie van motie R), is dat geprobeerd moet worden het percentage van de
schuldpositie zover mogelijk naar beneden te brengen. Wij streven er niet naar om de schuldpositie
volgend jaar al 65% te laten zijn, maar het streven moet wel zijn dat het percentage verlaagd wordt.
Dan wordt de schuldpositie lager, loopt de gemeente minder risico's en heeft ze een kleinere
risicoreserve nodig. Kortom, een lagere schuld heeft alleen maar voordelen.

        De heer VAN MARLE (wethouder): Uit de beraadslagingen die wij richting de voorjaarsnota
voeren omtrent de positie van het grondbedrijf, zal blijken hoe de vermindering van de
langetermijnschuldpositie er eventueel uit komt te zien.

        De heer GERRITSEN (D66): Dat ben ik met u eens. Dat betekent dat wij bij de
voorjaarsnotabehandeling keuzes kunnen maken. Als de nog in te dienen gewijzigde versie van motie R



INT - 1201417                                                                                        86
</p><p>een meerderheid krijgt in de raad, moet geprobeerd worden om het percentage van de schuldpositie
naar beneden te brengen. Dat is een uitdaging, en het resultaat daarvan zien wij bij de voorjaarsnota.

        De heer SCHUUR (PvdA): Voorzitter! Dit vind ik toch iets te gemakkelijk gesteld door de heer
Gerritsen. Hij zei dat wij het aan Heracles geleende geld terugkrijgen. Nee, het is een risicodragende
lening, en die zorgt ervoor dat de schuld in eerste instantie vergroot wordt. Daar zijn wij als raad mee
akkoord gegaan, en daar is niets mis mee, maar ik wijs er wel op dat de fractie van D66 de eerste was
die met de handen in de lucht riep dat de lening verstrekt moest worden. Gelet daarop, vind ik het een
beetje jammer dat de D66-fractie nu ongelooflijk moeilijk doet over de schuldpositie.

         De heer GERRITSEN (D66): Ik praat over de schuldpositie van het grondbedrijf. Dat is het
grootste probleem waar wij nu mee zitten. Daar heb ik mij ook op gericht in mijn bijdragen, waarin ik
gesteld heb dat de schuldpositie van het grondbedrijf verlaagd moet worden omdat daar de grote
risico's zitten. Het gaat nu niet alleen maar om de lening aan Heracles, want straks moet de gemeente
ook weer geld lenen voor het nieuwe stadhuis. Daar heeft u toch ook voor gestemd? Dat is ook een
risicovergroting. Wij zeggen: Nu de meerderheid daarvoor gekozen heeft, zullen wij de schuldpositie
van het grondbedrijf naar beneden moeten halen.

       De heer SCHUUR (PvdA): Voorzitter! Natuurlijk hebben wij voor het nieuwe stadhuis gekozen,
zoals wij ook voor de lening aan Heracles hebben gekozen. Wij hijsen op dit moment echter niet de
stormbal en zeggen niet: De schuldpositie van de gemeente moet naar 65% van het begrotingstotaal.
Dat doet de D66-fractie wel, en dat is niet consistent.

        De heer VAN MARLE (wethouder): Voorzitter!              Wat ik wilde zeggen over de schuldpositie
van de gemeente, heb ik inmiddels gezegd.
        Ik wil nu een opmerking maken over de financiële positie van het Sportbedrijf, waar de heer
Schuur naar gevraagd heeft. Het college ontvangt nog dit jaar de exploitatiebegroting 2012 van het
Sportbedrijf. Die begroting ontvangt de raad ook. Ik kan nu al zeggen dat de exploitatiebegroting 2012
die wij ontvangen hebben, sluitend is. Weliswaar moet die nog nader geanalyseerd worden, maar de
raad zal die ook ontvangen.
        De heer Hümmels stelde dat hij met betrekking tot de begroting 2012 geen keuzes en
afwegingen kon maken. Die hebben wij echter al gemaakt met elkaar, namelijk bij de behandeling van
de voorjaarsnota 2011. Toen is een pakket bezuinigingsmaatregelen voorgelegd aan de raad, en de
behandeling daarvan heeft geleid tot een bepaalde uitkomst, namelijk € 9,5 miljoen bezuinigen. Die
uitkomst hebben wij meegenomen in de begroting voor 2012.
        Verder deed de heer Hümmels de suggestie om de norm voor de algemene reserve te verlagen
van 10% van de algemene uitkering naar 8%. Daar wil ik in dit stadium nog niet aan toegeven, gezien
de onzekerheid met betrekking tot de rijksbezuinigingen, de economische onzekerheid en de Europese
financiële onzekerheid, die uiteindelijk ook op de lidstaten van invloed is en zal zijn.
        Dan de toeristenbelasting. Ik heb vanmorgen al gezegd dat daarover afspraken zijn gemaakt met
de raad bij de behandeling van de najaarsnota 2010. Toen is afgesproken om de opbrengsten van de
toeristenbelasting in te zetten voor nieuw beleid. Daar is uitdrukkelijk voor gekozen. De opbrengsten
van de toeristenbelasting zijn dus niet bedoeld om de bezuinigingen mee glad te strijken. Verder heeft
de raad bij de behandeling van de voorjaarsnota 2011 besluiten genomen over de inzet van deze
opbrengsten. In de voorjaarsnota waren de opbrengsten opgenomen als enerzijds ontvangsten en
anderzijds diverse intensiveringen. Ik noem daarvan de bedragen voor de topsport, cultuur, greenflag
parken en het zwembad. Als ik deze bedragen bij elkaar optel, kom ik zelfs boven het bedrag van
€ 250.000,-- uit. Ik weet dat deze zaken niet direct, maar zeker wel indirect bestemd zijn voor de
toeristen die de toeristenbelasting gaan betalen. Ik ontraad dan ook met klem het amendement (SA) dat
hierover is ingediend. De reden daarvoor is dat de consequentie van dat amendement is dat er een
structureel gat van € 250.000,-- wordt geschoten in onze begroting, tenminste als al zou kunnen wat
in het amendement staat, want dan wordt er een bestemmingsreserve aangelegd van de opbrengst van
algemene heffingsmaatregelen, maar dat terzijde.

         De heer NIJHUIS (VVD): Daarom zullen wij ook met een dekkingsvoorstel komen. Het zal
duidelijk zijn dat wij de opmerking dat wij ergens een financieel gat in schieten, niet over onze kant
laten gaan. Als dat zo zou zijn, dan komen wij ook met een dekkingsvoorstel.
         Verder wil ik tegen de wethouder zeggen dat wij de innovatie vooral zagen in de toeristische
sector (ik praat nu over onze beleving, maar die zal waarschijnlijk fout zijn geweest). Wij zagen de



INT - 1201417                                                                                            87
</p><p>opbrengst van de toeristenbelasting niet als een algemeen dekkingsmiddel. Af en toe bekruipt mij het
gevoel dat die opbrengst nu toch een beetje gebruikt is als stopverf voor de gemeentebegroting (zoals
onze partijleden in de Tweede Kamer dat uitdrukken). Dat moeten wij vooral niet willen. Wij moeten
wat de VVD-fractie betreft, de opbrengst van de toeristenbelasting gaan gebruiken voor innovaties in
vooral de toeristische sector. Maar nogmaals, de VVD-fractie zal met een dekkingsvoorstel daarvoor
komen, dat is duidelijk.

        De heer VAN MARLE (wethouder): Ik heb duidelijk aangegeven dat wij afgesproken hebben dat
wij de opbrengst van de toeristenbelasting gaan gebruiken voor nieuw beleid. Dat is nu uitgerold in de
intensiveringen die ik genoemd heb.

        De heer SCHUUR (PvdA): Voorzitter! Het zou fijn zijn als wij nog tijdens dit debat van de heer
Nijhuis te horen krijgen op welke manier hij het voorstel inzake de toeristenbelasting denkt te dekken in
de begroting. Anders zitten wij straks alleen maar met het amendement daarover en hebben wij geen
mogelijkheid meer om daarover van gedachten te wisselen.

        De heer NIJHUIS (VVD): Voorzitter! Daar kom ik straks eventueel op terug bij de indiening van
de nieuwe versie van het amendement (SA) inzake de toeristenbelasting. Ik wil dat namelijk eerst goed
nakijken voordat ik hier iets roep.

         Mevrouw TEN SELDAM-EVEN (CDA): Wat de CDA-fractie betreft, is het vanaf het begin klip en
klaar geweest - want zo staat het ook in de nota's - dat de opbrengst van de toeristenbelasting naar de
algemene middelen gaat. Door de inkomstenverhoging die daardoor ontstaat, zou nieuw beleid mogelijk
zijn. Zo is het geformuleerd. Dat de inkomsten uiteindelijk hebben geleid tot de door de wethouder
genoemde intensiveringen, kun je ook zien als nieuw beleid. In die zin is er dus geen verschil met wat
eerder afgesproken is. Lees de handelingen er op na, zou ik zeggen.

        De heer HÜMMELS (Leefbaar Almelo): Ik wil wel een deel van de vraag over de besteding van
de opbrengst van de toeristenbelasting beantwoorden, omdat ik volgens mij daar ook over meegedacht
heb. Ik hoop dat de VVD-fractie mij dat niet kwalijk neemt, maar dat zal wel meevallen (dat hoor ik
later wel). Volgens wat ik heb voorgesteld, zou de investering in de toeristische sector onder andere
bekostigd kunnen worden uit een verlaging van de algemene reserve en een verhoging van de ozb.
Over de dekking hoeft men zich dus geen zorgen te maken, want van mijn kant is er een dekking
aangegeven. Of die mogelijk is, hangt ook af van de besluitvorming door de raad, maar de VVD-fractie
is mogelijk nog creatiever dan ik ben!

        De heer NIJHUIS (VVD): Juist daarom, voorzitter, lijkt het mij verstandig om nu nog niet direct
wat te roepen over de dekking.

        De heer SCHUUR (PvdA): Voorzitter! Ik vind dit toch wel een gekke gang van zaken. Op het
laatste moment is nog even een amendement ingeschoten dat een gat van € 250.000,-- schiet in de
begroting. Dit betekent dat wij straks wellicht een derde termijn nodig hebben om te kunnen debatteren
over de dekking daarvoor. Ik vind het apart dat wij daar nu geen inzicht in krijgen.

       De heer GERRITSEN (D66): Voorzitter! Er wordt geen gat in de begroting geschoten, want er
komt straks een dekkingsvoorstel. Dan is het probleem gewoon opgelost. Overigens zullen wij straks
ook nog een gewijzigde versie van een motie indienen. Dan mag de heer Schuur daar rustig nog even
over nadenken. Dat is geen enkel probleem.

         De heer SCHUUR (PvdA): Zoals ik zojuist aangaf, moeten wij nu al inzicht in de dekking krijgen
als wij serieus willen nadenken en debatteren. Straks zijn er misschien fracties die wel voor het voorstel
inzake de toeristenbelasting zijn, maar die de dekking niet zien zitten. Zo kom je natuurlijk niet verder
met elkaar.

        De heer VAN MARLE (wethouder): Voorzitter! Ik wil ingaan op de vraag van de heer Buitenweg
waarom wij niet meedoen aan de stresstest voor gemeenten. Die vraag had ik in de eerste instantie
niet beantwoord. Ik kan nu zeggen dat wij als college van oordeel zijn dat het meedoen aan een
stresstest geen nieuwe of aanvullende inzichten zal opleveren, omdat wij de financiële problematiek en
de risico's goed in beeld denken te hebben. Daarbij gaat het om inzicht in de schuldpositie (die hebben



INT - 1201417                                                                                          88
</p><p>wij goed in beeld) en de weerstandscapaciteit in de vorm van reserves en voorzieningen. Ook zijn wij
zelf in staat om de financiële effecten door te rekenen indien zich concrete risico's zouden voordoen.
Het lijkt ons effectiever en efficiënter om de energie nu te richten op de korte termijn en om in nauw
samenspel met de raad voorstellen en scenario’s voor te bereiden om de bestaande financiële situatie
het hoofd te bieden.
         De heer Gerritsen heeft mij uitgenodigd om bij de volgende voorjaarsnota te komen met het
percentage waarmee de ozb verhoogd zou moeten worden. Ik kan op dit moment niet op de zaken
vooruitlopen. Ik heb wel inzicht in hoe 2012 eruit gaat zien, maar ik heb nog geen inzicht in hoe 2013
eruit zal zien. Daar wil ik op dit moment niets over zeggen.
         Dit laatste geldt ook voor de vraag van de heer Gerritsen of bij de volgende voorjaarsnota
radicale keuzes (zijn terminologie) gemaakt zullen worden en wat de kosten zullen zijn van de
openeinderegelingen. Ik kan en ik wil daar op dit moment geen inschatting van maken. Dat ik dat niet
kan, moet men mij maar niet kwalijk nemen.
         Ook vroeg de heer Gerritsen, naar aanleiding van mijn afwijzing van de motie (O) over extra
bezuinigen op alle beleidsterreinen, of ik alvast wat kan zeggen over het bedrag dat dat op kan leveren
(volgens de motie is dat € 3 miljoen). In de loop van de tijd naar de voorjaarsnotabehandeling zal ik
daarover graag in debat gaan met de heer Gerritsen. Dan moeten wij maar zien op welk bedrag wij
uitkomen, maar op dit moment maak ik daar geen inschatting van. Het bedrag is ook afhankelijk van de
beoordeling van het grondbedrijf en de grote projecten, want die beoordeling heeft daar natuurlijk alles
mee te maken. Maar nogmaals, ik ga graag met de heer Gerritsen en zijn collega's in de raad in gesprek
over de vraag hoe wij de financiële consequenties van een en ander kunnen wegwerken c.q. tackelen.
         De heer Nijhuis heeft een vraag gesteld over de mogelijkheid om de kantoorruimte van het
Sportbedrijf te verplaatsen naar de IISPA. Ik kan melden dat de commerciële ruimten van de IISPA,
waar de heer Stork over sprak, inderdaad eigendom zijn van de gemeente en dat de huuropbrengsten
ten goede komen aan de gemeente. Het Sportbedrijf is nu gevestigd in het pand Jacobs Kranenburg
(zoals dat nog steeds heet in de wandelgangen). Ook dat is een pand van de gemeente. Daarom is het
wat het verhuren betreft, lood om oud ijzer of het Sportbedrijf nu in de commerciële ruimten van de
IISPA gehuisvest is of in het pand Jacobs Kranenburg. Wij zijn aandeelhouder van het Sportbedrijf.
Formeel is de zorg voor de huisvesting een taak van de bedrijfsleiding en de Raad van Commissarissen
van het Sportbedrijf. Wij zijn slechts aandeelhouder. Ik kan echter wel zeggen dat wij met de
bedrijfsleiding/directie en de Raad van Commissarissen in gesprek zijn over de huisvesting. Als daar
uitsluitsel over is, krijgt de raad dat te horen.

        De heer STORK (ALA-AOV): Voorzitter! Ik denk toch dat het juister zou zijn als het Sportbedrijf
verhuist naar de commerciële ruimten in de IISPA, en wel om de volgende redenen:
- Op de eerste plaats leidt dat tot enige beweging in het hele spul, want dat is op dit moment
   natuurlijk een ramp.
- Het pand Jacobs Kranenburg komt dan vrij. Dat zou de gemeente dan misschien op de markt
   kunnen gooien.
        Hoe denkt de wethouder hierover?

       De heer VAN MARLE (wethouder): Het gaat wel erg ver om het hier nu te hebben over het pand
Jacobs Kranenburg. Het Sportbedrijf is overigens niet de enige huurder in dat pand.

       De heer STORK (ALA-AOV): Misschien kunnen dan nog wel meer huurders naar de commerciële
ruimtes in de IISPA getransporteerd worden. Dan hebben wij die vol.

        De heer SCHUUR (PvdA): Voorzitter! Ik heb nog een vraag aan de wethouder met betrekking
tot de bezuiniging op Scoop van 3 ton (ik heb namelijk het gevoel dat hij bijna aan het eind van zijn
betoog is). Gelet op de wijze waarop deze bezuiniging nu in de begroting is verwerkt, is dat een
incidentele bezuiniging en is het de bedoeling om nader te bekijken of de bezuiniging structureel
gemaakt kan worden voor de jaren 2013 en 2014. Volgens mij is dit het traject. Is dat correct?

        De heer VAN MARLE (wethouder): Ja.

        De heer SCHUUR (PvdA): Oké.

      De heer VAN MARLE (wethouder): Ik heb de heer Schuur geantwoord dat het bedrag dat
gemoeid is met de bezuiniging op Scoop, als een incidentele post is opgenomen voor de jaren 2012,



INT - 1201417                                                                                           89
</p><p>2013 en 2014. Voor het jaar 2012 hebben wij daar een concrete invulling aan gegeven, want dan
wordt het bedrag gefinancierd uit de eigen middelen van Scoop. Verder heb ik begrepen van mijn
collega mevrouw Kuik dat zij bij de voorjaarsnota 2012 komt met een dekkingsvoorstel voor de
bedragen voor de jaren 2013 en 2014.

        Mevrouw KUIK-VERWEG (wethouder): Mag ik daar iets over zeggen, voorzitter?

        De VOORZITTER: Nee.

        De heer SCHUUR (PvdA): Voorzitter! Ik heb begrepen, maar misschien is dat niet helemaal
correct, dat nu volgens de begroting voor 2012 een bedrag van € 300.000,-- ingeleverd wordt door
Scoop en dat overlegd wordt met die organisatie om te bekijken hoe invulling gegeven kan worden aan
de bezuinigingsvoorstellen voor de jaren daarna, maar dat het resultaat daarvan nog niet zeker is.

       De heer VAN MARLE (wethouder): Dat gaat gebeuren bij de voorjaarsnota 2012. Dan praten wij
over de jaren 2013 en 2014.

       De heer SCHUUR (PvdA): Maar er vindt dus nog eerst overleg plaats met Scoop over hoe die
eventueel kan komen tot de bezuiniging? Dat is toch correct, of niet?

        De heer VAN MARLE (wethouder): Correct.

        De heer SCHUUR (PvdA): Oké.

       De heer MAATHUIS (VVD): Voorzitter! Ik heb een vraag aan de heer Van Marle. Hij heeft een
toezegging gedaan aan de heer Schuur over de scenario's voor het grondbedrijf. Die toezegging wringt
volgens mij een beetje met de beleidsvoornemens die zijn verwoord in de begroting voor 2012.
Waarom heeft het college daar niet in opgeschreven wat de wethouder heeft toegezegd?

        De heer VAN MARLE (wethouder): Ik heb gezegd, naar aanleiding van de amendementen en
moties die daarover zijn ingediend, dat wij op weg naar de voorjaarsnota 2012 de situatie van het
grondbedrijf nader onder de loep zullen nemen. Dat betreft Marsroute 2. Wij zullen opnieuw beoordelen
of de aantallen die in dat kader genoemd zijn, realistisch zijn voor de toekomst. Voorts betreft dit punt
de grote projecten, zoals Waterrijk en de binnenstad.

       De heer MAATHUIS (VVD): Voorzitter! Volgens mij is dit geen antwoord op mijn vraag. De
wethouder heeft zelf in de stukken aangegeven dat de rek er een beetje uit is (om het maar kort door
de bocht te zeggen). Daarom vraag ik waarom hij niet zelf met het nu toegezegde beleidsvoornemen is
gekomen?

        De heer VAN MARLE (wethouder): Op basis van de situatie van het grondbedrijf zeggen wij (en
dat heb ik al eerder gezegd): De aantallen die in Marsroute 2 zijn aangegeven voor het jaar 2012, lijken
op dit moment nog realistisch, gezien de ontwikkeling van een aantal projecten die van start gaan in
2011 en 2012. Wij hebben natuurlijk nog wel terdege rekening te houden met de situatie van de
afgelopen jaren (waarover ik de heer Stork nog zal antwoorden). Ook hebben wij te maken met een
tweede financiële crisis, waarmee evenmin rekening gehouden is. Wij moeten wel bekijken wat de
consequenties daarvan zijn, of kunnen zijn, voor ons grondbedrijf. Dat heb ik bedoeld te zeggen.

        Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Om verwarring te voorkomen, kom ik nog
even terug op wat de wethouder zojuist geantwoord heeft met betrekking tot Scoop. Eerder is een
amendement aangenomen waarin een taakstelling was opgenomen voor Scoop voor de jaren 2012,
2013 en 2014. In de begroting die nu voorligt, is daar invulling aan gegeven voor de jaarschijf 2012. Ik
heb zojuist gezegd dat de taakstelling een structureel karakter heeft, omdat wij hierbij praten over drie
jaren. De discussie gaat nu over de vraag of de taakstelling voor 2012 incidenteel is, dan wel of de
taakstelling een structureel karakter heeft. Het antwoord hierop is van belang voor waar het geld
naartoe gaat, namelijk naar de risicovoorziening of niet. Dit moet men los zien van de inhoudelijke
uitvoering van het eerder aangenomen amendement. Als blijkt dat er bij de voorjaarsnota 2012 andere
voorstellen komen, dan zien wij die op dat moment wel. Nu gaat het erom hoe met het oog op de
reserves de bezuiniging op Scoop wordt geduid.



INT - 1201417                                                                                         90
</p><p>        De heer SCHUUR (PvdA): Voorzitter! Ik stel vast dat wanneer de wethouder zo meteen klaar is
met de beantwoording, de tweede termijn voorbij is. Daarom kondig ik nu alvast aan dat de PvdA-
fractie graag een schorsing van de vergadering wil voordat de tweede termijn wordt gesloten.

        De VOORZITTER: Ik zal straks inventariseren welke behoeften er bestaan met betrekking tot de
tijd.

        De heer VAN MARLE (wethouder): Voorzitter! Ik kom nu bij de vraag die de heer Nijhuis heeft
gesteld over het toepassen van de residuele grondwaardebenadering. Dat doen wij niet als het gaat om
de verkoop van grond in individuele gevallen. Wij doen dat wel bij verkopen aan projectontwikkelaars
en woningbouwcorporaties. Dan is het heel gebruikelijk om de residuele grondwaardebenadering toe te
passen. Die splitsing hanteren wij, en daar blijven wij voorlopig van uitgaan.

        De heer NIJHUIS (VVD): Voorzitter! Dit betekent dat deze systematiek niet kan worden
toegepast in Almelo Noordoost, omdat daar niet wordt gebouwd door woningbouwcorporaties of
projectontwikkelaars (afgezien misschien van de enkele ontwikkelaar die daar wellicht kleinere
hoeveelheden woningen wil bouwen). Voor individuen is deze systematiek dus niet toepasbaar. Ik
probeer juist om te bereiken dat er meebewogen wordt met de markt. Dat kan via het voor particulieren
en andere partijen toepassen van de systematiek van de residuele grondwaardebenadering. Ik heb
eerder al eens aangegeven dat wanneer men het uitmiddelt met het totaal van het aantal jaren dat de
exploitatie loopt, men op een redelijk gemiddelde uitkomt. Het zou de moeite waard zijn om, zoals de
wethouder indertijd toegezegd heeft, dit nog even door te rekenen, om te zien waar wij dan op
uitkomen. Die berekening heb ik tot op heden nog niet gezien.

        De heer VAN MARLE (wethouder): Daar heeft u gelijk in. Het gaat hierbij voornamelijk om de
kavels in Almelo Noordoost. Daaromtrent hebben wij een aantal vragen uitstaan. Daarbij hebben wij
gemerkt dat de residuele grondwaardebenadering daar gewoon niet van toepassing is. Die vraag is daar
niet. Mensen gaan gewoon uit van de marktwaarde van de kavels die zij willen kopen. Bij een eerdere
gelegenheid heb ik inderdaad toegezegd dat wij daar naar zullen kijken. Ik zal dat nogmaals op mij
nemen.
        Voorzitter! De heer Stork heeft gesproken over de waardebepaling van de lopende projecten.
Daarvoor houden wij ons aan de regels van het BBV (Besluit Begroting en Verantwoording. Dat wil
zeggen dat gronden in de lopende grondexploitaties niet als grondvoorraad worden gewaardeerd maar
als onderhanden werk, conform de BBV-voorschriften. Uiteindelijk is dat natuurlijk ook een
marktwaarde, zo mag men van mij aannemen. De grondvoorraad die niet begrepen is in de lopende
grondexploitaties, is ondergebracht in het complex verspreide bezittingen. Die voorraad wordt
gewaardeerd op basis van de economische waarde bij de huidige bestemming. Dat is ook de
marktwaarde. De conclusie is dat wij op basis van marktwaarde de grondwaarden bepalen, zij het dat
wij daarbij wel een splitsing aanbrengen tussen de lopende projecten en de grondvoorraad.

        De heer STORK (ALA-AOV): Wat doet u dan met de opmerking die Deloitte naar aanleiding van
de jaarrekening heeft gemaakt over de taxaties en waardebepalingen?

       De heer VAN MARLE (wethouder): Ik ken die opmerking van Deloitte niet exact, maar die heeft
ons geen aanleiding gegeven om dingen te wijzigen. Wij bepalen de zaken namelijk volgens de
voorschriften van het BBV. Daar heeft ook Deloitte zich aan te houden. Zoals ik zei, gaan wij wel uit
van de marktwaarden, zij het uitgesplitst naar lopende projecten en de grondvoorraad opgenomen in
het complex verspreide bezittingen.

        De heer STORK (ALA-AOV): Dan verzoek ik u toch om het rapport van Deloitte eens te
bekijken. Dat dateert van 26 april 2011. Daarin staat dat de getaxeerde grondwaarden moeten worden
opgenomen. Dat advies volgt u niet op.

       De heer VAN MARLE (wethouder): Ik zei dat wij de marktwaarden volgen. Voor onze
grondvoorraad hanteren wij de economische waarde bij de huidige bestemming, oftewel: de
marktwaarde op dit moment. Dat kan ook de getaxeerde waarde zijn. Dat zal ook de getaxeerde
waarde zijn.




INT - 1201417                                                                                      91
</p><p>        De heer STORK (ALA-AOV): Het spijt mij, maar nu veroorzaakt u mist door te zeggen dat het
dit of dat kan zijn. Het kan de economische waarde zijn, het kan de marktwaarde zijn…

       De heer VAN MARLE (wethouder): Ik heb het duidelijk niet over de getaxeerde waarden. Ik heb
het over marktwaarden.

      De heer STORK (ALA-AOV): Het gaat mij er alleen om dat u moet uitvoeren wat in het door mij
genoemde rapport staat. Daarin staat dat de waarden moeten worden opgenomen zoals ik heb
aangegeven. Dat doet u niet.

       De heer VAN MARLE (wethouder): Wij doen dat zo, ook conform de BBV-voorschriften. Daar
heeft Deloitte geen op- of aanmerkingen over gemaakt bij de jaarrekening, noch bij de Perspectiefnota
Grondbedrijf.

      De VOORZITTER: Volgens mij is dit debat rond. Wethouder, gaat u verder met uw
beantwoording.

        De heer VAN MARLE (wethouder): Voorzitter! Ik heb het al gehad over de septembercirculaire.
De raad is via een brief op de hoogte gesteld van de uitkomsten daarvan. De verwerking van de
wijzigingen die voortvloeien uit de septembercirculaire, vindt plaats in de begroting 2012, zij het niet in
de versie die voorligt, want dat is de primitieve begroting. Nee, ze zullen worden verwerkt in de
begrotingswijzigingen die hierna komen. Structurele wijzigingen worden meegenomen in de
voorjaarsnota 2012.
        Als laatste de vraag van de heer Stork over de bouwvolumes. Ik heb die paraat voor het jaar
2011, het lopende jaar. Gebouwd zijn 67 woningen en er zijn acht kavels verkocht in Almelo Noordoost
en één in Ooster-Es. Verder zijn op dit moment 80 woningen in aanbouw. Daarnaast is nog een
bouwterrein voor woningen verkocht in Weggeler.

       De heer STORK (ALA-AOV): Voorzitter! Ik had ook een vraag gesteld over de woningbouw in
2010. De cijfers daarover moet de wethouder toch ook hebben?

        De heer VAN MARLE (wethouder): Die heb ik niet paraat.
        Voorzitter! Tot zover de beantwoording.

       De VOORZITTER: Wethouder! Ik beschouw uw beantwoording in de tweede termijn nog niet als
afgerond, omdat de PvdA-fractie heeft gevraagd om een schorsing. Ik meen dat die gegund moet
worden.

        Hierna schorst de VOORZITTER de vergadering.

        Na hervatting der vergadering verleent de VOORZITTER het woord aan de heer Schuur.

       De heer SCHUUR (PvdA): Mevrouw de voorzitter! Ik dank u voor de flexibiliteit om ons nog
even de tijd te gunnen om een en ander nader te overwegen voordat de tweede termijn beëindigd was.

        De VOORZITTER: Dit is geen derde termijn.

        De heer SCHUUR (PvdA): Ik realiseer mij dat dit geen derde termijn is. Dat zal ik er ook niet van
maken. Wij hebben de tijd tijdens de schorsing nuttig besteed en zijn tot goede inzichten gekomen. Wat
mij betreft, mag de tweede termijn bij dezen beëindigd worden.

       De VOORZITTER: Dames en heren! Ik geef nu het woord aan de heer Veenstra, want hij heeft
nog een vraag aan wethouder Van Marle.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Ik heb in de tweede termijn aangegeven dat wat
mij betreft op een andere manier naar de reserves en voorzieningen moet worden gekeken dan wij tot
nu toe gewend zijn. Ik wil namelijk graag dat van alle reserves en voorzieningen wordt aangegeven
waarom ze niet vrij kunnen vallen. Dat is volgens mij de enige serieuze mogelijkheid om nog wat geld te




INT - 1201417                                                                                           92
</p><p>vinden. Daar heb ik van de wethouder geen antwoord op gekregen. Dat krijg ik graag alsnog. Daarna
heb ik nog een tweede vraag aan de wethouder.

       De heer VAN MARLE (wethouder): Voorzitter! De heer Veenstra spreekt nu over het beoordelen
van de reserves en voorzieningen. Ik heb gezegd dat deze beoordeling wordt betrokken bij het opstellen
van de voorjaarsnota 2012; dan kijken wij daar kritisch naar. Ik wil echter wel toezeggen dat wij ook in
het kader van onze zoektocht naar de wijze waarop het grondbedrijf er vanaf 2012 uit dient te gaan
zien, met een heel kritische bril alle posten langs zullen lopen. Dat deden wij natuurlijk altijd al (wij
hebben immers niet voor niets bestemmingsreserves en voorzieningen), maar wij zullen dat nu met een
dubbele bril op doen.

       De heer VEENSTRA (ChristenUnie): Communiceert u dan ook richting de raad waarom een
bepaalde post niet vrij zou kunnen vallen?

        De heer VAN MARLE (wethouder): In de jaarrekening wordt opgenomen welke vrijval er is van
reserves en voorzieningen. Daar krijgt de raad dan dus automatisch bericht van, maar in dit geval geven
wij dat bij de voorjaarsnota 2012.

        De heer VEENSTRA (ChristenUnie): Voorzitter! Dan mijn tweede vraag. Ik heb verzocht om
bezuinigingstaakstellingen per maatschappelijke opgave te formuleren. Ik begrijp dat het begrotings-
technisch richting de provincie heel lastig is om bezuinigingstaakstellingen in de begroting op te nemen.
Aan de andere kant heb ik begrepen dat bij het opstellen van de Sociale Agenda 2020 eigenlijk wel
datgene gebeurt wat ik vraag. Daarvoor wordt wel gezegd: Dat is de bezuinigingstaakstelling, dit zijn
de voorstellen, maar wij willen gezamenlijk komen tot eventueel betere voorstellen waarmee ook de
bezuiniging gehaald wordt. Het resultaat daarvan is dat uiteindelijk bekend wordt hoeveel geld besteed
wordt aan de desbetreffende maatschappelijke opgave, wat de kosten voor de gemeente zijn en wat er
bespaard wordt. Het klopt toch dat dit de aanpak is voor de Sociale Agenda 2020?

        De heer VAN MARLE (wethouder): Ik vind het een beetje lastig om terug te gaan en
taakstellingen op te nemen in de begroting. Ik meen dat wij dat niet moeten doen. Dan creëren wij
namelijk onduidelijkheid, niet zozeer voor ons eigen apparaat en de maatschappelijke organisaties, want
(in het geval van mijn portefeuille) wij betrekken de maatschappelijke organisaties wel degelijk bij de
bezuinigingen op de sport. Daarvoor ligt er een taakstelling, en een selectie van de verenigingen wordt
daarbij betrokken om hun inbreng te leveren met betrekking tot de vraag hoe de bezuiniging kan
worden ingevuld. Hetzelfde geldt wellicht voor de andere portefeuillehouders die zo gaan werken. Maar
om nu te zeggen dat wij alleen maar met taakstellingen gaan werken en dat wij die taakstellingen
opnemen in de begroting, gaat mij een beetje te ver.

         De heer VEENSTRA (ChristenUnie): Voorzitter! De wethouder heeft gezegd dat in de begroting
geen taakstellingen per maatschappelijke opgave kunnen worden opgenomen. Daarom probeer ik voor
mijzelf te zoeken naar een bepaalde vorm. De wethouder zei dat hij de door mij voorgestane aanpak
wel volgt op het gebied van de sport. Hij neemt besparingsvoorstellen mee naar de organisaties, maar
laat daarbij voldoende ruimte voor andere voorstellen. Uiteindelijk komt daar dan uit hoe de verdeling
wordt van de taken, verantwoordelijkheden en kosten over de gemeente en de sportverenigingen. Dat
zegt de wethouder toch?

       De heer VAN MARLE (wethouder): Dat klopt. Ik zei echter dat ik alleen namens mijzelf sprak; ik
gaf aan hoe ik het zelf aanpak. Ik kan niet voor de andere wethouders spreken.

         De VOORZITTER: Dames en heren! Ik sluit de beantwoording door het college in de tweede
termijn af. Daarmee zijn wij aan het eind gekomen van de beraadslaging over de begroting. Ik vraag u
of u nog ruimte en tijd nodig hebt voordat u laatste wijzigingen van moties en amendementen inlevert.
Ik stel vast dat dat inderdaad het geval is. Heeft u voldoende aan tien minuten?

        De heer NIJHUIS (VVD): Nee, voorzitter.

        De VOORZITTER: Mijnheer Nijhuis! Doet u eens een suggestie.

        De heer NIJHUIS (VVD): Een halfuur.



INT - 1201417                                                                                         93
</p><p>        De VOORZITTER: Akkoord. Dames en heren! Ik vraag u om alles wat u al klaar heeft, alvast in
te leveren bij de griffie. Ik hoop dat wij dan zo snel mogelijk de definitieve lijst van moties en
amendementen klaar kunnen maken.

        Mevrouw TEN SELDAM-EVEN (CDA): Voorzitter! Ik wil bij dezen melden dat wij amendement C
(Leerlingenvervoer) intrekken. Wat wij daarin stellen, komt of in een motie of in een amendement terug,
maar gehoord de wethouder, nu niet.

        De VOORZITTER: Dames en heren! Dat is duidelijk. Ik stel vast dat amendement C is
ingetrokken.

        De heer GERRITSEN (D66): Voorzitter! Wij trekken motie S (Parkeren Waterrijk) in. Voorts
dienen wij gewijzigde versies in van motie R (Schuld maximaal 65% van het begrotingstotaal) en P1
(Extra bezuinigen door focus op samenwerking). Die nieuwe moties luiden:

                "Motie P2.
                Extra bezuinigen door focus op samenwerking
                De raad van de gemeente Almelo in vergadering bijeen op 8 en 10 november 2011;
                Constateert:
                a. dat de schuldenlast van de gemeente toeneemt;
                b. dat de inkomsten van de gemeente afnemen;
                c. dat de schuldenlast bij ongewijzigd beleid het begrotingstotaal in 2013 zal
                   overtreffen;
                d. dat het college niet voornemens is extra maatregelen te treffen;
                Overweegt:
                a. dat extra maatregelen nodig zijn om aan de verplichtingen te kunnen blijven voldoen;
                b. dat extra maatregelen nodig zijn om nieuwe ontwikkelingen mogelijk te maken;
                c. dat extra bezuinigingen, het verhogen van de inkomsten of een mix van beide
                   onontkoombaar zijn;
                Besluit:
                a. het college op te dragen een sterkere focus op samenwerking en uitbesteding van
                   werk in de regio te realiseren en daarbij te streven naar een structurele besparing;
                b. de bevindingen te bespreken bij de behandeling van de voorjaarsnota 2012;
                c. dit bedrag aan te wenden voor fase 2 van het binnenstadsplan;
                En gaat over tot de orde van de dag."

                "Motie R1.
                Schuld van het begrotingstotaal
                Begroting 2012.
                De raad van de gemeente Almelo in vergadering bijeen op 8 en 10 november 2011;
                Constateert:
                a. dat de vaste schuld in % van het begrotingstotaal in 2011, 72% bedroeg;
                b. dat de vaste schuld in % van het begrotingstotaal in de conceptbegroting 2012,
                   88% bedraagt;
                Overweegt:
                a. dat bij ongewijzigd beleid de vaste schuld in % van het begrotingstotaal in 2013,
                   meer dan 100% zal bedragen;
                b. dat het gemiddelde van de Nederlandse gemeenten op ca. 65% ligt;
                c. dat ombuigen van bovengenoemde trend een positief effect zal hebben op het
                   weerstandsvermogen van de gemeente;
                Besluit:
                a. het college op te dragen er voor te zorgen dat er een trendbreuk tot stand wordt ge-
                   bracht, te beginnen bij de begroting 2013;
                b. verzoekt het college om de gemeenteraad, ruim voor de behandeling van de
                   Voorjaarsnota 2012, te informeren over de manier waarop en op welke wijze een
                   substantieel lager percentage kan worden bereikt;
                En gaat over tot de orde van de dag."




INT - 1201417                                                                                        94
</p><p></p><p></p>
"""
poliflow_html = """<div class="node-content clearfix"> &#13;
  &#13;
	 <p>Op 20 juni 2023 was de laatste etappe van de Voorjaarsnota 2024-2027 in de gemeenteraad van Zaanstad. Er werd gedebatteerd over moties en amendementen. E&#233;n motie van de SP, over het aansluiten bij Raden in Verzet, werd unaniem aangenomen.</p>
<p>Lekker belangrijk? Aansluiten bij Raden in Verzet? Hoezo en waarom?</p>
<p>De gemeenten hebben een groot financieel probleem. Natuurlijk ook Zaanstad. Zo'n twee jaar geleden werd door raadsleden in Zoetermeer een actiecomit&#233; opgericht om eens flink bij de Rijksoverheid te gaan protesteren tegen de financi&#235;le problemen. Een paar weken geleden riep het actiecomit&#233; Raden in Verzet gemeenteraadsleden op om ook in verzet te komen en aan te sluiten.</p>
<p>Het grootste knelpunt in de huidige raadsperiode is het zogeheten<br/>
financi&#235;le ravijn in 2026: gemeenteraden kunnen geen sluitende<br/>
meerjarenbegroting maken omdat er voor alle gemeenteraden in het land drie miljard euro ontbreekt. De rijksoverheid geeft vooralsnog niet thuis en laat de gemeenten met de problemen zitten. Dat kunnen we als gemeenteraden er niet bij laten zitten. Ook in Zaanstad niet. De gemeenteraad nam unaniem de motie van de SP aan, zodat de gemeenteraad vanaf nu ook in verzet gaat. Want het is voor de inwoners van Zaanstad heel erg belangrijk dat de rijksoverheid over de brug komt. We willen immers alle voorzieningen, vooral de sociale voorzieningen behouden. Want die zijn nodig om inwoners, die door de prijsstijgingen en de gasrekening het toch al moeilijk hebben, te geven waar zij recht op hebben.&#160;</p>
<p>Natuurlijk had de SP ook nog een aantal voorstellen die ervoor moesten zorgen dat de luchtkwaliteit, de natuur en het landschap gespaard blijven en bovendien een besparing realiseren die de inwoners van Zaanstad niet raakt. Wij hadden voorstellen om te stoppen met de 24- uurs openstelling van busbrug De Binding (zal hoogstwaarschijnlijk toch niet lukken en kost alleen maar steeds meer geld), de ondertunneling van de spoorwegovergang Guisweg (lost weinig problemen op en veroorzaakt veel andere problemen, vooral op het gebied van verkeer en natuur) en om te stoppen met het geldpotje voor de A8-A9 (die zal, nu die meer dan een miljard zal gaan kosten en geen toestemming krijgt van de werelderfgoedorganisatie Unesco t&#243;ch niet doorgaan). Helaas vonden wij alleen de Partij voor de Dieren aan onze zijde. Wij wachten de ontwikkelingen af.</p>
<p>Met steun van de SP komt er een onderzoek naar het slavernijverleden van Zaanstad, gaat het college ook op zoek naar geschikte banen voor mensen die willen werken maar moeilijk aan werk kunnen komen, worden jonge bomen voortaan gered en herplant, komt er een uitleg over de februaristaking en de beeldengroep bij het Zaantheater voor toeristen, wordt er alsnog &#8364; 16.000,- als verlate noodhulp aan de Molukken geschonken, krijgen veteranen een perkje met anjers en gaat het college een poging doen om asielzoekers eerder aan werk te helpen. &#203;n de groenadoptie wordt beter geregeld.</p>
<p>Het waren weer vier veelbewogen avonden in de gemeenteraad van Zaanstad.........&#160;</p>
<p>&#160;&#160; . &#160;</p>
 <div class="field field-name-field-bijlage field-type-file field-label-above"><div class="field-label">Bijlage:&#160;</div><div class="field-items"><div class="field-item even"><span class="file"><img class="file-icon" alt="PDF icon" title="application/pdf" src="/modules/file/icons/application-pdf.png"/> <a href="https://zaanstreek.sp.nl/sites/zaanstreek.sp.nl/files/attachments/motie_voorjaarsnota_2024_raden_in_verzet3.pdf" type="application/pdf; length=355374">motie_voorjaarsnota_2024_raden_in_verzet3.pdf</a></span></div></div></div>        </div>&#13;
&#13;
		
"""
woogle_txt = """De burgemeester heeft besloten om op grond van artikel 2:25 van de Algemene plaatselijke Verordening Waterland 2010 een evenementen vergunning te verlenen en op grond van artikel 35 van de Drank- en Horecawet een ontheffing te verlenen aan Ondernemersvereniging Waterland,  voor:  het      organiseren en houden van het evenement Wijnroute Monnickendam 2017  op zaterdag 14      oktober van 19.00- 22.00 uur in de binnenstad van Monnickendam;  voor het verstrekken van zwak-alcoholhoudende dranken bij bijzondere gelegenheden van zeer tijdelijke aard, namelijk het evenement Wijnroute Monnickendam 2017. Deze ontheffing geldt op zaterdag 14 oktober 2017 van 19.00-21.00 uur in de binnenstad van Monnickendam.

Evenementenvergunning en ontheffing artikel 35 van de Drank- en Horecawet voor het evenement Wijnroute Monnickendam, 14 oktober 2017, Waterland



Evenementenvergunning en ontheffing artikel 35 van de Drank- en Horecawet
voor het evenement ‘Wijnroute Monnickendam’, 14 oktober 2017, Waterland
De burgemeester heeft besloten om op grond van artikel 2:25 van de Algemene plaatselijke Verordening
Waterland 2010 een evenementen vergunning te verlenen en op grond van artikel 35 van de Drank- en
Horecawet een ontheffing te verlenen aan Ondernemersvereniging Waterland,  voor:
 
•
het organiseren en houden van het evenement “Wijnroute Monnickendam 2017 ” op zaterdag
14 oktober van 19.00- 22.00 uur in de binnenstad van Monnickendam;
•
voor het verstrekken van zwak-alcoholhoudende dranken bij bijzondere gelegenheden van zeer
tijdelijke aard, namelijk het evenement ‘Wijnroute Monnickendam 2017’. Deze ontheffing geldt
op zaterdag 14 oktober 2017 van 19.00-21.00 uur in de binnenstad van Monnickendam.
 
Tijdens dit evenement worden de Kerkstraat (vanaf de Kermergracht), het Noordeinde (tot de Molenstraat)
de Middendam/Damsluis, het Zuideinde, de Gooische Kaai en de Havenstraat (vanaf Wijnhuis Monnic-
kendam) afgesloten voor doorgaand verkeer.
 
Heeft u bezwaar?
Bent u het niet eens met deze beslissing? Dan kunt u een bezwaarschrift indienen bij de burgemeester.
Dit geldt ook voor andere belanghebbenden die het niet eens zijn met dit besluit. Het adres is:
Burgemeester van Waterland, Postbus 1000, 1140 BA MONNICKENDAM.
 
Zorgt u ervoor dat u het bezwaarschrift indient binnen zes weken na de dag waarop dit besluit is ver-
zonden. Schrijf in uw bezwaarschrift ten minste:
•
uw naam, adres en handtekening;
•
een omschrijving van het besluit waartegen u bezwaar tegen maakt
•
de reden waarom u bezwaar maakt.
 
Wilt u een voorlopige voorziening treffen of de beschikking schorsen? Dit kunt u vragen aan de voorzie-
ningenrechter van de rechtbank Noord-Holland. Het adres is:
Rechtbank Noord-Holland
Postbus 1621
2003 BR HAARLEM
Na ontvangst van het verzoekschrift geeft de rechtbank informatie over de heffing van griffierecht.
 
Digitaal indienen? Dit kan met een DigiD inlogcode.
Bij een bezwaarschrift via het digitale loket op www.waterland.nl
Bij een voorlopige voorziening via www.loket.rechtspraak.nl/Burgers/.
Nr. 177358
GEMEENTEBLAD
12 oktober
2017
Officiële uitgave van de gemeente Waterland
Gemeenteblad 2017 nr. 177358
12 oktober 2017
1"""
obk_txt = """

Besluit omgevingsvergunning reguliere procedure verleend Beethovenstraat 
70 1077JL Amsterdam, President Kennedylaan 15 1079MC Amsterdam 
Omschrijving: Kappen van twee iepen staande in de openbare ruimte op de locatie Beethovenstraat 
70 en President Kennedylaan 15 in Amsterdam, er geldt een herplantplicht 
Besluit: Verleend 
Besluit verzonden op: 25-09-2024 
Zaakadres: Beethovenstraat 70 1077JL Amsterdam, President Kennedylaan 15 1079MC Amsterdam 
Zaaknummer: Z2024-026310 
DSO-nummer: 2024090400410 
Meer informatie 
Het besluit en bijbehorende stukken kunt u per e-mail ontvangen. Stuur een verzoek naar vth.admini- 
stratie.sdz@amsterdam.nl en vermeld daarin: zaakadres, zaaknummer en DSO-nummer. 
Bezwaar tegen dit besluit 
Misschien bent u het niet eens met dit besluit? Als belanghebbende mag u bezwaar indienen. Dit moet 
binnen 6 weken na 25-09-2024. U dient uw bezwaar in: 
1. via: www.amsterdam.nl/bezwaar/jb met uw DigiD of E-herkenning 
2. per post naar Gemeente Amsterdam; T.a.v. Juridisch Bureau; Postbus 483; 1000AL Amsterdam 
Vermeld in uw bezwaarschrift altijd: 
- uw naam, adres, telefoonnummer en eventueel uw e-mailadres 
- de datum waarop u het bezwaarschrift schrijft en uw handtekening 
- het besluit waartegen u bezwaar maakt, met datum, zaaknummer en DSO-nummer van het besluit 
- waarom u bezwaar indient 
- misschien dient iemand anders namens u het bezwaar in. Stuur dan een machtiging mee. 
U vindt meer informatie op: https://www.amsterdam.nl/veelgevraagd onder het tabblad contact. 
Voorlopige voorziening 
Indienen van een bezwaar heeft geen schorsende werking. Dat betekent dat het besluit waartegen u 
bezwaar maakt geldig blijft totdat er een besluit is genomen op uw bezwaar. U kunt dat voorkomen 
door een voorlopige voorziening bij de bestuursrechter te vragen. Daar moet u voor betalen. Ga voor 
informatie over de kosten en over de voorwaarden naar https://www.rechtspraak.nl. U dient een verzoek 
om een voorlopige voorziening in via https://mijn.rechtspraak.nl met uw DigiD of E-herkenning. 
Nr. 417348 
GEMEENTEBLAD 
2 oktober 
2024 
Officiële uitgave van de gemeente Amsterdam 
Gemeenteblad 2024 nr. 417348
2 oktober 2024 
1 """

html_doc_array = [cvdr_html, poliflow_html]  
html_txt_doc_array = [openbesluitvorming_html_2]  
txt_doc_array = [oor_txt, woogle_txt, obk_txt]  

# Pattern to match "Pagina X van Y" where X and Y are numbers
pagina_pattern_1 = r'Pagina \d+ van \d+'

def clean_page_numbers(text):
    return re.sub(pagina_pattern_1, '', text)
 
def has_page_numbers(text):    
    return bool(re.search(pagina_pattern_1, text))
 
def clean_extra_whitespace(text):
    return " ".join(text.split())

def html_table_to_markdown(html_table):
    """Convert HTML table to markdown format.
    
    Args:
        html_table (str): HTML string containing a table
        
    Returns:
        str: Markdown formatted table
    """
    # Parse HTML
    soup = BeautifulSoup(html_table, 'html.parser')
    table = soup.find('table')
    
    if not table:
        return ""
    
    markdown_lines = []
    
    # Process header row if exists
    headers = []
    header_row = table.find('thead')
    if header_row:
        headers = [cell.get_text(strip=True) for cell in header_row.find_all(['th', 'td'])]
    else:
        # Try to get headers from first row
        first_row = table.find('tr')
        if first_row:
            headers = [cell.get_text(strip=True) for cell in first_row.find_all(['th', 'td'])]
    
    if headers:
        # Add header row
        markdown_lines.append('| ' + ' | '.join(headers) + ' |')
        # Add separator row
        markdown_lines.append('|' + '|'.join(['---' for _ in headers]) + '|')
    
    # Process data rows
    rows = table.find_all('tr')
    start_index = 1 if headers and not header_row else 0  # Skip first row if used as header
    
    for row in rows[start_index:]:
        cells = row.find_all(['td', 'th'])
        # Clean and format cell text
        cell_text = []
        for cell in cells:
            text = cell.get_text(strip=True)
            # Replace newlines with spaces
            text = re.sub(r'\s+', ' ', text)
            cell_text.append(text)
        
        if any(cell_text):  # Only add row if it contains any text
            markdown_lines.append('| ' + ' | '.join(cell_text) + ' |')
    
    return '\n'.join(markdown_lines)

def element_to_markdown(element):
    if has_page_numbers(element.text):
       return ""
    
    text = ""
    if element.category == 'Title':
        if element.metadata.category_depth == 0:
            text = f"## {element.text}\n\n"
        elif element.metadata.category_depth == 1:
            text = f"### {element.text}\n\n"
        elif element.metadata.category_depth == 2:
            text = f"#### {element.text}\n\n"
        elif element.metadata.category_depth == 3:
            text = f"##### {element.text}\n\n"
        else:
            text = f"## {element.text}\n\n"
    elif element.category == 'Header':
        text = f"## {element.text}\n\n"
    elif element.category == 'SubHeader':
        text = f"### {element.text}\n\n"
    elif element.category == 'ListItem':
        text = f"- {element.text}\n"
    elif element.category == 'Paragraph':
        text = f"{element.text} PARAGRAPH\n\n"
    elif element.category == 'NarrativeText':
        text = f"{element.text}\n\n"        
    elif element.category == 'CompositeElement':   
        text = f"{element.text}\n\n" 
    elif element.category == 'UncategorizedText':  
        text = f"{element.text}\n\n" 
    elif element.category == 'Table':
        text = html_table_to_markdown(element.metadata.text_as_html)
    else:
        print(f"Other:{element.category} - {element.text}")
        text = f"{element.text}\n" 

    return text

def elements_to_markdown(elements):
    # Convert all elements to Markdown
    return "".join([element_to_markdown(el) for el in elements])

def custom_clean(text):
    text = clean_non_ascii_chars(text)
    text = text.replace("•\n", "")
    text = text.replace("- \n", "- ")
    text = replace_unicode_quotes(text)
    return text


/home/bizon/.pyenv/versions/3.11.10/envs/cohere-haystack/lib/python3.11/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/bizon/.config/sagemaker/config.yaml


In [3]:
COMBINE_TEXT_UNDER_N_CHARS=800
MAX_CHARACTERS=1000
NEW_AFTER_N_CHARS=1000
MAX_PARTITION=1000
OVERLAP=100

def html_partition(docs):
    markdown_chunks = []    
    
    for doc in docs:        
        narative_text_elements = partition_html(
            text=doc,
            chunking_strategy='by_title',
            paragraph_grouper=group_broken_paragraphs,
            combine_text_under_n_chars=COMBINE_TEXT_UNDER_N_CHARS,
            max_characters=MAX_CHARACTERS,
            new_after_n_chars=NEW_AFTER_N_CHARS,
            max_partition=MAX_PARTITION,
            overlap=OVERLAP
        )
        
        for composite_element in narative_text_elements:
            markdown = elements_to_markdown(composite_element.metadata.orig_elements)
            cleaned_markdown = custom_clean(markdown)
            
            if not is_empty(cleaned_markdown):
                markdown_chunks.append(cleaned_markdown)
        
    return markdown_chunks


def txt_partition(docs):
    markdown_chunks = []
    
    for doc in docs:  
      narative_text_elements = partition_text(
          text=doc,
          chunking_strategy='by_title',
          paragraph_grouper=group_broken_paragraphs,
          combine_text_under_n_chars=COMBINE_TEXT_UNDER_N_CHARS,
          max_characters=MAX_CHARACTERS,
          new_after_n_chars=NEW_AFTER_N_CHARS,
          max_partition=MAX_PARTITION,
          overlap=OVERLAP
      )
      
      for composite_element in narative_text_elements:
          markdown = elements_to_markdown(composite_element.metadata.orig_elements)
          cleaned_markdown = custom_clean(markdown)
          if not is_empty(cleaned_markdown):
              markdown_chunks.append(cleaned_markdown)
    
    return markdown_chunks  

def is_empty(text):    
    if len(text) == 0:
      return True
    
    return False

def html_txt_partition(docs):
    markdown_chunks = []
    
    for doc in docs:
        doc = doc.replace("<p>", "").replace("</p>", "")
        # narative_text_elements = partition_html(
        #     text=doc,
        #     paragraph_grouper=group_broken_paragraphs,
        # )
        
        narative_text_elements = partition_text(
            text=doc,
            chunking_strategy='by_title',
            paragraph_grouper=group_broken_paragraphs,
            combine_text_under_n_chars=COMBINE_TEXT_UNDER_N_CHARS,
            max_characters=MAX_CHARACTERS,
            new_after_n_chars=NEW_AFTER_N_CHARS,
            max_partition=MAX_PARTITION,
            overlap=OVERLAP
        )
        
        for composite_element in narative_text_elements:
            markdown = elements_to_markdown(composite_element.metadata.orig_elements)
            cleaned_markdown = custom_clean(markdown)
            if not is_empty(cleaned_markdown):
                markdown_chunks.append(cleaned_markdown)
                print(f"============================= CHUNK: {cleaned_markdown}")
    
    print(f"markdown_chunks length: {len(markdown_chunks)}")
    
    over_1000 = sum(1 for chunk in markdown_chunks if len(chunk) > 1000)
    print(f"Number of chunks over 1000 characters: {over_1000}")
                    
    return markdown_chunks

all_chunks = []

all_chunks.extend(html_txt_partition(html_txt_doc_array))
# all_chunks.extend(txt_partition(txt_doc_array))
# all_chunks.extend(html_partition(html_doc_array))



============================= CHUNK: Notulen Raadsvergadering Gemeenteraad Almelo Datum: 10 november 2011 (voortzetting van de vergadering op dinsdag 8 november 2011 te 19.00 uur) Aanvang: 10.00 uur Locatie: Raadzaal Voorzitter: afwisselend mevrouw J.H.M. Hermans-Vloedbeld, burgemeester, en de heer J.J. Stapel, vicevoorzitter. Griffier: mevrouw C.M. Steenbergen, griffier. Aanwezige leden: mevrouw A. Abdullatif, R.B.M. Borgman, J. Brand, J. Buitenweg, U. ete, F.J. Cornelissen (later), mevrouw C.N. Davis, mevrouw I. Dikmen-Yali, mevrouw M.J.C. van Doorn, F.J. Gerritsen, J.J.H. Hammink (later), A. Heite, mevrouw H.J. Heitink, F.E. Van der Horst, A.G.P. Hmmels, H.C. Isendoorn, G.F. Kamphuis, mevrouw N. Kirkaga, B. Kozijn (later), A. Maathuis, H.K. Nijhuis, H.G. de Olde, J. Pauwels, mevrouw H.R. Rettensberger-Stevens, mevrouw H.E. Rozema-Jungjohann, J.L. Schuur, mevrouw I.A.M. ten Seldam-Even, L.K.J. Simmering, J.J. Stapel, G.W.L. Stork, J. Veenstra, G.P. Vos en mevrouw M. Wessels. Afwezige